In [1]:
import os,sys
sys.path.append("../../")
from analysis_common import *
from root_common import *
from analysis_helpers import *
%jsroot on

#Definitions
lep_accept_pt = 0.4 #GeV
lep_accept_angle = radians(40)
hadron_accept_ke = 0.060 #GeV
hadron_accept_angle = radians(40)

dir_files_G18_02a = "/Data/ldmx_eTi_4GeV_G18_02a_00"
dir_files_GEM21_11b = "/Data/ldmx_eTi_4GeV_GEM21_11b_00"
gst_files_G18_02a = glob.glob(f"{dir_files_G18_02a}/*gst.root")
gst_files_GEM21_11b = glob.glob(f"{dir_files_GEM21_11b}/*gst.root")

gst_chain_GEM21_11b = create_gst_chain(gst_files_GEM21_11b,verbose=True)
gst_chain_G18_02a = create_gst_chain(gst_files_G18_02a,verbose=True)

events = 100000
df_gst_GEM21_11b_all = ROOT.RDataFrame(gst_chain_GEM21_11b)
df_gst_GEM21_11b_all = df_gst_GEM21_11b_all.Range(events)

df_gst_G18_02a_all = ROOT.RDataFrame(gst_chain_G18_02a)
df_gst_G18_02a_all = df_gst_G18_02a_all.Range(events)

Welcome to JupyROOT 6.24/06
Created gst chain from 10 files with 1000000 total events.
Created gst chain from 10 files with 1000000 total events.


In [2]:
df_gst_G18_02a_all = define_df_gst_lep_vars(df_gst_G18_02a_all)
df_gst_G18_02a_all = define_df_gst_hadron_vars(df_gst_G18_02a_all)
df_gst_G18_02a_all = define_df_gst_hadrons_by_pdg(df_gst_G18_02a_all)
df_gst_G18_02a_all = define_df_gst_pi0decay(df_gst_G18_02a_all)
df_gst_G18_02a_all = define_df_gst_hadron_acceptance(df_gst_G18_02a_all)
df_gst_G18_02a_all = define_df_gst_photon_acceptance(df_gst_G18_02a_all)
df_gst_G18_02a_all = define_df_gst_hadron_sums(df_gst_G18_02a_all,sfx=["i","f","fa"])
df_gst_G18_02a_all = define_df_gst_momentum_imbalance(df_gst_G18_02a_all,suffix_list=["i"],cname="i")
df_gst_G18_02a_all = define_df_gst_momentum_imbalance(df_gst_G18_02a_all,suffix_list=["f"],cname="f")
df_gst_G18_02a_all = define_df_gst_momentum_imbalance(df_gst_G18_02a_all,cname="fa_all", suffix_list=["fa_proton","fa_neutron", "fa_piplus","fa_piminus","fa_pi0_ph"])
df_gst_G18_02a_all = define_df_gst_momentum_imbalance(df_gst_G18_02a_all,cname="fa_chhad",suffix_list=["fa_proton","fa_piplus","fa_piminus"])

In [3]:
df_gst_GEM21_11b_all = define_df_gst_lep_vars(df_gst_GEM21_11b_all)
df_gst_GEM21_11b_all = define_df_gst_hadron_vars(df_gst_GEM21_11b_all)
df_gst_GEM21_11b_all = define_df_gst_hadrons_by_pdg(df_gst_GEM21_11b_all)
df_gst_GEM21_11b_all = define_df_gst_pi0decay(df_gst_GEM21_11b_all)
df_gst_GEM21_11b_all = define_df_gst_hadron_acceptance(df_gst_GEM21_11b_all)
df_gst_GEM21_11b_all = define_df_gst_photon_acceptance(df_gst_GEM21_11b_all)
df_gst_GEM21_11b_all = define_df_gst_hadron_sums(df_gst_GEM21_11b_all,sfx=["i","f","fa"])
df_gst_GEM21_11b_all = define_df_gst_momentum_imbalance(df_gst_GEM21_11b_all,suffix_list=["i"],cname="i")
df_gst_GEM21_11b_all = define_df_gst_momentum_imbalance(df_gst_GEM21_11b_all,suffix_list=["f"],cname="f")
df_gst_GEM21_11b_all = define_df_gst_momentum_imbalance(df_gst_GEM21_11b_all,cname="fa_all", suffix_list=["fa_proton","fa_neutron", "fa_piplus","fa_piminus","fa_pi0_ph"])
df_gst_GEM21_11b_all = define_df_gst_momentum_imbalance(df_gst_GEM21_11b_all,cname="fa_chhad",suffix_list=["fa_proton","fa_piplus","fa_piminus"])

In [4]:
#Giana's added definitions
#num of final state pions (+ & - & neutral)
df_gst_GEM21_11b_all = df_gst_GEM21_11b_all.Define("num_pions", "nfpip+nfpim+nfpi0") 
df_gst_G18_02a_all = df_gst_G18_02a_all.Define("num_pions", "nfpip+nfpim+nfpi0") 
#sum of initial state pions (+ & - & neutral)
df_gst_GEM21_11b_all = df_gst_GEM21_11b_all.Define("numi_pions", "nipip+nipim+nipi0")
df_gst_G18_02a_all = df_gst_G18_02a_all.Define("numi_pions", "nipip+nipim+nipi0")

#most energetic proton final state
df_gst_GEM21_11b_all = df_gst_GEM21_11b_all.Filter("nfp > 0").Define("kef_proton_max", "Max(kef[pdgf==2212])")
df_gst_G18_02a_all = df_gst_G18_02a_all.Filter("nfp > 0").Define("kef_proton_max", "Max(kef[pdgf==2212])")
#most energetic proton in final state acceptance
df_gst_GEM21_11b_all = df_gst_GEM21_11b_all.Filter("nfp > 0").Define("kefa_proton_max", "Max(kefa_proton)")
df_gst_G18_02a_all = df_gst_G18_02a_all.Filter("nfp > 0").Define("kefa_proton_max", "Max(kefa_proton)")
#most energetic neutron in final state
df_gst_GEM21_11b_all = df_gst_GEM21_11b_all.Filter("nfn > 0").Define("kef_neutron_max", "Max(kef[pdgf==2112])")
df_gst_G18_02a_all = df_gst_G18_02a_all.Filter("nfn > 0").Define("kef_neutron_max", "Max(kef[pdgf==2112])")
#most energetic neutron in final state acceptance
df_gst_GEM21_11b_all = df_gst_GEM21_11b_all.Filter("nfn > 0").Define("kefa_neutron_max", "Max(kefa_neutron)")
df_gst_G18_02a_all = df_gst_G18_02a_all.Filter("nfn > 0").Define("kefa_neutron_max", "Max(kefa_neutron)")

#kef of protons above cut
df_gst_GEM21_11b_all = df_gst_GEM21_11b_all.Define("kef_above_threshold_protons",f"kef[kef>{hadron_accept_ke}&&pdgf==2212]") #kefa_proton
df_gst_G18_02a_all = df_gst_G18_02a_all.Define("kef_above_threshold_protons",f"kef[kef>{hadron_accept_ke}&&pdgf==2212]")
#kef of neutrons above cut
df_gst_GEM21_11b_all = df_gst_GEM21_11b_all.Define("kef_above_threshold_neutrons",f"kef[kef>{hadron_accept_ke}&&pdgf==2112]")
df_gst_G18_02a_all = df_gst_G18_02a_all.Define("kef_above_threshold_neutrons",f"kef[kef>{hadron_accept_ke}&&pdgf==2112]")
#kef of pi0 above cut
df_gst_GEM21_11b_all = df_gst_GEM21_11b_all.Define("kef_above_threshold_pi0",f"kef[kef>{hadron_accept_ke}&&pdgf==111]")
df_gst_G18_02a_all = df_gst_G18_02a_all.Define("kef_above_threshold_pi0",f"kef[kef>{hadron_accept_ke}&&pdgf==111]")
#kef of charged pions above cut
df_gst_GEM21_11b_all = df_gst_GEM21_11b_all.Define("kef_above_threshold_pions",f"kef[kef>{hadron_accept_ke}&&(pdgf==211||pdgf==-211)]")
df_gst_G18_02a_all = df_gst_G18_02a_all.Define("kef_above_threshold_pions",f"kef[kef>{hadron_accept_ke}&&(pdgf==211||pdgf==-211)]")
#num of protons and neutrons
df_gst_GEM21_11b_all = df_gst_GEM21_11b_all.Define("n_proton_neutron","nfn + nfp")
df_gst_G18_02a_all = df_gst_G18_02a_all.Define("n_proton_neutron","nfn + nfp")
#sum of sum of ke of protons and neutrons
df_gst_GEM21_11b_all = df_gst_GEM21_11b_all.Define("sum_kef_pn","sum_kef_proton + sum_kef_neutron")
df_gst_G18_02a_all = df_gst_G18_02a_all.Define("sum_kef_pn","sum_kef_proton + sum_kef_neutron")
#nfa pions nfa_piplus+nfa_piminus+nfa_pi0
df_gst_GEM21_11b_all = df_gst_GEM21_11b_all.Define("nfa_pions","nfa_piplus+nfa_piminus+nfa_pi0")
df_gst_G18_02a_all = df_gst_G18_02a_all.Define("nfa_pions","nfa_piplus+nfa_piminus+nfa_pi0")

#leading proton angle in final state
df_gst_G18_02a_all = df_gst_G18_02a_all.Filter("nfp > 0").Define("leading_proton_f", "ArgMax(kef_proton)").Define("leading_protonf_angle", "thetazf_proton.at(leading_proton_f)")
df_gst_GEM21_11b_all = df_gst_GEM21_11b_all.Filter("nfp > 0").Define("leading_proton_f", "ArgMax(kef_proton)").Define("leading_protonf_angle", "thetazf_proton.at(leading_proton_f)")
#leading proton angle in final state acceptance
df_gst_G18_02a_all = df_gst_G18_02a_all.Filter("nfp > 0").Define("leading_proton_fa", "ArgMax(kefa_proton)").Define("leading_protonfa_angle", "thetazfa_proton.at(leading_proton_fa)")
df_gst_GEM21_11b_all = df_gst_GEM21_11b_all.Filter("nfp > 0").Define("leading_proton_fa", "ArgMax(kefa_proton)").Define("leading_protonfa_angle", "thetazfa_proton.at(leading_proton_fa)")
#leading neutron angle
df_gst_G18_02a_all = df_gst_G18_02a_all.Filter("nfn > 0").Define("leading_neutron_f", "ArgMax(kef_neutron)").Define("leading_neutronf_angle", "thetazf_neutron.at(leading_neutron_f)")
df_gst_GEM21_11b_all = df_gst_GEM21_11b_all.Filter("nfn > 0").Define("leading_neutron_f", "ArgMax(kef_neutron)").Define("leading_neutronf_angle", "thetazf_neutron.at(leading_neutron_f)")
#leading neutron angle
df_gst_G18_02a_all = df_gst_G18_02a_all.Filter("nfn > 0").Define("leading_neutron_fa", "ArgMax(kefa_neutron)").Define("leading_neutronfa_angle", "thetazfa_neutron.at(leading_neutron_fa)")
df_gst_GEM21_11b_all = df_gst_GEM21_11b_all.Filter("nfn > 0").Define("leading_neutron_fa", "ArgMax(kefa_neutron)").Define("leading_neutronfa_angle", "thetazfa_neutron.at(leading_neutron_fa)")

In [ ]:
# Plots of Lepton Pt Momenta for interaction types
x_start = 0 
x_end = 1.6
bins = 100
lep_pt_cut = 0

In [ ]:
#g18 no lep pt cut 
b = ROOT.TCanvas("b")
df_gst_G18_02a_leppt = df_gst_G18_02a_all.Filter(f"ptl>{lep_pt_cut}")
G18_lep_pt_QEL = df_gst_G18_02a_leppt.Filter("qel==1").Histo1D(("G18_lep_pt_QEL", "Transverse Momenta in Interaction Types in G18 Model", bins, x_start, x_end), "ptl")
G18_lep_pt_QEL.SetLineColor(ROOT.kViolet)
G18_lep_pt_QEL.GetXaxis().SetTitle("Transverse Momenta (GeV)")
G18_lep_pt_QEL.GetYaxis().SetTitle("Counts")
G18_lep_pt_QEL.Draw("same")

G18_lep_pt_MEC = df_gst_G18_02a_leppt.Filter("mec==1").Histo1D(("G18_lep_pt_MEC", "", bins, x_start, x_end), "ptl")
G18_lep_pt_MEC.SetLineColor(ROOT.kRed)
G18_lep_pt_MEC.Draw("same")

G18_lep_pt_RES = df_gst_G18_02a_leppt.Filter("res==1").Histo1D(("G18_lep_pt_RES", "", bins, x_start, x_end), "ptl")
G18_lep_pt_RES.SetLineColor(ROOT.kGreen)
G18_lep_pt_RES.Draw("same")

G18_lep_pt_DIS = df_gst_G18_02a_leppt.Filter("dis==1").Histo1D(("G18_lep_pt_DIS", "", bins, x_start, x_end), "ptl")
G18_lep_pt_DIS.SetLineColor(ROOT.kBlue)
G18_lep_pt_DIS.Draw("same")

G18_lep_pt_none = df_gst_G18_02a_leppt.Filter("qel==0&&mec==0&&res==0&&dis==0").Histo1D(("G18_lep_pt_none", "", bins, x_start, x_end), "ptl")
G18_lep_pt_none.SetLineColor(ROOT.kGray)
G18_lep_pt_none.Draw("same")

G18_lep_pt_leg = ROOT.TLegend(.6, .66, .87, .87)
G18_lep_pt_leg.AddEntry("G18_lep_pt_QEL", "QE", "l")
G18_lep_pt_leg.AddEntry("G18_lep_pt_MEC", "MEC", "l")
G18_lep_pt_leg.AddEntry("G18_lep_pt_RES", "RES", "l")
G18_lep_pt_leg.AddEntry("G18_lep_pt_DIS", "DIS", "l")
G18_lep_pt_leg.AddEntry("G18_lep_pt_none", "Uncatagorized", "l")
G18_lep_pt_leg.Draw()

b.SetLogy()
b.Draw()

In [ ]:
#gem21
e = ROOT.TCanvas("e")
df_gst_GEM21_11b_leppt = df_gst_GEM21_11b_all.Filter(f"ptl>{lep_pt_cut}")
GEM_lep_pt_QEL = df_gst_GEM21_11b_leppt.Filter("qel==1").Histo1D(("GEM_lep_pt_QEL", "Transverse Momenta in Interaction Types in GEM21 Model", bins, x_start, x_end), "ptl")
GEM_lep_pt_QEL.SetLineColor(ROOT.kViolet)
GEM_lep_pt_QEL.GetXaxis().SetTitle("Transverse Momenta (GeV)")
GEM_lep_pt_QEL.GetYaxis().SetTitle("Counts")
GEM_lep_pt_QEL.Draw("same")

GEM_lep_pt_MEC = df_gst_GEM21_11b_leppt.Filter("mec==1").Histo1D(("GEM_lep_pt_MEC", "", bins, x_start, x_end), "ptl")
GEM_lep_pt_MEC.SetLineColor(ROOT.kRed)
GEM_lep_pt_MEC.Draw("same")

GEM_lep_pt_RES = df_gst_GEM21_11b_leppt.Filter("res==1").Histo1D(("GEM_lep_pt_RES", "", bins, x_start, x_end), "ptl")
GEM_lep_pt_RES.SetLineColor(ROOT.kGreen)
GEM_lep_pt_RES.Draw("same")

GEM_lep_pt_DIS = df_gst_GEM21_11b_leppt.Filter("dis==1").Histo1D(("GEM_lep_pt_DIS", "", bins, x_start, x_end), "ptl")
GEM_lep_pt_DIS.SetLineColor(ROOT.kBlue)
GEM_lep_pt_DIS.Draw("same")

GEM_lep_pt_none = df_gst_GEM21_11b_leppt.Filter("qel==0&&mec==0&&res==0&&dis==0").Histo1D(("GEM_lep_pt_none", "", bins, x_start, x_end), "ptl")
GEM_lep_pt_none.SetLineColor(ROOT.kGray)
GEM_lep_pt_none.Draw("same")

GEM_lep_pt_leg = ROOT.TLegend(.6, .66, .87, .87)
GEM_lep_pt_leg.AddEntry("GEM_lep_pt_QEL", "QE", "l")
GEM_lep_pt_leg.AddEntry("GEM_lep_pt_MEC", "MEC", "l")
GEM_lep_pt_leg.AddEntry("GEM_lep_pt_RES", "RES", "l")
GEM_lep_pt_leg.AddEntry("GEM_lep_pt_DIS", "DIS", "l")
GEM_lep_pt_leg.AddEntry("GEM_lep_pt_none", "Uncatagorized", "l")
GEM_lep_pt_leg.Draw()

e.SetLogy()
e.Draw()

In [ ]:
#g18 without initial state pions and no lep pt cut
b = ROOT.TCanvas("b")
df_gst_G18_02a_leppt_NIP = df_gst_G18_02a_all.Filter(f"ptl>{lep_pt_cut}").Filter("numi_pions==0")
G18_lep_pt_QEL_NIP = df_gst_G18_02a_leppt_NIP.Filter("qel==1").Histo1D(("G18_lep_pt_QEL_NIP", "Transverse Momenta in Interaction Types w/o Initial State Pions in G18 Model", bins, x_start, x_end), "ptl")
G18_lep_pt_QEL_NIP.SetLineColor(ROOT.kViolet)
G18_lep_pt_QEL_NIP.GetXaxis().SetTitle("Transverse Momenta (GeV)")
G18_lep_pt_QEL_NIP.GetYaxis().SetTitle("Counts")
G18_lep_pt_QEL_NIP.Draw("same")

G18_lep_pt_MEC_NIP = df_gst_G18_02a_leppt_NIP.Filter("mec==1").Histo1D(("G18_lep_pt_MEC_NIP", "", bins, x_start, x_end), "ptl")
G18_lep_pt_MEC_NIP.SetLineColor(ROOT.kRed)
G18_lep_pt_MEC_NIP.Draw("same")

G18_lep_pt_RES_NIP = df_gst_G18_02a_leppt_NIP.Filter("res==1").Histo1D(("G18_lep_pt_RES_NIP", "", bins, x_start, x_end), "ptl")
G18_lep_pt_RES_NIP.SetLineColor(ROOT.kGreen)
G18_lep_pt_RES_NIP.Draw("same")

G18_lep_pt_DIS_NIP = df_gst_G18_02a_leppt_NIP.Filter("dis==1").Histo1D(("G18_lep_pt_DIS_NIP", "", bins, x_start, x_end), "ptl")
G18_lep_pt_DIS_NIP.SetLineColor(ROOT.kBlue)
G18_lep_pt_DIS_NIP.Draw("same")

G18_lep_pt_none_NIP = df_gst_G18_02a_leppt_NIP.Filter("qel==0&&mec==0&&res==0&&dis==0").Histo1D(("G18_lep_pt_none_NIP", "", bins, x_start, x_end), "ptl")
G18_lep_pt_none_NIP.SetLineColor(ROOT.kGray)
G18_lep_pt_none_NIP.Draw("same")

G18_lep_pt_leg_NIP = ROOT.TLegend(.6, .66, .87, .87)
G18_lep_pt_leg_NIP.AddEntry("G18_lep_pt_QEL_NIP", "QE", "l")
G18_lep_pt_leg_NIP.AddEntry("G18_lep_pt_MEC_NIP", "MEC", "l")
G18_lep_pt_leg_NIP.AddEntry("G18_lep_pt_RES_NIP", "RES", "l")
G18_lep_pt_leg_NIP.AddEntry("G18_lep_pt_DIS_NIP", "DIS", "l")
G18_lep_pt_leg_NIP.AddEntry("G18_lep_pt_none_NIP", "Uncatagorized", "l")
G18_lep_pt_leg_NIP.Draw()

b.SetLogy()
b.Draw()

In [ ]:
#g21 without initial state pions and no lep pt cut
b = ROOT.TCanvas("b")
df_gst_GEM21_11b_leppt_NIP = df_gst_GEM21_11b_all.Filter(f"ptl>{lep_pt_cut}").Filter("numi_pions==0")
G21_lep_pt_QEL_NIP = df_gst_GEM21_11b_leppt_NIP.Filter("qel==1").Histo1D(("G21_lep_pt_QEL_NIP", "Transverse Momenta in Interaction Types w/o Initial State Pions in GEM21 Model", bins, x_start, x_end), "ptl")
G21_lep_pt_QEL_NIP.SetLineColor(ROOT.kViolet)
G21_lep_pt_QEL_NIP.GetXaxis().SetTitle("Transverse Momenta (GeV)")
G21_lep_pt_QEL_NIP.GetYaxis().SetTitle("Counts")
G21_lep_pt_QEL_NIP.Draw("same")

G21_lep_pt_MEC_NIP = df_gst_GEM21_11b_leppt_NIP.Filter("mec==1").Histo1D(("G21_lep_pt_MEC_NIP", "", bins, x_start, x_end), "ptl")
G21_lep_pt_MEC_NIP.SetLineColor(ROOT.kRed)
G21_lep_pt_MEC_NIP.Draw("same")

G21_lep_pt_RES_NIP = df_gst_GEM21_11b_leppt_NIP.Filter("res==1").Histo1D(("G21_lep_pt_RES_NIP", "", bins, x_start, x_end), "ptl")
G21_lep_pt_RES_NIP.SetLineColor(ROOT.kGreen)
G21_lep_pt_RES_NIP.Draw("same")

G21_lep_pt_DIS_NIP = df_gst_GEM21_11b_leppt_NIP.Filter("dis==1").Histo1D(("G21_lep_pt_DIS_NIP", "", bins, x_start, x_end), "ptl")
G21_lep_pt_DIS_NIP.SetLineColor(ROOT.kBlue)
G21_lep_pt_DIS_NIP.Draw("same")

G21_lep_pt_none_NIP = df_gst_GEM21_11b_leppt_NIP.Filter("qel==0&&mec==0&&res==0&&dis==0").Histo1D(("G21_lep_pt_none_NIP", "", bins, x_start, x_end), "ptl")
G21_lep_pt_none_NIP.SetLineColor(ROOT.kGray)
G21_lep_pt_none_NIP.Draw("same")

G21_lep_pt_leg_NIP = ROOT.TLegend(.6, .66, .87, .87)
G21_lep_pt_leg_NIP.AddEntry("G21_lep_pt_QEL_NIP", "QE", "l")
G21_lep_pt_leg_NIP.AddEntry("G21_lep_pt_MEC_NIP", "MEC", "l")
G21_lep_pt_leg_NIP.AddEntry("G21_lep_pt_RES_NIP", "RES", "l")
G21_lep_pt_leg_NIP.AddEntry("G21_lep_pt_DIS_NIP", "DIS", "l")
G21_lep_pt_leg_NIP.AddEntry("G21_lep_pt_none_NIP", "Uncatagorized", "l")
G21_lep_pt_leg_NIP.Draw()

b.SetLogy()
b.Draw()

In [ ]:
#g18 without final state pions and no lep pt cut
b = ROOT.TCanvas("b")
df_gst_G18_02a_leppt_NFP = df_gst_G18_02a_all.Filter(f"ptl>{lep_pt_cut}").Filter("num_pions==0")
G18_lep_pt_QEL_NFP = df_gst_G18_02a_leppt_NFP.Filter("qel==1").Histo1D(("G18_lep_pt_QEL_NFP", "Transverse Momenta in Interaction Types w/o Final State Pions in G18 Model", bins, x_start, x_end), "ptl")
G18_lep_pt_QEL_NFP.SetLineColor(ROOT.kViolet)
G18_lep_pt_QEL_NFP.GetXaxis().SetTitle("Transverse Momenta (GeV)")
G18_lep_pt_QEL_NFP.GetYaxis().SetTitle("Counts")
G18_lep_pt_QEL_NFP.Draw("same")

G18_lep_pt_MEC_NFP = df_gst_G18_02a_leppt_NFP.Filter("mec==1").Histo1D(("G18_lep_pt_MEC_NFP", "", bins, x_start, x_end), "ptl")
G18_lep_pt_MEC_NFP.SetLineColor(ROOT.kRed)
G18_lep_pt_MEC_NFP.Draw("same")

G18_lep_pt_RES_NFP = df_gst_G18_02a_leppt_NFP.Filter("res==1").Histo1D(("G18_lep_pt_RES_NFP", "", bins, x_start, x_end), "ptl")
G18_lep_pt_RES_NFP.SetLineColor(ROOT.kGreen)
G18_lep_pt_RES_NFP.Draw("same")

G18_lep_pt_DIS_NFP = df_gst_G18_02a_leppt_NFP.Filter("dis==1").Histo1D(("G18_lep_pt_DIS_NFP", "", bins, x_start, x_end), "ptl")
G18_lep_pt_DIS_NFP.SetLineColor(ROOT.kBlue)
G18_lep_pt_DIS_NFP.Draw("same")

G18_lep_pt_none_NFP = df_gst_G18_02a_leppt_NFP.Filter("qel==0&&mec==0&&res==0&&dis==0").Histo1D(("G18_lep_pt_none_NFP", "", bins, x_start, x_end), "ptl")
G18_lep_pt_none_NFP.SetLineColor(ROOT.kGray)
G18_lep_pt_none_NFP.Draw("same")

G18_lep_pt_leg_NFP = ROOT.TLegend(.6, .66, .87, .87)
G18_lep_pt_leg_NFP.AddEntry("G18_lep_pt_QEL_NFP", "QE", "l")
G18_lep_pt_leg_NFP.AddEntry("G18_lep_pt_MEC_NFP", "MEC", "l")
G18_lep_pt_leg_NFP.AddEntry("G18_lep_pt_RES_NFP", "RES", "l")
G18_lep_pt_leg_NFP.AddEntry("G18_lep_pt_DIS_NFP", "DIS", "l")
G18_lep_pt_leg_NFP.AddEntry("G18_lep_pt_none_NFP", "Uncatagorized", "l")
G18_lep_pt_leg_NFP.Draw()

b.SetLogy()
b.Draw()

In [ ]:
#g21 without final state pions and no lep pt cut
b = ROOT.TCanvas("b")
df_gst_GEM21_11b_leppt_NFP = df_gst_GEM21_11b_all.Filter(f"ptl>{lep_pt_cut}").Filter("num_pions==0")
G21_lep_pt_QEL_NFP = df_gst_GEM21_11b_leppt_NFP.Filter("qel==1").Histo1D(("G21_lep_pt_QEL_NFP", "Transverse Momenta in Interaction Types w/o Final State Pions in GEM21 Model", bins, x_start, x_end), "ptl")
G21_lep_pt_QEL_NFP.SetLineColor(ROOT.kViolet)
G21_lep_pt_QEL_NFP.GetXaxis().SetTitle("Transverse Momenta (GeV)")
G21_lep_pt_QEL_NFP.GetYaxis().SetTitle("Counts")
G21_lep_pt_QEL_NFP.Draw("same")

G21_lep_pt_MEC_NFP = df_gst_GEM21_11b_leppt_NFP.Filter("mec==1").Histo1D(("G21_lep_pt_MEC_NFP", "", bins, x_start, x_end), "ptl")
G21_lep_pt_MEC_NFP.SetLineColor(ROOT.kRed)
G21_lep_pt_MEC_NFP.Draw("same")

G21_lep_pt_RES_NFP = df_gst_GEM21_11b_leppt_NFP.Filter("res==1").Histo1D(("G21_lep_pt_RES_NFP", "", bins, x_start, x_end), "ptl")
G21_lep_pt_RES_NFP.SetLineColor(ROOT.kGreen)
G21_lep_pt_RES_NFP.Draw("same")

G21_lep_pt_DIS_NFP = df_gst_GEM21_11b_leppt_NFP.Filter("dis==1").Histo1D(("G21_lep_pt_DIS_NFP", "", bins, x_start, x_end), "ptl")
G21_lep_pt_DIS_NFP.SetLineColor(ROOT.kBlue)
G21_lep_pt_DIS_NFP.Draw("same")

G21_lep_pt_none_NFP = df_gst_GEM21_11b_leppt_NFP.Filter("qel==0&&mec==0&&res==0&&dis==0").Histo1D(("G21_lep_pt_none_NFP", "", bins, x_start, x_end), "ptl")
G21_lep_pt_none_NFP.SetLineColor(ROOT.kGray)
G21_lep_pt_none_NFP.Draw("same")

G21_lep_pt_leg_NFP = ROOT.TLegend(.6, .66, .87, .87)
G21_lep_pt_leg_NFP.AddEntry("G21_lep_pt_QEL_NFP", "QE", "l")
G21_lep_pt_leg_NFP.AddEntry("G21_lep_pt_MEC_NFP", "MEC", "l")
G21_lep_pt_leg_NFP.AddEntry("G21_lep_pt_RES_NFP", "RES", "l")
G21_lep_pt_leg_NFP.AddEntry("G21_lep_pt_DIS_NFP", "DIS", "l")
G21_lep_pt_leg_NFP.AddEntry("G21_lep_pt_none_NFP", "Uncatagorized", "l")
G21_lep_pt_leg_NFP.Draw()

b.SetLogy()
b.Draw()

In [ ]:
#g18 without nfa pions and no lep pt cut
b = ROOT.TCanvas("b")
df_gst_G18_02a_leppt_NFA = df_gst_G18_02a_all.Filter(f"ptl>{lep_pt_cut}").Filter("nfa_pions==0")
G18_lep_pt_QEL_NFA = df_gst_G18_02a_leppt_NFA.Filter("qel==1").Histo1D(("G18_lep_pt_QEL_NFA", "Transverse Momenta in Interaction Types w/o NFA Pions in G18 Model", bins, x_start, x_end), "ptl")
G18_lep_pt_QEL_NFA.SetLineColor(ROOT.kViolet)
G18_lep_pt_QEL_NFA.GetXaxis().SetTitle("Transverse Momenta (GeV)")
G18_lep_pt_QEL_NFA.GetYaxis().SetTitle("Counts")
G18_lep_pt_QEL_NFA.Draw("same")

G18_lep_pt_MEC_NFA = df_gst_G18_02a_leppt_NFA.Filter("mec==1").Histo1D(("G18_lep_pt_MEC_NFA", "", bins, x_start, x_end), "ptl")
G18_lep_pt_MEC_NFA.SetLineColor(ROOT.kRed)
G18_lep_pt_MEC_NFA.Draw("same")

G18_lep_pt_RES_NFA = df_gst_G18_02a_leppt_NFA.Filter("res==1").Histo1D(("G18_lep_pt_RES_NFA", "", bins, x_start, x_end), "ptl")
G18_lep_pt_RES_NFA.SetLineColor(ROOT.kGreen)
G18_lep_pt_RES_NFA.Draw("same")

G18_lep_pt_DIS_NFA = df_gst_G18_02a_leppt_NFA.Filter("dis==1").Histo1D(("G18_lep_pt_DIS_NFA", "", bins, x_start, x_end), "ptl")
G18_lep_pt_DIS_NFA.SetLineColor(ROOT.kBlue)
G18_lep_pt_DIS_NFA.Draw("same")

G18_lep_pt_none_NFA = df_gst_G18_02a_leppt_NFA.Filter("qel==0&&mec==0&&res==0&&dis==0").Histo1D(("G18_lep_pt_none_NFA", "", bins, x_start, x_end), "ptl")
G18_lep_pt_none_NFA.SetLineColor(ROOT.kGray)
G18_lep_pt_none_NFA.Draw("same")

G18_lep_pt_leg_NFA = ROOT.TLegend(.6, .66, .87, .87)
G18_lep_pt_leg_NFA.AddEntry("G18_lep_pt_QEL_NFA", "QE", "l")
G18_lep_pt_leg_NFA.AddEntry("G18_lep_pt_MEC_NFA", "MEC", "l")
G18_lep_pt_leg_NFA.AddEntry("G18_lep_pt_RES_NFA", "RES", "l")
G18_lep_pt_leg_NFA.AddEntry("G18_lep_pt_DIS_NFA", "DIS", "l")
G18_lep_pt_leg_NFA.AddEntry("G18_lep_pt_none_NFA", "Uncatagorized", "l")
G18_lep_pt_leg_NFA.Draw()

b.SetLogy()
b.Draw()

In [ ]:
#g21 without nfa pions and no lep pt cut
b = ROOT.TCanvas("b")
df_gst_GEM21_11b_leppt_NFA = df_gst_GEM21_11b_all.Filter(f"ptl>{lep_pt_cut}").Filter("nfa_pions==0")
G21_lep_pt_QEL_NFA = df_gst_GEM21_11b_leppt_NFA.Filter("qel==1").Histo1D(("G21_lep_pt_QEL_NFA", "Transverse Momenta in Interaction Types w/o NFA Pions in GEM21 Model", bins, x_start, x_end), "ptl")
G21_lep_pt_QEL_NFA.SetLineColor(ROOT.kViolet)
G21_lep_pt_QEL_NFA.GetXaxis().SetTitle("Transverse Momenta (GeV)")
G21_lep_pt_QEL_NFA.GetYaxis().SetTitle("Counts")
G21_lep_pt_QEL_NFA.Draw("same")

G21_lep_pt_MEC_NFA = df_gst_GEM21_11b_leppt_NFA.Filter("mec==1").Histo1D(("G21_lep_pt_MEC_NFA", "", bins, x_start, x_end), "ptl")
G21_lep_pt_MEC_NFA.SetLineColor(ROOT.kRed)
G21_lep_pt_MEC_NFA.Draw("same")

G21_lep_pt_RES_NFA = df_gst_GEM21_11b_leppt_NFA.Filter("res==1").Histo1D(("G21_lep_pt_RES_NFA", "", bins, x_start, x_end), "ptl")
G21_lep_pt_RES_NFA.SetLineColor(ROOT.kGreen)
G21_lep_pt_RES_NFA.Draw("same")

G21_lep_pt_DIS_NFA = df_gst_GEM21_11b_leppt_NFA.Filter("dis==1").Histo1D(("G21_lep_pt_DIS_NFA", "", bins, x_start, x_end), "ptl")
G21_lep_pt_DIS_NFA.SetLineColor(ROOT.kBlue)
G21_lep_pt_DIS_NFA.Draw("same")

G21_lep_pt_none_NFA = df_gst_GEM21_11b_leppt_NFA.Filter("qel==0&&mec==0&&res==0&&dis==0").Histo1D(("G21_lep_pt_none_NFA", "", bins, x_start, x_end), "ptl")
G21_lep_pt_none_NFA.SetLineColor(ROOT.kGray)
G21_lep_pt_none_NFA.Draw("same")

G21_lep_pt_leg_NFA = ROOT.TLegend(.6, .66, .87, .87)
G21_lep_pt_leg_NFA.AddEntry("G21_lep_pt_QEL_NFA", "QE", "l")
G21_lep_pt_leg_NFA.AddEntry("G21_lep_pt_MEC_NFA", "MEC", "l")
G21_lep_pt_leg_NFA.AddEntry("G21_lep_pt_RES_NFA", "RES", "l")
G21_lep_pt_leg_NFA.AddEntry("G21_lep_pt_DIS_NFA", "DIS", "l")
G21_lep_pt_leg_NFA.AddEntry("G21_lep_pt_none_NFA", "Uncatagorized", "l")
G21_lep_pt_leg_NFA.Draw()

b.SetLogy()
b.Draw()

In [ ]:
# Plots of num_pions per interaction event
x_start = 0
x_end = 10
bins = 10

In [ ]:
#g18
g = ROOT.TCanvas("g")

G18_pion_QEL = df_gst_G18_02a_all.Filter("qel==1").Histo1D(("G18_pion_QEL", "Pion Amounts Per Interaction Type in G18 Model", bins, x_start, x_end), "num_pions")
G18_pion_QEL.SetLineColor(ROOT.kViolet)
G18_pion_QEL.Draw("same")

G18_pion_MEC = df_gst_G18_02a_all.Filter("mec==1").Histo1D(("G18_pion_MEC", "", bins, x_start, x_end), "num_pions")
G18_pion_MEC.SetLineColor(ROOT.kRed)
G18_pion_MEC.Draw("same")

G18_pion_RES = df_gst_G18_02a_all.Filter("res==1").Histo1D(("G18_pion_RES", "", bins, x_start, x_end), "num_pions")
G18_pion_RES.SetLineColor(ROOT.kGreen)
G18_pion_RES.Draw("same")

G18_pion_DIS = df_gst_G18_02a_all.Filter("dis==1").Histo1D(("G18_pion_DIS", "", bins, x_start, x_end), "num_pions")
G18_pion_DIS.SetLineColor(ROOT.kBlue)
G18_pion_DIS.Draw("same")

G18_pion_none = df_gst_G18_02a_all.Filter("qel==0&&mec==0&&res==0&&dis==0").Histo1D(("G18_pion_none", "", bins, x_start, x_end), "num_pions")
G18_pion_none.SetLineColor(ROOT.kGray)
G18_pion_none.Draw("same")

G18_pion_leg = ROOT.TLegend(.6, .66, .87, .87)
G18_pion_leg.AddEntry("G18_pion_QEL", "QEL", "l")
G18_pion_leg.AddEntry("G18_pion_MEC", "MEC", "l")
G18_pion_leg.AddEntry("G18_pion_RES", "RES", "l")
G18_pion_leg.AddEntry("G18_pion_DIS", "DIS", "l")
G18_pion_leg.AddEntry("G18_pion_none", "Uncatagorized", "l")

G18_pion_leg.Draw()
g.Draw()

In [ ]:
#gem21
f = ROOT.TCanvas("f")
GEM_pion_QEL = df_gst_GEM21_11b_all.Filter("qel==1").Histo1D(("GEM_pion_QEL", "Pion Amounts Per Interaction Type in GEM21 Model", bins, x_start, x_end), "num_pions")
GEM_pion_QEL.SetLineColor(ROOT.kViolet)
GEM_pion_QEL.Draw("same")

GEM_pion_MEC = df_gst_GEM21_11b_all.Filter("mec==1").Histo1D(("GEM_pion_MEC", "", bins, x_start, x_end), "num_pions")
GEM_pion_MEC.SetLineColor(ROOT.kRed)
GEM_pion_MEC.Draw("same")

GEM_pion_RES = df_gst_GEM21_11b_all.Filter("res==1").Histo1D(("GEM_pion_RES", "", bins, x_start, x_end), "num_pions")
GEM_pion_RES.SetLineColor(ROOT.kGreen)
GEM_pion_RES.Draw("same")

GEM_pion_DIS = df_gst_GEM21_11b_all.Filter("dis==1").Histo1D(("GEM_pion_DIS", "", bins, x_start, x_end), "num_pions")
GEM_pion_DIS.SetLineColor(ROOT.kBlue)
GEM_pion_DIS.Draw("same")

GEM_pion_none = df_gst_GEM21_11b_all.Filter("qel==0 && mec==0 && res==0 && dis==0").Histo1D(("GEM_pion_none", "", bins, x_start, x_end), "num_pions")
GEM_pion_none.SetLineColor(ROOT.kGray)
GEM_pion_none.Draw("same")

GEM_pion_leg = ROOT.TLegend(.6, .66, .87, .87)
GEM_pion_leg.AddEntry("GEM_pion_QEL", "QEL", "l")
GEM_pion_leg.AddEntry("GEM_pion_MEC", "MEC", "l")
GEM_pion_leg.AddEntry("GEM_pion_RES", "RES", "l")
GEM_pion_leg.AddEntry("GEM_pion_DIS", "DIS", "l")
GEM_pion_leg.AddEntry("GEM_pion_none", "Uncatagorized", "l")

GEM_pion_leg.Draw()
f.Draw()

In [ ]:
#num of protons per interaction event
x_start = 0
x_end = 10
bins = 10

In [ ]:
#g18
i = ROOT.TCanvas("i")

G18_proton_QEL = df_gst_G18_02a_all.Filter("qel==1").Histo1D(("G18_proton_QEL", "Proton Amounts Per Interaction Type in G18 Model", bins, x_start, x_end), "nfp")
G18_proton_QEL.SetLineColor(ROOT.kViolet)
G18_proton_QEL.Draw("same")

G18_proton_MEC = df_gst_G18_02a_all.Filter("mec==1").Histo1D(("G18_proton_MEC", "", bins, x_start, x_end), "nfp")
G18_proton_MEC.SetLineColor(ROOT.kRed)
G18_proton_MEC.Draw("same")

G18_proton_RES = df_gst_G18_02a_all.Filter("res==1").Histo1D(("G18_proton_RES", "", bins, x_start, x_end), "nfp")
G18_proton_RES.SetLineColor(ROOT.kGreen)
G18_proton_RES.Draw("same")

G18_proton_DIS = df_gst_G18_02a_all.Filter("dis==1").Histo1D(("G18_proton_DIS", "", bins, x_start, x_end), "nfp")
G18_proton_DIS.SetLineColor(ROOT.kBlue)
G18_proton_DIS.Draw("same")

G18_proton_none = df_gst_G18_02a_all.Filter("qel==0&&mec==0&&res==0&&dis==0").Histo1D(("G18_proton_none", "", bins, x_start, x_end), "nfp")
G18_proton_none.SetLineColor(ROOT.kGray)
G18_proton_none.Draw("same")

G18_proton_leg = ROOT.TLegend(.6, .66, .87, .87)
G18_proton_leg.AddEntry("G18_proton_QEL", "QEL", "l")
G18_proton_leg.AddEntry("G18_proton_MEC", "MEC", "l")
G18_proton_leg.AddEntry("G18_proton_RES", "RES", "l")
G18_proton_leg.AddEntry("G18_proton_DIS", "DIS", "l")
G18_proton_leg.AddEntry("G18_proton_none", "Uncatagorized", "l")

G18_proton_leg.Draw()
i.Draw()

In [ ]:
#gem21
h = ROOT.TCanvas("h")

GEM_proton_QEL = df_gst_GEM21_11b_all.Filter("qel==1").Histo1D(("GEM_proton_QEL", "Proton Amounts Per Interaction Type in GEM21 Model", bins, x_start, x_end), "nfp")
GEM_proton_QEL.SetLineColor(ROOT.kViolet)
GEM_proton_QEL.Draw("same")

GEM_proton_MEC = df_gst_GEM21_11b_all.Filter("mec==1").Histo1D(("GEM_proton_MEC", "", bins, x_start, x_end), "nfp")
GEM_proton_MEC.SetLineColor(ROOT.kRed)
GEM_proton_MEC.Draw("same")

GEM_proton_RES = df_gst_GEM21_11b_all.Filter("res==1").Histo1D(("GEM_proton_RES", "", bins, x_start, x_end), "nfp")
GEM_proton_RES.SetLineColor(ROOT.kGreen)
GEM_proton_RES.Draw("same")

GEM_proton_DIS = df_gst_GEM21_11b_all.Filter("dis==1").Histo1D(("GEM_proton_DIS", "", bins, x_start, x_end), "nfp")
GEM_proton_DIS.SetLineColor(ROOT.kBlue)
GEM_proton_DIS.Draw("same")

GEM_proton_none = df_gst_GEM21_11b_all.Filter("qel==0&&mec==0&&res==0&&dis==0").Histo1D(("GEM_proton_none", "", bins, x_start, x_end), "nfp")
GEM_proton_none.SetLineColor(ROOT.kGray)
GEM_proton_none.Draw("same")

GEM_proton_leg = ROOT.TLegend(.6, .66, .87, .87)
GEM_proton_leg.AddEntry("GEM_proton_QEL", "QEL", "l")
GEM_proton_leg.AddEntry("GEM_proton_MEC", "MEC", "l")
GEM_proton_leg.AddEntry("GEM_proton_RES", "RES", "l")
GEM_proton_leg.AddEntry("GEM_proton_DIS", "DIS", "l")
GEM_proton_leg.AddEntry("GEM_proton_none", "Uncatagorized", "l")

GEM_proton_leg.Draw()
h.Draw()

In [ ]:
#num of neutrons per interaction event
x_start = 0
x_end = 10
bins = 10

In [ ]:
#g18
k = ROOT.TCanvas("k")

G18_neutron_QEL = df_gst_G18_02a_all.Filter("qel==1").Histo1D(("G18_neutron_QEL", "Neutron Amounts Per Interaction Type in G18 Model", bins, x_start, x_end), "nfn")
G18_neutron_QEL.SetLineColor(ROOT.kViolet)
G18_neutron_QEL.Draw("same")

G18_neutron_MEC = df_gst_G18_02a_all.Filter("mec==1").Histo1D(("G18_neutron_MEC", "", bins, x_start, x_end), "nfn")
G18_neutron_MEC.SetLineColor(ROOT.kRed)
G18_neutron_MEC.Draw("same")

G18_neutron_RES = df_gst_G18_02a_all.Filter("res==1").Histo1D(("G18_neutron_RES", "", bins, x_start, x_end), "nfn")
G18_neutron_RES.SetLineColor(ROOT.kGreen)
G18_neutron_RES.Draw("same")

G18_neutron_DIS = df_gst_G18_02a_all.Filter("dis==1").Histo1D(("G18_neutron_DIS", "", bins, x_start, x_end), "nfn")
G18_neutron_DIS.SetLineColor(ROOT.kBlue)
G18_neutron_DIS.Draw("same")

G18_neutron_none = df_gst_G18_02a_all.Filter("qel==0&&mec==0&&res==0&&dis==0").Histo1D(("G18_neutron_none", "", bins, x_start, x_end), "nfn")
G18_neutron_none.SetLineColor(ROOT.kGray)
G18_neutron_none.Draw("same")

G18_neutron_leg = ROOT.TLegend(.6, .66, .87, .87)
G18_neutron_leg.AddEntry("G18_neutron_QEL", "QEL", "l")
G18_neutron_leg.AddEntry("G18_neutron_MEC", "MEC", "l")
G18_neutron_leg.AddEntry("G18_neutron_RES", "RES", "l")
G18_neutron_leg.AddEntry("G18_neutron_DIS", "DIS", "l")
G18_neutron_leg.AddEntry("G18_neutron_none", "Uncatagorized", "l")

G18_neutron_leg.Draw()
k.Draw()

In [ ]:
#gem21
j = ROOT.TCanvas("j")

GEM_neutron_QEL = df_gst_GEM21_11b_all.Filter("qel==1").Histo1D(("GEM_neutron_QEL", "Neutron Amounts Per Interaction Type in GEM21 Model", bins, x_start, x_end), "nfn")
GEM_neutron_QEL.SetLineColor(ROOT.kViolet)
GEM_neutron_QEL.Draw("same")

GEM_neutron_MEC = df_gst_GEM21_11b_all.Filter("mec==1").Histo1D(("GEM_neutron_MEC", "", bins, x_start, x_end), "nfn")
GEM_neutron_MEC.SetLineColor(ROOT.kRed)
GEM_neutron_MEC.Draw("same")

GEM_neutron_RES = df_gst_GEM21_11b_all.Filter("res==1").Histo1D(("GEM_neutron_RES", "", bins, x_start, x_end), "nfn")
GEM_neutron_RES.SetLineColor(ROOT.kGreen)
GEM_neutron_RES.Draw("same")

GEM_neutron_DIS = df_gst_GEM21_11b_all.Filter("dis==1").Histo1D(("GEM_neutron_DIS", "", bins, x_start, x_end), "nfn")
GEM_neutron_DIS.SetLineColor(ROOT.kBlue)
GEM_neutron_DIS.Draw("same")

GEM_neutron_none = df_gst_GEM21_11b_all.Filter("qel==0&&mec==0&&res==0&&dis==0").Histo1D(("GEM_neutron_none", "", bins, x_start, x_end), "nfn")
GEM_neutron_none.SetLineColor(ROOT.kGray)
GEM_neutron_none.Draw("same")

GEM_neutron_leg = ROOT.TLegend(.6, .66, .87, .87)
GEM_neutron_leg.AddEntry("GEM_neutron_QEL", "QEL", "l")
GEM_neutron_leg.AddEntry("GEM_neutron_MEC", "MEC", "l")
GEM_neutron_leg.AddEntry("GEM_neutron_RES", "RES", "l")
GEM_neutron_leg.AddEntry("GEM_neutron_DIS", "DIS", "l")
GEM_neutron_leg.AddEntry("GEM_neutron_none", "Uncatagorized", "l")

GEM_neutron_leg.Draw()
j.Draw()

In [ ]:
#relativistic kinetic energy of protons in diff interaction types with kef cut
bins = 100
x_start = 0
x_end = 0.5

In [ ]:
#g18
m = ROOT.TCanvas("m")

g18_ke_p_qel = df_gst_G18_02a_all.Filter("qel==1").Histo1D(("g18_ke_p_qel", "Kinetic Energy of Protons In G18 Model", bins, x_start, x_end) ,"kef_above_threshold_protons")
g18_ke_p_qel.SetLineColor(ROOT.kViolet)
g18_ke_p_qel.Draw("same")

g18_ke_p_mec = df_gst_G18_02a_all.Filter("mec==1").Histo1D(("g18_ke_p_mec", "", bins, x_start, x_end) ,"kef_above_threshold_protons")
g18_ke_p_mec.SetLineColor(ROOT.kRed)
g18_ke_p_mec.Draw("same")

g18_ke_p_res = df_gst_G18_02a_all.Filter("res==1").Histo1D(("g18_ke_p_res", "", bins, x_start, x_end) ,"kef_above_threshold_protons")
g18_ke_p_res.SetLineColor(ROOT.kGreen)
g18_ke_p_res.Draw("same")

g18_ke_p_dis = df_gst_G18_02a_all.Filter("dis==1").Histo1D(("g18_ke_p_dis", "", bins, x_start, x_end) ,"kef_above_threshold_protons")
g18_ke_p_dis.SetLineColor(ROOT.kBlue)
g18_ke_p_dis.Draw("same")

g18_ke_p_none = df_gst_G18_02a_all.Filter("qel==0&&mec==0&&res==0&&dis==0").Histo1D(("g18_ke_p_none", "", bins, x_start, x_end) ,"kef_above_threshold_protons")
g18_ke_p_none.SetLineColor(ROOT.kGray)
g18_ke_p_none.Draw("same")

g18_ke_p_leg = ROOT.TLegend(.6, .66, .87, .87)
g18_ke_p_leg.AddEntry("g18_ke_p_qel", "QEL", "l")
g18_ke_p_leg.AddEntry("g18_ke_p_mec", "MEC", "l")
g18_ke_p_leg.AddEntry("g18_ke_p_res", "RES", "l")
g18_ke_p_leg.AddEntry("g18_ke_p_dis", "DIS", "l")
g18_ke_p_leg.AddEntry("g18_ke_p_none", "Uncatagorized", "l")

g18_ke_p_leg.Draw()
m.SetLogy()
m.Draw()

In [ ]:
#gem21
l = ROOT.TCanvas("l")

gem_ke_p_qel = df_gst_GEM21_11b_all.Filter("qel==1").Histo1D(("gem_ke_p_qel", "Kinetic Energy of Protons In GEM21 Model", bins, x_start, x_end), "kef_above_threshold_protons")
gem_ke_p_qel.SetLineColor(ROOT.kViolet)
gem_ke_p_qel.Draw("same")

gem_ke_p_mec = df_gst_GEM21_11b_all.Filter("mec==1").Histo1D(("gem_ke_p_mec", "", bins, x_start, x_end),"kef_above_threshold_protons")
gem_ke_p_mec.SetLineColor(ROOT.kRed)
gem_ke_p_mec.Draw("same")

gem_ke_p_res = df_gst_GEM21_11b_all.Filter("res==1").Histo1D(("gem_ke_p_res", "", bins, x_start, x_end), "kef_above_threshold_protons")
gem_ke_p_res.SetLineColor(ROOT.kGreen)
gem_ke_p_res.Draw("same")

gem_ke_p_dis = df_gst_GEM21_11b_all.Filter("dis==1").Histo1D(("gem_ke_p_dis", "", bins, x_start, x_end), "kef_above_threshold_protons")
gem_ke_p_dis.SetLineColor(ROOT.kBlue)
gem_ke_p_dis.Draw("same")

gem_ke_p_none = df_gst_GEM21_11b_all.Filter("qel==0&&mec==0&&res==0&&dis==0").Histo1D(("gem_ke_p_none", "", bins, x_start, x_end),"kef_above_threshold_protons")
gem_ke_p_none.SetLineColor(ROOT.kGray)
gem_ke_p_none.Draw("same")

gem_ke_p_leg = ROOT.TLegend(.6, .66, .87, .87)
gem_ke_p_leg.AddEntry("gem_ke_p_qel", "QEL", "l")
gem_ke_p_leg.AddEntry("gem_ke_p_mec", "MEC", "l")
gem_ke_p_leg.AddEntry("gem_ke_p_res", "RES", "l")
gem_ke_p_leg.AddEntry("gem_ke_p_dis", "DIS", "l")
gem_ke_p_leg.AddEntry("gem_ke_p_none", "Uncatagorized", "l")

gem_ke_p_leg.Draw()
l.SetLogy()
l.Draw()

In [ ]:
#relativistic kinetic energy of neutrons in diff interaction types with kef cut
l = ROOT.TCanvas("l")
bins = 100
x_start = 0
x_end = 0.5

In [ ]:
#g18
m = ROOT.TCanvas("m")

g18_ke_n_qel = df_gst_G18_02a_all.Filter("qel==1").Histo1D(("g18_ke_n_qel", "Kinetic Energy of Neutrons In G18 Model", bins, x_start, x_end) ,"kef_above_threshold_neutrons")
g18_ke_n_qel.SetLineColor(ROOT.kViolet)
g18_ke_n_qel.Draw("same")

g18_ke_n_mec = df_gst_G18_02a_all.Filter("mec==1").Histo1D(("g18_ke_n_mec", "", bins, x_start, x_end) ,"kef_above_threshold_neutrons")
g18_ke_n_mec.SetLineColor(ROOT.kRed)
g18_ke_n_mec.Draw("same")

g18_ke_n_res = df_gst_G18_02a_all.Filter("res==1").Histo1D(("g18_ke_n_res", "", bins, x_start, x_end) ,"kef_above_threshold_neutrons")
g18_ke_n_res.SetLineColor(ROOT.kGreen)
g18_ke_n_res.Draw("same")

g18_ke_n_dis = df_gst_G18_02a_all.Filter("dis==1").Histo1D(("g18_ke_n_dis", "", bins, x_start, x_end) ,"kef_above_threshold_neutrons")
g18_ke_n_dis.SetLineColor(ROOT.kBlue)
g18_ke_n_dis.Draw("same")

g18_ke_n_none = df_gst_G18_02a_all.Filter("qel==0&&mec==0&&res==0&&dis==0").Histo1D(("g18_ke_n_none", "", bins, x_start, x_end) ,"kef_above_threshold_neutrons")
g18_ke_n_none.SetLineColor(ROOT.kGray)
g18_ke_n_none.Draw("same")

g18_ke_n_leg = ROOT.TLegend(.6, .66, .87, .87)
g18_ke_n_leg.AddEntry("g18_ke_n_qel", "QEL", "l")
g18_ke_n_leg.AddEntry("g18_ke_n_mec", "MEC", "l")
g18_ke_n_leg.AddEntry("g18_ke_n_res", "RES", "l")
g18_ke_n_leg.AddEntry("g18_ke_n_dis", "DIS", "l")
g18_ke_n_leg.AddEntry("g18_ke_n_none", "Uncatagorized", "l")

g18_ke_n_leg.Draw()
m.SetLogy()
m.Draw()

In [ ]:
#gem21
l = ROOT.TCanvas("l")

gem_ke_n_qel = df_gst_GEM21_11b_all.Filter("qel==1").Histo1D(("gem_ke_n_qel", "Kinetic Energy of Neutrons In GEM21 Model", bins, x_start, x_end), "kef_above_threshold_neutrons")
gem_ke_n_qel.SetLineColor(ROOT.kViolet)
gem_ke_n_qel.Draw("same")

gem_ke_n_mec = df_gst_GEM21_11b_all.Filter("mec==1").Histo1D(("gem_ke_n_mec", "", bins, x_start, x_end),"kef_above_threshold_neutrons")
gem_ke_n_mec.SetLineColor(ROOT.kRed)
gem_ke_n_mec.Draw("same")

gem_ke_n_res = df_gst_GEM21_11b_all.Filter("res==1").Histo1D(("gem_ke_n_res", "", bins, x_start, x_end), "kef_above_threshold_neutrons")
gem_ke_n_res.SetLineColor(ROOT.kGreen)
gem_ke_n_res.Draw("same")

gem_ke_n_dis = df_gst_GEM21_11b_all.Filter("dis==1").Histo1D(("gem_ke_n_dis", "", bins, x_start, x_end), "kef_above_threshold_neutrons")
gem_ke_n_dis.SetLineColor(ROOT.kBlue)
gem_ke_n_dis.Draw("same")

gem_ke_n_none = df_gst_GEM21_11b_all.Filter("qel==0&&mec==0&&res==0&&dis==0").Histo1D(("gem_ke_n_none", "", bins, x_start, x_end),"kef_above_threshold_neutrons")
gem_ke_n_none.SetLineColor(ROOT.kGray)
gem_ke_n_none.Draw("same")

gem_ke_n_leg = ROOT.TLegend(.6, .66, .87, .87)
gem_ke_n_leg.AddEntry("gem_ke_n_qel", "QEL", "l")
gem_ke_n_leg.AddEntry("gem_ke_n_mec", "MEC", "l")
gem_ke_n_leg.AddEntry("gem_ke_n_res", "RES", "l")
gem_ke_n_leg.AddEntry("gem_ke_n_dis", "DIS", "l")
gem_ke_n_leg.AddEntry("gem_ke_n_none", "Uncatagorized", "l")

gem_ke_n_leg.Draw()
l.SetLogy()
l.Draw()

In [ ]:
#relativistic kinetic energy of neutral pions in diff interaction types with kef cut
bins = 100
x_start = 0
x_end = 5

In [ ]:
#g18
m = ROOT.TCanvas("m")
g18_ke_pi0_qel = df_gst_G18_02a_all.Filter("qel==1").Histo1D(("g18_ke_pi0_qel", "Kinetic Energy of Protons In G18 Model", bins, x_start, x_end) ,"kef_above_threshold_pi0")
g18_ke_pi0_qel.SetLineColor(ROOT.kViolet)
g18_ke_pi0_qel.Draw("same")

g18_ke_pi0_mec = df_gst_G18_02a_all.Filter("mec==1").Histo1D(("g18_ke_pi0_mec", "", bins, x_start, x_end) ,"kef_above_threshold_pi0")
g18_ke_pi0_mec.SetLineColor(ROOT.kRed)
g18_ke_pi0_mec.Draw("same")

g18_ke_pi0_res = df_gst_G18_02a_all.Filter("res==1").Histo1D(("g18_ke_pi0_res", "", bins, x_start, x_end) ,"kef_above_threshold_pi0")
g18_ke_pi0_res.SetLineColor(ROOT.kGreen)
g18_ke_pi0_res.Draw("same")

g18_ke_pi0_dis = df_gst_G18_02a_all.Filter("dis==1").Histo1D(("g18_ke_pi0_dis", "", bins, x_start, x_end) ,"kef_above_threshold_pi0")
g18_ke_pi0_dis.SetLineColor(ROOT.kBlue)
g18_ke_pi0_dis.Draw("same")

g18_ke_pi0_none = df_gst_G18_02a_all.Filter("qel==0&&mec==0&&res==0&&dis==0").Histo1D(("g18_ke_pi0_none", "", bins, x_start, x_end) ,"kef_above_threshold_pi0")
g18_ke_pi0_none.SetLineColor(ROOT.kGray)
g18_ke_pi0_none.Draw("same")

g18_ke_pi0_leg = ROOT.TLegend(.6, .66, .87, .87)
g18_ke_pi0_leg.AddEntry("g18_ke_pi0_qel", "QEL", "l")
g18_ke_pi0_leg.AddEntry("g18_ke_pi0_mec", "MEC", "l")
g18_ke_pi0_leg.AddEntry("g18_ke_pi0_res", "RES", "l")
g18_ke_pi0_leg.AddEntry("g18_ke_pi0_dis", "DIS", "l")
g18_ke_pi0_leg.AddEntry("g18_ke_pi0_none", "Uncatagorized", "l")

g18_ke_pi0_leg.Draw()
m.SetLogy()
m.Draw()

In [ ]:
#gem21
l = ROOT.TCanvas("l")

gem_ke_pi0_qel = df_gst_GEM21_11b_all.Filter("qel==1").Histo1D(("gem_ke_pi0_qel", "Kinetic Energy of Neutral Pions In GEM21 Model", bins, x_start, x_end), "kef_above_threshold_pi0")
gem_ke_pi0_qel.SetLineColor(ROOT.kViolet)
gem_ke_pi0_qel.Draw("same")

gem_ke_pi0_mec = df_gst_GEM21_11b_all.Filter("mec==1").Histo1D(("gem_ke_pi0_mec", "", bins, x_start, x_end),"kef_above_threshold_pi0")
gem_ke_pi0_mec.SetLineColor(ROOT.kRed)
gem_ke_pi0_mec.Draw("same")

gem_ke_pi0_res = df_gst_GEM21_11b_all.Filter("res==1").Histo1D(("gem_ke_pi0_res", "", bins, x_start, x_end), "kef_above_threshold_pi0")
gem_ke_pi0_res.SetLineColor(ROOT.kGreen)
gem_ke_pi0_res.Draw("same")

gem_ke_pi0_dis = df_gst_GEM21_11b_all.Filter("dis==1").Histo1D(("gem_ke_pi0_dis", "", bins, x_start, x_end), "kef_above_threshold_pi0")
gem_ke_pi0_dis.SetLineColor(ROOT.kBlue)
gem_ke_pi0_dis.Draw("same")

gem_ke_pi0_none = df_gst_GEM21_11b_all.Filter("qel==0&&mec==0&&res==0&&dis==0").Histo1D(("gem_ke_pi0_none", "", bins, x_start, x_end),"kef_above_threshold_pi0")
gem_ke_pi0_none.SetLineColor(ROOT.kGray)
gem_ke_pi0_none.Draw("same")

gem_ke_pi0_leg = ROOT.TLegend(.6, .66, .87, .87)
gem_ke_pi0_leg.AddEntry("gem_ke_pi0_qel", "QEL", "l")
gem_ke_pi0_leg.AddEntry("gem_ke_pi0_mec", "MEC", "l")
gem_ke_pi0_leg.AddEntry("gem_ke_pi0_res", "RES", "l")
gem_ke_pi0_leg.AddEntry("gem_ke_pi0_dis", "DIS", "l")
gem_ke_pi0_leg.AddEntry("gem_ke_pi0_none", "Uncatagorized", "l")

gem_ke_pi0_leg.Draw()
l.SetLogy()
l.Draw()

In [ ]:
#relativistic kinetic energy of charged pions in diff interaction types with kef cut
bins = 100
x_start = 0
x_end = 5

In [ ]:
#g18
m = ROOT.TCanvas("m")

g18_ke_pi_qel = df_gst_G18_02a_all.Filter("qel==1").Histo1D(("g18_ke_pi_qel", "Kinetic Energy of Protons In G18 Model", bins, x_start, x_end) ,"kef_above_threshold_pions")
g18_ke_pi_qel.SetLineColor(ROOT.kViolet)
g18_ke_pi_qel.Draw("same")

g18_ke_pi_mec = df_gst_G18_02a_all.Filter("mec==1").Histo1D(("g18_ke_pi_mec", "", bins, x_start, x_end) ,"kef_above_threshold_pions")
g18_ke_pi_mec.SetLineColor(ROOT.kRed)
g18_ke_pi_mec.Draw("same")

g18_ke_pi_res = df_gst_G18_02a_all.Filter("res==1").Histo1D(("g18_ke_pi_res", "", bins, x_start, x_end) ,"kef_above_threshold_pions")
g18_ke_pi_res.SetLineColor(ROOT.kGreen)
g18_ke_pi_res.Draw("same")

g18_ke_pi_dis = df_gst_G18_02a_all.Filter("dis==1").Histo1D(("g18_ke_pi_dis", "", bins, x_start, x_end) ,"kef_above_threshold_pions")
g18_ke_pi_dis.SetLineColor(ROOT.kBlue)
g18_ke_pi_dis.Draw("same")

g18_ke_pi_none = df_gst_G18_02a_all.Filter("qel==0&&mec==0&&res==0&&dis==0").Histo1D(("g18_ke_pi_none", "", bins, x_start, x_end) ,"kef_above_threshold_pions")
g18_ke_pi_none.SetLineColor(ROOT.kGray)
g18_ke_pi_none.Draw("same")

g18_ke_pi_leg = ROOT.TLegend(.6, .66, .87, .87)
g18_ke_pi_leg.AddEntry("g18_ke_pi0_qel", "QEL", "l")
g18_ke_pi_leg.AddEntry("g18_ke_pi0_mec", "MEC", "l")
g18_ke_pi_leg.AddEntry("g18_ke_pi0_res", "RES", "l")
g18_ke_pi_leg.AddEntry("g18_ke_pi0_dis", "DIS", "l")
g18_ke_pi_leg.AddEntry("g18_ke_pi0_none", "Uncatagorized", "l")

g18_ke_pi_leg.Draw()
m.SetLogy()
m.Draw()

In [ ]:
#gem21
l = ROOT.TCanvas("l")

gem_ke_pi_qel = df_gst_GEM21_11b_all.Filter("qel==1").Histo1D(("gem_ke_pi_qel", "Kinetic Energy of Neutral Pions In GEM21 Model", bins, x_start, x_end), "kef_above_threshold_pions")
gem_ke_pi_qel.SetLineColor(ROOT.kViolet)
gem_ke_pi_qel.Draw("same")

gem_ke_pi_mec = df_gst_GEM21_11b_all.Filter("mec==1").Histo1D(("gem_ke_pi_mec", "", bins, x_start, x_end),"kef_above_threshold_pions")
gem_ke_pi_mec.SetLineColor(ROOT.kRed)
gem_ke_pi_mec.Draw("same")

gem_ke_pi_res = df_gst_GEM21_11b_all.Filter("res==1").Histo1D(("gem_ke_pi_res", "", bins, x_start, x_end), "kef_above_threshold_pions")
gem_ke_pi_res.SetLineColor(ROOT.kGreen)
gem_ke_pi_res.Draw("same")

gem_ke_pi_dis = df_gst_GEM21_11b_all.Filter("dis==1").Histo1D(("gem_ke_pi_dis", "", bins, x_start, x_end), "kef_above_threshold_pions")
gem_ke_pi_dis.SetLineColor(ROOT.kBlue)
gem_ke_pi_dis.Draw("same")

gem_ke_pi_none = df_gst_GEM21_11b_all.Filter("qel==0&&mec==0&&res==0&&dis==0").Histo1D(("gem_ke_pi_none", "", bins, x_start, x_end),"kef_above_threshold_pions")
gem_ke_pi_none.SetLineColor(ROOT.kGray)
gem_ke_pi_none.Draw("same")

gem_ke_pi_leg = ROOT.TLegend(.6, .66, .87, .87)
gem_ke_pi_leg.AddEntry("gem_ke_pi_qel", "QEL", "l")
gem_ke_pi_leg.AddEntry("gem_ke_pi_mec", "MEC", "l")
gem_ke_pi_leg.AddEntry("gem_ke_pi_res", "RES", "l")
gem_ke_pi_leg.AddEntry("gem_ke_pi_dis", "DIS", "l")
gem_ke_pi_leg.AddEntry("gem_ke_pi_none", "Uncatagorized", "l")

gem_ke_pi_leg.Draw()
l.SetLogy()
l.Draw()

In [ ]:
#plotting etrans no cuts
bins = 100
x_start = 0
x_end = 4

In [ ]:
#g18 
b = ROOT.TCanvas("b")
G18_ETrans_QEL_Plot = df_gst_G18_02a_all.Filter("qel==1").Histo1D(("G18_ETrans_QEL_Plot", "Energy Transfer in Interaction Types in G18 Model", bins, x_start, x_end), "energy_transfer")
G18_ETrans_QEL_Plot.SetLineColor(ROOT.kViolet)
G18_ETrans_QEL_Plot.GetXaxis().SetTitle("Energy Transfer(GeV)")
G18_ETrans_QEL_Plot.GetYaxis().SetTitle("Counts")
G18_ETrans_QEL_Plot.Draw("same")

G18_ETrans_MEC_Plot = df_gst_G18_02a_all.Filter("mec==1").Histo1D(("G18_ETrans_MEC_Plot", "", bins, x_start, x_end), "energy_transfer")
G18_ETrans_MEC_Plot.SetLineColor(ROOT.kRed)
G18_ETrans_MEC_Plot.Draw("same")

G18_ETrans_RES_Plot = df_gst_G18_02a_all.Filter("res==1").Histo1D(("G18_ETrans_RES_Plot", "", bins, x_start, x_end), "energy_transfer")
G18_ETrans_RES_Plot.SetLineColor(ROOT.kGreen)
G18_ETrans_RES_Plot.Draw("same")

G18_ETrans_DIS_Plot = df_gst_G18_02a_all.Filter("dis==1").Histo1D(("G18_ETrans_DIS_Plot", "", bins, x_start, x_end), "energy_transfer")
G18_ETrans_DIS_Plot.SetLineColor(ROOT.kBlue)
G18_ETrans_DIS_Plot.Draw("same")

G18_ETrans_None_Plot = df_gst_G18_02a_all.Filter("qel==0&&mec==0&&res==0&&dis==0").Histo1D(("G18_ETrans_None_Plot", "", bins, x_start, x_end), "energy_transfer")
G18_ETrans_None_Plot.SetLineColor(ROOT.kGray)
G18_ETrans_None_Plot.Draw("same")

G18_Etrans_Leg = ROOT.TLegend(.6, .66, .87, .87)
G18_Etrans_Leg.AddEntry("G18_ETrans_QEL_Plot", "QE", "l")
G18_Etrans_Leg.AddEntry("G18_ETrans_MEC_Plot", "MEC", "l")
G18_Etrans_Leg.AddEntry("G18_ETrans_RES_Plot", "RES", "l")
G18_Etrans_Leg.AddEntry("G18_ETrans_DIS_Plot", "DIS", "l")
G18_Etrans_Leg.AddEntry("G18_ETrans_None_Plot", "Uncatagorized", "l")
G18_Etrans_Leg.Draw()

b.SetLogy()
b.Draw()

In [ ]:
#g21 
b = ROOT.TCanvas("b")
G21_ETrans_QEL_Plot = df_gst_GEM21_11b_all.Filter("qel==1").Histo1D(("G21_ETrans_QEL_Plot", "Energy Transfer in Interaction Types in GEM21 Model", bins, x_start, x_end), "energy_transfer")
G21_ETrans_QEL_Plot.SetLineColor(ROOT.kViolet)
G21_ETrans_QEL_Plot.GetXaxis().SetTitle("Energy Transfer(GeV)")
G21_ETrans_QEL_Plot.GetYaxis().SetTitle("Counts")
G21_ETrans_QEL_Plot.Draw("same")

G21_ETrans_MEC_Plot = df_gst_GEM21_11b_all.Filter("mec==1").Histo1D(("G21_ETrans_MEC_Plot", "", bins, x_start, x_end), "energy_transfer")
G21_ETrans_MEC_Plot.SetLineColor(ROOT.kRed)
G21_ETrans_MEC_Plot.Draw("same")

G21_ETrans_RES_Plot = df_gst_GEM21_11b_all.Filter("res==1").Histo1D(("G21_ETrans_RES_Plot", "", bins, x_start, x_end), "energy_transfer")
G21_ETrans_RES_Plot.SetLineColor(ROOT.kGreen)
G21_ETrans_RES_Plot.Draw("same")

G21_ETrans_DIS_Plot = df_gst_GEM21_11b_all.Filter("dis==1").Histo1D(("G21_ETrans_DIS_Plot", "", bins, x_start, x_end), "energy_transfer")
G21_ETrans_DIS_Plot.SetLineColor(ROOT.kBlue)
G21_ETrans_DIS_Plot.Draw("same")

G21_ETrans_None_Plot = df_gst_GEM21_11b_all.Filter("qel==0&&mec==0&&res==0&&dis==0").Histo1D(("G21_ETrans_None_Plot", "", bins, x_start, x_end), "energy_transfer")
G21_ETrans_None_Plot.SetLineColor(ROOT.kGray)
G21_ETrans_None_Plot.Draw("same")

G21_Etrans_Leg = ROOT.TLegend(.6, .66, .87, .87)
G21_Etrans_Leg.AddEntry("G21_ETrans_QEL_Plot", "QE", "l")
G21_Etrans_Leg.AddEntry("G21_ETrans_MEC_Plot", "MEC", "l")
G21_Etrans_Leg.AddEntry("G21_ETrans_RES_Plot", "RES", "l")
G21_Etrans_Leg.AddEntry("G21_ETrans_DIS_Plot", "DIS", "l")
G21_Etrans_Leg.AddEntry("G21_ETrans_None_Plot", "Uncatagorized", "l")
G21_Etrans_Leg.Draw()

b.SetLogy()
b.Draw()

In [ ]:
#histograms of energy transfer for events w lepton pt>400
bins = 100
x_start = 0
x_end = 4

In [ ]:
#g18 
b = ROOT.TCanvas("b")
df_gst_G18_02a_ETrans_Lep = df_gst_G18_02a_all.Filter(f"ptl>{lep_accept_pt}")
G18_ETrans_QEL_Lep_Plot = df_gst_G18_02a_ETrans_Lep.Filter("qel==1").Histo1D(("G18_ETrans_QEL_Lep_Plot", "Energy Transfer in Interaction Types with Lepton PT > 400 MeV in G18 Model", bins, x_start, x_end), "energy_transfer")
G18_ETrans_QEL_Lep_Plot.SetLineColor(ROOT.kViolet)
G18_ETrans_QEL_Lep_Plot.GetXaxis().SetTitle("Energy Transfer(GeV)")
G18_ETrans_QEL_Lep_Plot.GetYaxis().SetTitle("Counts")
G18_ETrans_QEL_Lep_Plot.Draw("same")

G18_ETrans_MEC_Lep_Plot = df_gst_G18_02a_ETrans_Lep.Filter("mec==1").Histo1D(("G18_ETrans_MEC_Lep_Plot", "", bins, x_start, x_end), "energy_transfer")
G18_ETrans_MEC_Lep_Plot.SetLineColor(ROOT.kRed)
G18_ETrans_MEC_Lep_Plot.Draw("same")

G18_ETrans_RES_Lep_Plot = df_gst_G18_02a_ETrans_Lep.Filter("res==1").Histo1D(("G18_ETrans_RES_Lep_Plot", "", bins, x_start, x_end), "energy_transfer")
G18_ETrans_RES_Lep_Plot.SetLineColor(ROOT.kGreen)
G18_ETrans_RES_Lep_Plot.Draw("same")

G18_ETrans_DIS_Lep_Plot = df_gst_G18_02a_ETrans_Lep.Filter("dis==1").Histo1D(("G18_ETrans_DIS_Lep_Plot", "", bins, x_start, x_end), "energy_transfer")
G18_ETrans_DIS_Lep_Plot.SetLineColor(ROOT.kBlue)
G18_ETrans_DIS_Lep_Plot.Draw("same")

G18_ETrans_None_Lep_Plot = df_gst_G18_02a_ETrans_Lep.Filter("qel==0&&mec==0&&res==0&&dis==0").Histo1D(("G18_ETrans_None_Lep_Plot", "", bins, x_start, x_end), "energy_transfer")
G18_ETrans_None_Lep_Plot.SetLineColor(ROOT.kGray)
G18_ETrans_None_Lep_Plot.Draw("same")

G18_Etrans_Leg_Lep = ROOT.TLegend(.6, .66, .87, .87)
G18_Etrans_Leg_Lep.AddEntry("G18_ETrans_QEL_Lep_Plot", "QE", "l")
G18_Etrans_Leg_Lep.AddEntry("G18_ETrans_MEC_Lep_Plot", "MEC", "l")
G18_Etrans_Leg_Lep.AddEntry("G18_ETrans_RES_Lep_Plot", "RES", "l")
G18_Etrans_Leg_Lep.AddEntry("G18_ETrans_DIS_Lep_Plot", "DIS", "l")
G18_Etrans_Leg_Lep.AddEntry("G18_ETrans_None_Lep_Plot", "Uncatagorized", "l")
G18_Etrans_Leg_Lep.Draw()

b.SetLogy()
b.Draw()

In [ ]:
#g21 
b = ROOT.TCanvas("b")
df_gst_G21_02a_ETrans_Lep = df_gst_GEM21_11b_all.Filter(f"ptl>{lep_accept_pt}")
G21_ETrans_QEL_Lep_Plot = df_gst_G21_02a_ETrans_Lep.Filter("qel==1").Histo1D(("G21_ETrans_QEL_Lep_Plot", "Energy Transfer in Interaction Types with Lepton PT > 400 MeV in GEM21 Model", bins, x_start, x_end), "energy_transfer")
G21_ETrans_QEL_Lep_Plot.SetLineColor(ROOT.kViolet)
G21_ETrans_QEL_Lep_Plot.GetXaxis().SetTitle("Energy Transfer(GeV)")
G21_ETrans_QEL_Lep_Plot.GetYaxis().SetTitle("Counts")
G21_ETrans_QEL_Lep_Plot.Draw("same")

G21_ETrans_MEC_Lep_Plot = df_gst_G21_02a_ETrans_Lep.Filter("mec==1").Histo1D(("G21_ETrans_MEC_Lep_Plot", "", bins, x_start, x_end), "energy_transfer")
G21_ETrans_MEC_Lep_Plot.SetLineColor(ROOT.kRed)
G21_ETrans_MEC_Lep_Plot.Draw("same")

G21_ETrans_RES_Lep_Plot = df_gst_G21_02a_ETrans_Lep.Filter("res==1").Histo1D(("G21_ETrans_RES_Lep_Plot", "", bins, x_start, x_end), "energy_transfer")
G21_ETrans_RES_Lep_Plot.SetLineColor(ROOT.kGreen)
G21_ETrans_RES_Lep_Plot.Draw("same")

G21_ETrans_DIS_Lep_Plot = df_gst_G21_02a_ETrans_Lep.Filter("dis==1").Histo1D(("G21_ETrans_DIS_Lep_Plot", "", bins, x_start, x_end), "energy_transfer")
G21_ETrans_DIS_Lep_Plot.SetLineColor(ROOT.kBlue)
G21_ETrans_DIS_Lep_Plot.Draw("same")

G21_ETrans_None_Lep_Plot = df_gst_G21_02a_ETrans_Lep.Filter("qel==0&&mec==0&&res==0&&dis==0").Histo1D(("G21_ETrans_None_Lep_Plot", "", bins, x_start, x_end), "energy_transfer")
G21_ETrans_None_Lep_Plot.SetLineColor(ROOT.kGray)
G21_ETrans_None_Lep_Plot.Draw("same")

G21_Etrans_Leg_Lep = ROOT.TLegend(.6, .66, .87, .87)
G21_Etrans_Leg_Lep.AddEntry("G21_ETrans_QEL_Lep_Plot", "QE", "l")
G21_Etrans_Leg_Lep.AddEntry("G21_ETrans_MEC_Lep_Plot", "MEC", "l")
G21_Etrans_Leg_Lep.AddEntry("G21_ETrans_RES_Lep_Plot", "RES", "l")
G21_Etrans_Leg_Lep.AddEntry("G21_ETrans_DIS_Lep_Plot", "DIS", "l")
G21_Etrans_Leg_Lep.AddEntry("G21_ETrans_None_Lep_Plot", "Uncatagorized", "l")
G21_Etrans_Leg_Lep.Draw()

b.SetLogy()
b.Draw()

In [ ]:
#plotting etrans with no pion cuts
bins = 100
x_start = 0
x_end = 4

In [ ]:
#g18 etrans with no intitial state pions
b = ROOT.TCanvas("b")
df_gst_G18_02a_ETrans_NIP = df_gst_G18_02a_all.Filter("numi_pions==0")
G18_ETrans_QEL_NIP_Plot = df_gst_G18_02a_ETrans_NIP.Filter("qel==1").Histo1D(("G18_ETrans_QEL_NIP_Plot", "Energy Transfer in Interaction Types without Initial State Pions in G18 Model", bins, x_start, x_end), "energy_transfer")
G18_ETrans_QEL_NIP_Plot.SetLineColor(ROOT.kViolet)
G18_ETrans_QEL_NIP_Plot.GetXaxis().SetTitle("Energy Transfer(GeV)")
G18_ETrans_QEL_NIP_Plot.GetYaxis().SetTitle("Counts")
G18_ETrans_QEL_NIP_Plot.Draw("same")

G18_ETrans_MEC_NIP_Plot = df_gst_G18_02a_ETrans_NIP.Filter("mec==1").Histo1D(("G18_ETrans_MEC_NIP_Plot", "", bins, x_start, x_end), "energy_transfer")
G18_ETrans_MEC_NIP_Plot.SetLineColor(ROOT.kRed)
G18_ETrans_MEC_NIP_Plot.Draw("same")

G18_ETrans_RES_NIP_Plot = df_gst_G18_02a_ETrans_NIP.Filter("res==1").Histo1D(("G18_ETrans_RES_NIP_Plot", "", bins, x_start, x_end), "energy_transfer")
G18_ETrans_RES_NIP_Plot.SetLineColor(ROOT.kGreen)
G18_ETrans_RES_NIP_Plot.Draw("same")

G18_ETrans_DIS_NIP_Plot = df_gst_G18_02a_ETrans_NIP.Filter("dis==1").Histo1D(("G18_ETrans_DIS_NIP_Plot", "", bins, x_start, x_end), "energy_transfer")
G18_ETrans_DIS_NIP_Plot.SetLineColor(ROOT.kBlue)
G18_ETrans_DIS_NIP_Plot.Draw("same")

G18_ETrans_None_NIP_Plot = df_gst_G18_02a_ETrans_NIP.Filter("qel==0&&mec==0&&res==0&&dis==0").Histo1D(("G18_ETrans_None_NIP_Plot", "", bins, x_start, x_end), "energy_transfer")
G18_ETrans_None_NIP_Plot.SetLineColor(ROOT.kGray)
G18_ETrans_None_NIP_Plot.Draw("same")

G18_Etrans_Leg_NIP = ROOT.TLegend(.6, .66, .87, .87)
G18_Etrans_Leg_NIP.AddEntry("G18_ETrans_QEL_NIP_Plot", "QE", "l")
G18_Etrans_Leg_NIP.AddEntry("G18_ETrans_MEC_NIP_Plot", "MEC", "l")
G18_Etrans_Leg_NIP.AddEntry("G18_ETrans_RES_NIP_Plot", "RES", "l")
G18_Etrans_Leg_NIP.AddEntry("G18_ETrans_DIS_NIP_Plot", "DIS", "l")
G18_Etrans_Leg_NIP.AddEntry("G18_ETrans_None_NIP_Plot", "Uncatagorized", "l")
G18_Etrans_Leg_NIP.Draw()

b.SetLogy()
b.Draw()

In [ ]:
#g21 with no intitial state pions
b = ROOT.TCanvas("b")
df_gst_GEM21_11b_ETrans_NIP = df_gst_GEM21_11b_all.Filter("numi_pions==0")
G21_ETrans_QEL_NIP_Plot = df_gst_GEM21_11b_ETrans_NIP.Filter("qel==1").Histo1D(("G21_ETrans_QEL_NIP_Plot", "Energy Transfer in Interaction Types without Initial State Pions in GEM21 Model", bins, x_start, x_end), "energy_transfer")
G21_ETrans_QEL_NIP_Plot.SetLineColor(ROOT.kViolet)
G21_ETrans_QEL_NIP_Plot.GetXaxis().SetTitle("Energy Transfer(GeV)")
G21_ETrans_QEL_NIP_Plot.GetYaxis().SetTitle("Counts")
G21_ETrans_QEL_NIP_Plot.Draw("same")

G21_ETrans_MEC_NIP_Plot = df_gst_GEM21_11b_ETrans_NIP.Filter("mec==1").Histo1D(("G21_ETrans_MEC_NIP_Plot", "", bins, x_start, x_end), "energy_transfer")
G21_ETrans_MEC_NIP_Plot.SetLineColor(ROOT.kRed)
G21_ETrans_MEC_NIP_Plot.Draw("same")

G21_ETrans_RES_NIP_Plot = df_gst_GEM21_11b_ETrans_NIP.Filter("res==1").Histo1D(("G21_ETrans_RES_NIP_Plot", "", bins, x_start, x_end), "energy_transfer")
G21_ETrans_RES_NIP_Plot.SetLineColor(ROOT.kGreen)
G21_ETrans_RES_NIP_Plot.Draw("same")

G21_ETrans_DIS_NIP_Plot = df_gst_GEM21_11b_ETrans_NIP.Filter("dis==1").Histo1D(("G21_ETrans_DIS_NIP_Plot", "", bins, x_start, x_end), "energy_transfer")
G21_ETrans_DIS_NIP_Plot.SetLineColor(ROOT.kBlue)
G21_ETrans_DIS_NIP_Plot.Draw("same")

G21_ETrans_None_NIP_Plot = df_gst_GEM21_11b_ETrans_NIP.Filter("qel==0&&mec==0&&res==0&&dis==0").Histo1D(("G21_ETrans_None_NIP_Plot", "", bins, x_start, x_end), "energy_transfer")
G21_ETrans_None_NIP_Plot.SetLineColor(ROOT.kGray)
G21_ETrans_None_NIP_Plot.Draw("same")

G21_Etrans_Leg_NIP = ROOT.TLegend(.6, .66, .87, .87)
G21_Etrans_Leg_NIP.AddEntry("G21_ETrans_QEL_NIP_Plot", "QE", "l")
G21_Etrans_Leg_NIP.AddEntry("G21_ETrans_MEC_NIP_Plot", "MEC", "l")
G21_Etrans_Leg_NIP.AddEntry("G21_ETrans_RES_NIP_Plot", "RES", "l")
G21_Etrans_Leg_NIP.AddEntry("G21_ETrans_DIS_NIP_Plot", "DIS", "l")
G21_Etrans_Leg_NIP.AddEntry("G21_ETrans_None_NIP_Plot", "Uncatagorized", "l")
G21_Etrans_Leg_NIP.Draw()

b.SetLogy()
b.Draw()

In [ ]:
#g18 with no final state pions
b = ROOT.TCanvas("b")
df_gst_G18_02a_ETrans_NFP = df_gst_G18_02a_all.Filter("num_pions==0")
G18_ETrans_QEL_NFP_Plot = df_gst_G18_02a_ETrans_NFP.Filter("qel==1").Histo1D(("G18_ETrans_QEL_NFP_Plot", "Energy Transfer in Interaction Types without Final State Pions in G18 Model", bins, x_start, x_end), "energy_transfer")
G18_ETrans_QEL_NFP_Plot.SetLineColor(ROOT.kViolet)
G18_ETrans_QEL_NFP_Plot.GetXaxis().SetTitle("Energy Transfer(GeV)")
G18_ETrans_QEL_NFP_Plot.GetYaxis().SetTitle("Counts")
G18_ETrans_QEL_NFP_Plot.Draw("same")

G18_ETrans_MEC_NFP_Plot = df_gst_G18_02a_ETrans_NFP.Filter("mec==1").Histo1D(("G18_ETrans_MEC_NFP_Plot", "", bins, x_start, x_end), "energy_transfer")
G18_ETrans_MEC_NFP_Plot.SetLineColor(ROOT.kRed)
G18_ETrans_MEC_NFP_Plot.Draw("same")

G18_ETrans_RES_NFP_Plot = df_gst_G18_02a_ETrans_NFP.Filter("res==1").Histo1D(("G18_ETrans_RES_NFP_Plot", "", bins, x_start, x_end), "energy_transfer")
G18_ETrans_RES_NFP_Plot.SetLineColor(ROOT.kGreen)
G18_ETrans_RES_NFP_Plot.Draw("same")

G18_ETrans_DIS_NFP_Plot = df_gst_G18_02a_ETrans_NFP.Filter("dis==1").Histo1D(("G18_ETrans_DIS_NFP_Plot", "", bins, x_start, x_end), "energy_transfer")
G18_ETrans_DIS_NFP_Plot.SetLineColor(ROOT.kBlue)
G18_ETrans_DIS_NFP_Plot.Draw("same")

G18_ETrans_None_NFP_Plot = df_gst_G18_02a_ETrans_NFP.Filter("qel==0&&mec==0&&res==0&&dis==0").Histo1D(("G18_ETrans_None_NFP_Plot", "", bins, x_start, x_end), "energy_transfer")
G18_ETrans_None_NFP_Plot.SetLineColor(ROOT.kGray)
G18_ETrans_None_NFP_Plot.Draw("same")

G18_Etrans_Leg_NFP = ROOT.TLegend(.6, .66, .87, .87)
G18_Etrans_Leg_NFP.AddEntry("G18_ETrans_QEL_NFP_Plot", "QE", "l")
G18_Etrans_Leg_NFP.AddEntry("G18_ETrans_MEC_NFP_Plot", "MEC", "l")
G18_Etrans_Leg_NFP.AddEntry("G18_ETrans_RES_NFP_Plot", "RES", "l")
G18_Etrans_Leg_NFP.AddEntry("G18_ETrans_DIS_NFP_Plot", "DIS", "l")
G18_Etrans_Leg_NFP.AddEntry("G18_ETrans_None_NFP_Plot", "Uncatagorized", "l")
G18_Etrans_Leg_NFP.Draw()

b.SetLogy()
b.Draw()

In [ ]:
#g21 with no final state pions
b = ROOT.TCanvas("b")
df_gst_GEM21_11b_ETrans_NFP = df_gst_GEM21_11b_all.Filter("num_pions==0")
G21_ETrans_QEL_NFP_Plot = df_gst_GEM21_11b_ETrans_NFP.Filter("qel==1").Histo1D(("G21_ETrans_QEL_NFP_Plot", "Energy Transfer in Interaction Types without Final State Pions in GEM21 Model", bins, x_start, x_end), "energy_transfer")
G21_ETrans_QEL_NFP_Plot.SetLineColor(ROOT.kViolet)
G21_ETrans_QEL_NFP_Plot.GetXaxis().SetTitle("Energy Transfer(GeV)")
G21_ETrans_QEL_NFP_Plot.GetYaxis().SetTitle("Counts")
G21_ETrans_QEL_NFP_Plot.Draw("same")

G21_ETrans_MEC_NFP_Plot = df_gst_GEM21_11b_ETrans_NFP.Filter("mec==1").Histo1D(("G21_ETrans_MEC_NFP_Plot", "", bins, x_start, x_end), "energy_transfer")
G21_ETrans_MEC_NFP_Plot.SetLineColor(ROOT.kRed)
G21_ETrans_MEC_NFP_Plot.Draw("same")

G21_ETrans_RES_NFP_Plot = df_gst_GEM21_11b_ETrans_NFP.Filter("res==1").Histo1D(("G21_ETrans_RES_NFP_Plot", "", bins, x_start, x_end), "energy_transfer")
G21_ETrans_RES_NFP_Plot.SetLineColor(ROOT.kGreen)
G21_ETrans_RES_NFP_Plot.Draw("same")

G21_ETrans_DIS_NFP_Plot = df_gst_GEM21_11b_ETrans_NFP.Filter("dis==1").Histo1D(("G21_ETrans_DIS_NFP_Plot", "", bins, x_start, x_end), "energy_transfer")
G21_ETrans_DIS_NFP_Plot.SetLineColor(ROOT.kBlue)
G21_ETrans_DIS_NFP_Plot.Draw("same")

G21_ETrans_None_NFP_Plot = df_gst_GEM21_11b_ETrans_NFP.Filter("qel==0&&mec==0&&res==0&&dis==0").Histo1D(("G21_ETrans_None_NFP_Plot", "", bins, x_start, x_end), "energy_transfer")
G21_ETrans_None_NFP_Plot.SetLineColor(ROOT.kGray)
G21_ETrans_None_NFP_Plot.Draw("same")

G21_Etrans_Leg_NFP = ROOT.TLegend(.6, .66, .87, .87)
G21_Etrans_Leg_NFP.AddEntry("G21_ETrans_QEL_NFP_Plot", "QE", "l")
G21_Etrans_Leg_NFP.AddEntry("G21_ETrans_MEC_NFP_Plot", "MEC", "l")
G21_Etrans_Leg_NFP.AddEntry("G21_ETrans_RES_NFP_Plot", "RES", "l")
G21_Etrans_Leg_NFP.AddEntry("G21_ETrans_DIS_NFP_Plot", "DIS", "l")
G21_Etrans_Leg_NFP.AddEntry("G21_ETrans_None_NFP_Plot", "Uncatagorized", "l")
G21_Etrans_Leg_NFP.Draw()

b.SetLogy()
b.Draw()

In [ ]:
#g18 with no nfa pions
b = ROOT.TCanvas("b")
df_gst_G18_02a_ETrans_NFA = df_gst_G18_02a_all.Filter("nfa_pions==0")
G18_ETrans_QEL_NFA_Plot = df_gst_G18_02a_ETrans_NFA.Filter("qel==1").Histo1D(("G18_ETrans_QEL_NFA_Plot", "Energy Transfer in Interaction Types without NFA Pions in G18 Model", bins, x_start, x_end), "energy_transfer")
G18_ETrans_QEL_NFA_Plot.SetLineColor(ROOT.kViolet)
G18_ETrans_QEL_NFA_Plot.GetXaxis().SetTitle("Energy Transfer(GeV)")
G18_ETrans_QEL_NFA_Plot.GetYaxis().SetTitle("Counts")
G18_ETrans_QEL_NFA_Plot.Draw("same")

G18_ETrans_MEC_NFA_Plot = df_gst_G18_02a_ETrans_NFA.Filter("mec==1").Histo1D(("G18_ETrans_MEC_NFA_Plot", "", bins, x_start, x_end), "energy_transfer")
G18_ETrans_MEC_NFA_Plot.SetLineColor(ROOT.kRed)
G18_ETrans_MEC_NFA_Plot.Draw("same")

G18_ETrans_RES_NFA_Plot = df_gst_G18_02a_ETrans_NFA.Filter("res==1").Histo1D(("G18_ETrans_RES_NFA_Plot", "", bins, x_start, x_end), "energy_transfer")
G18_ETrans_RES_NFA_Plot.SetLineColor(ROOT.kGreen)
G18_ETrans_RES_NFA_Plot.Draw("same")

G18_ETrans_DIS_NFA_Plot = df_gst_G18_02a_ETrans_NFA.Filter("dis==1").Histo1D(("G18_ETrans_DIS_NFA_Plot", "", bins, x_start, x_end), "energy_transfer")
G18_ETrans_DIS_NFA_Plot.SetLineColor(ROOT.kBlue)
G18_ETrans_DIS_NFA_Plot.Draw("same")

G18_ETrans_None_NFA_Plot = df_gst_G18_02a_ETrans_NFA.Filter("qel==0&&mec==0&&res==0&&dis==0").Histo1D(("G18_ETrans_None_NFA_Plot", "", bins, x_start, x_end), "energy_transfer")
G18_ETrans_None_NFA_Plot.SetLineColor(ROOT.kGray)
G18_ETrans_None_NFA_Plot.Draw("same")

G18_Etrans_Leg_NFA = ROOT.TLegend(.6, .66, .87, .87)
G18_Etrans_Leg_NFA.AddEntry("G18_ETrans_QEL_NFA_Plot", "QE", "l")
G18_Etrans_Leg_NFA.AddEntry("G18_ETrans_MEC_NFA_Plot", "MEC", "l")
G18_Etrans_Leg_NFA.AddEntry("G18_ETrans_RES_NFA_Plot", "RES", "l")
G18_Etrans_Leg_NFA.AddEntry("G18_ETrans_DIS_NFA_Plot", "DIS", "l")
G18_Etrans_Leg_NFA.AddEntry("G18_ETrans_None_NFA_Plot", "Uncatagorized", "l")
G18_Etrans_Leg_NFA.Draw()

b.SetLogy()
b.Draw()

In [ ]:
#g21 with no final state pions
b = ROOT.TCanvas("b")
df_gst_GEM21_11b_ETrans_NFA = df_gst_GEM21_11b_all.Filter("nfa_pions==0")
G21_ETrans_QEL_NFA_Plot = df_gst_GEM21_11b_ETrans_NFA.Filter("qel==1").Histo1D(("G21_ETrans_QEL_NFA_Plot", "Energy Transfer in Interaction Types without NFA Pions in GEM21 Model", bins, x_start, x_end), "energy_transfer")
G21_ETrans_QEL_NFA_Plot.SetLineColor(ROOT.kViolet)
G21_ETrans_QEL_NFA_Plot.GetXaxis().SetTitle("Energy Transfer(GeV)")
G21_ETrans_QEL_NFA_Plot.GetYaxis().SetTitle("Counts")
G21_ETrans_QEL_NFA_Plot.Draw("same")

G21_ETrans_MEC_NFA_Plot = df_gst_GEM21_11b_ETrans_NFA.Filter("mec==1").Histo1D(("G21_ETrans_MEC_NFA_Plot", "", bins, x_start, x_end), "energy_transfer")
G21_ETrans_MEC_NFA_Plot.SetLineColor(ROOT.kRed)
G21_ETrans_MEC_NFA_Plot.Draw("same")

G21_ETrans_RES_NFA_Plot = df_gst_GEM21_11b_ETrans_NFA.Filter("res==1").Histo1D(("G21_ETrans_RES_NFA_Plot", "", bins, x_start, x_end), "energy_transfer")
G21_ETrans_RES_NFA_Plot.SetLineColor(ROOT.kGreen)
G21_ETrans_RES_NFA_Plot.Draw("same")

G21_ETrans_DIS_NFA_Plot = df_gst_GEM21_11b_ETrans_NFA.Filter("dis==1").Histo1D(("G21_ETrans_DIS_NFA_Plot", "", bins, x_start, x_end), "energy_transfer")
G21_ETrans_DIS_NFA_Plot.SetLineColor(ROOT.kBlue)
G21_ETrans_DIS_NFA_Plot.Draw("same")

G21_ETrans_None_NFA_Plot = df_gst_GEM21_11b_ETrans_NFA.Filter("qel==0&&mec==0&&res==0&&dis==0").Histo1D(("G21_ETrans_None_NFA_Plot", "", bins, x_start, x_end), "energy_transfer")
G21_ETrans_None_NFA_Plot.SetLineColor(ROOT.kGray)
G21_ETrans_None_NFA_Plot.Draw("same")

G21_Etrans_Leg_NFA = ROOT.TLegend(.6, .66, .87, .87)
G21_Etrans_Leg_NFA.AddEntry("G21_ETrans_QEL_NFA_Plot", "QE", "l")
G21_Etrans_Leg_NFA.AddEntry("G21_ETrans_MEC_NFA_Plot", "MEC", "l")
G21_Etrans_Leg_NFA.AddEntry("G21_ETrans_RES_NFA_Plot", "RES", "l")
G21_Etrans_Leg_NFA.AddEntry("G21_ETrans_DIS_NFA_Plot", "DIS", "l")
G21_Etrans_Leg_NFA.AddEntry("G21_ETrans_None_NFA_Plot", "Uncatagorized", "l")
G21_Etrans_Leg_NFA.Draw()

b.SetLogy()
b.Draw()

In [ ]:
#plotting thetazl w lepton pt >200 MeV
bins = 100
x_start = 0
x_end = 0.42

In [ ]:
#g18 model
l = ROOT.TCanvas("l")
df_gst_G18_02a_lep200 = df_gst_G18_02a_all.Filter("ptl>.2")
G18_Thetazl_QEL_Plot = df_gst_G18_02a_lep200.Filter("qel==1").Histo1D(("G18_Thetazl_QEL_Plot", "Thetazl of Interaction Types with PTL > 200 MeV In G18 Model", bins, x_start, x_end), "thetazl")
G18_Thetazl_QEL_Plot.SetLineColor(ROOT.kViolet)
G18_Thetazl_QEL_Plot.GetXaxis().SetTitle("Lepton Angle (Radians)")
G18_Thetazl_QEL_Plot.GetYaxis().SetTitle("Counts")
G18_Thetazl_QEL_Plot.Draw("same")

G18_Thetazl_MEC_Plot = df_gst_G18_02a_lep200.Filter("mec==1").Histo1D(("G18_Thetazl_MEC_Plot", "", bins, x_start, x_end),"thetazl")
G18_Thetazl_MEC_Plot.SetLineColor(ROOT.kRed)
G18_Thetazl_MEC_Plot.Draw("same")

G18_Thetazl_RES_Plot = df_gst_G18_02a_lep200.Filter("res==1").Histo1D(("G18_Thetazl_RES_Plot", "", bins, x_start, x_end), "thetazl")
G18_Thetazl_RES_Plot.SetLineColor(ROOT.kGreen)
G18_Thetazl_RES_Plot.Draw("same")

G18_Thetazl_DIS_Plot = df_gst_G18_02a_lep200.Filter("dis==1").Histo1D(("G18_Thetazl_DIS_Plot", "", bins, x_start, x_end), "thetazl")
G18_Thetazl_DIS_Plot.SetLineColor(ROOT.kBlue)
G18_Thetazl_DIS_Plot.Draw("same")

G18_Thetazl_None_Plot = df_gst_G18_02a_lep200.Filter("qel==0&&mec==0&&res==0&&dis==0").Histo1D(("G18_Thetazl_None_Plot", "", bins, x_start, x_end),"thetazl")
G18_Thetazl_None_Plot.SetLineColor(ROOT.kGray)
G18_Thetazl_None_Plot.Draw("same")

G18_Thetazl_Leg = ROOT.TLegend(.6, .66, .87, .87)
G18_Thetazl_Leg.AddEntry("G18_Thetazl_QEL_Plot", "QEL", "l")
G18_Thetazl_Leg.AddEntry("G18_Thetazl_MEC_Plot", "MEC", "l")
G18_Thetazl_Leg.AddEntry("G18_Thetazl_RES_Plot", "RES", "l")
G18_Thetazl_Leg.AddEntry("G18_Thetazl_DIS_Plot", "DIS", "l")
G18_Thetazl_Leg.AddEntry("G18_Thetazl_None_Plot", "Uncatagorized", "l")

G18_Thetazl_Leg.Draw()
l.Draw()

In [ ]:
#gem21 model
l = ROOT.TCanvas("l")
df_gst_GEM21_11b_lep200 = df_gst_GEM21_11b_all.Filter("ptl>.2")
G21_Thetazl_QEL_Plot = df_gst_GEM21_11b_lep200.Filter("qel==1").Histo1D(("G21_Thetazl_QEL_Plot", "Thetazl of Interaction Types with PTL > 200 MeV In GEM21 Model", bins, x_start, x_end), "thetazl")
G21_Thetazl_QEL_Plot.SetLineColor(ROOT.kViolet)
G21_Thetazl_QEL_Plot.GetXaxis().SetTitle("Lepton Angle (Radians)")
G21_Thetazl_QEL_Plot.GetYaxis().SetTitle("Counts")
G21_Thetazl_QEL_Plot.Draw("same")

G21_Thetazl_MEC_Plot = df_gst_GEM21_11b_lep200.Filter("mec==1").Histo1D(("G21_Thetazl_MEC_Plot", "", bins, x_start, x_end),"thetazl")
G21_Thetazl_MEC_Plot.SetLineColor(ROOT.kRed)
G21_Thetazl_MEC_Plot.Draw("same")

G21_Thetazl_RES_Plot = df_gst_GEM21_11b_lep200.Filter("res==1").Histo1D(("G21_Thetazl_RES_Plot", "", bins, x_start, x_end), "thetazl")
G21_Thetazl_RES_Plot.SetLineColor(ROOT.kGreen)
G21_Thetazl_RES_Plot.Draw("same")

G21_Thetazl_DIS_Plot = df_gst_GEM21_11b_lep200.Filter("dis==1").Histo1D(("G21_Thetazl_DIS_Plot", "", bins, x_start, x_end), "thetazl")
G21_Thetazl_DIS_Plot.SetLineColor(ROOT.kBlue)
G21_Thetazl_DIS_Plot.Draw("same")

G21_Thetazl_None_Plot = df_gst_GEM21_11b_lep200.Filter("qel==0&&mec==0&&res==0&&dis==0").Histo1D(("G21_Thetazl_None_Plot", "", bins, x_start, x_end),"thetazl")
G21_Thetazl_None_Plot.SetLineColor(ROOT.kGray)
G21_Thetazl_None_Plot.Draw("same")

G21_Thetazl_Leg = ROOT.TLegend(.6, .66, .87, .87)
G21_Thetazl_Leg.AddEntry("G21_Thetazl_QEL_Plot", "QEL", "l")
G21_Thetazl_Leg.AddEntry("G21_Thetazl_MEC_Plot", "MEC", "l")
G21_Thetazl_Leg.AddEntry("G21_Thetazl_RES_Plot", "RES", "l")
G21_Thetazl_Leg.AddEntry("G21_Thetazl_DIS_Plot", "DIS", "l")
G21_Thetazl_Leg.AddEntry("G21_Thetazl_None_Plot", "Uncatagorized", "l")
G21_Thetazl_Leg.Draw()

l.Draw()

In [ ]:
#plotting thetazl w lepton pt > 400 MeV
bins = 100
x_start = 0
x_end = 0.6

In [ ]:
#g18 model
l = ROOT.TCanvas("l")
df_gst_G18_02a_lep200 = df_gst_G18_02a_all.Filter("ptl>.4")
G18_Thetazl_QEL_Plot = df_gst_G18_02a_lep200.Filter("qel==1").Histo1D(("G18_Thetazl_QEL_Plot", "Thetazl of Interaction Types with PTL > 400 MeV In G18 Model", bins, x_start, x_end), "thetazl")
G18_Thetazl_QEL_Plot.SetLineColor(ROOT.kViolet)
G18_Thetazl_QEL_Plot.GetXaxis().SetTitle("Lepton Angle (Radians)")
G18_Thetazl_QEL_Plot.GetYaxis().SetTitle("Counts")
G18_Thetazl_QEL_Plot.Draw("same")

G18_Thetazl_MEC_Plot = df_gst_G18_02a_lep200.Filter("mec==1").Histo1D(("G18_Thetazl_MEC_Plot", "", bins, x_start, x_end),"thetazl")
G18_Thetazl_MEC_Plot.SetLineColor(ROOT.kRed)
G18_Thetazl_MEC_Plot.Draw("same")

G18_Thetazl_RES_Plot = df_gst_G18_02a_lep200.Filter("res==1").Histo1D(("G18_Thetazl_RES_Plot", "", bins, x_start, x_end), "thetazl")
G18_Thetazl_RES_Plot.SetLineColor(ROOT.kGreen)
G18_Thetazl_RES_Plot.Draw("same")

G18_Thetazl_DIS_Plot = df_gst_G18_02a_lep200.Filter("dis==1").Histo1D(("G18_Thetazl_DIS_Plot", "", bins, x_start, x_end), "thetazl")
G18_Thetazl_DIS_Plot.SetLineColor(ROOT.kBlue)
G18_Thetazl_DIS_Plot.Draw("same")

G18_Thetazl_None_Plot = df_gst_G18_02a_lep200.Filter("qel==0&&mec==0&&res==0&&dis==0").Histo1D(("G18_Thetazl_None_Plot", "", bins, x_start, x_end),"thetazl")
G18_Thetazl_None_Plot.SetLineColor(ROOT.kGray)
G18_Thetazl_None_Plot.Draw("same")

G18_Thetazl_Leg = ROOT.TLegend(.6, .66, .87, .87)
G18_Thetazl_Leg.AddEntry("G18_Thetazl_QEL_Plot", "QEL", "l")
G18_Thetazl_Leg.AddEntry("G18_Thetazl_MEC_Plot", "MEC", "l")
G18_Thetazl_Leg.AddEntry("G18_Thetazl_RES_Plot", "RES", "l")
G18_Thetazl_Leg.AddEntry("G18_Thetazl_DIS_Plot", "DIS", "l")
G18_Thetazl_Leg.AddEntry("G18_Thetazl_None_Plot", "Uncatagorized", "l")

G18_Thetazl_Leg.Draw()
l.Draw()

In [ ]:
#gem21 model
l = ROOT.TCanvas("l")
df_gst_GEM21_11b_lep200 = df_gst_GEM21_11b_all.Filter("ptl>.4")
G21_Thetazl_QEL_Plot = df_gst_GEM21_11b_lep200.Filter("qel==1").Histo1D(("G21_Thetazl_QEL_Plot", "Thetazl of Interaction Types with PTL > 400 MeV In GEM21 Model", bins, x_start, x_end), "thetazl")
G21_Thetazl_QEL_Plot.SetLineColor(ROOT.kViolet)
G21_Thetazl_QEL_Plot.GetXaxis().SetTitle("Lepton Angle (Radians)")
G21_Thetazl_QEL_Plot.GetYaxis().SetTitle("Counts")
G21_Thetazl_QEL_Plot.Draw("same")

G21_Thetazl_MEC_Plot = df_gst_GEM21_11b_lep200.Filter("mec==1").Histo1D(("G21_Thetazl_MEC_Plot", "", bins, x_start, x_end),"thetazl")
G21_Thetazl_MEC_Plot.SetLineColor(ROOT.kRed)
G21_Thetazl_MEC_Plot.Draw("same")

G21_Thetazl_RES_Plot = df_gst_GEM21_11b_lep200.Filter("res==1").Histo1D(("G21_Thetazl_RES_Plot", "", bins, x_start, x_end), "thetazl")
G21_Thetazl_RES_Plot.SetLineColor(ROOT.kGreen)
G21_Thetazl_RES_Plot.Draw("same")

G21_Thetazl_DIS_Plot = df_gst_GEM21_11b_lep200.Filter("dis==1").Histo1D(("G21_Thetazl_DIS_Plot", "", bins, x_start, x_end), "thetazl")
G21_Thetazl_DIS_Plot.SetLineColor(ROOT.kBlue)
G21_Thetazl_DIS_Plot.Draw("same")

G21_Thetazl_None_Plot = df_gst_GEM21_11b_lep200.Filter("qel==0&&mec==0&&res==0&&dis==0").Histo1D(("G21_Thetazl_None_Plot", "", bins, x_start, x_end),"thetazl")
G21_Thetazl_None_Plot.SetLineColor(ROOT.kGray)
G21_Thetazl_None_Plot.Draw("same")

G21_Thetazl_Leg = ROOT.TLegend(.6, .66, .87, .87)
G21_Thetazl_Leg.AddEntry("G21_Thetazl_QEL_Plot", "QEL", "l")
G21_Thetazl_Leg.AddEntry("G21_Thetazl_MEC_Plot", "MEC", "l")
G21_Thetazl_Leg.AddEntry("G21_Thetazl_RES_Plot", "RES", "l")
G21_Thetazl_Leg.AddEntry("G21_Thetazl_DIS_Plot", "DIS", "l")
G21_Thetazl_Leg.AddEntry("G21_Thetazl_None_Plot", "Uncatagorized", "l")
G21_Thetazl_Leg.Draw()

l.Draw()

In [ ]:
#new things

In [ ]:
for c in df_gst_GEM21_11b_all.GetColumnNames():
    print(c)

In [ ]:
for c in df_gst_G18_02a_all.GetColumnNames():
    print(c)

In [ ]:
#For issues w plot, maybe all are low energy, place low energy cut to see if they go away 
#kef_proton > 60 MeV or just remove protons w angle at zero

In [ ]:
#make a quick check to see that the proton indices are filled, and sorted by KE
rows=10
disp = df_gst_GEM21_11b_all.Filter("nfp>0").Range(rows).Display(["nfp","kefa_proton","thetazf_proton","leading_protonfa_angle"],rows)
disp.Print()

In [ ]:
#reminder: make the ones for gem21
lep = .2
lepp = round(lep + .10, 2)
bins = 100
x_start = 0.0
x_end = .5

In [ ]:
#g18 leading proton ke for final state protons with final state pions and etrans > .3
l = ROOT.TCanvas("l")
df_gst_G18_02a_etcut = df_gst_G18_02a_all.Filter("energy_transfer < 0.3").Filter(f"ptl > {lep} && ptl < {lepp}")
G18_LeadFProton_QEL_P = df_gst_G18_02a_etcut.Filter("qel==1").Histo1D(("G18_LeadFProton_QEL_P", "", bins, x_start, x_end), "kef_proton_max")
G18_LeadFProton_QEL_P.SetLineColor(ROOT.kViolet)
G18_LeadFProton_QEL_P.SetTitle(f"Leading Kinetic Energy of Final State Protons with Lep PT [{lep} - {lepp}]GeV and ETrans < 0.3 GeV in G18")
G18_LeadFProton_QEL_P.GetXaxis().SetTitle("Kinetic Energy (GeV)")
G18_LeadFProton_QEL_P.GetYaxis().SetTitle("Counts")
G18_LeadFProton_QEL_P.Draw("same")

G18_LeadFProton_MEC_P = df_gst_G18_02a_etcut.Filter("mec==1").Histo1D(("G18_LeadFProton_MEC_P", "", bins, x_start, x_end), "kef_proton_max")
G18_LeadFProton_MEC_P.SetLineColor(ROOT.kRed)
G18_LeadFProton_MEC_P.Draw("same")

G18_LeadFProton_RES_P = df_gst_G18_02a_etcut.Filter("res==1").Histo1D(("G18_LeadFProton_RES_P", "", bins, x_start, x_end), "kef_proton_max")
G18_LeadFProton_RES_P.SetLineColor(ROOT.kGreen)
G18_LeadFProton_RES_P.Draw("same")

G18_LeadFProton_DIS_P = df_gst_G18_02a_etcut.Filter("dis==1").Histo1D(("G18_LeadFProton_DIS_P", "", bins, x_start, x_end), "kef_proton_max")
G18_LeadFProton_DIS_P.SetLineColor(ROOT.kBlue)
G18_LeadFProton_DIS_P.Draw("same")

G18_LeadFProton_None_P = df_gst_G18_02a_etcut.Filter("qel==0&&mec==0&&res==0&&dis==0").Histo1D(("G18_LeadFProton_None_P", "", bins, x_start, x_end), "kef_proton_max")
G18_LeadFProton_None_P.SetLineColor(ROOT.kGray)
G18_LeadFProton_None_P.Draw("same")

G18_LeadFProton_Leg = ROOT.TLegend(.6, .66, .87, .87)
G18_LeadFProton_Leg.AddEntry("G18_LeadFProton_QEL_P", "QEL", "l")
G18_LeadFProton_Leg.AddEntry("G18_LeadFProton_MEC_P", "MEC", "l")
G18_LeadFProton_Leg.AddEntry("G18_LeadFProton_RES_P", "RES", "l")
G18_LeadFProton_Leg.AddEntry("G18_LeadFProton_DIS_P", "DIS", "l")
G18_LeadFProton_Leg.AddEntry("G18_LeadFProton_None_P", "Uncatagorized", "l")
G18_LeadFProton_Leg.Draw()
l.SetLogy()
l.Draw()

In [ ]:
#g18 leading proton ke for final state protons without final state pions and etrans > .3
l = ROOT.TCanvas("l")
df_gst_G18_02a_etcut = df_gst_G18_02a_all.Filter("energy_transfer < 0.3").Filter(f"ptl > {lep} && ptl < {lepp}").Filter("num_pions==0")
G18_LeadFProton_QEL_NP = df_gst_G18_02a_etcut.Filter("qel==1").Histo1D(("G18_LeadFProton_QEL_NP", "", bins, x_start, x_end), "kef_proton_max")
G18_LeadFProton_QEL_NP.SetLineColor(ROOT.kViolet)
G18_LeadFProton_QEL_NP.SetTitle(f"Leading Kinetic Energy of Final State Protons with Lep PT [{lep} - {lepp}] GeV and ETrans < 0.3 GeV w/o Pions in G18")
G18_LeadFProton_QEL_NP.GetXaxis().SetTitle("Kinetic Energy (GeV)")
G18_LeadFProton_QEL_NP.GetYaxis().SetTitle("Counts")
G18_LeadFProton_QEL_NP.Draw("same")

G18_LeadFProton_MEC_NP = df_gst_G18_02a_etcut.Filter("mec==1").Histo1D(("G18_LeadFProton_MEC_NP", "", bins, x_start, x_end), "kef_proton_max")
G18_LeadFProton_MEC_NP.SetLineColor(ROOT.kRed)
G18_LeadFProton_MEC_NP.Draw("same")

G18_LeadFProton_RES_NP = df_gst_G18_02a_etcut.Filter("res==1").Histo1D(("G18_LeadFProton_RES_NP", "", bins, x_start, x_end), "kef_proton_max")
G18_LeadFProton_RES_NP.SetLineColor(ROOT.kGreen)
G18_LeadFProton_RES_NP.Draw("same")

G18_LeadFProton_DIS_NP = df_gst_G18_02a_etcut.Filter("dis==1").Histo1D(("G18_LeadFProton_DIS_NP", "", bins, x_start, x_end), "kef_proton_max")
G18_LeadFProton_DIS_NP.SetLineColor(ROOT.kBlue)
G18_LeadFProton_DIS_NP.Draw("same")

G18_LeadFProton_None_NP = df_gst_G18_02a_etcut.Filter("qel==0&&mec==0&&res==0&&dis==0").Histo1D(("G18_LeadFProton_None_NP", "", bins, x_start, x_end), "kef_proton_max")
G18_LeadFProton_None_NP.SetLineColor(ROOT.kGray)
G18_LeadFProton_None_NP.Draw("same")

G18_LeadFProton_Leg = ROOT.TLegend(.6, .66, .87, .87)
G18_LeadFProton_Leg.AddEntry("G18_LeadFProton_QEL_NP", "QEL", "l")
G18_LeadFProton_Leg.AddEntry("G18_LeadFProton_MEC_NP", "MEC", "l")
G18_LeadFProton_Leg.AddEntry("G18_LeadFProton_RES_NP", "RES", "l")
G18_LeadFProton_Leg.AddEntry("G18_LeadFProton_DIS_NP", "DIS", "l")
G18_LeadFProton_Leg.AddEntry("G18_LeadFProton_None_NP", "Uncatagorized", "l")
G18_LeadFProton_Leg.Draw()
l.SetLogy()
l.Draw()

In [ ]:
#g18 leading proton ke for final state accepted protons with final state pions and etrans > .3
l = ROOT.TCanvas("l")
df_gst_G18_02a_etcut = df_gst_G18_02a_all.Filter("energy_transfer < 0.3").Filter(f"ptl > {lep} && ptl < {lepp}")
G18_LeadFAProton_QEL_P = df_gst_G18_02a_etcut.Filter("qel==1").Histo1D(("G18_LeadFAProton_QEL_P", "", bins, x_start, x_end), "kefa_proton_max")
G18_LeadFAProton_QEL_P.SetLineColor(ROOT.kViolet)
G18_LeadFAProton_QEL_P.SetTitle(f"Leading Kinetic Energy of Final State Accepted Protons with Lep PT [{lep} - {lepp}] GeV and ETrans < 0.3 GeV in G18")
G18_LeadFAProton_QEL_P.GetXaxis().SetTitle("Kinetic Energy (GeV)")
G18_LeadFAProton_QEL_P.GetYaxis().SetTitle("Counts")
G18_LeadFAProton_QEL_P.Draw("same")

G18_LeadFAProton_MEC_P = df_gst_G18_02a_etcut.Filter("mec==1").Histo1D(("G18_LeadFAProton_MEC_P", "", bins, x_start, x_end), "kefa_proton_max")
G18_LeadFAProton_MEC_P.SetLineColor(ROOT.kRed)
G18_LeadFAProton_MEC_P.Draw("same")

G18_LeadFAProton_RES_P = df_gst_G18_02a_etcut.Filter("res==1").Histo1D(("G18_LeadFAProton_RES_P", "", bins, x_start, x_end), "kefa_proton_max")
G18_LeadFAProton_RES_P.SetLineColor(ROOT.kGreen)
G18_LeadFAProton_RES_P.Draw("same")

G18_LeadFAProton_DIS_P = df_gst_G18_02a_etcut.Filter("dis==1").Histo1D(("G18_LeadFAProton_DIS_P", "", bins, x_start, x_end), "kefa_proton_max")
G18_LeadFAProton_DIS_P.SetLineColor(ROOT.kBlue)
G18_LeadFAProton_DIS_P.Draw("same")

G18_LeadFAProton_None_P = df_gst_G18_02a_etcut.Filter("qel==0&&mec==0&&res==0&&dis==0").Histo1D(("G18_LeadFAProton_None_P", "", bins, x_start, x_end), "kefa_proton_max")
G18_LeadFAProton_None_P.SetLineColor(ROOT.kGray)
G18_LeadFAProton_None_P.Draw("same")

G18_LeadFAProton_Leg = ROOT.TLegend(.6, .66, .87, .87)
G18_LeadFAProton_Leg.AddEntry("G18_LeadFAProton_QEL_P", "QEL", "l")
G18_LeadFAProton_Leg.AddEntry("G18_LeadFAProton_MEC_P", "MEC", "l")
G18_LeadFAProton_Leg.AddEntry("G18_LeadFAProton_RES_P", "RES", "l")
G18_LeadFAProton_Leg.AddEntry("G18_LeadFAProton_DIS_P", "DIS", "l")
G18_LeadFAProton_Leg.AddEntry("G18_LeadFAProton_None_P", "Uncatagorized", "l")
G18_LeadFAProton_Leg.Draw()
l.SetLogy()
l.Draw()

In [ ]:
#g18 leading proton ke for final state accepted protons without final state pions and etrans > .3
l = ROOT.TCanvas("l")
df_gst_G18_02a_etcut = df_gst_G18_02a_all.Filter("energy_transfer < 0.3").Filter(f"ptl > {lep} && ptl < {lepp}").Filter("num_pions==0")
G18_LeadFAProton_QEL_P = df_gst_G18_02a_etcut.Filter("qel==1").Histo1D(("G18_LeadFAProton_QEL_P", "", bins, x_start, x_end), "kefa_proton_max")
G18_LeadFAProton_QEL_P.SetLineColor(ROOT.kViolet)
G18_LeadFAProton_QEL_P.SetTitle(f"Leading Kinetic Energy of Final State Accepted Protons with Lep PT [{lep} - {lepp}] GeV and ETrans < 0.3 GeV w/o Pions in G18")
G18_LeadFAProton_QEL_P.GetXaxis().SetTitle("Kinetic Energy (GeV)")
G18_LeadFAProton_QEL_P.GetYaxis().SetTitle("Counts")
G18_LeadFAProton_QEL_P.Draw("same")

G18_LeadFAProton_MEC_P = df_gst_G18_02a_etcut.Filter("mec==1").Histo1D(("G18_LeadFAProton_MEC_P", "", bins, x_start, x_end), "kefa_proton_max")
G18_LeadFAProton_MEC_P.SetLineColor(ROOT.kRed)
G18_LeadFAProton_MEC_P.Draw("same")

G18_LeadFAProton_RES_P = df_gst_G18_02a_etcut.Filter("res==1").Histo1D(("G18_LeadFAProton_RES_P", "", bins, x_start, x_end), "kefa_proton_max")
G18_LeadFAProton_RES_P.SetLineColor(ROOT.kGreen)
G18_LeadFAProton_RES_P.Draw("same")

G18_LeadFAProton_DIS_P = df_gst_G18_02a_etcut.Filter("dis==1").Histo1D(("G18_LeadFAProton_DIS_P", "", bins, x_start, x_end), "kefa_proton_max")
G18_LeadFAProton_DIS_P.SetLineColor(ROOT.kBlue)
G18_LeadFAProton_DIS_P.Draw("same")

G18_LeadFAProton_None_P = df_gst_G18_02a_etcut.Filter("qel==0&&mec==0&&res==0&&dis==0").Histo1D(("G18_LeadFAProton_None_P", "", bins, x_start, x_end), "kefa_proton_max")
G18_LeadFAProton_None_P.SetLineColor(ROOT.kGray)
G18_LeadFAProton_None_P.Draw("same")

G18_LeadFAProton_Leg = ROOT.TLegend(.6, .66, .87, .87)
G18_LeadFAProton_Leg.AddEntry("G18_LeadFAProton_QEL_P", "QEL", "l")
G18_LeadFAProton_Leg.AddEntry("G18_LeadFAProton_MEC_P", "MEC", "l")
G18_LeadFAProton_Leg.AddEntry("G18_LeadFAProton_RES_P", "RES", "l")
G18_LeadFAProton_Leg.AddEntry("G18_LeadFAProton_DIS_P", "DIS", "l")
G18_LeadFAProton_Leg.AddEntry("G18_LeadFAProton_None_P", "Uncatagorized", "l")
G18_LeadFAProton_Leg.Draw()
l.SetLogy()
l.Draw()

In [5]:
#leading angle of proton
#Filter("kef_proton > 0.06 && nfp > 0 && thetazf_proton > 0") 
bins = 100
x_start = 0
x_end = 4

In [ ]:
#g18 plotting leading angle for final state protons with pions and etrans > .3
l = ROOT.TCanvas()
df_gst_G18_02a_highE = df_gst_G18_02a_all.Filter("energy_transfer > 0.3")
G18_LeadFProtonAng_QEL_P = df_gst_G18_02a_highE.Filter("qel == 1").Histo1D(("G18_LeadFProtonAng_QEL_P", "", bins, x_start, x_end), "leading_protonf_angle")
G18_LeadFProtonAng_QEL_P.SetTitle("Leading Proton Angle with Etrans > 0.3 GeV in G18")
G18_LeadFProtonAng_QEL_P.GetXaxis().SetTitle("Angle (Radians)")
G18_LeadFProtonAng_QEL_P.GetYaxis().SetTitle("Counts")
G18_LeadFProtonAng_QEL_P.GetYaxis().SetRange(0, 100000) #
G18_LeadFProtonAng_QEL_P.SetLineColor(ROOT.kViolet)
G18_LeadFProtonAng_QEL_P.Draw("same")

G18_LeadFProtonAng_MEC_P = df_gst_G18_02a_highE.Filter("mec == 1").Histo1D(("G18_LeadFProtonAng_MEC_P", "", bins, x_start, x_end), "leading_protonf_angle")
G18_LeadFProtonAng_MEC_P.SetLineColor(ROOT.kRed)
G18_LeadFProtonAng_MEC_P.Draw("same")

G18_LeadFProtonAng_RES_P = df_gst_G18_02a_highE.Filter("res == 1").Histo1D(("G18_LeadFProtonAng_RES_P", "", bins, x_start, x_end), "leading_protonf_angle")
G18_LeadFProtonAng_RES_P.SetLineColor(ROOT.kGreen)
G18_LeadFProtonAng_RES_P.Draw("same")

G18_LeadFProtonAng_DIS_P = df_gst_G18_02a_highE.Filter("dis == 1").Histo1D(("G18_LeadFProtonAng_DIS_P", "", bins, x_start, x_end), "leading_protonf_angle")
G18_LeadFProtonAng_DIS_P.SetLineColor(ROOT.kBlue)
G18_LeadFProtonAng_DIS_P.Draw("same")

G18_LeadFProtonAng_None_P = df_gst_G18_02a_highE.Filter("qel==0 && mec==0 && res==0 && dis==0").Histo1D(("G18_LeadFProtonAng_None_P", "", bins, x_start, x_end), "leading_protonf_angle")
G18_LeadFProtonAng_None_P.SetLineColor(ROOT.kGray)
G18_LeadFProtonAng_None_P.Draw("same")

G18_LeadFProtonAng_Leg_P = ROOT.TLegend(.6, .66, .87, .87)
G18_LeadFProtonAng_Leg_P.AddEntry("G18_LeadFProtonAng_QEL_P", "QEL", "l")
G18_LeadFProtonAng_Leg_P.AddEntry("G18_LeadFProtonAng_MEC_P", "MEC", "l")
G18_LeadFProtonAng_Leg_P.AddEntry("G18_LeadFProtonAng_RES_P", "RES", "l")
G18_LeadFProtonAng_Leg_P.AddEntry("G18_LeadFProtonAng_DIS_P", "DIS", "l")
G18_LeadFProtonAng_Leg_P.AddEntry("G18_LeadFProtonAng_None_P", "Uncatagorized", "l")
G18_LeadFProtonAng_Leg_P.Draw()

l.SetLogy()
l.Draw()

In [ ]:
#g18 plotting leading angle for final state protons without pions and etrans > .3
l = ROOT.TCanvas()
df_gst_G18_02a_highE = df_gst_G18_02a_all.Filter("energy_transfer > 0.3").Filter("num_pions==0")
G18_LeadFProtonAng_QEL_NP = df_gst_G18_02a_highE.Filter("qel == 1").Histo1D(("G18_LeadFProtonAng_QEL_NP", "", bins, x_start, x_end), "leading_protonf_angle")
G18_LeadFProtonAng_QEL_NP.SetTitle("Leading Proton Angle with Etrans > 0.3 GeV w/o Pions in G18")
G18_LeadFProtonAng_QEL_NP.GetXaxis().SetTitle("Angle (Radians)")
G18_LeadFProtonAng_QEL_NP.GetYaxis().SetTitle("Counts")
G18_LeadFProtonAng_QEL_NP.GetYaxis().SetRange(0, 100000) #?
G18_LeadFProtonAng_QEL_NP.SetLineColor(ROOT.kViolet)
G18_LeadFProtonAng_QEL_NP.Draw("same")

G18_LeadFProtonAng_MEC_NP = df_gst_G18_02a_highE.Filter("mec == 1").Histo1D(("G18_LeadFProtonAng_MEC_NP", "", bins, x_start, x_end), "leading_protonf_angle")
G18_LeadFProtonAng_MEC_NP.SetLineColor(ROOT.kRed)
G18_LeadFProtonAng_MEC_NP.Draw("same")

G18_LeadFProtonAng_RES_NP = df_gst_G18_02a_highE.Filter("res == 1").Histo1D(("G18_LeadFProtonAng_RES_NP", "", bins, x_start, x_end), "leading_protonf_angle")
G18_LeadFProtonAng_RES_NP.SetLineColor(ROOT.kGreen)
G18_LeadFProtonAng_RES_NP.Draw("same")

G18_LeadFProtonAng_DIS_NP = df_gst_G18_02a_highE.Filter("dis == 1").Histo1D(("G18_LeadFProtonAng_DIS_NP", "", bins, x_start, x_end), "leading_protonf_angle")
G18_LeadFProtonAng_DIS_NP.SetLineColor(ROOT.kBlue)
G18_LeadFProtonAng_DIS_NP.Draw("same")

G18_LeadFProtonAng_None_NP = df_gst_G18_02a_highE.Filter("qel==0 && mec==0 && res==0 && dis==0").Histo1D(("G18_LeadFProtonAng_None_NP", "", bins, x_start, x_end), "leading_protonf_angle")
G18_LeadFProtonAng_None_NP.SetLineColor(ROOT.kGray)
G18_LeadFProtonAng_None_NP.Draw("same")

G18_LeadFProtonAng_Leg_NP = ROOT.TLegend(.6, .66, .87, .87)
G18_LeadFProtonAng_Leg_NP.AddEntry("G18_LeadFProtonAng_QEL_NP", "QEL", "l")
G18_LeadFProtonAng_Leg_NP.AddEntry("G18_LeadFProtonAng_MEC_NP", "MEC", "l")
G18_LeadFProtonAng_Leg_NP.AddEntry("G18_LeadFProtonAng_RES_NP", "RES", "l")
G18_LeadFProtonAng_Leg_NP.AddEntry("G18_LeadFProtonAng_DIS_NP", "DIS", "l")
G18_LeadFProtonAng_Leg_NP.AddEntry("G18_LeadFProtonAng_None_NP", "Uncatagorized", "l")
G18_LeadFProtonAng_Leg_NP.Draw()

l.SetLogy()
l.Draw()

In [6]:
#g18 plotting leading angle for final state accepted protons with pions and etrans > .3
l = ROOT.TCanvas()
df_gst_G18_02a_highE = df_gst_G18_02a_all.Filter("energy_transfer > 0.3")
G18_LeadFAProtonAng_QEL_P = df_gst_G18_02a_highE.Filter("qel == 1").Histo1D(("G18_LeadFAProtonAng_QEL_P", "", bins, x_start, x_end), "leading_protonfa_angle")
G18_LeadFAProtonAng_QEL_P.SetTitle("Leading Proton Angle in Acceptance Range with Etrans > 0.3 GeV in G18")
G18_LeadFAProtonAng_QEL_P.GetXaxis().SetTitle("Angle (Radians)")
G18_LeadFAProtonAng_QEL_P.GetYaxis().SetTitle("Counts")
G18_LeadFAProtonAng_QEL_P.GetYaxis().SetRange(0, 100000) #
G18_LeadFAProtonAng_QEL_P.SetLineColor(ROOT.kViolet)
G18_LeadFAProtonAng_QEL_P.Draw("same")

G18_LeadFAProtonAng_MEC_P = df_gst_G18_02a_highE.Filter("mec == 1").Histo1D(("G18_LeadFAProtonAng_MEC_P", "", bins, x_start, x_end), "leading_protonfa_angle")
G18_LeadFAProtonAng_MEC_P.SetLineColor(ROOT.kRed)
G18_LeadFAProtonAng_MEC_P.Draw("same")

G18_LeadFAProtonAng_RES_P = df_gst_G18_02a_highE.Filter("res == 1").Histo1D(("G18_LeadFAProtonAng_RES_P", "", bins, x_start, x_end), "leading_protonfa_angle")
G18_LeadFAProtonAng_RES_P.SetLineColor(ROOT.kGreen)
G18_LeadFAProtonAng_RES_P.Draw("same")

G18_LeadFAProtonAng_DIS_P = df_gst_G18_02a_highE.Filter("dis == 1").Histo1D(("G18_LeadFAProtonAng_DIS_P", "", bins, x_start, x_end), "leading_protonfa_angle")
G18_LeadFAProtonAng_DIS_P.SetLineColor(ROOT.kBlue)
G18_LeadFAProtonAng_DIS_P.Draw("same")

G18_LeadFAProtonAng_None_P = df_gst_G18_02a_highE.Filter("qel==0 && mec==0 && res==0 && dis==0").Histo1D(("G18_LeadFAProtonAng_None_P", "", bins, x_start, x_end), "leading_protonfa_angle")
G18_LeadFAProtonAng_None_P.SetLineColor(ROOT.kGray)
G18_LeadFAProtonAng_None_P.Draw("same")

G18_LeadFAProtonAng_Leg_P = ROOT.TLegend(.6, .66, .87, .87)
G18_LeadFAProtonAng_Leg_P.AddEntry("G18_LeadFAProtonAng_QEL_P", "QEL", "l")
G18_LeadFAProtonAng_Leg_P.AddEntry("G18_LeadFAProtonAng_MEC_P", "MEC", "l")
G18_LeadFAProtonAng_Leg_P.AddEntry("G18_LeadFAProtonAng_RES_P", "RES", "l")
G18_LeadFAProtonAng_Leg_P.AddEntry("G18_LeadFAProtonAng_DIS_P", "DIS", "l")
G18_LeadFAProtonAng_Leg_P.AddEntry("G18_LeadFAProtonAng_None_P", "Uncatagorized", "l")
G18_LeadFAProtonAng_Leg_P.Draw()

l.SetLogy()
l.Draw()

out_of_range: TH1D& ROOT::RDF::RResultPtr<TH1D>::operator*() =>
    out_of_range: vector::_M_range_check

RDataFrame::Run: event loop was interrupted


In [ ]:
#g18 plotting leading angle for final state accepted protons without pions and etrans > .3
l = ROOT.TCanvas()
df_gst_G18_02a_highE = df_gst_G18_02a_all.Filter("energy_transfer > 0.3").Filter("num_pions==0")
G18_LeadFAProtonAng_QEL_NP = df_gst_G18_02a_highE.Filter("qel == 1").Histo1D(("G18_LeadFAProtonAng_QEL_NP", "", bins, x_start, x_end), "leading_protonfa_angle")
G18_LeadFAProtonAng_QEL_NP.SetTitle("Leading Proton Angle in Acceptance Range with Etrans > 0.3 GeV w/o Pions in G18")
G18_LeadFAProtonAng_QEL_NP.GetXaxis().SetTitle("Angle (Radians)")
G18_LeadFAProtonAng_QEL_NP.GetYaxis().SetTitle("Counts")
G18_LeadFAProtonAng_QEL_NP.GetYaxis().SetRange(0, 100000) #
G18_LeadFAProtonAng_QEL_NP.SetLineColor(ROOT.kViolet)
G18_LeadFAProtonAng_QEL_NP.Draw("same")

G18_LeadFAProtonAng_MEC_NP = df_gst_G18_02a_highE.Filter("mec == 1").Histo1D(("G18_LeadFAProtonAng_MEC_NP", "", bins, x_start, x_end), "leading_protonfa_angle")
G18_LeadFAProtonAng_MEC_NP.SetLineColor(ROOT.kRed)
G18_LeadFAProtonAng_MEC_NP.Draw("same")

G18_LeadFAProtonAng_RES_NP = df_gst_G18_02a_highE.Filter("res == 1").Histo1D(("G18_LeadFAProtonAng_RES_NP", "", bins, x_start, x_end), "leading_protonfa_angle")
G18_LeadFAProtonAng_RES_NP.SetLineColor(ROOT.kGreen)
G18_LeadFAProtonAng_RES_NP.Draw("same")

G18_LeadFAProtonAng_DIS_NP = df_gst_G18_02a_highE.Filter("dis == 1").Histo1D(("G18_LeadFAProtonAng_DIS_NP", "", bins, x_start, x_end), "leading_protonfa_angle")
G18_LeadFAProtonAng_DIS_NP.SetLineColor(ROOT.kBlue)
G18_LeadFAProtonAng_DIS_NP.Draw("same")

G18_LeadFAProtonAng_None_NP = df_gst_G18_02a_highE.Filter("qel==0 && mec==0 && res==0 && dis==0").Histo1D(("G18_LeadFAProtonAng_None_NP", "", bins, x_start, x_end), "leading_protonfa_angle")
G18_LeadFAProtonAng_None_NP.SetLineColor(ROOT.kGray)
G18_LeadFAProtonAng_None_NP.Draw("same")

G18_LeadFAProtonAng_Leg_NP = ROOT.TLegend(.6, .66, .87, .87)
G18_LeadFAProtonAng_Leg_NP.AddEntry("G18_LeadFAProtonAng_QEL_NP", "QEL", "l")
G18_LeadFAProtonAng_Leg_NP.AddEntry("G18_LeadFAProtonAng_MEC_NP", "MEC", "l")
G18_LeadFAProtonAng_Leg_NP.AddEntry("G18_LeadFAProtonAng_RES_NP", "RES", "l")
G18_LeadFAProtonAng_Leg_NP.AddEntry("G18_LeadFAProtonAng_DIS_NP", "DIS", "l")
G18_LeadFAProtonAng_Leg_NP.AddEntry("G18_LeadFAProtonAng_None_NP", "Uncatagorized", "l")
G18_LeadFAProtonAng_Leg_NP.Draw()

l.SetLogy()
l.Draw()

In [ ]:
#neutron version

In [ ]:
#g18 leading Neutron ke for final state Neutrons with final state pions and etrans > .3
bins = 100
x_start = 0.0
x_end = 3.5
l = ROOT.TCanvas("l")
df_gst_G18_02a_etcut = df_gst_G18_02a_all.Filter("energy_transfer < 0.3")
G18_LeadFNeutron_QEL_P = df_gst_G18_02a_etcut.Filter("qel == 1").Histo1D(("G18_LeadFNeutron_QEL_P", "Leading Kinetic Energy of Final State Neutrons in G18", bins, x_start, x_end), "leading_neutron_f")
G18_LeadFNeutron_QEL_P.SetLineColor(ROOT.kViolet)
G18_LeadFNeutron_QEL_P.GetXaxis().SetTitle("Kinetic Energy (GeV)")
G18_LeadFNeutron_QEL_P.GetYaxis().SetTitle("Counts")
G18_LeadFNeutron_QEL_P.Draw("same")

G18_LeadFNeutron_MEC_P = df_gst_G18_02a_etcut.Filter("mec == 1").Histo1D(("G18_LeadFNeutron_MEC_P", "", bins, x_start, x_end), "leading_neutron_f")
G18_LeadFNeutron_MEC_P.SetLineColor(ROOT.kRed)
G18_LeadFNeutron_MEC_P.Draw("same")

G18_LeadFNeutron_RES_P = df_gst_G18_02a_etcut.Filter("res == 1").Histo1D(("G18_LeadFNeutron_RES_P", "", bins, x_start, x_end), "leading_neutron_f")
G18_LeadFNeutron_RES_P.SetLineColor(ROOT.kGreen)
G18_LeadFNeutron_RES_P.Draw("same")

G18_LeadFNeutron_DIS_P = df_gst_G18_02a_etcut.Filter("dis == 1").Histo1D(("G18_LeadFNeutron_DIS_P", "", bins, x_start, x_end), "leading_neutron_f")
G18_LeadFNeutron_DIS_P.SetLineColor(ROOT.kBlue)
G18_LeadFNeutron_DIS_P.Draw("same")

G18_LeadFNeutron_None_P = df_gst_G18_02a_etcut.Filter("qel == 0 && mec == 0 && res == 0 && dis == 0").Histo1D(("G18_LeadFNeutron_None_P", "", bins, x_start, x_end), "leading_neutron_f")
G18_LeadFNeutron_None_P.SetLineColor(ROOT.kGray)
G18_LeadFNeutron_None_P.Draw("same")

G18_LeadFNeutron_Leg = ROOT.TLegend(.6, .66, .87, .87)
G18_LeadFNeutron_Leg.AddEntry("G18_LeadFNeutron_QEL_P", "QEL", "l")
G18_LeadFNeutron_Leg.AddEntry("G18_LeadFNeutron_MEC_P", "MEC", "l")
G18_LeadFNeutron_Leg.AddEntry("G18_LeadFNeutron_RES_P", "RES", "l")
G18_LeadFNeutron_Leg.AddEntry("G18_LeadFNeutron_DIS_P", "DIS", "l")
G18_LeadFNeutron_Leg.AddEntry("G18_LeadFNeutron_None_P", "Uncatagorized", "l")
G18_LeadFNeutron_Leg.Draw()

l.SetLogy()
l.Draw()

In [ ]:
#g18 leading Neutron ke for final state Neutrons without final state pions and etrans > .3
bins = 100
x_start = 0.0
x_end = 3.5
l = ROOT.TCanvas("l")
df_gst_G18_02a_etcut = df_gst_G18_02a_all.Filter("energy_transfer < 0.3").Filter("num_pions==0")
G18_LeadFNeutron_QEL_NP = df_gst_G18_02a_etcut.Filter("qel == 1").Histo1D(("G18_LeadFNeutron_QEL_NP", "Leading Kinetic Energy of Final State Neutrons w/o Final State Pions in G18", bins, x_start, x_end), "leading_neutron_f")
G18_LeadFNeutron_QEL_NP.SetLineColor(ROOT.kViolet)
G18_LeadFNeutron_QEL_NP.GetXaxis().SetTitle("Kinetic Energy (GeV)")
G18_LeadFNeutron_QEL_NP.GetYaxis().SetTitle("Counts")
G18_LeadFNeutron_QEL_NP.Draw("same")

G18_LeadFNeutron_MEC_NP = df_gst_G18_02a_etcut.Filter("mec == 1").Histo1D(("G18_LeadFNeutron_MEC_NP", "", bins, x_start, x_end), "leading_neutron_f")
G18_LeadFNeutron_MEC_NP.SetLineColor(ROOT.kRed)
G18_LeadFNeutron_MEC_NP.Draw("same")

G18_LeadFNeutron_RES_NP = df_gst_G18_02a_etcut.Filter("res == 1").Histo1D(("G18_LeadFNeutron_RES_NP", "", bins, x_start, x_end), "leading_neutron_f")
G18_LeadFNeutron_RES_NP.SetLineColor(ROOT.kGreen)
G18_LeadFNeutron_RES_NP.Draw("same")

G18_LeadFNeutron_DIS_NP = df_gst_G18_02a_etcut.Filter("dis == 1").Histo1D(("G18_LeadFNeutron_DIS_NP", "", bins, x_start, x_end), "leading_neutron_f")
G18_LeadFNeutron_DIS_NP.SetLineColor(ROOT.kBlue)
G18_LeadFNeutron_DIS_NP.Draw("same")

G18_LeadFNeutron_None_NP = df_gst_G18_02a_etcut.Filter("qel == 0 && mec == 0 && res == 0 && dis == 0").Histo1D(("G18_LeadFNeutron_None_NP", "", bins, x_start, x_end), "leading_neutron_f")
G18_LeadFNeutron_None_NP.SetLineColor(ROOT.kGray)
G18_LeadFNeutron_None_NP.Draw("same")

G18_LeadFNeutron_Leg = ROOT.TLegend(.6, .66, .87, .87)
G18_LeadFNeutron_Leg.AddEntry("G18_LeadFNeutron_QEL_NP", "QEL", "l")
G18_LeadFNeutron_Leg.AddEntry("G18_LeadFNeutron_MEC_NP", "MEC", "l")
G18_LeadFNeutron_Leg.AddEntry("G18_LeadFNeutron_RES_NP", "RES", "l")
G18_LeadFNeutron_Leg.AddEntry("G18_LeadFNeutron_DIS_NP", "DIS", "l")
G18_LeadFNeutron_Leg.AddEntry("G18_LeadFNeutron_None_NP", "Uncatagorized", "l")
G18_LeadFNeutron_Leg.Draw()

l.SetLogy()
l.Draw()

In [ ]:
#g18 leading Neutron ke for final state accepted Neutrons with final state pions and etrans > .3
bins = 100
x_start = 0.0
x_end = 3.5
l = ROOT.TCanvas("l")
df_gst_G18_02a_etcut = df_gst_G18_02a_all.Filter("energy_transfer < 0.3")
G18_LeadFANeutron_QEL_P = df_gst_G18_02a_etcut.Filter("qel == 1").Histo1D(("G18_LeadFANeutron_QEL_P", "Leading Kinetic Energy of Final State Accepted Neutrons in G18", bins, x_start, x_end), "leading_neutron_fa")
G18_LeadFANeutron_QEL_P.SetLineColor(ROOT.kViolet)
G18_LeadFANeutron_QEL_P.GetXaxis().SetTitle("Kinetic Energy (GeV)")
G18_LeadFANeutron_QEL_P.GetYaxis().SetTitle("Counts")
G18_LeadFANeutron_QEL_P.Draw("same")

G18_LeadFANeutron_MEC_P = df_gst_G18_02a_etcut.Filter("mec == 1").Histo1D(("G18_LeadFANeutron_MEC_P", "", bins, x_start, x_end), "leading_neutron_fa")
G18_LeadFANeutron_MEC_P.SetLineColor(ROOT.kRed)
G18_LeadFANeutron_MEC_P.Draw("same")

G18_LeadFANeutron_RES_P = df_gst_G18_02a_etcut.Filter("res == 1").Histo1D(("G18_LeadFANeutron_RES_P", "", bins, x_start, x_end), "leading_neutron_fa")
G18_LeadFANeutron_RES_P.SetLineColor(ROOT.kGreen)
G18_LeadFANeutron_RES_P.Draw("same")

G18_LeadFANeutron_DIS_P = df_gst_G18_02a_etcut.Filter("dis == 1").Histo1D(("G18_LeadFANeutron_DIS_P", "", bins, x_start, x_end), "leading_neutron_fa")
G18_LeadFANeutron_DIS_P.SetLineColor(ROOT.kBlue)
G18_LeadFANeutron_DIS_P.Draw("same")

G18_LeadFANeutron_None_P = df_gst_G18_02a_etcut.Filter("qel == 0 && mec == 0 && res == 0 && dis == 0").Histo1D(("G18_LeadFANeutron_None_P", "", bins, x_start, x_end), "leading_neutron_fa")
G18_LeadFANeutron_None_P.SetLineColor(ROOT.kGray)
G18_LeadFANeutron_None_P.Draw("same")

G18_LeadFANeutron_Leg = ROOT.TLegend(.6, .66, .87, .87)
G18_LeadFANeutron_Leg.AddEntry("G18_LeadFANeutron_QEL_P", "QEL", "l")
G18_LeadFANeutron_Leg.AddEntry("G18_LeadFANeutron_MEC_P", "MEC", "l")
G18_LeadFANeutron_Leg.AddEntry("G18_LeadFANeutron_RES_P", "RES", "l")
G18_LeadFANeutron_Leg.AddEntry("G18_LeadFANeutron_DIS_P", "DIS", "l")
G18_LeadFANeutron_Leg.AddEntry("G18_LeadFANeutron_None_P", "Uncatagorized", "l")
G18_LeadFANeutron_Leg.Draw()

l.SetLogy()
l.Draw()

In [ ]:
#g18 leading Neutron ke for final state accepted Neutrons without final state pions and etrans > .3
bins = 100
x_start = 0.0
x_end = 3.5
l = ROOT.TCanvas("l")
df_gst_G18_02a_etcut = df_gst_G18_02a_all.Filter("energy_transfer < 0.3").Filter("num_pions==0")
G18_LeadFANeutron_QEL_NP = df_gst_G18_02a_etcut.Filter("qel == 1").Histo1D(("G18_LeadFANeutron_QEL_NP", "Leading Kinetic Energy of Final State Accepted Neutrons w/o Final State Pions in G18", bins, x_start, x_end), "leading_neutron_fa")
G18_LeadFANeutron_QEL_NP.SetLineColor(ROOT.kViolet)
G18_LeadFANeutron_QEL_NP.GetXaxis().SetTitle("Kinetic Energy (GeV)")
G18_LeadFANeutron_QEL_NP.GetYaxis().SetTitle("Counts")
G18_LeadFANeutron_QEL_NP.Draw("same")

G18_LeadFANeutron_MEC_NP = df_gst_G18_02a_etcut.Filter("mec == 1").Histo1D(("G18_LeadFANeutron_MEC_NP", "", bins, x_start, x_end), "leading_neutron_fa")
G18_LeadFANeutron_MEC_NP.SetLineColor(ROOT.kRed)
G18_LeadFANeutron_MEC_NP.Draw("same")

G18_LeadFANeutron_RES_NP = df_gst_G18_02a_etcut.Filter("res == 1").Histo1D(("G18_LeadFANeutron_RES_NP", "", bins, x_start, x_end), "leading_neutron_fa")
G18_LeadFANeutron_RES_NP.SetLineColor(ROOT.kGreen)
G18_LeadFANeutron_RES_NP.Draw("same")

G18_LeadFANeutron_DIS_NP = df_gst_G18_02a_etcut.Filter("dis == 1").Histo1D(("G18_LeadFANeutron_DIS_NP", "", bins, x_start, x_end), "leading_neutron_fa")
G18_LeadFANeutron_DIS_NP.SetLineColor(ROOT.kBlue)
G18_LeadFANeutron_DIS_NP.Draw("same")

G18_LeadFANeutron_None_NP = df_gst_G18_02a_etcut.Filter("qel == 0 && mec == 0 && res == 0 && dis == 0").Histo1D(("G18_LeadFANeutron_None_NP", "", bins, x_start, x_end), "leading_neutron_fa")
G18_LeadFANeutron_None_NP.SetLineColor(ROOT.kGray)
G18_LeadFANeutron_None_NP.Draw("same")

G18_LeadFANeutron_Leg = ROOT.TLegend(.6, .66, .87, .87)
G18_LeadFANeutron_Leg.AddEntry("G18_LeadFANeutron_QEL_NP", "QEL", "l")
G18_LeadFANeutron_Leg.AddEntry("G18_LeadFANeutron_MEC_NP", "MEC", "l")
G18_LeadFANeutron_Leg.AddEntry("G18_LeadFANeutron_RES_NP", "RES", "l")
G18_LeadFANeutron_Leg.AddEntry("G18_LeadFANeutron_DIS_NP", "DIS", "l")
G18_LeadFANeutron_Leg.AddEntry("G18_LeadFANeutron_None_NP", "Uncatagorized", "l")
G18_LeadFANeutron_Leg.Draw()

l.SetLogy()
l.Draw()

In [ ]:
#deltaptf and deltaptfa plots for events w etrans < .3 for all and w/o pions
bins = 100
x_start = 0
x_end = 4

In [ ]:
#deltaptf for etrans < .3 w pions
a = ROOT.TCanvas()
df_gst_G18_02a_etcut = df_gst_G18_02a_all.Filter("energy_transfer < 0.3")
G18_Delta_Ptf_QEL_Plot = df_gst_G18_02a_etcut.Filter("qel == 1").Histo1D(("G18_Delta_Ptf_QEL_Plot", "Transverse Momentum Imbalance Magnitude in G18", bins, x_start, x_end), "delta_ptf")
G18_Delta_Ptf_QEL_Plot.GetXaxis().SetTitle("")
G18_Delta_Ptf_QEL_Plot.GetYaxis().SetTitle("Counts")
G18_Delta_Ptf_QEL_Plot.SetLineColor(ROOT.kViolet)
G18_Delta_Ptf_QEL_Plot.Draw("same")

G18_Delta_Ptf_MEC_Plot = df_gst_G18_02a_etcut.Filter("mec == 1").Histo1D(("G18_Delta_Ptf_MEC_Plot", "", bins, x_start, x_end), "delta_ptf")
G18_Delta_Ptf_MEC_Plot.SetLineColor(ROOT.kRed)
G18_Delta_Ptf_MEC_Plot.Draw("same")

G18_Delta_Ptf_RES_Plot = df_gst_G18_02a_etcut.Filter("res == 1").Histo1D(("G18_Delta_Ptf_RES_Plot", "", bins, x_start, x_end), "delta_ptf")
G18_Delta_Ptf_RES_Plot.SetLineColor(ROOT.kGreen)
G18_Delta_Ptf_RES_Plot.Draw("same")

G18_Delta_Ptf_DIS_Plot = df_gst_G18_02a_etcut.Filter("dis == 1").Histo1D(("G18_Delta_Ptf_DIS_Plot", "", bins, x_start, x_end), "delta_ptf")
G18_Delta_Ptf_DIS_Plot.SetLineColor(ROOT.kBlue)
G18_Delta_Ptf_DIS_Plot.Draw("same")

G18_Delta_Ptf_None_Plot = df_gst_G18_02a_etcut.Filter("qel==0&&mec==0&&res==0&&dis==0").Histo1D(("G18_Delta_Ptf_None_Plot", "", bins, x_start, x_end), "delta_ptf")
G18_Delta_Ptf_None_Plot.SetLineColor(ROOT.kGray)
G18_Delta_Ptf_None_Plot.Draw("same")

G18_Delta_Ptf_Leg = ROOT.TLegend(.6, .66, .87, .87)
G18_Delta_Ptf_Leg.AddEntry("G18_Delta_Ptf_QEL_Plot", "QEL", "l")
G18_Delta_Ptf_Leg.AddEntry("G18_Delta_Ptf_MEC_Plot", "MEC", "l")
G18_Delta_Ptf_Leg.AddEntry("G18_Delta_Ptf_RES_Plot", "RES", "l")
G18_Delta_Ptf_Leg.AddEntry("G18_Delta_Ptf_DIS_Plot", "DIS", "l")
G18_Delta_Ptf_Leg.AddEntry("G18_Delta_Ptf_None_Plot", "Uncatagorized", "l")
G18_Delta_Ptf_Leg.Draw()
a.SetLogy()
a.Draw()

In [ ]:
#deltaptf for etrans < .3 without pions
a = ROOT.TCanvas()
df_gst_G18_02a_etcut = df_gst_G18_02a_all.Filter("energy_transfer < 0.3 && num_pions==0")
G18_Delta_Ptf_NP_QEL_Plot = df_gst_G18_02a_etcut.Filter("qel == 1").Histo1D(("G18_Delta_Ptf_NP_QEL_Plot", "Transverse Momentum Imbalance Magnitude w/o Final State Pions in G18", bins, x_start, x_end), "delta_ptf")
G18_Delta_Ptf_NP_QEL_Plot.GetXaxis().SetTitle("")
G18_Delta_Ptf_NP_QEL_Plot.GetYaxis().SetTitle("Counts")
G18_Delta_Ptf_NP_QEL_Plot.SetLineColor(ROOT.kViolet)
G18_Delta_Ptf_NP_QEL_Plot.Draw("same")

G18_Delta_Ptf_NP_MEC_Plot = df_gst_G18_02a_etcut.Filter("mec == 1").Histo1D(("G18_Delta_Ptf_NP_MEC_Plot", "", bins, x_start, x_end), "delta_ptf")
G18_Delta_Ptf_NP_MEC_Plot.SetLineColor(ROOT.kRed)
G18_Delta_Ptf_NP_MEC_Plot.Draw("same")

G18_Delta_Ptf_NP_RES_Plot = df_gst_G18_02a_etcut.Filter("res == 1").Histo1D(("G18_Delta_Ptf_NP_RES_Plot", "", bins, x_start, x_end), "delta_ptf")
G18_Delta_Ptf_NP_RES_Plot.SetLineColor(ROOT.kGreen)
G18_Delta_Ptf_NP_RES_Plot.Draw("same")

G18_Delta_Ptf_NP_DIS_Plot = df_gst_G18_02a_etcut.Filter("dis == 1").Histo1D(("G18_Delta_Ptf_NP_DIS_Plot", "", bins, x_start, x_end), "delta_ptf")
G18_Delta_Ptf_NP_DIS_Plot.SetLineColor(ROOT.kBlue)
G18_Delta_Ptf_NP_DIS_Plot.Draw("same")

G18_Delta_Ptf_NP_None_Plot = df_gst_G18_02a_etcut.Filter("qel==0&&mec==0&&res==0&&dis==0").Histo1D(("G18_Delta_Ptf_NP_None_Plot", "", bins, x_start, x_end), "delta_ptf")
G18_Delta_Ptf_NP_None_Plot.SetLineColor(ROOT.kGray)
G18_Delta_Ptf_NP_None_Plot.Draw("same")

G18_Delta_Ptf_NP_Leg = ROOT.TLegend(.6, .66, .87, .87)
G18_Delta_Ptf_NP_Leg.AddEntry("G18_Delta_Ptf_NP_QEL_Plot", "QEL", "l")
G18_Delta_Ptf_NP_Leg.AddEntry("G18_Delta_Ptf_NP_MEC_Plot", "MEC", "l")
G18_Delta_Ptf_NP_Leg.AddEntry("G18_Delta_Ptf_NP_RES_Plot", "RES", "l")
G18_Delta_Ptf_NP_Leg.AddEntry("G18_Delta_Ptf_NP_DIS_Plot", "DIS", "l")
G18_Delta_Ptf_NP_Leg.AddEntry("G18_Delta_Ptf_NP_None_Plot", "Uncatagorized", "l")
G18_Delta_Ptf_NP_Leg.Draw()
a.SetLogy()
a.Draw()

In [ ]:
#deltaptfa for etrans < .3 w pions
a = ROOT.TCanvas()
df_gst_G18_02a_etcut = df_gst_G18_02a_all.Filter("energy_transfer < 0.3")
G18_Delta_Ptfa_QEL_Plot = df_gst_G18_02a_etcut.Filter("qel == 1").Histo1D(("G18_Delta_Ptfa_QEL_Plot", "Transverse Momentum Imbalance Magnitude in Final Acceptance in G18", bins, x_start, x_end), "delta_ptfa_all")
G18_Delta_Ptfa_QEL_Plot.GetXaxis().SetTitle("")
G18_Delta_Ptfa_QEL_Plot.GetYaxis().SetTitle("Counts")
G18_Delta_Ptfa_QEL_Plot.SetLineColor(ROOT.kViolet)
G18_Delta_Ptfa_QEL_Plot.Draw("same")

G18_Delta_Ptfa_MEC_Plot = df_gst_G18_02a_etcut.Filter("mec == 1").Histo1D(("G18_Delta_Ptfa_MEC_Plot", "", bins, x_start, x_end), "delta_ptfa_all")
G18_Delta_Ptfa_MEC_Plot.SetLineColor(ROOT.kRed)
G18_Delta_Ptfa_MEC_Plot.Draw("same")

G18_Delta_Ptfa_RES_Plot = df_gst_G18_02a_etcut.Filter("res == 1").Histo1D(("G18_Delta_Ptfa_RES_Plot", "", bins, x_start, x_end), "delta_ptfa_all")
G18_Delta_Ptfa_RES_Plot.SetLineColor(ROOT.kGreen)
G18_Delta_Ptfa_RES_Plot.Draw("same")

G18_Delta_Ptfa_DIS_Plot = df_gst_G18_02a_etcut.Filter("dis == 1").Histo1D(("G18_Delta_Ptfa_DIS_Plot", "", bins, x_start, x_end), "delta_ptfa_all")
G18_Delta_Ptfa_DIS_Plot.SetLineColor(ROOT.kBlue)
G18_Delta_Ptfa_DIS_Plot.Draw("same")

G18_Delta_Ptfa_None_Plot = df_gst_G18_02a_etcut.Filter("qel==0&&mec==0&&res==0&&dis==0").Histo1D(("G18_Delta_Ptfa_None_Plot", "", bins, x_start, x_end), "delta_ptfa_all")
G18_Delta_Ptfa_None_Plot.SetLineColor(ROOT.kGray)
G18_Delta_Ptfa_None_Plot.Draw("same")

G18_Delta_Ptfa_Leg = ROOT.TLegend(.6, .66, .87, .87)
G18_Delta_Ptfa_Leg.AddEntry("G18_Delta_Ptfa_QEL_Plot", "QEL", "l")
G18_Delta_Ptfa_Leg.AddEntry("G18_Delta_Ptfa_MEC_Plot", "MEC", "l")
G18_Delta_Ptfa_Leg.AddEntry("G18_Delta_Ptfa_RES_Plot", "RES", "l")
G18_Delta_Ptfa_Leg.AddEntry("G18_Delta_Ptfa_DIS_Plot", "DIS", "l")
G18_Delta_Ptfa_Leg.AddEntry("G18_Delta_Ptfa_None_Plot", "Uncatagorized", "l")
G18_Delta_Ptfa_Leg.Draw()
a.SetLogy()
a.Draw()

In [ ]:
#deltaptfa for etrans < .3 without pions
a = ROOT.TCanvas()
df_gst_G18_02a_etcut = df_gst_G18_02a_all.Filter("energy_transfer < 0.3 && num_pions==0")
G18_Delta_Ptfa_NP_QEL_Plot = df_gst_G18_02a_etcut.Filter("qel == 1").Histo1D(("G18_Delta_Ptfa_NP_QEL_Plot", "Transverse Momentum Imbalance Magnitude in Final Acceptance w/o Final State Pions in G18", bins, x_start, x_end), "delta_ptfa_all")
G18_Delta_Ptfa_NP_QEL_Plot.GetXaxis().SetTitle("")
G18_Delta_Ptfa_NP_QEL_Plot.GetYaxis().SetTitle("Counts")
G18_Delta_Ptfa_NP_QEL_Plot.SetLineColor(ROOT.kViolet)
G18_Delta_Ptfa_NP_QEL_Plot.Draw("same")

G18_Delta_Ptfa_NP_MEC_Plot = df_gst_G18_02a_etcut.Filter("mec == 1").Histo1D(("G18_Delta_Ptfa_NP_MEC_Plot", "", bins, x_start, x_end), "delta_ptfa_all")
G18_Delta_Ptfa_NP_MEC_Plot.SetLineColor(ROOT.kRed)
G18_Delta_Ptfa_NP_MEC_Plot.Draw("same")

G18_Delta_Ptfa_NP_RES_Plot = df_gst_G18_02a_etcut.Filter("res == 1").Histo1D(("G18_Delta_Ptfa_NP_RES_Plot", "", bins, x_start, x_end), "delta_ptfa_all")
G18_Delta_Ptfa_NP_RES_Plot.SetLineColor(ROOT.kGreen)
G18_Delta_Ptfa_NP_RES_Plot.Draw("same")

G18_Delta_Ptfa_NP_DIS_Plot = df_gst_G18_02a_etcut.Filter("dis == 1").Histo1D(("G18_Delta_Ptfa_NP_DIS_Plot", "", bins, x_start, x_end), "delta_ptfa_all")
G18_Delta_Ptfa_NP_DIS_Plot.SetLineColor(ROOT.kBlue)
G18_Delta_Ptfa_NP_DIS_Plot.Draw("same")

G18_Delta_Ptfa_NP_None_Plot = df_gst_G18_02a_etcut.Filter("qel==0&&mec==0&&res==0&&dis==0").Histo1D(("G18_Delta_Ptfa_NP_None_Plot", "", bins, x_start, x_end), "delta_ptfa_all")
G18_Delta_Ptfa_NP_None_Plot.SetLineColor(ROOT.kGray)
G18_Delta_Ptfa_NP_None_Plot.Draw("same")

G18_Delta_Ptfa_NP_Leg = ROOT.TLegend(.6, .66, .87, .87)
G18_Delta_Ptfa_NP_Leg.AddEntry("G18_Delta_Ptfa_NP_QEL_Plot", "QEL", "l")
G18_Delta_Ptfa_NP_Leg.AddEntry("G18_Delta_Ptfa_NP_MEC_Plot", "MEC", "l")
G18_Delta_Ptfa_NP_Leg.AddEntry("G18_Delta_Ptfa_NP_RES_Plot", "RES", "l")
G18_Delta_Ptfa_NP_Leg.AddEntry("G18_Delta_Ptfa_NP_DIS_Plot", "DIS", "l")
G18_Delta_Ptfa_NP_Leg.AddEntry("G18_Delta_Ptfa_NP_None_Plot", "Uncatagorized", "l")
G18_Delta_Ptfa_NP_Leg.Draw()
a.SetLogy()
a.Draw()

In [ ]:
#sum of hadronic elements for momentum imbalance not working?
a = ROOT.TCanvas()
df_gst_G18_02a_1ProNeu = df_gst_G18_02a_all.Filter("pdgf[2212] < 1 || pdgf[2112] > 1 ")
G18_HSum_Pfa_QEL_Plot = df_gst_G18_02a_1ProNeu.Filter("qel == 1").Histo1D(("G18_HSum_Pfa_QEL_Plot", "Transverse Momentum Imbalance Angle in Final State G18", bins, x_start, x_end), "hsum_pf")
G18_HSum_Pfa_QEL_Plot.GetXaxis().SetTitle("Delta Angle (Radians)")
G18_HSum_Pfa_QEL_Plot.GetYaxis().SetTitle("Counts")
G18_HSum_Pfa_QEL_Plot.SetLineColor(ROOT.kViolet)
G18_HSum_Pfa_QEL_Plot.Draw("same")

G18_HSum_Pfa_MEC_Plot = df_gst_G18_02a_1ProNeu.Filter("mec == 1").Histo1D(("G18_HSum_Pfa_MEC_Plot", "", bins, x_start, x_end), "delta_alphatf")
G18_HSum_Pfa_MEC_Plot.SetLineColor(ROOT.kRed)
G18_HSum_Pfa_MEC_Plot.Draw("same")

G18_HSum_Pfa_RES_Plot = df_gst_G18_02a_1ProNeu.Filter("res == 1").Histo1D(("G18_HSum_Pfa_RES_Plot", "", bins, x_start, x_end), "delta_alphatf")
G18_HSum_Pfa_RES_Plot.SetLineColor(ROOT.kGreen)
G18_HSum_Pfa_RES_Plot.Draw("same")

G18_HSum_Pfa_DIS_Plot = df_gst_G18_02a_1ProNeu.Filter("dis == 1").Histo1D(("G18_HSum_Pfa_DIS_Plot", "", bins, x_start, x_end), "delta_alphatf")
G18_HSum_Pfa_DIS_Plot.SetLineColor(ROOT.kBlue)
G18_HSum_Pfa_DIS_Plot.Draw("same")

G18_HSum_Pfa_None_Plot = df_gst_G18_02a_1ProNeu.Filter("qel==0&&mec==0&&res==0&&dis==0").Histo1D(("G18_HSum_Pfa_None_Plot", "", bins, x_start, x_end), "delta_alphatf")
G18_HSum_Pfa_None_Plot.SetLineColor(ROOT.kGray)
G18_HSum_Pfa_None_Plot.Draw("same")

a.Draw()

In [ ]:
#delta_alpha final state
a = ROOT.TCanvas()
df_gst_G18_02a_1ProNeu = df_gst_G18_02a_all.Filter("pdgf[2212] < 1 || pdgf[2112] > 1 ")
G18_HSum_Pfa_QEL_Plot = df_gst_G18_02a_1ProNeu.Filter("qel == 1").Histo1D(("G18_HSum_Pfa_QEL_Plot", "Delta Alpha in Final State G18", bins, x_start, x_end), "delta_alphafa_all")
G18_HSum_Pfa_QEL_Plot.GetXaxis().SetTitle("Delta Angle (Radians)")
G18_HSum_Pfa_QEL_Plot.GetYaxis().SetRange(0, 3)
G18_HSum_Pfa_QEL_Plot.GetYaxis().SetTitle("Counts")
G18_HSum_Pfa_QEL_Plot.SetLineColor(ROOT.kViolet)
G18_HSum_Pfa_QEL_Plot.Draw("same")

G18_HSum_Pfa_MEC_Plot = df_gst_G18_02a_1ProNeu.Filter("mec == 1").Histo1D(("G18_HSum_Pfa_MEC_Plot", "", bins, x_start, x_end), "delta_alphafa_all")
G18_HSum_Pfa_MEC_Plot.SetLineColor(ROOT.kRed)
G18_HSum_Pfa_MEC_Plot.Draw("same")

G18_HSum_Pfa_RES_Plot = df_gst_G18_02a_1ProNeu.Filter("res == 1").Histo1D(("G18_HSum_Pfa_RES_Plot", "", bins, x_start, x_end), "delta_alphafa_all")
G18_HSum_Pfa_RES_Plot.SetLineColor(ROOT.kGreen)
G18_HSum_Pfa_RES_Plot.Draw("same")

G18_HSum_Pfa_DIS_Plot = df_gst_G18_02a_1ProNeu.Filter("dis == 1").Histo1D(("G18_HSum_Pfa_DIS_Plot", "", bins, x_start, x_end), "delta_alphafa_all")
G18_HSum_Pfa_DIS_Plot.SetLineColor(ROOT.kBlue)
G18_HSum_Pfa_DIS_Plot.Draw("same")

G18_HSum_Pfa_None_Plot = df_gst_G18_02a_1ProNeu.Filter("qel==0&&mec==0&&res==0&&dis==0").Histo1D(("G18_HSum_Pfa_None_Plot", "", bins, x_start, x_end), "delta_alphafa_all")
G18_HSum_Pfa_None_Plot.SetLineColor(ROOT.kGray)
G18_HSum_Pfa_None_Plot.Draw("same")

a.Draw()

In [ ]:
#cosalpha
x_end = 1.2
a = ROOT.TCanvas()
df_gst_G18_02a_1ProNeu = df_gst_G18_02a_all.Filter("pdgf[2212] < 1 || pdgf[2112] > 1 ")
G18_HSum_Pfa_QEL_Plot = df_gst_G18_02a_1ProNeu.Filter("qel == 1").Histo1D(("G18_HSum_Pfa_QEL_Plot", "Delta Alpha Cos in Final State G18", bins, x_start, x_end), "delta_cosalphaf")
G18_HSum_Pfa_QEL_Plot.GetXaxis().SetTitle("Delta Angle (Radians)")
G18_HSum_Pfa_QEL_Plot.GetYaxis().SetTitle("Counts")
G18_HSum_Pfa_QEL_Plot.SetLineColor(ROOT.kViolet)
G18_HSum_Pfa_QEL_Plot.Draw("same")

G18_HSum_Pfa_MEC_Plot = df_gst_G18_02a_1ProNeu.Filter("mec == 1").Histo1D(("G18_HSum_Pfa_MEC_Plot", "", bins, x_start, x_end), "delta_cosalphaf")
G18_HSum_Pfa_MEC_Plot.SetLineColor(ROOT.kRed)
G18_HSum_Pfa_MEC_Plot.Draw("same")

G18_HSum_Pfa_RES_Plot = df_gst_G18_02a_1ProNeu.Filter("res == 1").Histo1D(("G18_HSum_Pfa_RES_Plot", "", bins, x_start, x_end), "delta_cosalphaf")
G18_HSum_Pfa_RES_Plot.SetLineColor(ROOT.kGreen)
G18_HSum_Pfa_RES_Plot.Draw("same")

G18_HSum_Pfa_DIS_Plot = df_gst_G18_02a_1ProNeu.Filter("dis == 1").Histo1D(("G18_HSum_Pfa_DIS_Plot", "", bins, x_start, x_end), "delta_cosalphaf")
G18_HSum_Pfa_DIS_Plot.SetLineColor(ROOT.kBlue)
G18_HSum_Pfa_DIS_Plot.Draw("same")

G18_HSum_Pfa_None_Plot = df_gst_G18_02a_1ProNeu.Filter("qel==0&&mec==0&&res==0&&dis==0").Histo1D(("G18_HSum_Pfa_None_Plot", "", bins, x_start, x_end), "delta_cosalphaf")
G18_HSum_Pfa_None_Plot.SetLineColor(ROOT.kGray)
G18_HSum_Pfa_None_Plot.Draw("same")

a.Draw()

In [ ]:
#plots for poster
bins = 100
x_start = 0
x_end = 2.0 #GeV
#Show lepton pt and lepton angular distrib of leptons of interaction types, combine qe and mec from g18 
#then qel and mec from g21 etc  qel ==1 || mec==1 then draw lines where the triggers are pt 400 MeV and angle 40 degrees

In [7]:
#lep pt of interaction types of qel+mec and res+dis from g18 and g21
x_end = 2.0
bruh = ROOT.TCanvas("bruh")
G18_LepPt_QM = df_gst_G18_02a_all.Filter("qel == 1 || mec == 1").Histo1D(("G18_LepPt_QM", "Lepton Transverse Momentum of QEL/MEC and RES/DIS Events of G18 and G21 Models", bins, x_start, x_end), "ptl")
G18_LepPt_QM.SetLineColor(ROOT.kBlack)
G18_LepPt_QM.GetXaxis().SetTitle("Momentum (GeV)")
G18_LepPt_QM.GetYaxis().SetTitle("Counts")
G18_LepPt_QM.Draw("same")

G21_LepPt_QM = df_gst_GEM21_11b_all.Filter("qel == 1 || mec == 1").Histo1D(("G21_LepPt_QM", "", bins, x_start, x_end), "ptl")
G21_LepPt_QM.SetLineColor(ROOT.kRed)
G21_LepPt_QM.Draw("same")

G18_LepPt_RD = df_gst_G18_02a_all.Filter("res == 1 || dis == 1").Histo1D(("G18_LepPt_RD", "", bins, x_start, x_end), "ptl")
G18_LepPt_RD.SetLineColor(ROOT.kBlue)
G18_LepPt_RD.Draw("same")

G21_LepPt_RD = df_gst_GEM21_11b_all.Filter("res == 1 || dis == 1").Histo1D(("G21_LepPt_RD", "", bins, x_start, x_end), "ptl")
G21_LepPt_RD.SetLineColor(ROOT.kGreen)
G21_LepPt_RD.Draw("same")

G18_G21_LepPt_Leg = ROOT.TLegend(0.2, 0.80, 0.80, 0.90)
G18_G21_LepPt_Leg.SetNColumns(2)
G18_G21_LepPt_Leg.AddEntry("G18_LepPt_QM", "QEL and MEC Events in G18", "l")
G18_G21_LepPt_Leg.AddEntry("G21_LepPt_QM", "QEL and MEC Events in GEM21", "l")
G18_G21_LepPt_Leg.AddEntry("G18_LepPt_RD", "RES and DIS Events in G18", "l")
G18_G21_LepPt_Leg.AddEntry("G21_LepPt_RD", "RES and DIS Events in GEM21", "l")
G18_G21_LepPt_Leg.Draw()

bruh.SetLogy()
bruh.Draw()

out_of_range: TH1D& ROOT::RDF::RResultPtr<TH1D>::operator*() =>
    out_of_range: vector::_M_range_check

RDataFrame::Run: event loop was interrupted


In [8]:
#thetazl of interaction types of qel+mec and res+dis from g18 and g21
x_end = 1.5
b = ROOT.TCanvas("b")
df_gst_G18_02a_triggers = df_gst_G18_02a_all.Filter("ptl > .4")
df_gst_GEM21_11b_triggers = df_gst_GEM21_11b_all.Filter("ptl > .4")

G18_Thetazl_QM = df_gst_G18_02a_triggers.Filter("qel == 1 || mec == 1").Histo1D(("G18_Thetazl_QM", "Lepton Angle of QEL+MEC and RES+DIS Events", bins, x_start, x_end), "thetazl")
G18_Thetazl_QM.SetLineColor(ROOT.kBlack)
G18_Thetazl_QM.GetXaxis().SetTitle("Angle (Radians)")
G18_Thetazl_QM.GetYaxis().SetTitle("Counts")
G18_Thetazl_QM.Draw("same")

G21_Thetazl_QM = df_gst_GEM21_11b_triggers.Filter("qel == 1 || mec == 1").Histo1D(("G21_Thetazl_QM", "", bins, x_start, x_end), "thetazl")
G21_Thetazl_QM.SetLineColor(ROOT.kRed)
G21_Thetazl_QM.Draw("same")

G18_Thetazl_RD = df_gst_G18_02a_triggers.Filter("res == 1 || dis == 1").Histo1D(("G18_Thetazl_RD", "", bins, x_start, x_end), "thetazl")
G18_Thetazl_RD.SetLineColor(ROOT.kBlue)
G18_Thetazl_RD.Draw("same")

G21_Thetazl_RD = df_gst_GEM21_11b_triggers.Filter("res == 1 || dis == 1").Histo1D(("G21_Thetazl_RD", "", bins, x_start, x_end), "thetazl")
G21_Thetazl_RD.SetLineColor(ROOT.kGreen)
G21_Thetazl_RD.Draw("same")

G18_G21_Thetazl_Leg = ROOT.TLegend(0.2, 0.80, 0.80, 0.90)
G18_G21_Thetazl_Leg.SetNColumns(2)
G18_G21_Thetazl_Leg.AddEntry("G18_Thetazl_QM", "QEL and MEC Events in G18", "l")
G18_G21_Thetazl_Leg.AddEntry("G21_Thetazl_QM", "QEL and MEC Events in GEM21", "l")
G18_G21_Thetazl_Leg.AddEntry("G18_Thetazl_RD", "RES and DIS Events in G18", "l")
G18_G21_Thetazl_Leg.AddEntry("G21_Thetazl_RD", "RES and DIS Events in GEM21", "l")
G18_G21_Thetazl_Leg.Draw()

b.Draw()

out_of_range: TH1D& ROOT::RDF::RResultPtr<TH1D>::operator*() =>
    out_of_range: vector::_M_range_check

RDataFrame::Run: event loop was interrupted


In [ ]:
#etrans of interaction types of qel+mec and res+dis from g18 and g21
x_end = 4.0
b = ROOT.TCanvas("b")
df_gst_G18_02a_triggers = df_gst_G18_02a_all.Filter("ptl > .4").Filter("thetazl < .69")
df_gst_GEM21_11b_triggers = df_gst_GEM21_11b_all.Filter("ptl > .4").Filter("thetazl < .69")
G18_Etrans_QM = df_gst_G18_02a_triggers.Filter("qel == 1 || mec == 1").Histo1D(("G18_Etrans_QM", "Lepton Energy Transfer of QEL+MEC and RES+DIS Events", bins, x_start, x_end), "energy_transfer")
G18_Etrans_QM.SetLineColor(ROOT.kBlack)
G18_Etrans_QM.GetXaxis().SetTitle("Energy Transfer (GeV)")
G18_Etrans_QM.GetYaxis().SetTitle("Counts")
G18_Etrans_QM.Draw("same")

G21_Etrans_QM = df_gst_GEM21_11b_triggers.Filter("qel == 1 || mec == 1").Histo1D(("G21_Etrans_QM", "", bins, x_start, x_end), "energy_transfer")
G21_Etrans_QM.SetLineColor(ROOT.kRed)
G21_Etrans_QM.Draw("same")

G18_Etrans_RD = df_gst_G18_02a_triggers.Filter("res == 1 || dis == 1").Histo1D(("G18_Etrans_RD", "", bins, x_start, x_end), "energy_transfer")
G18_Etrans_RD.SetLineColor(ROOT.kBlue)
G18_Etrans_RD.Draw("same")

G21_Etrans_RD = df_gst_GEM21_11b_triggers.Filter("res == 1 || dis == 1").Histo1D(("G21_Etrans_RD", "", bins, x_start, x_end), "energy_transfer")
G21_Etrans_RD.SetLineColor(ROOT.kGreen)
G21_Etrans_RD.Draw("same")

G18_G21_Etrans_Leg = ROOT.TLegend(0.2, 0.80, 0.80, 0.90)
G18_G21_Etrans_Leg.SetNColumns(2)
G18_G21_Etrans_Leg.AddEntry("G18_Etrans_QM", "QEL and MEC Events in G18", "l")
G18_G21_Etrans_Leg.AddEntry("G21_Etrans_QM", "QEL and MEC Events in GEM21", "l")
G18_G21_Etrans_Leg.AddEntry("G18_Etrans_RD", "RES and DIS Events in G18", "l")
G18_G21_Etrans_Leg.AddEntry("G21_Etrans_RD", "RES and DIS Events in GEM21", "l")
G18_G21_Etrans_Leg.Draw()

b.SetLogy()
b.Draw()

# Arrays

In [ ]:
import uproot
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from math import *
import scipy as sp
import scipy.stats
import glob
import time
import array
%jsroot on

In [ ]:
#relative fraction arrays of events for lepton pt cuts
bins = 100
x_start = 0
x_end = 4.0 #GeV
values = [0.0, 0.05, 0.1, 0.15, 0.20, 0.25, 0.30, 0.35, 0.40, 0.45, 0.50, 0.55, 0.60, 0.65, 0.70, 0.75, 0.80, 0.85, 0.90, 0.95, 1.00]
n_event_cut = 20

In [ ]:
#g18 with pions v2
G18_QEL_List_P = []
G18_MEC_List_P = []
G18_RES_List_P = []
G18_DIS_List_P = []
G18_LepCut_Vals_List = []
G18_Total_Events_P = 0
for value in values:
    df_gst_G18_02a_P = df_gst_G18_02a_all.Filter(f"ptl>{value} && abs(thetazl)<{lep_accept_angle}")
    h18_lep_pt_QEL_P = df_gst_G18_02a_P.Filter("qel==1").Histo1D(("h18_lep_pt_QEL_P", "", bins, x_start, x_end), "ptl")
    h18_lep_pt_MEC_P = df_gst_G18_02a_P.Filter("mec==1").Histo1D(("h18_lep_pt_MEC_P", "", bins, x_start, x_end), "ptl")
    h18_lep_pt_RES_P = df_gst_G18_02a_P.Filter("res==1").Histo1D(("h18_lep_pt_RES_P", "", bins, x_start, x_end), "ptl")
    h18_lep_pt_DIS_P = df_gst_G18_02a_P.Filter("dis==1").Histo1D(("h18_lep_pt_DIS_P", "", bins, x_start, x_end), "ptl")
    gev18P = [h18_lep_pt_QEL_P.Integral(), h18_lep_pt_MEC_P.Integral(), h18_lep_pt_RES_P.Integral(), h18_lep_pt_DIS_P.Integral()]
    total18P = gev18P[0] + gev18P[1] + gev18P[2] + gev18P[3]
    event_lists_18P = [G18_QEL_List_P, G18_MEC_List_P, G18_RES_List_P, G18_DIS_List_P]
    G18_Total_Events_P += total18P
    for i in range(len(gev18P)):
        if total18P > n_event_cut: 
            event_lists_18P[i].append(gev18P[i]/total18P)
            if value not in G18_LepCut_Vals_List:
                G18_LepCut_Vals_List.append(value)  
G18_QEL_Pions = array.array('f', G18_QEL_List_P)
G18_MEC_Pions = array.array('f', G18_MEC_List_P)
G18_RES_Pions = array.array('f', G18_RES_List_P)
G18_DIS_Pions = array.array('f', G18_DIS_List_P)
G18_LepCut_Vals_Array = array.array('f', G18_LepCut_Vals_List)

G18_Yerr_QEL_Pions_List = []
G18_Yerr_MEC_Pions_List = []
G18_Yerr_RES_Pions_List = []
G18_Yerr_DIS_Pions_List = []
G18_Yerr_Events_Lists = [G18_Yerr_QEL_Pions_List, G18_Yerr_MEC_Pions_List, G18_Yerr_RES_Pions_List, G18_Yerr_DIS_Pions_List]
event_lists_18P = [G18_QEL_List_P, G18_MEC_List_P, G18_RES_List_P, G18_DIS_List_P]
for event in event_lists_18P:
        for fraction in event:
            num = event_lists_18P.index(event)
            G18_Yerr_Events_Lists[num].append(sqrt(fraction*(1-fraction)/G18_Total_Events_P))
G18_Yerr_QEL_Pions_Array = array.array('f', G18_Yerr_QEL_Pions_List)
G18_Yerr_MEC_Pions_Array = array.array('f', G18_Yerr_MEC_Pions_List)
G18_Yerr_RES_Pions_Array = array.array('f', G18_Yerr_RES_Pions_List)
G18_Yerr_DIS_Pions_Array = array.array('f', G18_Yerr_DIS_Pions_List)

In [ ]:
#g18 without initial state pions v2
G18_QEL_List_NIP = []
G18_MEC_List_NIP = []
G18_RES_List_NIP = []
G18_DIS_List_NIP = []
G18_LepCut_Vals_List_NIP = []
G18_Total_Events_NIP = 0
for value in values:
    df_gst_G18_02a_NIP = df_gst_G18_02a_all.Filter(f"ptl>{value} && abs(thetazl)<{lep_accept_angle}").Filter("numi_pions==0")
    h18_lep_pt_QEL_NIP = df_gst_G18_02a_NIP.Filter("qel==1").Histo1D(("h18_lep_pt_QEL_NIP", "", bins, x_start, x_end), "ptl")
    h18_lep_pt_MEC_NIP = df_gst_G18_02a_NIP.Filter("mec==1").Histo1D(("h18_lep_pt_MEC_NIP", "", bins, x_start, x_end), "ptl")
    h18_lep_pt_RES_NIP = df_gst_G18_02a_NIP.Filter("res==1").Histo1D(("h18_lep_pt_RES_NIP", "", bins, x_start, x_end), "ptl")
    h18_lep_pt_DIS_NIP = df_gst_G18_02a_NIP.Filter("dis==1").Histo1D(("h18_lep_pt_DIS_NIP", "", bins, x_start, x_end), "ptl")
    G18_Integral_List_NIP = [h18_lep_pt_QEL_NIP.Integral(), h18_lep_pt_MEC_NIP.Integral(), h18_lep_pt_RES_NIP.Integral(), h18_lep_pt_DIS_NIP.Integral()]
    G18_Total_NIP = G18_Integral_List_NIP[0] + G18_Integral_List_NIP[1] + G18_Integral_List_NIP[2] + G18_Integral_List_NIP[3]
    G18_Event_Lists_NIP = [G18_QEL_List_NIP, G18_MEC_List_NIP, G18_RES_List_NIP, G18_DIS_List_NIP]
    G18_Total_Events_NIP += G18_Total_NIP
    for i in range(len(G18_Integral_List_NIP)):
        if G18_Total_NIP > n_event_cut: 
            G18_Event_Lists_NIP[i].append(G18_Integral_List_NIP[i]/G18_Total_NIP)
            if value not in G18_LepCut_Vals_List_NIP:
                G18_LepCut_Vals_List_NIP.append(value)  
G18_QEL_NIP = array.array('f', G18_QEL_List_NIP)
G18_MEC_NIP = array.array('f', G18_MEC_List_NIP)
G18_RES_NIP = array.array('f', G18_RES_List_NIP)
G18_DIS_NIP = array.array('f', G18_DIS_List_NIP)
G18_LepCut_Vals_NIP_Array = array.array('f', G18_LepCut_Vals_List_NIP)

G18_Yerr_QEL_NIP_List = []
G18_Yerr_MEC_NIP_List = []
G18_Yerr_RES_NIP_List = []
G18_Yerr_DIS_NIP_List = []
G18_Yerr_Events_Lists_NIP = [G18_Yerr_QEL_NIP_List, G18_Yerr_MEC_NIP_List, G18_Yerr_RES_NIP_List, G18_Yerr_DIS_NIP_List]
G18_Event_Lists_NIP = [G18_QEL_List_NIP, G18_MEC_List_NIP, G18_RES_List_NIP, G18_DIS_List_NIP]
for event in G18_Event_Lists_NIP:
        for fraction in event:
            num = G18_Event_Lists_NIP.index(event)
            G18_Yerr_Events_Lists_NIP[num].append(sqrt(fraction*(1-fraction)/G18_Total_Events_NIP))
G18_Yerr_QEL_NIP_Array = array.array('f', G18_Yerr_QEL_NIP_List)
G18_Yerr_MEC_NIP_Array = array.array('f', G18_Yerr_MEC_NIP_List)
G18_Yerr_RES_NIP_Array = array.array('f', G18_Yerr_RES_NIP_List)
G18_Yerr_DIS_NIP_Array = array.array('f', G18_Yerr_DIS_NIP_List)

In [ ]:
#g18 without final state pions v2
G18_QEL_List_NP = []
G18_MEC_List_NP = []
G18_RES_List_NP = []
G18_DIS_List_NP = []
G18_LepCut_Vals_List_NP = []
G18_Total_Events_NP = 0
for value in values:
    df_gst_G18_02a_NP = df_gst_G18_02a_all.Filter(f"ptl>{value} && abs(thetazl)<{lep_accept_angle}").Filter("num_pions==0")
    h18_lep_pt_QEL_NP = df_gst_G18_02a_NP.Filter("qel==1").Histo1D(("h18_lep_pt_QEL_NP", "", bins, x_start, x_end), "ptl")
    h18_lep_pt_MEC_NP = df_gst_G18_02a_NP.Filter("mec==1").Histo1D(("h18_lep_pt_MEC_NP", "", bins, x_start, x_end), "ptl")
    h18_lep_pt_RES_NP = df_gst_G18_02a_NP.Filter("res==1").Histo1D(("h18_lep_pt_RES_NP", "", bins, x_start, x_end), "ptl")
    h18_lep_pt_DIS_NP = df_gst_G18_02a_NP.Filter("dis==1").Histo1D(("h18_lep_pt_DIS_NP", "", bins, x_start, x_end), "ptl")
    G18_Integral_List_NP = [h18_lep_pt_QEL_NP.Integral(), h18_lep_pt_MEC_NP.Integral(), h18_lep_pt_RES_NP.Integral(), h18_lep_pt_DIS_NP.Integral()]
    G18_Total_NP = G18_Integral_List_NP[0] + G18_Integral_List_NP[1] + G18_Integral_List_NP[2] + G18_Integral_List_NP[3]
    G18_Event_Lists_NP = [G18_QEL_List_NP, G18_MEC_List_NP, G18_RES_List_NP, G18_DIS_List_NP]
    G18_Total_Events_NP += G18_Total_NP
    for i in range(len(G18_Integral_List_NP)):
        if G18_Total_NP > n_event_cut: 
            G18_Event_Lists_NP[i].append(G18_Integral_List_NP[i]/G18_Total_NP)
            if value not in G18_LepCut_Vals_List_NP:
                G18_LepCut_Vals_List_NP.append(value)  
G18_QEL_NP = array.array('f', G18_QEL_List_NP)
G18_MEC_NP = array.array('f', G18_MEC_List_NP)
G18_RES_NP = array.array('f', G18_RES_List_NP)
G18_DIS_NP = array.array('f', G18_DIS_List_NP)
G18_LepCut_Vals_NP_Array = array.array('f', G18_LepCut_Vals_List_NP)

G18_Yerr_QEL_NP_List = []
G18_Yerr_MEC_NP_List = []
G18_Yerr_RES_NP_List = []
G18_Yerr_DIS_NP_List = []
G18_Yerr_Events_Lists_NP = [G18_Yerr_QEL_NP_List, G18_Yerr_MEC_NP_List, G18_Yerr_RES_NP_List, G18_Yerr_DIS_NP_List]
G18_Event_Lists_NP = [G18_QEL_List_NP, G18_MEC_List_NP, G18_RES_List_NP, G18_DIS_List_NP]
for event in G18_Event_Lists_NP:
        for fraction in event:
            num = G18_Event_Lists_NP.index(event)
            G18_Yerr_Events_Lists_NP[num].append(sqrt(fraction*(1-fraction)/G18_Total_Events_NP))
G18_Yerr_QEL_NP_Array = array.array('f', G18_Yerr_QEL_NP_List)
G18_Yerr_MEC_NP_Array = array.array('f', G18_Yerr_MEC_NP_List)
G18_Yerr_RES_NP_Array = array.array('f', G18_Yerr_RES_NP_List)
G18_Yerr_DIS_NP_Array = array.array('f', G18_Yerr_DIS_NP_List)

In [ ]:
#g18 without NFA pions 
G18_QEL_List_NFA = []
G18_MEC_List_NFA = []
G18_RES_List_NFA = []
G18_DIS_List_NFA = []
G18_LepCut_Vals_List_NFA = []
G18_Total_Events_NFA = 0
for value in values:
    df_gst_G18_02a_NFA = df_gst_G18_02a_all.Filter(f"ptl>{value} && abs(thetazl)<{lep_accept_angle}").Filter("(nfa_piplus+nfa_piminus+nfa_pi0)==0")
    h18_lep_pt_QEL_NFA = df_gst_G18_02a_NFA.Filter("qel==1").Histo1D(("h18_lep_pt_QEL_NFA", "", bins, x_start, x_end), "ptl")
    h18_lep_pt_MEC_NFA = df_gst_G18_02a_NFA.Filter("mec==1").Histo1D(("h18_lep_pt_MEC_NFA", "", bins, x_start, x_end), "ptl")
    h18_lep_pt_RES_NFA = df_gst_G18_02a_NFA.Filter("res==1").Histo1D(("h18_lep_pt_RES_NFA", "", bins, x_start, x_end), "ptl")
    h18_lep_pt_DIS_NFA = df_gst_G18_02a_NFA.Filter("dis==1").Histo1D(("h18_lep_pt_DIS_NFA", "", bins, x_start, x_end), "ptl")
    G18_Integral_List_NFA = [h18_lep_pt_QEL_NFA.Integral(), h18_lep_pt_MEC_NFA.Integral(), h18_lep_pt_RES_NFA.Integral(), h18_lep_pt_DIS_NFA.Integral()]
    G18_Total_NFA = G18_Integral_List_NFA[0] + G18_Integral_List_NFA[1] + G18_Integral_List_NFA[2] + G18_Integral_List_NFA[3]
    G18_Event_Lists_NFA = [G18_QEL_List_NFA, G18_MEC_List_NFA, G18_RES_List_NFA, G18_DIS_List_NFA]
    G18_Total_Events_NFA += G18_Total_NFA
    for i in range(len(G18_Integral_List_NFA)):
        if G18_Total_NFA > n_event_cut: 
            G18_Event_Lists_NFA[i].append(G18_Integral_List_NFA[i]/G18_Total_NFA)
            if value not in G18_LepCut_Vals_List_NFA:
                G18_LepCut_Vals_List_NFA.append(value)  
G18_QEL_NFA = array.array('f', G18_QEL_List_NFA)
G18_MEC_NFA = array.array('f', G18_MEC_List_NFA)
G18_RES_NFA = array.array('f', G18_RES_List_NFA)
G18_DIS_NFA = array.array('f', G18_DIS_List_NFA)
G18_LepCut_Vals_NFA_Array = array.array('f', G18_LepCut_Vals_List_NFA)

G18_Yerr_QEL_NFA_List = []
G18_Yerr_MEC_NFA_List = []
G18_Yerr_RES_NFA_List = []
G18_Yerr_DIS_NFA_List = []
G18_Yerr_Events_Lists_NFA = [G18_Yerr_QEL_NFA_List, G18_Yerr_MEC_NFA_List, G18_Yerr_RES_NFA_List, G18_Yerr_DIS_NFA_List]
G18_Event_Lists_NFA = [G18_QEL_List_NFA, G18_MEC_List_NFA, G18_RES_List_NFA, G18_DIS_List_NFA]
for event in G18_Event_Lists_NFA:
        for fraction in event:
            num = G18_Event_Lists_NFA.index(event)
            G18_Yerr_Events_Lists_NFA[num].append(sqrt(fraction*(1-fraction)/G18_Total_Events_NFA))
G18_Yerr_QEL_NFA_Array = array.array('f', G18_Yerr_QEL_NFA_List)
G18_Yerr_MEC_NFA_Array = array.array('f', G18_Yerr_MEC_NFA_List)
G18_Yerr_RES_NFA_Array = array.array('f', G18_Yerr_RES_NFA_List)
G18_Yerr_DIS_NFA_Array = array.array('f', G18_Yerr_DIS_NFA_List)

In [ ]:
#gem21 model arrays

In [ ]:
#gem21 with pions v2
G21_QEL_List_P = []
G21_MEC_List_P = []
G21_RES_List_P = []
G21_DIS_List_P = []
G21_LepCut_Vals_List = []
G21_Total_Events_P = 0
for value in values:
    df_gst_G21_02a_P = df_gst_GEM21_11b_all.Filter(f"ptl>{value} && abs(thetazl)<{lep_accept_angle}")
    h21_lep_pt_QEL_P = df_gst_G21_02a_P.Filter("qel==1").Histo1D(("h21_lep_pt_QEL_P", "", bins, x_start, x_end), "ptl")
    h21_lep_pt_MEC_P = df_gst_G21_02a_P.Filter("mec==1").Histo1D(("h21_lep_pt_MEC_P", "", bins, x_start, x_end), "ptl")
    h21_lep_pt_RES_P = df_gst_G21_02a_P.Filter("res==1").Histo1D(("h21_lep_pt_RES_P", "", bins, x_start, x_end), "ptl")
    h21_lep_pt_DIS_P = df_gst_G21_02a_P.Filter("dis==1").Histo1D(("h21_lep_pt_DIS_P", "", bins, x_start, x_end), "ptl")
    gev21P = [h21_lep_pt_QEL_P.Integral(), h21_lep_pt_MEC_P.Integral(), h21_lep_pt_RES_P.Integral(), h21_lep_pt_DIS_P.Integral()]
    total21P = gev21P[0] + gev21P[1] + gev21P[2] + gev21P[3]
    event_lists_21P = [G21_QEL_List_P, G21_MEC_List_P, G21_RES_List_P, G21_DIS_List_P]
    G21_Total_Events_P += total21P
    for i in range(len(gev21P)):
        if total21P > n_event_cut: 
            event_lists_21P[i].append(gev21P[i]/total21P)
            if value not in G21_LepCut_Vals_List:
                G21_LepCut_Vals_List.append(value)  
G21_QEL_Pions = array.array('f', G21_QEL_List_P)
G21_MEC_Pions = array.array('f', G21_MEC_List_P)
G21_RES_Pions = array.array('f', G21_RES_List_P)
G21_DIS_Pions = array.array('f', G21_DIS_List_P)
G21_LepCut_Vals_Array = array.array('f', G21_LepCut_Vals_List)

G21_Yerr_QEL_Pions_List = []
G21_Yerr_MEC_Pions_List = []
G21_Yerr_RES_Pions_List = []
G21_Yerr_DIS_Pions_List = []
G21_Yerr_Events_Lists = [G21_Yerr_QEL_Pions_List, G21_Yerr_MEC_Pions_List, G21_Yerr_RES_Pions_List, G21_Yerr_DIS_Pions_List]
event_lists_21P = [G21_QEL_List_P, G21_MEC_List_P, G21_RES_List_P, G21_DIS_List_P]
for event in event_lists_21P:
        for fraction in event:
            num = event_lists_21P.index(event)
            G21_Yerr_Events_Lists[num].append(sqrt(fraction*(1-fraction)/G21_Total_Events_P))
G21_Yerr_QEL_Pions_Array = array.array('f', G21_Yerr_QEL_Pions_List)
G21_Yerr_MEC_Pions_Array = array.array('f', G21_Yerr_MEC_Pions_List)
G21_Yerr_RES_Pions_Array = array.array('f', G21_Yerr_RES_Pions_List)
G21_Yerr_DIS_Pions_Array = array.array('f', G21_Yerr_DIS_Pions_List)

In [ ]:
#gem21 without initial state pions v2
G21_QEL_List_NIP = []
G21_MEC_List_NIP = []
G21_RES_List_NIP = []
G21_DIS_List_NIP = []
G21_LepCut_Vals_List_NIP = []
G21_Total_Events_NIP = 0
for value in values:
    df_gst_G21_02a_NIP = df_gst_GEM21_11b_all.Filter(f"ptl>{value} && abs(thetazl)<{lep_accept_angle}").Filter("numi_pions==0")
    h21_lep_pt_QEL_NIP = df_gst_G21_02a_NIP.Filter("qel==1").Histo1D(("h21_lep_pt_QEL_NIP", "", bins, x_start, x_end), "ptl")
    h21_lep_pt_MEC_NIP = df_gst_G21_02a_NIP.Filter("mec==1").Histo1D(("h21_lep_pt_MEC_NIP", "", bins, x_start, x_end), "ptl")
    h21_lep_pt_RES_NIP = df_gst_G21_02a_NIP.Filter("res==1").Histo1D(("h21_lep_pt_RES_NIP", "", bins, x_start, x_end), "ptl")
    h21_lep_pt_DIS_NIP = df_gst_G21_02a_NIP.Filter("dis==1").Histo1D(("h21_lep_pt_DIS_NIP", "", bins, x_start, x_end), "ptl")
    G21_Integral_List_NIP = [h21_lep_pt_QEL_NIP.Integral(), h21_lep_pt_MEC_NIP.Integral(), h21_lep_pt_RES_NIP.Integral(), h21_lep_pt_DIS_NIP.Integral()]
    G21_Total_NIP = G21_Integral_List_NIP[0] + G21_Integral_List_NIP[1] + G21_Integral_List_NIP[2] + G21_Integral_List_NIP[3]
    G21_Event_Lists_NIP = [G21_QEL_List_NIP, G21_MEC_List_NIP, G21_RES_List_NIP, G21_DIS_List_NIP]
    G21_Total_Events_NIP += G21_Total_NIP
    for i in range(len(G21_Integral_List_NIP)):
        if G21_Total_NIP > n_event_cut: 
            G21_Event_Lists_NIP[i].append(G21_Integral_List_NIP[i]/G21_Total_NIP)
            if value not in G21_LepCut_Vals_List_NIP:
                G21_LepCut_Vals_List_NIP.append(value)  
G21_QEL_NIP = array.array('f', G21_QEL_List_NIP)
G21_MEC_NIP = array.array('f', G21_MEC_List_NIP)
G21_RES_NIP = array.array('f', G21_RES_List_NIP)
G21_DIS_NIP = array.array('f', G21_DIS_List_NIP)
G21_LepCut_Vals_NIP_Array = array.array('f', G21_LepCut_Vals_List_NIP)

G21_Yerr_QEL_NIP_List = []
G21_Yerr_MEC_NIP_List = []
G21_Yerr_RES_NIP_List = []
G21_Yerr_DIS_NIP_List = []
G21_Yerr_Events_Lists_NIP = [G21_Yerr_QEL_NIP_List, G21_Yerr_MEC_NIP_List, G21_Yerr_RES_NIP_List, G21_Yerr_DIS_NIP_List]
G21_Event_Lists_NIP = [G21_QEL_List_NIP, G21_MEC_List_NIP, G21_RES_List_NIP, G21_DIS_List_NIP]
for event in G21_Event_Lists_NIP:
        for fraction in event:
            num = G21_Event_Lists_NIP.index(event)
            G21_Yerr_Events_Lists_NIP[num].append(sqrt(fraction*(1-fraction)/G21_Total_Events_NIP))
G21_Yerr_QEL_NIP_Array = array.array('f', G21_Yerr_QEL_NIP_List)
G21_Yerr_MEC_NIP_Array = array.array('f', G21_Yerr_MEC_NIP_List)
G21_Yerr_RES_NIP_Array = array.array('f', G21_Yerr_RES_NIP_List)
G21_Yerr_DIS_NIP_Array = array.array('f', G21_Yerr_DIS_NIP_List)

In [ ]:
#g21 without final state pions v2
G21_QEL_List_NP = []
G21_MEC_List_NP = []
G21_RES_List_NP = []
G21_DIS_List_NP = []
G21_LepCut_Vals_List_NP = []
G21_Total_Events_NP = 0
for value in values:
    df_gst_G21_02a_NP = df_gst_GEM21_11b_all.Filter(f"ptl>{value} && abs(thetazl)<{lep_accept_angle}").Filter("num_pions==0")
    h21_lep_pt_QEL_NP = df_gst_G21_02a_NP.Filter("qel==1").Histo1D(("h21_lep_pt_QEL_NP", "", bins, x_start, x_end), "ptl")
    h21_lep_pt_MEC_NP = df_gst_G21_02a_NP.Filter("mec==1").Histo1D(("h21_lep_pt_MEC_NP", "", bins, x_start, x_end), "ptl")
    h21_lep_pt_RES_NP = df_gst_G21_02a_NP.Filter("res==1").Histo1D(("h21_lep_pt_RES_NP", "", bins, x_start, x_end), "ptl")
    h21_lep_pt_DIS_NP = df_gst_G21_02a_NP.Filter("dis==1").Histo1D(("h21_lep_pt_DIS_NP", "", bins, x_start, x_end), "ptl")
    G21_Integral_List_NP = [h21_lep_pt_QEL_NP.Integral(), h21_lep_pt_MEC_NP.Integral(), h21_lep_pt_RES_NP.Integral(), h21_lep_pt_DIS_NP.Integral()]
    G21_Total_NP = G21_Integral_List_NP[0] + G21_Integral_List_NP[1] + G21_Integral_List_NP[2] + G21_Integral_List_NP[3]
    G21_Event_Lists_NP = [G21_QEL_List_NP, G21_MEC_List_NP, G21_RES_List_NP, G21_DIS_List_NP]
    G21_Total_Events_NP += G21_Total_NP
    for i in range(len(G21_Integral_List_NP)):
        if G21_Total_NP > n_event_cut: 
            G21_Event_Lists_NP[i].append(G21_Integral_List_NP[i]/G21_Total_NP)
            if value not in G21_LepCut_Vals_List_NP:
                G21_LepCut_Vals_List_NP.append(value)  
G21_QEL_NP = array.array('f', G21_QEL_List_NP)
G21_MEC_NP = array.array('f', G21_MEC_List_NP)
G21_RES_NP = array.array('f', G21_RES_List_NP)
G21_DIS_NP = array.array('f', G21_DIS_List_NP)
G21_LepCut_Vals_NP_Array = array.array('f', G21_LepCut_Vals_List_NP)

G21_Yerr_QEL_NP_List = []
G21_Yerr_MEC_NP_List = []
G21_Yerr_RES_NP_List = []
G21_Yerr_DIS_NP_List = []
G21_Yerr_Events_Lists_NP = [G21_Yerr_QEL_NP_List, G21_Yerr_MEC_NP_List, G21_Yerr_RES_NP_List, G21_Yerr_DIS_NP_List]
G21_Event_Lists_NP = [G21_QEL_List_NP, G21_MEC_List_NP, G21_RES_List_NP, G21_DIS_List_NP]
for event in G21_Event_Lists_NP:
        for fraction in event:
            num = G21_Event_Lists_NP.index(event)
            G21_Yerr_Events_Lists_NP[num].append(sqrt(fraction*(1-fraction)/G21_Total_Events_NP))
G21_Yerr_QEL_NP_Array = array.array('f', G21_Yerr_QEL_NP_List)
G21_Yerr_MEC_NP_Array = array.array('f', G21_Yerr_MEC_NP_List)
G21_Yerr_RES_NP_Array = array.array('f', G21_Yerr_RES_NP_List)
G21_Yerr_DIS_NP_Array = array.array('f', G21_Yerr_DIS_NP_List)

In [ ]:
#gem21 without NFA pions 
G21_QEL_List_NFA = []
G21_MEC_List_NFA = []
G21_RES_List_NFA = []
G21_DIS_List_NFA = []
G21_LepCut_Vals_List_NFA = []
G21_Total_Events_NFA = 0
for value in values:
    df_gst_GEM21_11b_NFA = df_gst_GEM21_11b_all.Filter(f"ptl>{value} && abs(thetazl)<{lep_accept_angle}").Filter("(nfa_piplus+nfa_piminus+nfa_pi0)==0")
    h21_lep_pt_QEL_NFA = df_gst_GEM21_11b_NFA.Filter("qel==1").Histo1D(("h21_lep_pt_QEL_NFA", "", bins, x_start, x_end), "ptl")
    h21_lep_pt_MEC_NFA = df_gst_GEM21_11b_NFA.Filter("mec==1").Histo1D(("h21_lep_pt_MEC_NFA", "", bins, x_start, x_end), "ptl")
    h21_lep_pt_RES_NFA = df_gst_GEM21_11b_NFA.Filter("res==1").Histo1D(("h21_lep_pt_RES_NFA", "", bins, x_start, x_end), "ptl")
    h21_lep_pt_DIS_NFA = df_gst_GEM21_11b_NFA.Filter("dis==1").Histo1D(("h21_lep_pt_DIS_NFA", "", bins, x_start, x_end), "ptl")
    G21_Integral_List_NFA = [h21_lep_pt_QEL_NFA.Integral(), h21_lep_pt_MEC_NFA.Integral(), h21_lep_pt_RES_NFA.Integral(), h21_lep_pt_DIS_NFA.Integral()]
    G21_Total_NFA = G21_Integral_List_NFA[0] + G21_Integral_List_NFA[1] + G21_Integral_List_NFA[2] + G21_Integral_List_NFA[3]
    G21_Event_Lists_NFA = [G21_QEL_List_NFA, G21_MEC_List_NFA, G21_RES_List_NFA, G21_DIS_List_NFA]
    G21_Total_Events_NFA += G21_Total_NFA
    for i in range(len(G21_Integral_List_NFA)):
        if G21_Total_NFA > n_event_cut: 
            G21_Event_Lists_NFA[i].append(G21_Integral_List_NFA[i]/G21_Total_NFA)
            if value not in G21_LepCut_Vals_List_NFA:
                G21_LepCut_Vals_List_NFA.append(value)  
G21_QEL_NFA = array.array('f', G21_QEL_List_NFA)
G21_MEC_NFA = array.array('f', G21_MEC_List_NFA)
G21_RES_NFA = array.array('f', G21_RES_List_NFA)
G21_DIS_NFA = array.array('f', G21_DIS_List_NFA)
G21_LepCut_Vals_NFA_Array = array.array('f', G21_LepCut_Vals_List_NFA)

G21_Yerr_QEL_NFA_List = []
G21_Yerr_MEC_NFA_List = []
G21_Yerr_RES_NFA_List = []
G21_Yerr_DIS_NFA_List = []
G21_Yerr_Events_Lists_NFA = [G21_Yerr_QEL_NFA_List, G21_Yerr_MEC_NFA_List, G21_Yerr_RES_NFA_List, G21_Yerr_DIS_NFA_List]
G21_Event_Lists_NFA = [G21_QEL_List_NFA, G21_MEC_List_NFA, G21_RES_List_NFA, G21_DIS_List_NFA]
for event in G21_Event_Lists_NFA:
        for fraction in event:
            num = G21_Event_Lists_NFA.index(event)
            G21_Yerr_Events_Lists_NFA[num].append(sqrt(fraction*(1-fraction)/G21_Total_Events_NFA))
G21_Yerr_QEL_NFA_Array = array.array('f', G21_Yerr_QEL_NFA_List)
G21_Yerr_MEC_NFA_Array = array.array('f', G21_Yerr_MEC_NFA_List)
G21_Yerr_RES_NFA_Array = array.array('f', G21_Yerr_RES_NFA_List)
G21_Yerr_DIS_NFA_Array = array.array('f', G21_Yerr_DIS_NFA_List)

In [ ]:
#plots of relative fraction of events per lepton pt cut

In [ ]:
#g18 model v2
G18_Pions_Canvas = ROOT.TCanvas()
G18_QEL_Pions_Plot = ROOT.TGraph(len(G18_LepCut_Vals_Array), G18_LepCut_Vals_Array, G18_QEL_Pions)
G18_QEL_Pions_Plot.SetTitle("Relative Fraction of Events Per Lepton PT Cut in G18 Model")
G18_QEL_Pions_Plot.GetXaxis().SetTitle("Momentum (GeV)")
G18_QEL_Pions_Plot.GetYaxis().SetTitle("Percentage")
G18_QEL_Pions_Plot.SetMarkerColor(4)
G18_QEL_Pions_Plot.SetMarkerStyle(29)
G18_QEL_Pions_Plot.SetMaximum(1.0)
G18_QEL_Pions_Plot.SetMinimum(0.0)
G18_QEL_Pions_Plot.Draw("")

G18_MEC_Pions_Plot = ROOT.TGraph(len(G18_LepCut_Vals_Array), G18_LepCut_Vals_Array, G18_MEC_Pions)
G18_MEC_Pions_Plot.SetMarkerColor(2)
G18_MEC_Pions_Plot.SetMarkerStyle(29)
G18_MEC_Pions_Plot.Draw("same")

G18_RES_Pions_Plot = ROOT.TGraph(len(G18_LepCut_Vals_Array), G18_LepCut_Vals_Array, G18_RES_Pions)
G18_RES_Pions_Plot.SetMarkerColor(3)
G18_RES_Pions_Plot.SetMarkerStyle(29)
G18_RES_Pions_Plot.Draw("same")

G18_DIS_Pions_Plot = ROOT.TGraph(len(G18_LepCut_Vals_Array), G18_LepCut_Vals_Array, G18_DIS_Pions)
G18_DIS_Pions_Plot.SetMarkerColor(6)
G18_DIS_Pions_Plot.SetMarkerStyle(29)
G18_DIS_Pions_Plot.Draw("same")

G18_Pions_leg = ROOT.TLegend(0.2, 0.85, 0.80, 0.90)
G18_Pions_leg.SetNColumns(4)
G18_Pions_leg.AddEntry(G18_QEL_Pions_Plot, "QEL")
G18_Pions_leg.AddEntry(G18_MEC_Pions_Plot, "MEC")
G18_Pions_leg.AddEntry(G18_RES_Pions_Plot, "RES")
G18_Pions_leg.AddEntry(G18_DIS_Pions_Plot, "DIS")
G18_Pions_leg.Draw()

G18_Pions_zeroes = []
for i in range(len(G18_LepCut_Vals_Array)):
    G18_Pions_zeroes.append(0)
G18_Pions_zeroes_array = array.array('f', G18_Pions_zeroes)

G18_Yerr_QEL_Pions_Bars = ROOT.TGraphErrors(len(G18_LepCut_Vals_Array), G18_LepCut_Vals_Array, G18_QEL_Pions, G18_Pions_zeroes_array, G18_Yerr_QEL_Pions_Array)
G18_Yerr_MEC_Pions_Bars = ROOT.TGraphErrors(len(G18_LepCut_Vals_Array), G18_LepCut_Vals_Array, G18_MEC_Pions, G18_Pions_zeroes_array, G18_Yerr_MEC_Pions_Array)
G18_Yerr_RES_Pions_Bars = ROOT.TGraphErrors(len(G18_LepCut_Vals_Array), G18_LepCut_Vals_Array, G18_RES_Pions, G18_Pions_zeroes_array, G18_Yerr_RES_Pions_Array)
G18_Yerr_DIS_Pions_Bars = ROOT.TGraphErrors(len(G18_LepCut_Vals_Array), G18_LepCut_Vals_Array, G18_DIS_Pions, G18_Pions_zeroes_array, G18_Yerr_DIS_Pions_Array)
G18_Yerr_QEL_Pions_Bars.Draw("same")
G18_Yerr_MEC_Pions_Bars.Draw("same")
G18_Yerr_RES_Pions_Bars.Draw("same")
G18_Yerr_DIS_Pions_Bars.Draw("same")
G18_Pions_Canvas.Draw()

In [ ]:
#g18 model without initial state pions v2
G18_NoIPions_Canvas = ROOT.TCanvas()
G18_QEL_NoIPions_Plot = ROOT.TGraph(len(G18_LepCut_Vals_NIP_Array), G18_LepCut_Vals_NIP_Array, G18_QEL_NIP)
G18_QEL_NoIPions_Plot.SetTitle("Relative Fraction of Events Per Lepton PT Cut w/o Initial State Pions in G18 Model")
G18_QEL_NoIPions_Plot.GetXaxis().SetTitle("Transverse Momenta (GeV)")
G18_QEL_NoIPions_Plot.GetYaxis().SetTitle("Percentage")
G18_QEL_NoIPions_Plot.SetMaximum(1.1)
G18_QEL_NoIPions_Plot.SetMinimum(0.0)
G18_QEL_NoIPions_Plot.SetMarkerColor(4)
G18_QEL_NoIPions_Plot.SetMarkerStyle(29)
G18_QEL_NoIPions_Plot.Draw("same")

G18_MEC_NoIPions_Plot = ROOT.TGraph(len(G18_LepCut_Vals_NIP_Array), G18_LepCut_Vals_NIP_Array, G18_MEC_NIP)
G18_MEC_NoIPions_Plot.Draw("same")
G18_MEC_NoIPions_Plot.SetMarkerColor(2)
G18_MEC_NoIPions_Plot.SetMarkerStyle(29)

G18_RES_NoIPions_Plot = ROOT.TGraph(len(G18_LepCut_Vals_NIP_Array), G18_LepCut_Vals_NIP_Array, G18_RES_NIP)
G18_RES_NoIPions_Plot.Draw("same")
G18_RES_NoIPions_Plot.SetMarkerColor(3)
G18_RES_NoIPions_Plot.SetMarkerStyle(29)

G18_DIS_NoIPions_Plot = ROOT.TGraph(len(G18_LepCut_Vals_NIP_Array), G18_LepCut_Vals_NIP_Array, G18_DIS_NIP)
G18_DIS_NoIPions_Plot.Draw("same")
G18_DIS_NoIPions_Plot.SetMarkerColor(6)
G18_DIS_NoIPions_Plot.SetMarkerStyle(29)

G18_NoIPions_leg = ROOT.TLegend(0.2, 0.85, 0.80, 0.90)
G18_NoIPions_leg.SetNColumns(4)
G18_NoIPions_leg.AddEntry(G18_QEL_NoIPions_Plot, "QEL")
G18_NoIPions_leg.AddEntry(G18_MEC_NoIPions_Plot, "MEC")
G18_NoIPions_leg.AddEntry(G18_RES_NoIPions_Plot, "RES")
G18_NoIPions_leg.AddEntry(G18_DIS_NoIPions_Plot, "DIS")
G18_NoIPions_leg.Draw()

G18_NIP_zeroes = []
for i in range(len(G18_LepCut_Vals_NIP_Array)):
    G18_NIP_zeroes.append(0)
G18_NIP_zeroes_array = array.array('f', G18_NIP_zeroes)

G18_Yerr_QEL_NIP_Bars = ROOT.TGraphErrors(len(G18_LepCut_Vals_NIP_Array), G18_LepCut_Vals_NIP_Array, G18_QEL_NIP, G18_NIP_zeroes_array, G18_Yerr_QEL_NIP_Array)
G18_Yerr_MEC_NIP_Bars = ROOT.TGraphErrors(len(G18_LepCut_Vals_NIP_Array), G18_LepCut_Vals_NIP_Array, G18_MEC_NIP, G18_NIP_zeroes_array, G18_Yerr_MEC_NIP_Array)
G18_Yerr_RES_NIP_Bars = ROOT.TGraphErrors(len(G18_LepCut_Vals_NIP_Array), G18_LepCut_Vals_NIP_Array, G18_RES_NIP, G18_NIP_zeroes_array, G18_Yerr_RES_NIP_Array)
G18_Yerr_DIS_NIP_Bars = ROOT.TGraphErrors(len(G18_LepCut_Vals_NIP_Array), G18_LepCut_Vals_NIP_Array, G18_DIS_NIP, G18_NIP_zeroes_array, G18_Yerr_DIS_NIP_Array)
G18_Yerr_QEL_NIP_Bars.Draw("same")
G18_Yerr_MEC_NIP_Bars.Draw("same")
G18_Yerr_RES_NIP_Bars.Draw("same")
G18_Yerr_DIS_NIP_Bars.Draw("same")

G18_NoIPions_Canvas.Draw()

In [ ]:
#g18 model without final state pions v2
G18_NoPions_Canvas = ROOT.TCanvas()
G18_QEL_NoPions_Plot = ROOT.TGraph(len(G18_LepCut_Vals_NP_Array), G18_LepCut_Vals_NP_Array, G18_QEL_NP)
G18_QEL_NoPions_Plot.SetTitle("Relative Fraction of Events Per Lepton PT Cut w/o Final State Pions in G18 Model")
G18_QEL_NoPions_Plot.GetXaxis().SetTitle("Transverse Momenta (GeV)")
G18_QEL_NoPions_Plot.GetYaxis().SetTitle("Percentage")
G18_QEL_NoPions_Plot.SetMaximum(1.0)
G18_QEL_NoPions_Plot.SetMinimum(0.0)
G18_QEL_NoPions_Plot.SetMarkerColor(4)
G18_QEL_NoPions_Plot.SetMarkerStyle(29)
G18_QEL_NoPions_Plot.Draw("same")

G18_MEC_NoPions_Plot = ROOT.TGraph(len(G18_LepCut_Vals_NP_Array), G18_LepCut_Vals_NP_Array, G18_MEC_NP)
G18_MEC_NoPions_Plot.Draw("same")
G18_MEC_NoPions_Plot.SetMarkerColor(2)
G18_MEC_NoPions_Plot.SetMarkerStyle(29)

G18_RES_NoPions_Plot = ROOT.TGraph(len(G18_LepCut_Vals_NP_Array), G18_LepCut_Vals_NP_Array, G18_RES_NP)
G18_RES_NoPions_Plot.Draw("same")
G18_RES_NoPions_Plot.SetMarkerColor(3)
G18_RES_NoPions_Plot.SetMarkerStyle(29)

G18_DIS_NoPions_Plot = ROOT.TGraph(len(G18_LepCut_Vals_NP_Array), G18_LepCut_Vals_NP_Array, G18_DIS_NP)
G18_DIS_NoPions_Plot.Draw("same")
G18_DIS_NoPions_Plot.SetMarkerColor(6)
G18_DIS_NoPions_Plot.SetMarkerStyle(29)

G18_NoPions_leg = ROOT.TLegend(0.2, 0.85, 0.80, 0.90)
G18_NoPions_leg.SetNColumns(4)
G18_NoPions_leg.AddEntry(G18_QEL_NoPions_Plot, "QEL")
G18_NoPions_leg.AddEntry(G18_MEC_NoPions_Plot, "MEC")
G18_NoPions_leg.AddEntry(G18_RES_NoPions_Plot, "RES")
G18_NoPions_leg.AddEntry(G18_DIS_NoPions_Plot, "DIS")
G18_NoPions_leg.Draw()

G18_NP_zeroes = []
for i in range(len(G18_LepCut_Vals_NP_Array)):
    G18_NP_zeroes.append(0)
G18_NP_zeroes_array = array.array('f', G18_NP_zeroes)

G18_Yerr_QEL_NP_Bars = ROOT.TGraphErrors(len(G18_LepCut_Vals_NP_Array), G18_LepCut_Vals_NP_Array, G18_QEL_NP, G18_NP_zeroes_array, G18_Yerr_QEL_NP_Array)
G18_Yerr_MEC_NP_Bars = ROOT.TGraphErrors(len(G18_LepCut_Vals_NP_Array), G18_LepCut_Vals_NP_Array, G18_MEC_NP, G18_NP_zeroes_array, G18_Yerr_MEC_NP_Array)
G18_Yerr_RES_NP_Bars = ROOT.TGraphErrors(len(G18_LepCut_Vals_NP_Array), G18_LepCut_Vals_NP_Array, G18_RES_NP, G18_NP_zeroes_array, G18_Yerr_RES_NP_Array)
G18_Yerr_DIS_NP_Bars = ROOT.TGraphErrors(len(G18_LepCut_Vals_NP_Array), G18_LepCut_Vals_NP_Array, G18_DIS_NP, G18_NP_zeroes_array, G18_Yerr_DIS_NP_Array)
G18_Yerr_QEL_NP_Bars.Draw("same")
G18_Yerr_MEC_NP_Bars.Draw("same")
G18_Yerr_RES_NP_Bars.Draw("same")
G18_Yerr_DIS_NP_Bars.Draw("same")

G18_NoPions_Canvas.Draw()

In [ ]:
#g18 model without nfa pions v2
G18_NoPions_Canvas = ROOT.TCanvas()
G18_QEL_NoPions_Plot = ROOT.TGraph(len(G18_LepCut_Vals_NFA_Array), G18_LepCut_Vals_NFA_Array, G18_QEL_NFA)
G18_QEL_NoPions_Plot.SetTitle("Relative Fraction of Events Per Lepton PT Cut w/o NFA Pions in G18 Model")
G18_QEL_NoPions_Plot.GetXaxis().SetTitle("Transverse Momenta (GeV)")
G18_QEL_NoPions_Plot.GetYaxis().SetTitle("Percentage")
G18_QEL_NoPions_Plot.SetMaximum(1.0)
G18_QEL_NoPions_Plot.SetMinimum(0.0)
G18_QEL_NoPions_Plot.SetMarkerColor(4)
G18_QEL_NoPions_Plot.SetMarkerStyle(29)
G18_QEL_NoPions_Plot.Draw("same")

G18_MEC_NoPions_Plot = ROOT.TGraph(len(G18_LepCut_Vals_NFA_Array), G18_LepCut_Vals_NFA_Array, G18_MEC_NFA)
G18_MEC_NoPions_Plot.Draw("same")
G18_MEC_NoPions_Plot.SetMarkerColor(2)
G18_MEC_NoPions_Plot.SetMarkerStyle(29)

G18_RES_NoPions_Plot = ROOT.TGraph(len(G18_LepCut_Vals_NFA_Array), G18_LepCut_Vals_NFA_Array, G18_RES_NFA)
G18_RES_NoPions_Plot.Draw("same")
G18_RES_NoPions_Plot.SetMarkerColor(3)
G18_RES_NoPions_Plot.SetMarkerStyle(29)

G18_DIS_NoPions_Plot = ROOT.TGraph(len(G18_LepCut_Vals_NFA_Array), G18_LepCut_Vals_NFA_Array, G18_DIS_NFA)
G18_DIS_NoPions_Plot.Draw("same")
G18_DIS_NoPions_Plot.SetMarkerColor(6)
G18_DIS_NoPions_Plot.SetMarkerStyle(29)

G18_NoPions_leg = ROOT.TLegend(0.2, 0.85, 0.80, 0.90)
G18_NoPions_leg.SetNColumns(4)
G18_NoPions_leg.AddEntry(G18_QEL_NoPions_Plot, "QEL")
G18_NoPions_leg.AddEntry(G18_MEC_NoPions_Plot, "MEC")
G18_NoPions_leg.AddEntry(G18_RES_NoPions_Plot, "RES")
G18_NoPions_leg.AddEntry(G18_DIS_NoPions_Plot, "DIS")
G18_NoPions_leg.Draw()

G18_NFA_zeroes = []
for i in range(len(G18_LepCut_Vals_NFA_Array)):
    G18_NFA_zeroes.append(0)
G18_NFA_zeroes_array = array.array('f', G18_NFA_zeroes)

G18_Yerr_QEL_NFA_Bars = ROOT.TGraphErrors(len(G18_LepCut_Vals_NFA_Array), G18_LepCut_Vals_NFA_Array, G18_QEL_NFA, G18_NFA_zeroes_array, G18_Yerr_QEL_NFA_Array)
G18_Yerr_MEC_NFA_Bars = ROOT.TGraphErrors(len(G18_LepCut_Vals_NFA_Array), G18_LepCut_Vals_NFA_Array, G18_MEC_NFA, G18_NFA_zeroes_array, G18_Yerr_MEC_NFA_Array)
G18_Yerr_RES_NFA_Bars = ROOT.TGraphErrors(len(G18_LepCut_Vals_NFA_Array), G18_LepCut_Vals_NFA_Array, G18_RES_NFA, G18_NFA_zeroes_array, G18_Yerr_RES_NFA_Array)
G18_Yerr_DIS_NFA_Bars = ROOT.TGraphErrors(len(G18_LepCut_Vals_NFA_Array), G18_LepCut_Vals_NFA_Array, G18_DIS_NFA, G18_NFA_zeroes_array, G18_Yerr_DIS_NFA_Array)
G18_Yerr_QEL_NFA_Bars.Draw("same")
G18_Yerr_MEC_NFA_Bars.Draw("same")
G18_Yerr_RES_NFA_Bars.Draw("same")
G18_Yerr_DIS_NFA_Bars.Draw("same")

G18_NoPions_Canvas.Draw()

In [ ]:
#gem21 model v2
G21_Pions_Canvas = ROOT.TCanvas()
G21_QEL_Pions_Plot = ROOT.TGraph(len(G21_LepCut_Vals_Array), G21_LepCut_Vals_Array, G21_QEL_Pions)
G21_QEL_Pions_Plot.SetTitle("Relative Fraction of Events Per Lepton PT Cut with Pions in GEM21 Model")
G21_QEL_Pions_Plot.GetXaxis().SetTitle("Transverse Momenta (GeV)")
G21_QEL_Pions_Plot.GetYaxis().SetTitle("Percentage")
G21_QEL_Pions_Plot.SetMarkerColor(4)
G21_QEL_Pions_Plot.SetMarkerStyle(29)
G21_QEL_Pions_Plot.SetMaximum(0.7)
G21_QEL_Pions_Plot.SetMinimum(0.0)
G21_QEL_Pions_Plot.Draw("")

G21_MEC_Pions_Plot = ROOT.TGraph(len(G21_LepCut_Vals_Array), G21_LepCut_Vals_Array, G21_MEC_Pions)
G21_MEC_Pions_Plot.SetMarkerColor(2)
G21_MEC_Pions_Plot.SetMarkerStyle(29)
G21_MEC_Pions_Plot.Draw("same")

G21_RES_Pions_Plot = ROOT.TGraph(len(G21_LepCut_Vals_Array), G21_LepCut_Vals_Array, G21_RES_Pions)
G21_RES_Pions_Plot.SetMarkerColor(3)
G21_RES_Pions_Plot.SetMarkerStyle(29)
G21_RES_Pions_Plot.Draw("same")

G21_DIS_Pions_Plot = ROOT.TGraph(len(G21_LepCut_Vals_Array), G21_LepCut_Vals_Array, G21_DIS_Pions)
G21_DIS_Pions_Plot.SetMarkerColor(6)
G21_DIS_Pions_Plot.SetMarkerStyle(29)
G21_DIS_Pions_Plot.Draw("same")

G21_Pions_leg = ROOT.TLegend(0.2, 0.85, 0.80, 0.90)
G21_Pions_leg.SetNColumns(4)
G21_Pions_leg.AddEntry(G21_QEL_Pions_Plot, "QEL")
G21_Pions_leg.AddEntry(G21_MEC_Pions_Plot, "MEC")
G21_Pions_leg.AddEntry(G21_RES_Pions_Plot, "RES")
G21_Pions_leg.AddEntry(G21_DIS_Pions_Plot, "DIS")
G21_Pions_leg.Draw()

G21_Pions_zeroes = []
for i in range(len(G21_LepCut_Vals_Array)):
    G21_Pions_zeroes.append(0)
G21_Pions_zeroes_array = array.array('f', G21_Pions_zeroes)

G21_Yerr_QEL_Pions_Bars = ROOT.TGraphErrors(len(G21_LepCut_Vals_Array), G21_LepCut_Vals_Array, G21_QEL_Pions, G21_Pions_zeroes_array, G21_Yerr_QEL_Pions_Array)
G21_Yerr_MEC_Pions_Bars = ROOT.TGraphErrors(len(G21_LepCut_Vals_Array), G21_LepCut_Vals_Array, G21_MEC_Pions, G21_Pions_zeroes_array, G21_Yerr_MEC_Pions_Array)
G21_Yerr_RES_Pions_Bars = ROOT.TGraphErrors(len(G21_LepCut_Vals_Array), G21_LepCut_Vals_Array, G21_RES_Pions, G21_Pions_zeroes_array, G21_Yerr_RES_Pions_Array)
G21_Yerr_DIS_Pions_Bars = ROOT.TGraphErrors(len(G21_LepCut_Vals_Array), G21_LepCut_Vals_Array, G21_DIS_Pions, G21_Pions_zeroes_array, G21_Yerr_DIS_Pions_Array)
G21_Yerr_QEL_Pions_Bars.Draw("same")
G21_Yerr_MEC_Pions_Bars.Draw("same")
G21_Yerr_RES_Pions_Bars.Draw("same")
G21_Yerr_DIS_Pions_Bars.Draw("same")
G21_Pions_Canvas.Draw()

In [ ]:
#g21 model without initial state pions v2
G21_NoIPions_Canvas = ROOT.TCanvas()
G21_QEL_NoIPions_Plot = ROOT.TGraph(len(G21_LepCut_Vals_NIP_Array), G21_LepCut_Vals_NIP_Array, G21_QEL_NIP)
G21_QEL_NoIPions_Plot.SetTitle("Relative Fraction of Events Per Lepton PT Cut w/o Initial State Pions in GEM21 Model")
G21_QEL_NoIPions_Plot.GetXaxis().SetTitle("Transverse Momenta (GeV)")
G21_QEL_NoIPions_Plot.GetYaxis().SetTitle("Percentage")
G21_QEL_NoIPions_Plot.SetMaximum(1.0)
G21_QEL_NoIPions_Plot.SetMinimum(0.0)
G21_QEL_NoIPions_Plot.SetMarkerColor(4)
G21_QEL_NoIPions_Plot.SetMarkerStyle(29)
G21_QEL_NoIPions_Plot.Draw("same")

G21_MEC_NoIPions_Plot = ROOT.TGraph(len(G21_LepCut_Vals_NIP_Array), G21_LepCut_Vals_NIP_Array, G21_MEC_NIP)
G21_MEC_NoIPions_Plot.Draw("same")
G21_MEC_NoIPions_Plot.SetMarkerColor(2)
G21_MEC_NoIPions_Plot.SetMarkerStyle(29)

G21_RES_NoIPions_Plot = ROOT.TGraph(len(G21_LepCut_Vals_NIP_Array), G21_LepCut_Vals_NIP_Array, G21_RES_NIP)
G21_RES_NoIPions_Plot.Draw("same")
G21_RES_NoIPions_Plot.SetMarkerColor(3)
G21_RES_NoIPions_Plot.SetMarkerStyle(29)

G21_DIS_NoIPions_Plot = ROOT.TGraph(len(G21_LepCut_Vals_NIP_Array), G21_LepCut_Vals_NIP_Array, G21_DIS_NIP)
G21_DIS_NoIPions_Plot.Draw("same")
G21_DIS_NoIPions_Plot.SetMarkerColor(6)
G21_DIS_NoIPions_Plot.SetMarkerStyle(29)

G21_NoIPions_leg = ROOT.TLegend(0.2, 0.85, 0.80, 0.90)
G21_NoIPions_leg.SetNColumns(4)
G21_NoIPions_leg.AddEntry(G21_QEL_NoIPions_Plot, "QEL")
G21_NoIPions_leg.AddEntry(G21_MEC_NoIPions_Plot, "MEC")
G21_NoIPions_leg.AddEntry(G21_RES_NoIPions_Plot, "RES")
G21_NoIPions_leg.AddEntry(G21_DIS_NoIPions_Plot, "DIS")
G21_NoIPions_leg.Draw()

G21_NIP_zeroes = []
for i in range(len(G21_LepCut_Vals_NIP_Array)):
    G21_NIP_zeroes.append(0)
G21_NIP_zeroes_array = array.array('f', G21_NIP_zeroes)

G21_Yerr_QEL_NIP_Bars = ROOT.TGraphErrors(len(G21_LepCut_Vals_NIP_Array), G21_LepCut_Vals_NIP_Array, G21_QEL_NIP, G21_NIP_zeroes_array, G21_Yerr_QEL_NIP_Array)
G21_Yerr_MEC_NIP_Bars = ROOT.TGraphErrors(len(G21_LepCut_Vals_NIP_Array), G21_LepCut_Vals_NIP_Array, G21_MEC_NIP, G21_NIP_zeroes_array, G21_Yerr_MEC_NIP_Array)
G21_Yerr_RES_NIP_Bars = ROOT.TGraphErrors(len(G21_LepCut_Vals_NIP_Array), G21_LepCut_Vals_NIP_Array, G21_RES_NIP, G21_NIP_zeroes_array, G21_Yerr_RES_NIP_Array)
G21_Yerr_DIS_NIP_Bars = ROOT.TGraphErrors(len(G21_LepCut_Vals_NIP_Array), G21_LepCut_Vals_NIP_Array, G21_DIS_NIP, G21_NIP_zeroes_array, G21_Yerr_DIS_NIP_Array)
G21_Yerr_QEL_NIP_Bars.Draw("same")
G21_Yerr_MEC_NIP_Bars.Draw("same")
G21_Yerr_RES_NIP_Bars.Draw("same")
G21_Yerr_DIS_NIP_Bars.Draw("same")

G21_NoIPions_Canvas.Draw()

In [ ]:
#gem21 model without final state pions v2
G21_NoPions_Canvas = ROOT.TCanvas()
G21_QEL_NoPions_Plot = ROOT.TGraph(len(G21_LepCut_Vals_NP_Array), G21_LepCut_Vals_NP_Array, G21_QEL_NP)
G21_QEL_NoPions_Plot.SetTitle("Relative Fraction of Events Per Lepton PT Cut w/o Final State Pions in GEM21 Model")
G21_QEL_NoPions_Plot.GetXaxis().SetTitle("Transverse Momenta (GeV)")
G21_QEL_NoPions_Plot.GetYaxis().SetTitle("Percentage")
G21_QEL_NoPions_Plot.SetMaximum(1.0)
G21_QEL_NoPions_Plot.SetMinimum(0.0)
G21_QEL_NoPions_Plot.SetMarkerColor(4)
G21_QEL_NoPions_Plot.SetMarkerStyle(29)
G21_QEL_NoPions_Plot.Draw("same")

G21_MEC_NoPions_Plot = ROOT.TGraph(len(G21_LepCut_Vals_NP_Array), G21_LepCut_Vals_NP_Array, G21_MEC_NP)
G21_MEC_NoPions_Plot.Draw("same")
G21_MEC_NoPions_Plot.SetMarkerColor(2)
G21_MEC_NoPions_Plot.SetMarkerStyle(29)

G21_RES_NoPions_Plot = ROOT.TGraph(len(G21_LepCut_Vals_NP_Array), G21_LepCut_Vals_NP_Array, G21_RES_NP)
G21_RES_NoPions_Plot.Draw("same")
G21_RES_NoPions_Plot.SetMarkerColor(3)
G21_RES_NoPions_Plot.SetMarkerStyle(29)

G21_DIS_NoPions_Plot = ROOT.TGraph(len(G21_LepCut_Vals_NP_Array), G21_LepCut_Vals_NP_Array, G21_DIS_NP)
G21_DIS_NoPions_Plot.Draw("same")
G21_DIS_NoPions_Plot.SetMarkerColor(6)
G21_DIS_NoPions_Plot.SetMarkerStyle(29)

G21_NoPions_leg = ROOT.TLegend(0.2, 0.85, 0.80, 0.90)
G21_NoPions_leg.SetNColumns(4)
G21_NoPions_leg.AddEntry(G21_QEL_NoPions_Plot, "QEL")
G21_NoPions_leg.AddEntry(G21_MEC_NoPions_Plot, "MEC")
G21_NoPions_leg.AddEntry(G21_RES_NoPions_Plot, "RES")
G21_NoPions_leg.AddEntry(G21_DIS_NoPions_Plot, "DIS")
G21_NoPions_leg.Draw()

G21_NP_zeroes = []
for i in range(len(G21_LepCut_Vals_NP_Array)):
    G21_NP_zeroes.append(0)
G21_NP_zeroes_array = array.array('f', G21_NP_zeroes)

G21_Yerr_QEL_NP_Bars = ROOT.TGraphErrors(len(G21_LepCut_Vals_NP_Array), G21_LepCut_Vals_NP_Array, G21_QEL_NP, G21_NP_zeroes_array, G21_Yerr_QEL_NP_Array)
G21_Yerr_MEC_NP_Bars = ROOT.TGraphErrors(len(G21_LepCut_Vals_NP_Array), G21_LepCut_Vals_NP_Array, G21_MEC_NP, G21_NP_zeroes_array, G21_Yerr_MEC_NP_Array)
G21_Yerr_RES_NP_Bars = ROOT.TGraphErrors(len(G21_LepCut_Vals_NP_Array), G21_LepCut_Vals_NP_Array, G21_RES_NP, G21_NP_zeroes_array, G21_Yerr_RES_NP_Array)
G21_Yerr_DIS_NP_Bars = ROOT.TGraphErrors(len(G21_LepCut_Vals_NP_Array), G21_LepCut_Vals_NP_Array, G21_DIS_NP, G21_NP_zeroes_array, G21_Yerr_DIS_NP_Array)
G21_Yerr_QEL_NP_Bars.Draw("same")
G21_Yerr_MEC_NP_Bars.Draw("same")
G21_Yerr_RES_NP_Bars.Draw("same")
G21_Yerr_DIS_NP_Bars.Draw("same")

G21_NoPions_Canvas.Draw()

In [ ]:
#gem21 model without nfa pions v2
G21_NoPions_Canvas = ROOT.TCanvas()
G21_QEL_NoPions_Plot = ROOT.TGraph(len(G21_LepCut_Vals_NFA_Array), G21_LepCut_Vals_NFA_Array, G21_QEL_NFA)
G21_QEL_NoPions_Plot.SetTitle("Relative Fraction of Events Per Lepton PT Cut w/o NFA Pions in GEM21 Model")
G21_QEL_NoPions_Plot.GetXaxis().SetTitle("Transverse Momenta (GeV)")
G21_QEL_NoPions_Plot.GetYaxis().SetTitle("Percentage")
G21_QEL_NoPions_Plot.SetMaximum(1.0)
G21_QEL_NoPions_Plot.SetMinimum(0.0)
G21_QEL_NoPions_Plot.SetMarkerColor(4)
G21_QEL_NoPions_Plot.SetMarkerStyle(29)
G21_QEL_NoPions_Plot.Draw("same")

G21_MEC_NoPions_Plot = ROOT.TGraph(len(G21_LepCut_Vals_NFA_Array), G21_LepCut_Vals_NFA_Array, G21_MEC_NFA)
G21_MEC_NoPions_Plot.Draw("same")
G21_MEC_NoPions_Plot.SetMarkerColor(2)
G21_MEC_NoPions_Plot.SetMarkerStyle(29)

G21_RES_NoPions_Plot = ROOT.TGraph(len(G21_LepCut_Vals_NFA_Array), G21_LepCut_Vals_NFA_Array, G21_RES_NFA)
G21_RES_NoPions_Plot.Draw("same")
G21_RES_NoPions_Plot.SetMarkerColor(3)
G21_RES_NoPions_Plot.SetMarkerStyle(29)

G21_DIS_NoPions_Plot = ROOT.TGraph(len(G21_LepCut_Vals_NFA_Array), G21_LepCut_Vals_NFA_Array, G21_DIS_NFA)
G21_DIS_NoPions_Plot.Draw("same")
G21_DIS_NoPions_Plot.SetMarkerColor(6)
G21_DIS_NoPions_Plot.SetMarkerStyle(29)

G21_NoPions_leg = ROOT.TLegend(0.2, 0.85, 0.80, 0.90)
G21_NoPions_leg.SetNColumns(4)
G21_NoPions_leg.AddEntry(G21_QEL_NoPions_Plot, "QEL")
G21_NoPions_leg.AddEntry(G21_MEC_NoPions_Plot, "MEC")
G21_NoPions_leg.AddEntry(G21_RES_NoPions_Plot, "RES")
G21_NoPions_leg.AddEntry(G21_DIS_NoPions_Plot, "DIS")
G21_NoPions_leg.Draw()

G21_NFA_zeroes = []
for i in range(len(G21_LepCut_Vals_NFA_Array)):
    G21_NFA_zeroes.append(0)
G21_NFA_zeroes_array = array.array('f', G21_NFA_zeroes)

G21_Yerr_QEL_NFA_Bars = ROOT.TGraphErrors(len(G21_LepCut_Vals_NFA_Array), G21_LepCut_Vals_NFA_Array, G21_QEL_NFA, G21_NFA_zeroes_array, G21_Yerr_QEL_NFA_Array)
G21_Yerr_MEC_NFA_Bars = ROOT.TGraphErrors(len(G21_LepCut_Vals_NFA_Array), G21_LepCut_Vals_NFA_Array, G21_MEC_NFA, G21_NFA_zeroes_array, G21_Yerr_MEC_NFA_Array)
G21_Yerr_RES_NFA_Bars = ROOT.TGraphErrors(len(G21_LepCut_Vals_NFA_Array), G21_LepCut_Vals_NFA_Array, G21_RES_NFA, G21_NFA_zeroes_array, G21_Yerr_RES_NFA_Array)
G21_Yerr_DIS_NFA_Bars = ROOT.TGraphErrors(len(G21_LepCut_Vals_NFA_Array), G21_LepCut_Vals_NFA_Array, G21_DIS_NFA, G21_NFA_zeroes_array, G21_Yerr_DIS_NFA_Array)
G21_Yerr_QEL_NFA_Bars.Draw("same")
G21_Yerr_MEC_NFA_Bars.Draw("same")
G21_Yerr_RES_NFA_Bars.Draw("same")
G21_Yerr_DIS_NFA_Bars.Draw("same")

G21_NoPions_Canvas.Draw()

In [ ]:
#lepton pt cuts like above but sorted into bins by 50 MeV/c for ni_pions=0 nf_pions=0 nfa_pions=0 v4
bins = 100
x_start = 0
x_end = 4.0 #GeV
pt_values = [0.00, 0.05, 0.10, 0.15, 0.20, 0.25, 0.30, 0.35, 0.40, 0.45, 0.50, 0.55, 0.60, 0.65, 0.70, 0.75, 0.80, 0.85, 0.90, 0.95, 1.00]
n_event_cut = 20
#v5 added sum plot and fixed error in value_min -> value_max and midpoint instead of 0.0

In [ ]:
#g18 without initial state pions v5
G18_QEL_NIP_List = []
G18_MEC_NIP_List = []
G18_RES_NIP_List = []
G18_DIS_NIP_List = []
G18_LepBins_XVals_NIP_List = []
G18_Total_Count_NIP = 0
for value in pt_values:
    value_min = value
    value_max = value + 0.05
    df_gst_G18_02a_LepNIP = df_gst_G18_02a_all.Filter(f"ptl>{value_min} && ptl<{value_max} && abs(thetazl)<{lep_accept_angle}").Filter("numi_pions==0")
    G18_LepPt_QEL_NIP = df_gst_G18_02a_LepNIP.Filter("qel==1").Histo1D(("G18_LepPt_QEL_NIP", "", bins, x_start, x_end), "ptl")
    G18_LepPt_MEC_NIP = df_gst_G18_02a_LepNIP.Filter("mec==1").Histo1D(("G18_LepPt_MEC_NIP", "", bins, x_start, x_end), "ptl")
    G18_LepPt_RES_NIP = df_gst_G18_02a_LepNIP.Filter("res==1").Histo1D(("G18_LepPt_RES_NIP", "", bins, x_start, x_end), "ptl")
    G18_LepPt_DIS_NIP = df_gst_G18_02a_LepNIP.Filter("dis==1").Histo1D(("G18_LepPt_DIS_NIP", "", bins, x_start, x_end), "ptl")
    G18_Int_NIP_List = [G18_LepPt_QEL_NIP.Integral(), G18_LepPt_MEC_NIP.Integral(), G18_LepPt_RES_NIP.Integral(), G18_LepPt_DIS_NIP.Integral()]
    G18_Total_Run_NIP = G18_Int_NIP_List[0] + G18_Int_NIP_List[1] + G18_Int_NIP_List[2] + G18_Int_NIP_List[3]
    G18_Events_NIP_List = [G18_QEL_NIP_List, G18_MEC_NIP_List, G18_RES_NIP_List, G18_DIS_NIP_List]
    G18_Total_Count_NIP += G18_Total_Run_NIP
    G18_MidPoint_NIP = (value_max - value_min)/2
    for i in range(len(G18_Int_NIP_List)):
        if G18_Total_Run_NIP < n_event_cut and G18_Events_NIP_List[i] == []:
            G18_Events_NIP_List[i].append(0.0)
        if G18_LepBins_XVals_NIP_List == []:
            G18_LepBins_XVals_NIP_List.append(G18_MidPoint_NIP)
        if G18_Total_Run_NIP > n_event_cut: 
            G18_Events_NIP_List[i].append(G18_Int_NIP_List[i]/G18_Total_Run_NIP)
            if (value_max - G18_MidPoint_NIP) not in G18_LepBins_XVals_NIP_List:
                G18_LepBins_XVals_NIP_List.append(value_max - G18_MidPoint_NIP)
G18_LepBins_XVals_NIP_Array = array.array('f', G18_LepBins_XVals_NIP_List)
G18_LepPt_QEL_NIP_Array = array.array('f', G18_QEL_NIP_List)
G18_LepPt_MEC_NIP_Array = array.array('f', G18_MEC_NIP_List)
G18_LepPt_RES_NIP_Array = array.array('f', G18_RES_NIP_List)
G18_LepPt_DIS_NIP_Array = array.array('f', G18_DIS_NIP_List)
G18_Yerr_QEL_NIP_List = []
G18_Yerr_MEC_NIP_List = []
G18_Yerr_RES_NIP_List = []
G18_Yerr_DIS_NIP_List = []
G18_Yerr_Events_NIP_Lists = [G18_Yerr_QEL_NIP_List, G18_Yerr_MEC_NIP_List, G18_Yerr_RES_NIP_List, G18_Yerr_DIS_NIP_List]
for event in G18_Events_NIP_List:
        for fraction in event:
            num = G18_Events_NIP_List.index(event)
            G18_Yerr_Events_NIP_Lists[num].append(sqrt(fraction*(1-fraction)/G18_Total_Count_NIP))
G18_Yerr_QEL_NIP_Array = array.array('f', G18_Yerr_QEL_NIP_List)
G18_Yerr_MEC_NIP_Array = array.array('f', G18_Yerr_MEC_NIP_List)
G18_Yerr_RES_NIP_Array = array.array('f', G18_Yerr_RES_NIP_List)
G18_Yerr_DIS_NIP_Array = array.array('f', G18_Yerr_DIS_NIP_List)

G18_LepPt_Sum_NIP = []
for i in range(len(G18_QEL_NIP_List)):
    count = 0
    count += G18_QEL_NIP_List[i] + G18_MEC_NIP_List[i] +  G18_RES_NIP_List[i] + G18_DIS_NIP_List[i]
    G18_LepPt_Sum_NIP.append(count)
G18_LepPt_Sum_NIP_Array = array.array('f', G18_LepPt_Sum_NIP)

In [ ]:
#g18 without final state pions v5
G18_QEL_NFP_List = []
G18_MEC_NFP_List = []
G18_RES_NFP_List = []
G18_DIS_NFP_List = []
G18_LepBins_XVals_NFP_List = []
G18_Total_Count_NFP = 0
for value in pt_values:
    value_min = value
    value_max = value + 0.05
    df_gst_G18_02a_LepNFP = df_gst_G18_02a_all.Filter(f"ptl>{value_min} && ptl<{value_max} && abs(thetazl)<{lep_accept_angle}").Filter("num_pions==0")
    G18_LepPt_QEL_NFP = df_gst_G18_02a_LepNFP.Filter("qel==1").Histo1D(("G18_LepPt_QEL_NFP", "", bins, x_start, x_end), "ptl")
    G18_LepPt_MEC_NFP = df_gst_G18_02a_LepNFP.Filter("mec==1").Histo1D(("G18_LepPt_MEC_NFP", "", bins, x_start, x_end), "ptl")
    G18_LepPt_RES_NFP = df_gst_G18_02a_LepNFP.Filter("res==1").Histo1D(("G18_LepPt_RES_NFP", "", bins, x_start, x_end), "ptl")
    G18_LepPt_DIS_NFP = df_gst_G18_02a_LepNFP.Filter("dis==1").Histo1D(("G18_LepPt_DIS_NFP", "", bins, x_start, x_end), "ptl")
    G18_Int_NFP_List = [G18_LepPt_QEL_NFP.Integral(), G18_LepPt_MEC_NFP.Integral(), G18_LepPt_RES_NFP.Integral(), G18_LepPt_DIS_NFP.Integral()]
    G18_Total_Run_NFP = G18_Int_NFP_List[0] + G18_Int_NFP_List[1] + G18_Int_NFP_List[2] + G18_Int_NFP_List[3]
    G18_Events_NFP_List = [G18_QEL_NFP_List, G18_MEC_NFP_List, G18_RES_NFP_List, G18_DIS_NFP_List]
    G18_Total_Count_NFP += G18_Total_Run_NFP
    G18_MidPoint_NFP = (value_max - value_min)/2
    for i in range(len(G18_Int_NFP_List)):
        if G18_Total_Run_NFP < n_event_cut and G18_Events_NFP_List[i] == []:
            G18_Events_NFP_List[i].append(0.0)
        if G18_Total_Run_NFP > n_event_cut: 
            G18_Events_NFP_List[i].append(G18_Int_NFP_List[i]/G18_Total_Run_NFP)
            if G18_LepBins_XVals_NFP_List == []:
                G18_LepBins_XVals_NFP_List.append(G18_MidPoint_NFP)
            if (value_max - G18_MidPoint_NFP) not in G18_LepBins_XVals_NFP_List:
                G18_LepBins_XVals_NFP_List.append(value_max - G18_MidPoint_NFP)
G18_LepBins_XVals_NFP_Array = array.array('f', G18_LepBins_XVals_NFP_List)
G18_LepPt_QEL_NFP_Array = array.array('f', G18_QEL_NFP_List)
G18_LepPt_MEC_NFP_Array = array.array('f', G18_MEC_NFP_List)
G18_LepPt_RES_NFP_Array = array.array('f', G18_RES_NFP_List)
G18_LepPt_DIS_NFP_Array = array.array('f', G18_DIS_NFP_List)
G18_Yerr_QEL_NFP_List = []
G18_Yerr_MEC_NFP_List = []
G18_Yerr_RES_NFP_List = []
G18_Yerr_DIS_NFP_List = []
G18_Yerr_Events_NFP_Lists = [G18_Yerr_QEL_NFP_List, G18_Yerr_MEC_NFP_List, G18_Yerr_RES_NFP_List, G18_Yerr_DIS_NFP_List]
for event in G18_Events_NFP_List:
        for fraction in event:
            num = G18_Events_NFP_List.index(event)
            G18_Yerr_Events_NFP_Lists[num].append(sqrt(fraction*(1-fraction)/G18_Total_Count_NFP))
G18_Yerr_QEL_NFP_Array = array.array('f', G18_Yerr_QEL_NFP_List)
G18_Yerr_MEC_NFP_Array = array.array('f', G18_Yerr_MEC_NFP_List)
G18_Yerr_RES_NFP_Array = array.array('f', G18_Yerr_RES_NFP_List)
G18_Yerr_DIS_NFP_Array = array.array('f', G18_Yerr_DIS_NFP_List)

G18_LepPt_Sum_NFP = []
for i in range(len(G18_QEL_NFP_List)):
    count = 0
    count += G18_QEL_NFP_List[i] + G18_MEC_NFP_List[i] +  G18_RES_NFP_List[i] + G18_DIS_NFP_List[i]
    G18_LepPt_Sum_NFP.append(count)
G18_LepPt_Sum_NFP_Array = array.array('f', G18_LepPt_Sum_NFP)

In [ ]:
#g18 without NFAP pions v5
G18_QEL_NFAP_List = []
G18_MEC_NFAP_List = []
G18_RES_NFAP_List = []
G18_DIS_NFAP_List = []
G18_LepBins_XVals_NFAP_List = []
G18_Total_Count_NFAP = 0
for value in pt_values:
    value_min = value
    value_max = value + 0.05
    df_gst_G18_02a_LepNFAP = df_gst_G18_02a_all.Filter(f"ptl>{value_min} && ptl<{value_max} && abs(thetazl)<{lep_accept_angle}").Filter("(nfa_piplus+nfa_piminus+nfa_pi0)==0")
    G18_LepPt_QEL_NFAP = df_gst_G18_02a_LepNFAP.Filter("qel==1").Histo1D(("G18_LepPt_QEL_NFAP", "", bins, x_start, x_end), "ptl")
    G18_LepPt_MEC_NFAP = df_gst_G18_02a_LepNFAP.Filter("mec==1").Histo1D(("G18_LepPt_MEC_NFAP", "", bins, x_start, x_end), "ptl")
    G18_LepPt_RES_NFAP = df_gst_G18_02a_LepNFAP.Filter("res==1").Histo1D(("G18_LepPt_RES_NFAP", "", bins, x_start, x_end), "ptl")
    G18_LepPt_DIS_NFAP = df_gst_G18_02a_LepNFAP.Filter("dis==1").Histo1D(("G18_LepPt_DIS_NFAP", "", bins, x_start, x_end), "ptl")
    G18_Int_NFAP_List = [G18_LepPt_QEL_NFAP.Integral(), G18_LepPt_MEC_NFAP.Integral(), G18_LepPt_RES_NFAP.Integral(), G18_LepPt_DIS_NFAP.Integral()]
    G18_Total_Run_NFAP = G18_Int_NFAP_List[0] + G18_Int_NFAP_List[1] + G18_Int_NFAP_List[2] + G18_Int_NFAP_List[3]
    G18_Events_NFAP_List = [G18_QEL_NFAP_List, G18_MEC_NFAP_List, G18_RES_NFAP_List, G18_DIS_NFAP_List]
    G18_Total_Count_NFAP += G18_Total_Run_NFAP
    G18_MidPoint_NFAP = (value_max - value_min)/2
    for i in range(len(G18_Int_NFAP_List)):
        if G18_Total_Run_NFAP < n_event_cut and G18_Events_NFAP_List[i] == []:
            G18_Events_NFAP_List[i].append(0.0)
        if G18_Total_Run_NFAP > n_event_cut: 
            G18_Events_NFAP_List[i].append(G18_Int_NFAP_List[i]/G18_Total_Run_NFAP)
            if G18_LepBins_XVals_NFAP_List == []:
                G18_LepBins_XVals_NFAP_List.append(G18_MidPoint_NFAP)
            if (value_max - G18_MidPoint_NFAP) not in G18_LepBins_XVals_NFAP_List:
                G18_LepBins_XVals_NFAP_List.append(value_max - G18_MidPoint_NFAP)
G18_LepBins_XVals_NFAP_Array = array.array('f', G18_LepBins_XVals_NFAP_List)
G18_LepPt_QEL_NFAP_Array = array.array('f', G18_QEL_NFAP_List)
G18_LepPt_MEC_NFAP_Array = array.array('f', G18_MEC_NFAP_List)
G18_LepPt_RES_NFAP_Array = array.array('f', G18_RES_NFAP_List)
G18_LepPt_DIS_NFAP_Array = array.array('f', G18_DIS_NFAP_List)
G18_Yerr_QEL_NFAP_List = []
G18_Yerr_MEC_NFAP_List = []
G18_Yerr_RES_NFAP_List = []
G18_Yerr_DIS_NFAP_List = []
G18_Yerr_Events_NFAP_Lists = [G18_Yerr_QEL_NFAP_List, G18_Yerr_MEC_NFAP_List, G18_Yerr_RES_NFAP_List, G18_Yerr_DIS_NFAP_List]
for event in G18_Events_NFAP_List:
        for fraction in event:
            num = G18_Events_NFAP_List.index(event)
            G18_Yerr_Events_NFAP_Lists[num].append(sqrt(fraction*(1-fraction)/G18_Total_Count_NFAP))
G18_Yerr_QEL_NFAP_Array = array.array('f', G18_Yerr_QEL_NFAP_List)
G18_Yerr_MEC_NFAP_Array = array.array('f', G18_Yerr_MEC_NFAP_List)
G18_Yerr_RES_NFAP_Array = array.array('f', G18_Yerr_RES_NFAP_List)
G18_Yerr_DIS_NFAP_Array = array.array('f', G18_Yerr_DIS_NFAP_List)

G18_LepPt_Sum_NFAP = []
for i in range(len(G18_QEL_NFAP_List)):
    count = 0
    count += G18_QEL_NFAP_List[i] + G18_MEC_NFAP_List[i] +  G18_RES_NFAP_List[i] + G18_DIS_NFAP_List[i]
    G18_LepPt_Sum_NFAP.append(count)
G18_LepPt_Sum_NFAP_Array = array.array('f', G18_LepPt_Sum_NFAP)

In [ ]:
#gem21 arrays

In [ ]:
#gem21 without initial state pions v5
G21_QEL_NIP_List = []
G21_MEC_NIP_List = []
G21_RES_NIP_List = []
G21_DIS_NIP_List = []
G21_LepBins_XVals_NIP_List = []
G21_Total_Count_NIP = 0
for value in pt_values:
    value_min = value
    value_max = value + 0.05
    df_gst_GEM21_11b_LepNIP = df_gst_GEM21_11b_all.Filter(f"ptl>{value_min} && ptl<{value_max} && abs(thetazl)<{lep_accept_angle}").Filter("numi_pions==0")
    G21_LepPt_QEL_NIP = df_gst_GEM21_11b_LepNIP.Filter("qel==1").Histo1D(("G21_LepPt_QEL_NIP", "", bins, x_start, x_end), "ptl")
    G21_LepPt_MEC_NIP = df_gst_GEM21_11b_LepNIP.Filter("mec==1").Histo1D(("G21_LepPt_MEC_NIP", "", bins, x_start, x_end), "ptl")
    G21_LepPt_RES_NIP = df_gst_GEM21_11b_LepNIP.Filter("res==1").Histo1D(("G21_LepPt_RES_NIP", "", bins, x_start, x_end), "ptl")
    G21_LepPt_DIS_NIP = df_gst_GEM21_11b_LepNIP.Filter("dis==1").Histo1D(("G21_LepPt_DIS_NIP", "", bins, x_start, x_end), "ptl")
    G21_Int_NIP_List = [G21_LepPt_QEL_NIP.Integral(), G21_LepPt_MEC_NIP.Integral(), G21_LepPt_RES_NIP.Integral(), G21_LepPt_DIS_NIP.Integral()]
    G21_Total_Run_NIP = G21_Int_NIP_List[0] + G21_Int_NIP_List[1] + G21_Int_NIP_List[2] + G21_Int_NIP_List[3]
    G21_Events_NIP_List = [G21_QEL_NIP_List, G21_MEC_NIP_List, G21_RES_NIP_List, G21_DIS_NIP_List]
    G21_Total_Count_NIP += G21_Total_Run_NIP
    G21_MidPoint_NIP = (value_max - value_min)/2
    for i in range(len(G21_Int_NIP_List)):
        if G21_Total_Run_NIP < n_event_cut and G21_Events_NIP_List[i] == []:
            G21_Events_NIP_List[i].append(0.0)#adds zero to beginning of list when events are too low
        if G21_LepBins_XVals_NIP_List == []:
            G21_LepBins_XVals_NIP_List.append(G21_MidPoint_NIP)
        if G21_Total_Run_NIP > n_event_cut: 
            G21_Events_NIP_List[i].append(G21_Int_NIP_List[i]/G21_Total_Run_NIP)
            if (value_max - G21_MidPoint_NIP) not in G21_LepBins_XVals_NIP_List:
                G21_LepBins_XVals_NIP_List.append(value_max - G21_MidPoint_NIP)
G21_LepBins_XVals_NIP_Array = array.array('f', G21_LepBins_XVals_NIP_List)
G21_LepPt_QEL_NIP_Array = array.array('f', G21_QEL_NIP_List)
G21_LepPt_MEC_NIP_Array = array.array('f', G21_MEC_NIP_List)
G21_LepPt_RES_NIP_Array = array.array('f', G21_RES_NIP_List)
G21_LepPt_DIS_NIP_Array = array.array('f', G21_DIS_NIP_List)
G21_Yerr_QEL_NIP_List = []
G21_Yerr_MEC_NIP_List = []
G21_Yerr_RES_NIP_List = []
G21_Yerr_DIS_NIP_List = []
G21_Yerr_Events_NIP_Lists = [G21_Yerr_QEL_NIP_List, G21_Yerr_MEC_NIP_List, G21_Yerr_RES_NIP_List, G21_Yerr_DIS_NIP_List]
for event in G21_Events_NIP_List:
        for fraction in event:
            num = G21_Events_NIP_List.index(event)
            G21_Yerr_Events_NIP_Lists[num].append(sqrt(fraction*(1-fraction)/G21_Total_Count_NIP))
G21_Yerr_QEL_NIP_Array = array.array('f', G21_Yerr_QEL_NIP_List)
G21_Yerr_MEC_NIP_Array = array.array('f', G21_Yerr_MEC_NIP_List)
G21_Yerr_RES_NIP_Array = array.array('f', G21_Yerr_RES_NIP_List)
G21_Yerr_DIS_NIP_Array = array.array('f', G21_Yerr_DIS_NIP_List)

G21_LepPt_Sum_NIP = []
for i in range(len(G21_QEL_NIP_List)):
    count = 0
    count += G21_QEL_NIP_List[i] + G21_MEC_NIP_List[i] +  G21_RES_NIP_List[i] + G21_DIS_NIP_List[i]
    G21_LepPt_Sum_NIP.append(count)
G21_LepPt_Sum_NIP_Array = array.array('f', G21_LepPt_Sum_NIP)

In [ ]:
#gem21 without final state pions v5
G21_QEL_NFP_List = []
G21_MEC_NFP_List = []
G21_RES_NFP_List = []
G21_DIS_NFP_List = []
G21_LepBins_XVals_NFP_List = []
G21_Total_Count_NFP = 0
for value in pt_values:
    value_min = value
    value_max = value + 0.05
    df_gst_GEM21_11b_LepNFP = df_gst_GEM21_11b_all.Filter(f"ptl>{value_min} && ptl<{value_max} && abs(thetazl)<{lep_accept_angle}").Filter("num_pions==0")
    G21_LepPt_QEL_NFP = df_gst_GEM21_11b_LepNFP.Filter("qel==1").Histo1D(("G21_LepPt_QEL_NFP", "", bins, x_start, x_end), "ptl")
    G21_LepPt_MEC_NFP = df_gst_GEM21_11b_LepNFP.Filter("mec==1").Histo1D(("G21_LepPt_MEC_NFP", "", bins, x_start, x_end), "ptl")
    G21_LepPt_RES_NFP = df_gst_GEM21_11b_LepNFP.Filter("res==1").Histo1D(("G21_LepPt_RES_NFP", "", bins, x_start, x_end), "ptl")
    G21_LepPt_DIS_NFP = df_gst_GEM21_11b_LepNFP.Filter("dis==1").Histo1D(("G21_LepPt_DIS_NFP", "", bins, x_start, x_end), "ptl")
    G21_Int_NFP_List = [G21_LepPt_QEL_NFP.Integral(), G21_LepPt_MEC_NFP.Integral(), G21_LepPt_RES_NFP.Integral(), G21_LepPt_DIS_NFP.Integral()]
    G21_Total_Run_NFP = G21_Int_NFP_List[0] + G21_Int_NFP_List[1] + G21_Int_NFP_List[2] + G21_Int_NFP_List[3]
    G21_Events_NFP_List = [G21_QEL_NFP_List, G21_MEC_NFP_List, G21_RES_NFP_List, G21_DIS_NFP_List]
    G21_Total_Count_NFP += G21_Total_Run_NFP
    G21_MidPoint_NFP = (value_max - value_min)/2
    for i in range(len(G21_Int_NFP_List)):
        if G21_Total_Run_NFP < n_event_cut and G21_Events_NFP_List[i] == []:
            G21_Events_NFP_List[i].append(0.0)
        if G21_Total_Run_NFP > n_event_cut: 
            G21_Events_NFP_List[i].append(G21_Int_NFP_List[i]/G21_Total_Run_NFP)
            if G21_LepBins_XVals_NFP_List == []:
                G21_LepBins_XVals_NFP_List.append(G21_MidPoint_NFP)
            if (value_max - G21_MidPoint_NFP) not in G21_LepBins_XVals_NFP_List:
                G21_LepBins_XVals_NFP_List.append(value_max - G21_MidPoint_NFP)
G21_LepBins_XVals_NFP_Array = array.array('f', G21_LepBins_XVals_NFP_List)
G21_LepPt_QEL_NFP_Array = array.array('f', G21_QEL_NFP_List)
G21_LepPt_MEC_NFP_Array = array.array('f', G21_MEC_NFP_List)
G21_LepPt_RES_NFP_Array = array.array('f', G21_RES_NFP_List)
G21_LepPt_DIS_NFP_Array = array.array('f', G21_DIS_NFP_List)
G21_Yerr_QEL_NFP_List = []
G21_Yerr_MEC_NFP_List = []
G21_Yerr_RES_NFP_List = []
G21_Yerr_DIS_NFP_List = []
G21_Yerr_Events_NFP_Lists = [G21_Yerr_QEL_NFP_List, G21_Yerr_MEC_NFP_List, G21_Yerr_RES_NFP_List, G21_Yerr_DIS_NFP_List]
for event in G21_Events_NFP_List:
        for fraction in event:
            num = G21_Events_NFP_List.index(event)
            G21_Yerr_Events_NFP_Lists[num].append(sqrt(fraction*(1-fraction)/G21_Total_Count_NFP))
G21_Yerr_QEL_NFP_Array = array.array('f', G21_Yerr_QEL_NFP_List)
G21_Yerr_MEC_NFP_Array = array.array('f', G21_Yerr_MEC_NFP_List)
G21_Yerr_RES_NFP_Array = array.array('f', G21_Yerr_RES_NFP_List)
G21_Yerr_DIS_NFP_Array = array.array('f', G21_Yerr_DIS_NFP_List)

G21_LepPt_Sum_NFP = []
for i in range(len(G21_QEL_NFP_List)):
    count = 0
    count += G21_QEL_NFP_List[i] + G21_MEC_NFP_List[i] +  G21_RES_NFP_List[i] + G21_DIS_NFP_List[i]
    G21_LepPt_Sum_NFP.append(count)
G21_LepPt_Sum_NFP_Array = array.array('f', G21_LepPt_Sum_NFP)

In [ ]:
#gem21 without NFAP pions v5
G21_QEL_NFAP_List = []
G21_MEC_NFAP_List = []
G21_RES_NFAP_List = []
G21_DIS_NFAP_List = []
G21_LepBins_XVals_NFAP_List = []
G21_Total_Count_NFAP = 0
for value in pt_values:
    value_min = value
    value_max = value + 0.05
    df_gst_GEM21_11b_LepNFAP = df_gst_GEM21_11b_all.Filter(f"ptl>{value_min} && ptl<{value_max} && abs(thetazl)<{lep_accept_angle}").Filter("(nfa_piplus+nfa_piminus+nfa_pi0)==0")
    G21_LepPt_QEL_NFAP = df_gst_GEM21_11b_LepNFAP.Filter("qel==1").Histo1D(("G21_LepPt_QEL_NFAP", "", bins, x_start, x_end), "ptl")
    G21_LepPt_MEC_NFAP = df_gst_GEM21_11b_LepNFAP.Filter("mec==1").Histo1D(("G21_LepPt_MEC_NFAP", "", bins, x_start, x_end), "ptl")
    G21_LepPt_RES_NFAP = df_gst_GEM21_11b_LepNFAP.Filter("res==1").Histo1D(("G21_LepPt_RES_NFAP", "", bins, x_start, x_end), "ptl")
    G21_LepPt_DIS_NFAP = df_gst_GEM21_11b_LepNFAP.Filter("dis==1").Histo1D(("G21_LepPt_DIS_NFAP", "", bins, x_start, x_end), "ptl")
    G21_Int_NFAP_List = [G21_LepPt_QEL_NFAP.Integral(), G21_LepPt_MEC_NFAP.Integral(), G21_LepPt_RES_NFAP.Integral(), G21_LepPt_DIS_NFAP.Integral()]
    G21_Total_Run_NFAP = G21_Int_NFAP_List[0] + G21_Int_NFAP_List[1] + G21_Int_NFAP_List[2] + G21_Int_NFAP_List[3]
    G21_Events_NFAP_List = [G21_QEL_NFAP_List, G21_MEC_NFAP_List, G21_RES_NFAP_List, G21_DIS_NFAP_List]
    G21_Total_Count_NFAP += G21_Total_Run_NFAP
    G21_MidPoint_NFAP = (value_max - value_min)/2
    for i in range(len(G21_Int_NFAP_List)):
        if G21_Total_Run_NFAP < n_event_cut and G21_Events_NFAP_List[i] == []:
            G21_Events_NFAP_List[i].append(0.0)
        if G21_Total_Run_NFAP > n_event_cut: 
            G21_Events_NFAP_List[i].append(G21_Int_NFAP_List[i]/G21_Total_Run_NFAP)
            if G21_LepBins_XVals_NFAP_List == []:
                G21_LepBins_XVals_NFAP_List.append(G21_MidPoint_NFAP)
            if (value_max - G21_MidPoint_NFAP) not in G21_LepBins_XVals_NFAP_List:
                G21_LepBins_XVals_NFAP_List.append(value_max - G21_MidPoint_NFAP)
G21_LepBins_XVals_NFAP_Array = array.array('f', G21_LepBins_XVals_NFAP_List)
G21_LepPt_QEL_NFAP_Array = array.array('f', G21_QEL_NFAP_List)
G21_LepPt_MEC_NFAP_Array = array.array('f', G21_MEC_NFAP_List)
G21_LepPt_RES_NFAP_Array = array.array('f', G21_RES_NFAP_List)
G21_LepPt_DIS_NFAP_Array = array.array('f', G21_DIS_NFAP_List)
G21_Yerr_QEL_NFAP_List = []
G21_Yerr_MEC_NFAP_List = []
G21_Yerr_RES_NFAP_List = []
G21_Yerr_DIS_NFAP_List = []
G21_Yerr_Events_NFAP_Lists = [G21_Yerr_QEL_NFAP_List, G21_Yerr_MEC_NFAP_List, G21_Yerr_RES_NFAP_List, G21_Yerr_DIS_NFAP_List]
for event in G21_Events_NFAP_List:
        for fraction in event:
            num = G21_Events_NFAP_List.index(event)
            G21_Yerr_Events_NFAP_Lists[num].append(sqrt(fraction*(1-fraction)/G21_Total_Count_NFAP))
G21_Yerr_QEL_NFAP_Array = array.array('f', G21_Yerr_QEL_NFAP_List)
G21_Yerr_MEC_NFAP_Array = array.array('f', G21_Yerr_MEC_NFAP_List)
G21_Yerr_RES_NFAP_Array = array.array('f', G21_Yerr_RES_NFAP_List)
G21_Yerr_DIS_NFAP_Array = array.array('f', G21_Yerr_DIS_NFAP_List)

G21_LepPt_Sum_NFAP = []
for i in range(len(G21_QEL_NFAP_List)):
    count = 0
    count += G21_QEL_NFAP_List[i] + G21_MEC_NFAP_List[i] +  G21_RES_NFAP_List[i] + G21_DIS_NFAP_List[i]
    G21_LepPt_Sum_NFAP.append(count)
G21_LepPt_Sum_NFAP_Array = array.array('f', G21_LepPt_Sum_NFAP)

In [ ]:
#plots for lepton pt cut binning

In [ ]:
#G18 without initial state pions v5
G18_LepCut_NIP_Canvas = ROOT.TCanvas()

G18_LepCut_QEL_NIP_Plot = ROOT.TGraph(len(G18_LepBins_XVals_NIP_Array), G18_LepBins_XVals_NIP_Array, G18_LepPt_QEL_NIP_Array)
G18_LepCut_QEL_NIP_Plot.SetTitle("Relative Fraction of Interaction Type Per Lepton PT Cut w/o Initial State Pions in G18 Model")
G18_LepCut_QEL_NIP_Plot.GetXaxis().SetTitle("Lepton PT (GeV)")
G18_LepCut_QEL_NIP_Plot.GetYaxis().SetTitle("Percentage")
G18_LepCut_QEL_NIP_Plot.SetMaximum(1.1)
G18_LepCut_QEL_NIP_Plot.SetMinimum(0.0)
G18_LepCut_QEL_NIP_Plot.SetMarkerColor(4)
G18_LepCut_QEL_NIP_Plot.SetMarkerStyle(29)
G18_LepCut_QEL_NIP_Plot.Draw("same")

G18_LepCut_MEC_NIP_Plot = ROOT.TGraph(len(G18_LepBins_XVals_NIP_Array), G18_LepBins_XVals_NIP_Array, G18_LepPt_MEC_NIP_Array)
G18_LepCut_MEC_NIP_Plot.Draw("same")
G18_LepCut_MEC_NIP_Plot.SetMarkerColor(2)
G18_LepCut_MEC_NIP_Plot.SetMarkerStyle(29)

G18_LepCut_RES_NIP_Plot = ROOT.TGraph(len(G18_LepBins_XVals_NIP_Array), G18_LepBins_XVals_NIP_Array, G18_LepPt_RES_NIP_Array)
G18_LepCut_RES_NIP_Plot.Draw("same")
G18_LepCut_RES_NIP_Plot.SetMarkerColor(3)
G18_LepCut_RES_NIP_Plot.SetMarkerStyle(29)

G18_LepCut_DIS_NIP_Plot = ROOT.TGraph(len(G18_LepBins_XVals_NIP_Array), G18_LepBins_XVals_NIP_Array, G18_LepPt_DIS_NIP_Array)
G18_LepCut_DIS_NIP_Plot.Draw("same")
G18_LepCut_DIS_NIP_Plot.SetMarkerColor(6)
G18_LepCut_DIS_NIP_Plot.SetMarkerStyle(29)

G18_LepCut_Sum_NIP_Plot = ROOT.TGraph(len(G18_LepBins_XVals_NIP_Array), G18_LepBins_XVals_NIP_Array, G18_LepPt_Sum_NIP_Array)
G18_LepCut_Sum_NIP_Plot.Draw("same")
G18_LepCut_Sum_NIP_Plot.SetMarkerColor(1)
G18_LepCut_Sum_NIP_Plot.SetMarkerStyle(34)

G18_LepCut_Leg_NIP = ROOT.TLegend(0.2, 0.85, 0.80, 0.90)
G18_LepCut_Leg_NIP.SetNColumns(5)
G18_LepCut_Leg_NIP.AddEntry(G18_LepCut_QEL_NIP_Plot, "QEL")
G18_LepCut_Leg_NIP.AddEntry(G18_LepCut_MEC_NIP_Plot, "MEC")
G18_LepCut_Leg_NIP.AddEntry(G18_LepCut_RES_NIP_Plot, "RES")
G18_LepCut_Leg_NIP.AddEntry(G18_LepCut_DIS_NIP_Plot, "DIS")
G18_LepCut_Leg_NIP.AddEntry(G18_LepCut_Sum_NIP_Plot, "SUM")
G18_LepCut_Leg_NIP.Draw()

G18_LepCut_Zeroes_NIP = []
for i in range(len(G18_LepBins_XVals_NIP_Array)):
    G18_LepCut_Zeroes_NIP.append(0.025)
G18_LepCut_Zeroes_NIP_array = array.array('f', G18_LepCut_Zeroes_NIP)

G18_Yerr_QEL_NIP_Bars = ROOT.TGraphErrors(len(G18_LepBins_XVals_NIP_Array), G18_LepBins_XVals_NIP_Array, G18_LepPt_QEL_NIP_Array, G18_LepCut_Zeroes_NIP_array, G18_Yerr_QEL_NIP_Array)
G18_Yerr_MEC_NIP_Bars = ROOT.TGraphErrors(len(G18_LepBins_XVals_NIP_Array), G18_LepBins_XVals_NIP_Array, G18_LepPt_MEC_NIP_Array, G18_LepCut_Zeroes_NIP_array, G18_Yerr_MEC_NIP_Array)
G18_Yerr_RES_NIP_Bars = ROOT.TGraphErrors(len(G18_LepBins_XVals_NIP_Array), G18_LepBins_XVals_NIP_Array, G18_LepPt_RES_NIP_Array, G18_LepCut_Zeroes_NIP_array, G18_Yerr_RES_NIP_Array)
G18_Yerr_DIS_NIP_Bars = ROOT.TGraphErrors(len(G18_LepBins_XVals_NIP_Array), G18_LepBins_XVals_NIP_Array, G18_LepPt_DIS_NIP_Array, G18_LepCut_Zeroes_NIP_array, G18_Yerr_DIS_NIP_Array)
G18_Yerr_QEL_NIP_Bars.Draw("same")
G18_Yerr_MEC_NIP_Bars.Draw("same")
G18_Yerr_RES_NIP_Bars.Draw("same")
G18_Yerr_DIS_NIP_Bars.Draw("same")

G18_LepCut_NIP_Canvas.Draw()

In [ ]:
#G18 without final state pions v5
G18_LepCut_NFP_Canvas = ROOT.TCanvas()

G18_LepCut_QEL_NFP_Plot = ROOT.TGraph(len(G18_LepBins_XVals_NFP_Array), G18_LepBins_XVals_NFP_Array, G18_LepPt_QEL_NFP_Array)
G18_LepCut_QEL_NFP_Plot.SetTitle("Relative Fraction of Interaction Type Per Lepton PT Cut w/o Final State Pions in G18 Model")
G18_LepCut_QEL_NFP_Plot.GetXaxis().SetTitle("Lepton PT (GeV)")
G18_LepCut_QEL_NFP_Plot.GetYaxis().SetTitle("Percentage")
G18_LepCut_QEL_NFP_Plot.SetMaximum(1.1)
G18_LepCut_QEL_NFP_Plot.SetMinimum(0.0)
G18_LepCut_QEL_NFP_Plot.SetMarkerColor(4)
G18_LepCut_QEL_NFP_Plot.SetMarkerStyle(29)
G18_LepCut_QEL_NFP_Plot.Draw("same")

G18_LepCut_MEC_NFP_Plot = ROOT.TGraph(len(G18_LepBins_XVals_NFP_Array), G18_LepBins_XVals_NFP_Array, G18_LepPt_MEC_NFP_Array)
G18_LepCut_MEC_NFP_Plot.Draw("same")
G18_LepCut_MEC_NFP_Plot.SetMarkerColor(2)
G18_LepCut_MEC_NFP_Plot.SetMarkerStyle(29)

G18_LepCut_RES_NFP_Plot = ROOT.TGraph(len(G18_LepBins_XVals_NFP_Array), G18_LepBins_XVals_NFP_Array, G18_LepPt_RES_NFP_Array)
G18_LepCut_RES_NFP_Plot.Draw("same")
G18_LepCut_RES_NFP_Plot.SetMarkerColor(3)
G18_LepCut_RES_NFP_Plot.SetMarkerStyle(29)

G18_LepCut_DIS_NFP_Plot = ROOT.TGraph(len(G18_LepBins_XVals_NFP_Array), G18_LepBins_XVals_NFP_Array, G18_LepPt_DIS_NFP_Array)
G18_LepCut_DIS_NFP_Plot.Draw("same")
G18_LepCut_DIS_NFP_Plot.SetMarkerColor(6)
G18_LepCut_DIS_NFP_Plot.SetMarkerStyle(29)

G18_LepCut_Sum_NFP_Plot = ROOT.TGraph(len(G18_LepBins_XVals_NFP_Array), G18_LepBins_XVals_NFP_Array, G18_LepPt_Sum_NFP_Array)
G18_LepCut_Sum_NFP_Plot.Draw("same")
G18_LepCut_Sum_NFP_Plot.SetMarkerColor(1)
G18_LepCut_Sum_NFP_Plot.SetMarkerStyle(34)

G18_LepCut_Leg_NFP = ROOT.TLegend(0.2, 0.85, 0.80, 0.90)
G18_LepCut_Leg_NFP.SetNColumns(5)
G18_LepCut_Leg_NFP.AddEntry(G18_LepCut_QEL_NFP_Plot, "QEL")
G18_LepCut_Leg_NFP.AddEntry(G18_LepCut_MEC_NFP_Plot, "MEC")
G18_LepCut_Leg_NFP.AddEntry(G18_LepCut_RES_NFP_Plot, "RES")
G18_LepCut_Leg_NFP.AddEntry(G18_LepCut_DIS_NFP_Plot, "DIS")
G18_LepCut_Leg_NFP.AddEntry(G18_LepCut_Sum_NFP_Plot, "SUM")
G18_LepCut_Leg_NFP.Draw()

G18_LepCut_Zeroes_NFP = []
for i in range(len(G18_LepBins_XVals_NFP_Array)):
    G18_LepCut_Zeroes_NFP.append(0.025)
G18_LepCut_Zeroes_NFP_array = array.array('f', G18_LepCut_Zeroes_NFP)

G18_Yerr_QEL_NFP_Bars = ROOT.TGraphErrors(len(G18_LepBins_XVals_NFP_Array), G18_LepBins_XVals_NFP_Array, G18_LepPt_QEL_NFP_Array, G18_LepCut_Zeroes_NFP_array, G18_Yerr_QEL_NFP_Array)
G18_Yerr_MEC_NFP_Bars = ROOT.TGraphErrors(len(G18_LepBins_XVals_NFP_Array), G18_LepBins_XVals_NFP_Array, G18_LepPt_MEC_NFP_Array, G18_LepCut_Zeroes_NFP_array, G18_Yerr_MEC_NFP_Array)
G18_Yerr_RES_NFP_Bars = ROOT.TGraphErrors(len(G18_LepBins_XVals_NFP_Array), G18_LepBins_XVals_NFP_Array, G18_LepPt_RES_NFP_Array, G18_LepCut_Zeroes_NFP_array, G18_Yerr_RES_NFP_Array)
G18_Yerr_DIS_NFP_Bars = ROOT.TGraphErrors(len(G18_LepBins_XVals_NFP_Array), G18_LepBins_XVals_NFP_Array, G18_LepPt_DIS_NFP_Array, G18_LepCut_Zeroes_NFP_array, G18_Yerr_DIS_NFP_Array)
G18_Yerr_QEL_NFP_Bars.Draw("same")
G18_Yerr_MEC_NFP_Bars.Draw("same")
G18_Yerr_RES_NFP_Bars.Draw("same")
G18_Yerr_DIS_NFP_Bars.Draw("same")

G18_LepCut_NFP_Canvas.Draw()

In [ ]:
#G18 without nfa pions v5
G18_LepCut_NFAP_Canvas = ROOT.TCanvas()

G18_LepCut_QEL_NFAP_Plot = ROOT.TGraph(len(G18_LepBins_XVals_NFAP_Array), G18_LepBins_XVals_NFAP_Array, G18_LepPt_QEL_NFAP_Array)
G18_LepCut_QEL_NFAP_Plot.SetTitle("Relative Fraction of Interaction Type Per Lepton PT Cut w/o NFA Pions in G18 Model")
G18_LepCut_QEL_NFAP_Plot.GetXaxis().SetTitle("Lepton PT (GeV)")
G18_LepCut_QEL_NFAP_Plot.GetYaxis().SetTitle("Percentage")
G18_LepCut_QEL_NFAP_Plot.SetMaximum(1.1)
G18_LepCut_QEL_NFAP_Plot.SetMinimum(0.0)
G18_LepCut_QEL_NFAP_Plot.SetMarkerColor(4)
G18_LepCut_QEL_NFAP_Plot.SetMarkerStyle(29)
G18_LepCut_QEL_NFAP_Plot.Draw("same")

G18_LepCut_MEC_NFAP_Plot = ROOT.TGraph(len(G18_LepBins_XVals_NFAP_Array), G18_LepBins_XVals_NFAP_Array, G18_LepPt_MEC_NFAP_Array)
G18_LepCut_MEC_NFAP_Plot.Draw("same")
G18_LepCut_MEC_NFAP_Plot.SetMarkerColor(2)
G18_LepCut_MEC_NFAP_Plot.SetMarkerStyle(29)

G18_LepCut_RES_NFAP_Plot = ROOT.TGraph(len(G18_LepBins_XVals_NFAP_Array), G18_LepBins_XVals_NFAP_Array, G18_LepPt_RES_NFAP_Array)
G18_LepCut_RES_NFAP_Plot.Draw("same")
G18_LepCut_RES_NFAP_Plot.SetMarkerColor(3)
G18_LepCut_RES_NFAP_Plot.SetMarkerStyle(29)

G18_LepCut_DIS_NFAP_Plot = ROOT.TGraph(len(G18_LepBins_XVals_NFAP_Array), G18_LepBins_XVals_NFAP_Array, G18_LepPt_DIS_NFAP_Array)
G18_LepCut_DIS_NFAP_Plot.Draw("same")
G18_LepCut_DIS_NFAP_Plot.SetMarkerColor(6)
G18_LepCut_DIS_NFAP_Plot.SetMarkerStyle(29)

G18_LepCut_Sum_NFAP_Plot = ROOT.TGraph(len(G18_LepBins_XVals_NFAP_Array), G18_LepBins_XVals_NFAP_Array, G18_LepPt_Sum_NFAP_Array)
G18_LepCut_Sum_NFAP_Plot.Draw("same")
G18_LepCut_Sum_NFAP_Plot.SetMarkerColor(1)
G18_LepCut_Sum_NFAP_Plot.SetMarkerStyle(34)

G18_LepCut_Leg_NFAP = ROOT.TLegend(0.2, 0.85, 0.80, 0.90)
G18_LepCut_Leg_NFAP.SetNColumns(5)
G18_LepCut_Leg_NFAP.AddEntry(G18_LepCut_QEL_NFAP_Plot, "QEL")
G18_LepCut_Leg_NFAP.AddEntry(G18_LepCut_MEC_NFAP_Plot, "MEC")
G18_LepCut_Leg_NFAP.AddEntry(G18_LepCut_RES_NFAP_Plot, "RES")
G18_LepCut_Leg_NFAP.AddEntry(G18_LepCut_DIS_NFAP_Plot, "DIS")
G18_LepCut_Leg_NFAP.AddEntry(G18_LepCut_Sum_NFAP_Plot, "SUM")
G18_LepCut_Leg_NFAP.Draw()

G18_LepCut_Zeroes_NFAP = []
for i in range(len(G18_LepBins_XVals_NFAP_Array)):
    G18_LepCut_Zeroes_NFAP.append(0.025)
G18_LepCut_Zeroes_NFAP_array = array.array('f', G18_LepCut_Zeroes_NFAP)

G18_Yerr_QEL_NFAP_Bars = ROOT.TGraphErrors(len(G18_LepBins_XVals_NFAP_Array), G18_LepBins_XVals_NFAP_Array, G18_LepPt_QEL_NFAP_Array, G18_LepCut_Zeroes_NFAP_array, G18_Yerr_QEL_NFAP_Array)
G18_Yerr_MEC_NFAP_Bars = ROOT.TGraphErrors(len(G18_LepBins_XVals_NFAP_Array), G18_LepBins_XVals_NFAP_Array, G18_LepPt_MEC_NFAP_Array, G18_LepCut_Zeroes_NFAP_array, G18_Yerr_MEC_NFAP_Array)
G18_Yerr_RES_NFAP_Bars = ROOT.TGraphErrors(len(G18_LepBins_XVals_NFAP_Array), G18_LepBins_XVals_NFAP_Array, G18_LepPt_RES_NFAP_Array, G18_LepCut_Zeroes_NFAP_array, G18_Yerr_RES_NFAP_Array)
G18_Yerr_DIS_NFAP_Bars = ROOT.TGraphErrors(len(G18_LepBins_XVals_NFAP_Array), G18_LepBins_XVals_NFAP_Array, G18_LepPt_DIS_NFAP_Array, G18_LepCut_Zeroes_NFAP_array, G18_Yerr_DIS_NFAP_Array)
G18_Yerr_QEL_NFAP_Bars.Draw("same")
G18_Yerr_MEC_NFAP_Bars.Draw("same")
G18_Yerr_RES_NFAP_Bars.Draw("same")
G18_Yerr_DIS_NFAP_Bars.Draw("same")

G18_LepCut_NFAP_Canvas.Draw()

In [ ]:
#gem21 plots

In [ ]:
#GEM21 without initial state pions v5
G21_LepCut_NIP_Canvas = ROOT.TCanvas()

G21_LepCut_QEL_NIP_Plot = ROOT.TGraph(len(G21_LepBins_XVals_NIP_Array), G21_LepBins_XVals_NIP_Array, G21_LepPt_QEL_NIP_Array)
G21_LepCut_QEL_NIP_Plot.SetTitle("Relative Fraction of Interaction Type Per Lepton PT Cut w/o Initial State Pions in GEM21 Model")
G21_LepCut_QEL_NIP_Plot.GetXaxis().SetTitle("Lepton PT (GeV)")
G21_LepCut_QEL_NIP_Plot.GetYaxis().SetTitle("Percentage")
G21_LepCut_QEL_NIP_Plot.SetMaximum(1.1)
G21_LepCut_QEL_NIP_Plot.SetMinimum(0.0)
G21_LepCut_QEL_NIP_Plot.SetMarkerColor(4)
G21_LepCut_QEL_NIP_Plot.SetMarkerStyle(29)
G21_LepCut_QEL_NIP_Plot.Draw("same")

G21_LepCut_MEC_NIP_Plot = ROOT.TGraph(len(G21_LepBins_XVals_NIP_Array), G21_LepBins_XVals_NIP_Array, G21_LepPt_MEC_NIP_Array)
G21_LepCut_MEC_NIP_Plot.Draw("same")
G21_LepCut_MEC_NIP_Plot.SetMarkerColor(2)
G21_LepCut_MEC_NIP_Plot.SetMarkerStyle(29)

G21_LepCut_RES_NIP_Plot = ROOT.TGraph(len(G21_LepBins_XVals_NIP_Array), G21_LepBins_XVals_NIP_Array, G21_LepPt_RES_NIP_Array)
G21_LepCut_RES_NIP_Plot.Draw("same")
G21_LepCut_RES_NIP_Plot.SetMarkerColor(3)
G21_LepCut_RES_NIP_Plot.SetMarkerStyle(29)

G21_LepCut_DIS_NIP_Plot = ROOT.TGraph(len(G21_LepBins_XVals_NIP_Array), G21_LepBins_XVals_NIP_Array, G21_LepPt_DIS_NIP_Array)
G21_LepCut_DIS_NIP_Plot.Draw("same")
G21_LepCut_DIS_NIP_Plot.SetMarkerColor(6)
G21_LepCut_DIS_NIP_Plot.SetMarkerStyle(29)

G21_LepCut_Sum_NIP_Plot = ROOT.TGraph(len(G21_LepBins_XVals_NIP_Array), G21_LepBins_XVals_NIP_Array, G21_LepPt_Sum_NIP_Array)
G21_LepCut_Sum_NIP_Plot.Draw("same")
G21_LepCut_Sum_NIP_Plot.SetMarkerColor(1)
G21_LepCut_Sum_NIP_Plot.SetMarkerStyle(34)

G21_LepCut_Leg_NIP = ROOT.TLegend(0.2, 0.85, 0.80, 0.90)
G21_LepCut_Leg_NIP.SetNColumns(5)
G21_LepCut_Leg_NIP.AddEntry(G21_LepCut_QEL_NIP_Plot, "QEL")
G21_LepCut_Leg_NIP.AddEntry(G21_LepCut_MEC_NIP_Plot, "MEC")
G21_LepCut_Leg_NIP.AddEntry(G21_LepCut_RES_NIP_Plot, "RES")
G21_LepCut_Leg_NIP.AddEntry(G21_LepCut_DIS_NIP_Plot, "DIS")
G21_LepCut_Leg_NIP.AddEntry(G21_LepCut_Sum_NIP_Plot, "SUM")
G21_LepCut_Leg_NIP.Draw()

G21_LepCut_Zeroes_NIP = []
for i in range(len(G21_LepBins_XVals_NIP_Array)):
    G21_LepCut_Zeroes_NIP.append(0.025)
G21_LepCut_Zeroes_NIP_array = array.array('f', G21_LepCut_Zeroes_NIP)

G21_Yerr_QEL_NIP_Bars = ROOT.TGraphErrors(len(G21_LepBins_XVals_NIP_Array), G21_LepBins_XVals_NIP_Array, G21_LepPt_QEL_NIP_Array, G21_LepCut_Zeroes_NIP_array, G21_Yerr_QEL_NIP_Array)
G21_Yerr_MEC_NIP_Bars = ROOT.TGraphErrors(len(G21_LepBins_XVals_NIP_Array), G21_LepBins_XVals_NIP_Array, G21_LepPt_MEC_NIP_Array, G21_LepCut_Zeroes_NIP_array, G21_Yerr_MEC_NIP_Array)
G21_Yerr_RES_NIP_Bars = ROOT.TGraphErrors(len(G21_LepBins_XVals_NIP_Array), G21_LepBins_XVals_NIP_Array, G21_LepPt_RES_NIP_Array, G21_LepCut_Zeroes_NIP_array, G21_Yerr_RES_NIP_Array)
G21_Yerr_DIS_NIP_Bars = ROOT.TGraphErrors(len(G21_LepBins_XVals_NIP_Array), G21_LepBins_XVals_NIP_Array, G21_LepPt_DIS_NIP_Array, G21_LepCut_Zeroes_NIP_array, G21_Yerr_DIS_NIP_Array)
G21_Yerr_QEL_NIP_Bars.Draw("same")
G21_Yerr_MEC_NIP_Bars.Draw("same")
G21_Yerr_RES_NIP_Bars.Draw("same")
G21_Yerr_DIS_NIP_Bars.Draw("same")

G21_LepCut_NIP_Canvas.Draw()

In [ ]:
#GEM21 without final state pions v5
G21_LepCut_NFP_Canvas = ROOT.TCanvas()

G21_LepCut_QEL_NFP_Plot = ROOT.TGraph(len(G21_LepBins_XVals_NFP_Array), G21_LepBins_XVals_NFP_Array, G21_LepPt_QEL_NFP_Array)
G21_LepCut_QEL_NFP_Plot.SetTitle("Relative Fraction of Interaction Type Per Lepton PT Cut w/o Final State Pions in GEM21 Model")
G21_LepCut_QEL_NFP_Plot.GetXaxis().SetTitle("Lepton PT (GeV)")
G21_LepCut_QEL_NFP_Plot.GetYaxis().SetTitle("Percentage")
G21_LepCut_QEL_NFP_Plot.SetMaximum(1.1)
G21_LepCut_QEL_NFP_Plot.SetMinimum(0.0)
G21_LepCut_QEL_NFP_Plot.SetMarkerColor(4)
G21_LepCut_QEL_NFP_Plot.SetMarkerStyle(29)
G21_LepCut_QEL_NFP_Plot.Draw("same")

G21_LepCut_MEC_NFP_Plot = ROOT.TGraph(len(G21_LepBins_XVals_NFP_Array), G21_LepBins_XVals_NFP_Array, G21_LepPt_MEC_NFP_Array)
G21_LepCut_MEC_NFP_Plot.Draw("same")
G21_LepCut_MEC_NFP_Plot.SetMarkerColor(2)
G21_LepCut_MEC_NFP_Plot.SetMarkerStyle(29)

G21_LepCut_RES_NFP_Plot = ROOT.TGraph(len(G21_LepBins_XVals_NFP_Array), G21_LepBins_XVals_NFP_Array, G21_LepPt_RES_NFP_Array)
G21_LepCut_RES_NFP_Plot.Draw("same")
G21_LepCut_RES_NFP_Plot.SetMarkerColor(3)
G21_LepCut_RES_NFP_Plot.SetMarkerStyle(29)

G21_LepCut_DIS_NFP_Plot = ROOT.TGraph(len(G21_LepBins_XVals_NFP_Array), G21_LepBins_XVals_NFP_Array, G21_LepPt_DIS_NFP_Array)
G21_LepCut_DIS_NFP_Plot.Draw("same")
G21_LepCut_DIS_NFP_Plot.SetMarkerColor(6)
G21_LepCut_DIS_NFP_Plot.SetMarkerStyle(29)

G18_LepCut_Sum_NFP_Plot = ROOT.TGraph(len(G18_LepBins_XVals_NFP_Array), G18_LepBins_XVals_NFP_Array, G18_LepPt_Sum_NFP_Array)
G18_LepCut_Sum_NFP_Plot.Draw("same")
G18_LepCut_Sum_NFP_Plot.SetMarkerColor(1)
G18_LepCut_Sum_NFP_Plot.SetMarkerStyle(34)

G21_LepCut_Leg_NFP = ROOT.TLegend(0.2, 0.85, 0.80, 0.90)
G21_LepCut_Leg_NFP.SetNColumns(5)
G21_LepCut_Leg_NFP.AddEntry(G21_LepCut_QEL_NFP_Plot, "QEL")
G21_LepCut_Leg_NFP.AddEntry(G21_LepCut_MEC_NFP_Plot, "MEC")
G21_LepCut_Leg_NFP.AddEntry(G21_LepCut_RES_NFP_Plot, "RES")
G21_LepCut_Leg_NFP.AddEntry(G21_LepCut_DIS_NFP_Plot, "DIS")
G18_LepCut_Leg_NFP.AddEntry(G18_LepCut_Sum_NFP_Plot, "SUM")
G21_LepCut_Leg_NFP.Draw()

G21_LepCut_Zeroes_NFP = []
for i in range(len(G21_LepBins_XVals_NFP_Array)):
    G21_LepCut_Zeroes_NFP.append(0.025)
G21_LepCut_Zeroes_NFP_array = array.array('f', G21_LepCut_Zeroes_NFP)

G21_Yerr_QEL_NFP_Bars = ROOT.TGraphErrors(len(G21_LepBins_XVals_NFP_Array), G21_LepBins_XVals_NFP_Array, G21_LepPt_QEL_NFP_Array, G21_LepCut_Zeroes_NFP_array, G21_Yerr_QEL_NFP_Array)
G21_Yerr_MEC_NFP_Bars = ROOT.TGraphErrors(len(G21_LepBins_XVals_NFP_Array), G21_LepBins_XVals_NFP_Array, G21_LepPt_MEC_NFP_Array, G21_LepCut_Zeroes_NFP_array, G21_Yerr_MEC_NFP_Array)
G21_Yerr_RES_NFP_Bars = ROOT.TGraphErrors(len(G21_LepBins_XVals_NFP_Array), G21_LepBins_XVals_NFP_Array, G21_LepPt_RES_NFP_Array, G21_LepCut_Zeroes_NFP_array, G21_Yerr_RES_NFP_Array)
G21_Yerr_DIS_NFP_Bars = ROOT.TGraphErrors(len(G21_LepBins_XVals_NFP_Array), G21_LepBins_XVals_NFP_Array, G21_LepPt_DIS_NFP_Array, G21_LepCut_Zeroes_NFP_array, G21_Yerr_DIS_NFP_Array)
G21_Yerr_QEL_NFP_Bars.Draw("same")
G21_Yerr_MEC_NFP_Bars.Draw("same")
G21_Yerr_RES_NFP_Bars.Draw("same")
G21_Yerr_DIS_NFP_Bars.Draw("same")

G21_LepCut_NFP_Canvas.Draw()

In [ ]:
#GEM21 without nfa pions v5
G21_LepCut_NFAP_Canvas = ROOT.TCanvas()

G21_LepCut_QEL_NFAP_Plot = ROOT.TGraph(len(G21_LepBins_XVals_NFAP_Array), G21_LepBins_XVals_NFAP_Array, G21_LepPt_QEL_NFAP_Array)
G21_LepCut_QEL_NFAP_Plot.SetTitle("Relative Fraction of Interaction Type Per Lepton PT Cut w/o NFA Pions in GEM21 Model")
G21_LepCut_QEL_NFAP_Plot.GetXaxis().SetTitle("Lepton PT (GeV)")
G21_LepCut_QEL_NFAP_Plot.GetYaxis().SetTitle("Percentage")
G21_LepCut_QEL_NFAP_Plot.SetMaximum(1.1)
G21_LepCut_QEL_NFAP_Plot.SetMinimum(0.0)
G21_LepCut_QEL_NFAP_Plot.SetMarkerColor(4)
G21_LepCut_QEL_NFAP_Plot.SetMarkerStyle(29)
G21_LepCut_QEL_NFAP_Plot.Draw("same")

G21_LepCut_MEC_NFAP_Plot = ROOT.TGraph(len(G21_LepBins_XVals_NFAP_Array), G21_LepBins_XVals_NFAP_Array, G21_LepPt_MEC_NFAP_Array)
G21_LepCut_MEC_NFAP_Plot.Draw("same")
G21_LepCut_MEC_NFAP_Plot.SetMarkerColor(2)
G21_LepCut_MEC_NFAP_Plot.SetMarkerStyle(29)

G21_LepCut_RES_NFAP_Plot = ROOT.TGraph(len(G21_LepBins_XVals_NFAP_Array), G21_LepBins_XVals_NFAP_Array, G21_LepPt_RES_NFAP_Array)
G21_LepCut_RES_NFAP_Plot.Draw("same")
G21_LepCut_RES_NFAP_Plot.SetMarkerColor(3)
G21_LepCut_RES_NFAP_Plot.SetMarkerStyle(29)

G21_LepCut_DIS_NFAP_Plot = ROOT.TGraph(len(G21_LepBins_XVals_NFAP_Array), G21_LepBins_XVals_NFAP_Array, G21_LepPt_DIS_NFAP_Array)
G21_LepCut_DIS_NFAP_Plot.Draw("same")
G21_LepCut_DIS_NFAP_Plot.SetMarkerColor(6)
G21_LepCut_DIS_NFAP_Plot.SetMarkerStyle(29)

G18_LepCut_Sum_NFAP_Plot = ROOT.TGraph(len(G18_LepBins_XVals_NFAP_Array), G18_LepBins_XVals_NFAP_Array, G18_LepPt_Sum_NFAP_Array)
G18_LepCut_Sum_NFAP_Plot.Draw("same")
G18_LepCut_Sum_NFAP_Plot.SetMarkerColor(1)
G18_LepCut_Sum_NFAP_Plot.SetMarkerStyle(34)

G21_LepCut_Leg_NFAP = ROOT.TLegend(0.2, 0.85, 0.80, 0.90)
G21_LepCut_Leg_NFAP.SetNColumns(5)
G21_LepCut_Leg_NFAP.AddEntry(G21_LepCut_QEL_NFAP_Plot, "QEL")
G21_LepCut_Leg_NFAP.AddEntry(G21_LepCut_MEC_NFAP_Plot, "MEC")
G21_LepCut_Leg_NFAP.AddEntry(G21_LepCut_RES_NFAP_Plot, "RES")
G21_LepCut_Leg_NFAP.AddEntry(G21_LepCut_DIS_NFAP_Plot, "DIS")
G18_LepCut_Leg_NFAP.AddEntry(G18_LepCut_Sum_NFAP_Plot, "SUM")
G21_LepCut_Leg_NFAP.Draw()

G21_LepCut_Zeroes_NFAP = []
for i in range(len(G21_LepBins_XVals_NFAP_Array)):
    G21_LepCut_Zeroes_NFAP.append(0.025)
G21_LepCut_Zeroes_NFAP_array = array.array('f', G21_LepCut_Zeroes_NFAP)

G21_Yerr_QEL_NFAP_Bars = ROOT.TGraphErrors(len(G21_LepBins_XVals_NFAP_Array), G21_LepBins_XVals_NFAP_Array, G21_LepPt_QEL_NFAP_Array, G21_LepCut_Zeroes_NFAP_array, G21_Yerr_QEL_NFAP_Array)
G21_Yerr_MEC_NFAP_Bars = ROOT.TGraphErrors(len(G21_LepBins_XVals_NFAP_Array), G21_LepBins_XVals_NFAP_Array, G21_LepPt_MEC_NFAP_Array, G21_LepCut_Zeroes_NFAP_array, G21_Yerr_MEC_NFAP_Array)
G21_Yerr_RES_NFAP_Bars = ROOT.TGraphErrors(len(G21_LepBins_XVals_NFAP_Array), G21_LepBins_XVals_NFAP_Array, G21_LepPt_RES_NFAP_Array, G21_LepCut_Zeroes_NFAP_array, G21_Yerr_RES_NFAP_Array)
G21_Yerr_DIS_NFAP_Bars = ROOT.TGraphErrors(len(G21_LepBins_XVals_NFAP_Array), G21_LepBins_XVals_NFAP_Array, G21_LepPt_DIS_NFAP_Array, G21_LepCut_Zeroes_NFAP_array, G21_Yerr_DIS_NFAP_Array)
G21_Yerr_QEL_NFAP_Bars.Draw("same")
G21_Yerr_MEC_NFAP_Bars.Draw("same")
G21_Yerr_RES_NFAP_Bars.Draw("same")
G21_Yerr_DIS_NFAP_Bars.Draw("same")

G21_LepCut_NFAP_Canvas.Draw()

In [ ]:
# G18 Model Angle Cut Arrays
#for a given pt cut (400 MeV), what fraction of interaction types are as a function of lepton angle up to 45 degrees
degree_values = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45]
bins = 100
x_start = 0
x_end = 4.0 #GeV
n_event_cut = 20

In [ ]:
#G18 Model v4
G18_PTCut_QEL_List = []
G18_PTCut_MEC_List = []
G18_PTCut_RES_List = []
G18_PTCut_DIS_List = []
G18_PTCut_Angs_List = []
G18_PTCut_Total = 0
for value in degree_values:
    value_min = radians(value)
    value_max = radians(value+5)
    df_gst_G18_02a_PTAng = df_gst_G18_02a_all.Filter(f"ptl>{lep_accept_pt} && thetazl>{value_min} && thetazl<{value_max}")
    G18_PTCut_QEL_Ang = df_gst_G18_02a_PTAng.Filter("qel==1").Histo1D(("G18_PTCut_QEL_Ang", "", bins, x_start, x_end), "ptl")
    G18_PTCut_MEC_Ang = df_gst_G18_02a_PTAng.Filter("mec==1").Histo1D(("G18_PTCut_MEC_Ang", "", bins, x_start, x_end), "ptl")
    G18_PTCut_RES_Ang = df_gst_G18_02a_PTAng.Filter("res==1").Histo1D(("G18_PTCut_RES_Ang", "", bins, x_start, x_end), "ptl")
    G18_PTCut_DIS_Ang = df_gst_G18_02a_PTAng.Filter("dis==1").Histo1D(("G18_PTCut_DIS_Ang", "", bins, x_start, x_end), "ptl")
    G18_theta_range = [G18_PTCut_QEL_Ang.Integral(), G18_PTCut_MEC_Ang.Integral(), G18_PTCut_RES_Ang.Integral(), G18_PTCut_DIS_Ang.Integral()]
    G18_event_lists = [G18_PTCut_QEL_List, G18_PTCut_MEC_List, G18_PTCut_RES_List, G18_PTCut_DIS_List]
    G18_event_total = G18_theta_range[0] + G18_theta_range[1] + G18_theta_range[2] + G18_theta_range[3]
    G18_difference = (value_max - value_min)/2
    G18_PTCut_Total += G18_event_total
    for i in range(len(G18_theta_range)):
        if G18_event_total < n_event_cut and G18_event_lists[i] == []:
            G18_event_lists[i].append(0.0)
        if G18_event_total > n_event_cut: 
            G18_event_lists[i].append(G18_theta_range[i]/G18_event_total)
            if G18_PTCut_Angs_List == []:
                G18_PTCut_Angs_List.append(G18_difference)
            if degrees(value_max - G18_difference) not in G18_PTCut_Angs_List:
                G18_PTCut_Angs_List.append(degrees(value_max - G18_difference))
G18_PTCut_Angs_Array = array.array('f', G18_PTCut_Angs_List)
G18_QEL_Ang = array.array('f', G18_PTCut_QEL_List)
G18_MEC_Ang = array.array('f', G18_PTCut_MEC_List)
G18_RES_Ang = array.array('f', G18_PTCut_RES_List)
G18_DIS_Ang = array.array('f', G18_PTCut_DIS_List)
G18_Yerr_QEL_List = []
G18_Yerr_MEC_List = []
G18_Yerr_RES_List = []
G18_Yerr_DIS_List = []
G18_Yerr_Events_Lists = [G18_Yerr_QEL_List, G18_Yerr_MEC_List, G18_Yerr_RES_List, G18_Yerr_DIS_List]
G18_Event_Lists = [G18_PTCut_QEL_List, G18_PTCut_MEC_List, G18_PTCut_RES_List, G18_PTCut_DIS_List]
for event in G18_Event_Lists:
        for fraction in event:
            num = G18_Event_Lists.index(event)
            G18_Yerr_Events_Lists[num].append(sqrt(fraction*(1-fraction)/G18_PTCut_Total))
G18_Yerr_QEL_Array = array.array('f', G18_Yerr_QEL_List)
G18_Yerr_MEC_Array = array.array('f', G18_Yerr_MEC_List)
G18_Yerr_RES_Array = array.array('f', G18_Yerr_RES_List)
G18_Yerr_DIS_Array = array.array('f', G18_Yerr_DIS_List)

G18_PTCut_Sum_P = []
for i in range(len(G18_PTCut_QEL_List)):
    count = 0
    count += G18_PTCut_QEL_List[i] + G18_PTCut_MEC_List[i] + G18_PTCut_RES_List[i] + G18_PTCut_DIS_List[i]
    G18_PTCut_Sum_P.append(count)
G18_PTCut_Sum_P_Array = array.array('f', G18_PTCut_Sum_P)

In [ ]:
#G18 Model without Initial State Pions v4
G18_PTCut_QEL_NIP_List = []
G18_PTCut_MEC_NIP_List = []
G18_PTCut_RES_NIP_List = []
G18_PTCut_DIS_NIP_List = []
G18_PTCut_Angs_NIP_List = []
G18_PTCut_NIP_Total = 0
for value in degree_values:
    value_min = radians(value)
    value_max = radians(value+5)
    df_gst_G18_02a_PTAng_NIP = df_gst_G18_02a_all.Filter(f"ptl>{lep_accept_pt} && thetazl>{value_min} && thetazl<{value_max} && numi_pions==0")
    G18_PTCut_QEL_Ang_NIP = df_gst_G18_02a_PTAng_NIP.Filter("qel==1").Histo1D(("G18_PTCut_QEL_Ang_NIP", "", bins, x_start, x_end), "ptl")
    G18_PTCut_MEC_Ang_NIP = df_gst_G18_02a_PTAng_NIP.Filter("mec==1").Histo1D(("G18_PTCut_MEC_Ang_NIP", "", bins, x_start, x_end), "ptl")
    G18_PTCut_RES_Ang_NIP = df_gst_G18_02a_PTAng_NIP.Filter("res==1").Histo1D(("G18_PTCut_RES_Ang_NIP", "", bins, x_start, x_end), "ptl")
    G18_PTCut_DIS_Ang_NIP = df_gst_G18_02a_PTAng_NIP.Filter("dis==1").Histo1D(("G18_PTCut_DIS_Ang_NIP", "", bins, x_start, x_end), "ptl")
    G18_theta_range_NIP = [G18_PTCut_QEL_Ang_NIP.Integral(), G18_PTCut_MEC_Ang_NIP.Integral(), G18_PTCut_RES_Ang_NIP.Integral(), G18_PTCut_DIS_Ang_NIP.Integral()]
    G18_event_lists_NIP = [G18_PTCut_QEL_NIP_List, G18_PTCut_MEC_NIP_List, G18_PTCut_RES_NIP_List, G18_PTCut_DIS_NIP_List]
    G18_range_total_NIP = G18_theta_range_NIP[0] + G18_theta_range_NIP[1] + G18_theta_range_NIP[2] + G18_theta_range_NIP[3]
    G18_difference_NIP = (value_max - value_min)/2
    G18_PTCut_NIP_Total += G18_range_total_NIP
    for i in range(len(G18_theta_range_NIP)):
        if G18_PTCut_NIP_Total < n_event_cut and G18_event_lists_NIP[i] == []:
            G18_event_lists_NIP[i].append(0.0)
        if G18_range_total_NIP > n_event_cut: 
            G18_event_lists_NIP[i].append(G18_theta_range_NIP[i]/G18_range_total_NIP)
            if G18_PTCut_Angs_NIP_List == []:
                G18_PTCut_Angs_NIP_List.append(G18_difference_NIP)
            if degrees(value_max - G18_difference_NIP) not in G18_PTCut_Angs_NIP_List:
                G18_PTCut_Angs_NIP_List.append(degrees(value_max - G18_difference_NIP))
G18_PTCut_Angs_NIP_Array = array.array('f', G18_PTCut_Angs_NIP_List)
G18_QEL_Ang_NIP = array.array('f', G18_PTCut_QEL_NIP_List)
G18_MEC_Ang_NIP = array.array('f', G18_PTCut_MEC_NIP_List)
G18_RES_Ang_NIP = array.array('f', G18_PTCut_RES_NIP_List)
G18_DIS_Ang_NIP = array.array('f', G18_PTCut_DIS_NIP_List)

G18_Yerr_QEL_NIP_List = []
G18_Yerr_MEC_NIP_List = []
G18_Yerr_RES_NIP_List = []
G18_Yerr_DIS_NIP_List = []
G18_Yerr_Events_NIP_Lists = [G18_Yerr_QEL_NIP_List, G18_Yerr_MEC_NIP_List, G18_Yerr_RES_NIP_List, G18_Yerr_DIS_NIP_List]
G18_Event_NIP_Lists = [G18_PTCut_QEL_NIP_List, G18_PTCut_MEC_NIP_List, G18_PTCut_RES_NIP_List, G18_PTCut_DIS_NIP_List]
for event in G18_Event_NIP_Lists:
        for fraction in event:
            num = G18_Event_NIP_Lists.index(event)
            G18_Yerr_Events_NIP_Lists[num].append(sqrt(fraction*(1-fraction)/G18_PTCut_NIP_Total))
G18_Yerr_QEL_NIP_Array = array.array('f', G18_Yerr_QEL_NIP_List)
G18_Yerr_MEC_NIP_Array = array.array('f', G18_Yerr_MEC_NIP_List)
G18_Yerr_RES_NIP_Array = array.array('f', G18_Yerr_RES_NIP_List)
G18_Yerr_DIS_NIP_Array = array.array('f', G18_Yerr_DIS_NIP_List)

G18_PTCut_Sum_NIP = []
for i in range(len(G18_PTCut_QEL_NIP_List)):
    count = 0
    count += G18_PTCut_QEL_NIP_List[i] + G18_PTCut_MEC_NIP_List[i] + G18_PTCut_RES_NIP_List[i] + G18_PTCut_DIS_NIP_List[i]
    G18_PTCut_Sum_NIP.append(count)
G18_PTCut_Sum_NIP_Array = array.array('f', G18_PTCut_Sum_NIP)

In [ ]:
#G18 Model without final State Pions v4
G18_PTCut_QEL_NP_List = []
G18_PTCut_MEC_NP_List = []
G18_PTCut_RES_NP_List = []
G18_PTCut_DIS_NP_List = []
G18_PTCut_Angs_NP_List = []
G18_PTCut_NP_Total = 0
for value in degree_values:
    value_min = radians(value)
    value_max = radians(value+5)
    df_gst_G18_02a_PTAng_NP = df_gst_G18_02a_all.Filter(f"ptl>{lep_accept_pt} && thetazl>{value_min} && thetazl<{value_max} && num_pions==0")
    G18_PTCut_QEL_Ang_NP = df_gst_G18_02a_PTAng_NP.Filter("qel==1").Histo1D(("G18_PTCut_QEL_Ang_NP", "", bins, x_start, x_end), "ptl")
    G18_PTCut_MEC_Ang_NP = df_gst_G18_02a_PTAng_NP.Filter("mec==1").Histo1D(("G18_PTCut_MEC_Ang_NP", "", bins, x_start, x_end), "ptl")
    G18_PTCut_RES_Ang_NP = df_gst_G18_02a_PTAng_NP.Filter("res==1").Histo1D(("G18_PTCut_RES_Ang_NP", "", bins, x_start, x_end), "ptl")
    G18_PTCut_DIS_Ang_NP = df_gst_G18_02a_PTAng_NP.Filter("dis==1").Histo1D(("G18_PTCut_DIS_Ang_NP", "", bins, x_start, x_end), "ptl")
    G18_theta_range_NP = [G18_PTCut_QEL_Ang_NP.Integral(), G18_PTCut_MEC_Ang_NP.Integral(), G18_PTCut_RES_Ang_NP.Integral(), G18_PTCut_DIS_Ang_NP.Integral()]
    G18_event_lists_NP = [G18_PTCut_QEL_NP_List, G18_PTCut_MEC_NP_List, G18_PTCut_RES_NP_List, G18_PTCut_DIS_NP_List]
    G18_range_total_NP = G18_theta_range_NP[0] + G18_theta_range_NP[1] + G18_theta_range_NP[2] + G18_theta_range_NP[3]
    G18_difference_NP = (value_max - value_min)/2
    G18_PTCut_NP_Total += G18_range_total_NP
    for i in range(len(G18_theta_range_NP)):
        if G18_PTCut_NP_Total < n_event_cut and G18_event_lists_NP[i] == []:
            G18_event_lists_NP[i].append(0.0)
        if G18_range_total_NP > n_event_cut: 
            G18_event_lists_NP[i].append(G18_theta_range_NP[i]/G18_range_total_NP)
            if G18_PTCut_Angs_NP_List == []:
                G18_PTCut_Angs_NP_List.append(G18_difference_NP) 
            if degrees(value_max - G18_difference_NP) not in G18_PTCut_Angs_NP_List:
                G18_PTCut_Angs_NP_List.append(degrees(value_max - G18_difference_NP))
G18_PTCut_Angs_NP_Array = array.array('f', G18_PTCut_Angs_NP_List)
G18_QEL_Ang_NP = array.array('f', G18_PTCut_QEL_NP_List)
G18_MEC_Ang_NP = array.array('f', G18_PTCut_MEC_NP_List)
G18_RES_Ang_NP = array.array('f', G18_PTCut_RES_NP_List)
G18_DIS_Ang_NP = array.array('f', G18_PTCut_DIS_NP_List)

G18_Yerr_QEL_NP_List = []
G18_Yerr_MEC_NP_List = []
G18_Yerr_RES_NP_List = []
G18_Yerr_DIS_NP_List = []
G18_Yerr_Events_NP_Lists = [G18_Yerr_QEL_NP_List, G18_Yerr_MEC_NP_List, G18_Yerr_RES_NP_List, G18_Yerr_DIS_NP_List]
G18_Event_NP_Lists = [G18_PTCut_QEL_NP_List, G18_PTCut_MEC_NP_List, G18_PTCut_RES_NP_List, G18_PTCut_DIS_NP_List]
for event in G18_Event_NP_Lists:
        for fraction in event:
            num = G18_Event_NP_Lists.index(event)
            G18_Yerr_Events_NP_Lists[num].append(sqrt(fraction*(1-fraction)/G18_PTCut_NP_Total))
G18_Yerr_QEL_NP_Array = array.array('f', G18_Yerr_QEL_NP_List)
G18_Yerr_MEC_NP_Array = array.array('f', G18_Yerr_MEC_NP_List)
G18_Yerr_RES_NP_Array = array.array('f', G18_Yerr_RES_NP_List)
G18_Yerr_DIS_NP_Array = array.array('f', G18_Yerr_DIS_NP_List)

G18_PTCut_Sum_NP = []
for i in range(len(G18_PTCut_QEL_NP_List)):
    count = 0
    count += G18_PTCut_QEL_NP_List[i] + G18_PTCut_MEC_NP_List[i] + G18_PTCut_RES_NP_List[i] + G18_PTCut_DIS_NP_List[i]
    G18_PTCut_Sum_NP.append(count)
G18_PTCut_Sum_NP_Array = array.array('f', G18_PTCut_Sum_NP)

In [ ]:
#G18 Model without res/dis v4
G18_PTCut_QEL_NRD_List = []
G18_PTCut_MEC_NRD_List = []
G18_PTCut_Angs_NRD_List = []
G18_PTCut_NRD_Total = 0
for value in degree_values:
    value_min = radians(value)
    value_max = radians(value+5)
    df_gst_G18_02a_PTAng_NRD = df_gst_G18_02a_all.Filter(f"ptl>{lep_accept_pt} && thetazl>{value_min} && thetazl<{value_max} && res==0 && dis==0")
    G18_PTCut_QEL_Ang_NRD = df_gst_G18_02a_PTAng_NRD.Filter("qel==1").Histo1D(("G18_PTCut_QEL_Ang_NRD", "", bins, x_start, x_end), "ptl")
    G18_PTCut_MEC_Ang_NRD = df_gst_G18_02a_PTAng_NRD.Filter("mec==1").Histo1D(("G18_PTCut_MEC_Ang_NRD", "", bins, x_start, x_end), "ptl")
    G18_theta_range_NRD = [G18_PTCut_QEL_Ang_NRD.Integral(), G18_PTCut_MEC_Ang_NRD.Integral()]
    G18_event_lists_NRD = [G18_PTCut_QEL_NRD_List, G18_PTCut_MEC_NRD_List]
    G18_range_total_NRD = G18_theta_range_NRD[0] + G18_theta_range_NRD[1]
    G18_difference_NRD = (value_max - value_min)/2
    G18_PTCut_NRD_Total += G18_range_total_NRD
    for i in range(len(G18_theta_range_NRD)):
        if G18_PTCut_NRD_Total < n_event_cut and G18_event_lists_NRD[i] == []:
            G18_event_lists_NRD[i].append(0.0)
        if G18_range_total_NRD > n_event_cut: 
            G18_event_lists_NRD[i].append(G18_theta_range_NRD[i]/G18_range_total_NRD)
            if G18_PTCut_Angs_NRD_List == []:
                G18_PTCut_Angs_NRD_List.append(G18_difference_NRD)
            if degrees(value_max - G18_difference_NRD) not in G18_PTCut_Angs_NRD_List:
                G18_PTCut_Angs_NRD_List.append(degrees(value_max - G18_difference_NRD))
G18_PTCut_Angs_NRD_Array = array.array('f', G18_PTCut_Angs_NRD_List)
G18_QEL_Ang_NRD = array.array('f', G18_PTCut_QEL_NRD_List)
G18_MEC_Ang_NRD = array.array('f', G18_PTCut_MEC_NRD_List)

G18_Yerr_QEL_NRD_List = []
G18_Yerr_MEC_NRD_List = []
G18_Yerr_Events_NRD_Lists = [G18_Yerr_QEL_NRD_List, G18_Yerr_MEC_NRD_List]
G18_Event_NRD_Lists = [G18_PTCut_QEL_NRD_List, G18_PTCut_MEC_NRD_List]
for event in G18_Event_NRD_Lists:
        for fraction in event:
            num = G18_Event_NRD_Lists.index(event)
            G18_Yerr_Events_NRD_Lists[num].append(sqrt(fraction*(1-fraction)/G18_PTCut_NRD_Total))
G18_Yerr_QEL_NRD_Array = array.array('f', G18_Yerr_QEL_NRD_List)
G18_Yerr_MEC_NRD_Array = array.array('f', G18_Yerr_MEC_NRD_List)

G18_PTCut_Sum_NRD = []
for i in range(len(G18_PTCut_QEL_NRD_List)):
    count = 0
    count += G18_PTCut_QEL_NRD_List[i] + G18_PTCut_MEC_NRD_List[i]
    G18_PTCut_Sum_NRD.append(count)
G18_PTCut_Sum_NRD_Array = array.array('f', G18_PTCut_Sum_NRD)

In [ ]:
#G18 Model without res/dis/final state pions v3
G18_PTCut_QEL_NRDP_List = []
G18_PTCut_MEC_NRDP_List = []
G18_PTCut_Angs_NRDP_List = []
G18_PTCut_NRDP_Total = 0
for value in degree_values:
    value_min = radians(value)
    value_max = radians(value+5)
    df_gst_G18_02a_PTAng_NRDP = df_gst_G18_02a_all.Filter(f"ptl>{lep_accept_pt} && thetazl>{value_min} && thetazl<{value_max} && num_pions==0 && res==0 && dis==0")
    G18_PTCut_QEL_Ang_NRDP = df_gst_G18_02a_PTAng_NRDP.Filter("qel==1").Histo1D(("G18_PTCut_QEL_Ang_NRDP", "", bins, x_start, x_end), "ptl")
    G18_PTCut_MEC_Ang_NRDP = df_gst_G18_02a_PTAng_NRDP.Filter("mec==1").Histo1D(("G18_PTCut_MEC_Ang_NRDP", "", bins, x_start, x_end), "ptl")
    G18_theta_range_NRDP = [G18_PTCut_QEL_Ang_NRDP.Integral(), G18_PTCut_MEC_Ang_NRDP.Integral()]
    G18_event_lists_NRDP = [G18_PTCut_QEL_NRDP_List, G18_PTCut_MEC_NRDP_List]
    G18_range_total_NRDP = G18_theta_range_NRDP[0] + G18_theta_range_NRDP[1]
    G18_difference_NRDP = (value_max - value_min)/2
    G18_PTCut_NRDP_Total += G18_range_total_NRDP
    for i in range(len(G18_theta_range_NRDP)):
        if G18_PTCut_NRDP_Total < n_event_cut and G18_event_lists_NRDP[i] == []:
            G18_event_lists_NRDP[i].append(0.0)
        if G18_range_total_NRDP > n_event_cut: 
            G18_event_lists_NRDP[i].append(G18_theta_range_NRDP[i]/G18_range_total_NRDP)
            if G18_PTCut_Angs_NRDP_List == []:
                G18_PTCut_Angs_NRDP_List.append(G18_difference_NRDP)
            if degrees(value_max - G18_difference_NRDP) not in G18_PTCut_Angs_NRDP_List:
                G18_PTCut_Angs_NRDP_List.append(degrees(value_max - G18_difference_NRDP))
G18_PTCut_Angs_NRDP_Array = array.array('f', G18_PTCut_Angs_NRDP_List)
G18_QEL_Ang_NRDP = array.array('f', G18_PTCut_QEL_NRDP_List)
G18_MEC_Ang_NRDP = array.array('f', G18_PTCut_MEC_NRDP_List)

G18_Yerr_QEL_NRDP_List = []
G18_Yerr_MEC_NRDP_List = []
G18_Yerr_Events_NRDP_Lists = [G18_Yerr_QEL_NRDP_List, G18_Yerr_MEC_NRDP_List]
G18_Event_NRDP_Lists = [G18_PTCut_QEL_NRDP_List, G18_PTCut_MEC_NRDP_List]
for event in G18_Event_NRDP_Lists:
        for fraction in event:
            num = G18_Event_NRDP_Lists.index(event)
            G18_Yerr_Events_NRDP_Lists[num].append(sqrt(fraction*(1-fraction)/G18_PTCut_NRDP_Total))
G18_Yerr_QEL_NRDP_Array = array.array('f', G18_Yerr_QEL_NRDP_List)
G18_Yerr_MEC_NRDP_Array = array.array('f', G18_Yerr_MEC_NRDP_List)

G18_PTCut_Sum_NRDP = []
for i in range(len(G18_PTCut_QEL_NRDP_List)):
    count = 0
    count += G18_PTCut_QEL_NRDP_List[i] + G18_PTCut_MEC_NRDP_List[i]
    G18_PTCut_Sum_NRDP.append(count)
G18_PTCut_Sum_NRDP_Array = array.array('f', G18_PTCut_Sum_NRDP)

In [ ]:
#G18 Model without nfa Pions v4
G18_PTCut_QEL_NFA_List = []
G18_PTCut_MEC_NFA_List = []
G18_PTCut_RES_NFA_List = []
G18_PTCut_DIS_NFA_List = []
G18_PTCut_Angs_NFA_List = []
G18_PTCut_NFA_Total = 0
for value in degree_values:
    value_min = radians(value)
    value_max = radians(value+5)
    df_gst_G18_02a_PTAng_NFA = df_gst_G18_02a_all.Filter(f"ptl>{lep_accept_pt} && thetazl>{value_min} && thetazl<{value_max} && (nfa_piplus+nfa_piminus+nfa_pi0)==0")
    G18_PTCut_QEL_Ang_NFA = df_gst_G18_02a_PTAng_NFA.Filter("qel==1").Histo1D(("G18_PTCut_QEL_Ang_NFA", "", bins, x_start, x_end), "ptl")
    G18_PTCut_MEC_Ang_NFA = df_gst_G18_02a_PTAng_NFA.Filter("mec==1").Histo1D(("G18_PTCut_MEC_Ang_NFA", "", bins, x_start, x_end), "ptl")
    G18_PTCut_RES_Ang_NFA = df_gst_G18_02a_PTAng_NFA.Filter("res==1").Histo1D(("G18_PTCut_RES_Ang_NFA", "", bins, x_start, x_end), "ptl")
    G18_PTCut_DIS_Ang_NFA = df_gst_G18_02a_PTAng_NFA.Filter("dis==1").Histo1D(("G18_PTCut_DIS_Ang_NFA", "", bins, x_start, x_end), "ptl")
    G18_theta_range_NFA = [G18_PTCut_QEL_Ang_NFA.Integral(), G18_PTCut_MEC_Ang_NFA.Integral(), G18_PTCut_RES_Ang_NFA.Integral(), G18_PTCut_DIS_Ang_NFA.Integral()]
    G18_event_lists_NFA = [G18_PTCut_QEL_NFA_List, G18_PTCut_MEC_NFA_List, G18_PTCut_RES_NFA_List, G18_PTCut_DIS_NFA_List]
    G18_range_total_NFA = G18_theta_range_NFA[0] + G18_theta_range_NFA[1] + G18_theta_range_NFA[2] + G18_theta_range_NFA[3]
    G18_PTCut_NFA_Total += G18_range_total_NFA
    G18_difference_NFA = (value_max - value_min)/2
    for i in range(len(G18_theta_range_NFA)):
        if G18_PTCut_NFA_Total < n_event_cut and G18_event_lists_NFA[i] == []:
            G18_event_lists_NFA[i].append(0.0)
        if G18_range_total_NFA > n_event_cut: 
            G18_event_lists_NFA[i].append(G18_theta_range_NFA[i]/G18_range_total_NFA)
            if G18_PTCut_Angs_NFA_List == []:
                G18_PTCut_Angs_NFA_List.append(G18_difference_NFA)
            if degrees(value_max - G18_difference_NFA) not in G18_PTCut_Angs_NFA_List:
                G18_PTCut_Angs_NFA_List.append(degrees(value_max - G18_difference_NFA))
G18_PTCut_Angs_NFA_Array = array.array('f', G18_PTCut_Angs_NFA_List)
G18_QEL_Ang_NFA = array.array('f', G18_PTCut_QEL_NFA_List)
G18_MEC_Ang_NFA = array.array('f', G18_PTCut_MEC_NFA_List)
G18_RES_Ang_NFA = array.array('f', G18_PTCut_RES_NFA_List)
G18_DIS_Ang_NFA = array.array('f', G18_PTCut_DIS_NFA_List)

G18_Yerr_QEL_NFA_List = []
G18_Yerr_MEC_NFA_List = []
G18_Yerr_RES_NFA_List = []
G18_Yerr_DIS_NFA_List = []
G18_Yerr_Events_NFA_Lists = [G18_Yerr_QEL_NFA_List, G18_Yerr_MEC_NFA_List, G18_Yerr_RES_NFA_List, G18_Yerr_DIS_NFA_List]
G18_Event_NFA_Lists = [G18_PTCut_QEL_NFA_List, G18_PTCut_MEC_NFA_List, G18_PTCut_RES_NFA_List, G18_PTCut_DIS_NFA_List]
for event in G18_Event_NFA_Lists:
        for fraction in event:
            num = G18_Event_NFA_Lists.index(event)
            G18_Yerr_Events_NFA_Lists[num].append(sqrt(fraction*(1-fraction)/G18_PTCut_NFA_Total))
G18_Yerr_QEL_NFA_Array = array.array('f', G18_Yerr_QEL_NFA_List)
G18_Yerr_MEC_NFA_Array = array.array('f', G18_Yerr_MEC_NFA_List)
G18_Yerr_RES_NFA_Array = array.array('f', G18_Yerr_RES_NFA_List)
G18_Yerr_DIS_NFA_Array = array.array('f', G18_Yerr_DIS_NFA_List)

G18_PTCut_Sum_NFA = []
for i in range(len(G18_PTCut_QEL_NFA_List)):
    count = 0
    count += G18_PTCut_QEL_NFA_List[i] + G18_PTCut_MEC_NFA_List[i] + G18_PTCut_RES_NFA_List[i] + G18_PTCut_DIS_NFA_List[i]
    G18_PTCut_Sum_NFA.append(count)
G18_PTCut_Sum_NFA_Array = array.array('f', G18_PTCut_Sum_NFA)

In [ ]:
# GEM21 Model Angle Cut Arrays V4

In [ ]:
#G21 Model v4
G21_PTCut_QEL_List = []
G21_PTCut_MEC_List = []
G21_PTCut_RES_List = []
G21_PTCut_DIS_List = []
G21_PTCut_Angs_List = []
G21_PTCut_Total = 0
for value in degree_values:
    value_min = radians(value)
    value_max = radians(value+5)
    df_gst_G21_02a_PTAng = df_gst_GEM21_11b_all.Filter(f"ptl>{lep_accept_pt} && thetazl>{value_min} && thetazl<{value_max}")
    G21_PTCut_QEL_Ang = df_gst_G21_02a_PTAng.Filter("qel==1").Histo1D(("G21_PTCut_QEL_Ang", "", bins, x_start, x_end), "ptl")
    G21_PTCut_MEC_Ang = df_gst_G21_02a_PTAng.Filter("mec==1").Histo1D(("G21_PTCut_MEC_Ang", "", bins, x_start, x_end), "ptl")
    G21_PTCut_RES_Ang = df_gst_G21_02a_PTAng.Filter("res==1").Histo1D(("G21_PTCut_RES_Ang", "", bins, x_start, x_end), "ptl")
    G21_PTCut_DIS_Ang = df_gst_G21_02a_PTAng.Filter("dis==1").Histo1D(("G21_PTCut_DIS_Ang", "", bins, x_start, x_end), "ptl")
    G21_theta_range = [G21_PTCut_QEL_Ang.Integral(), G21_PTCut_MEC_Ang.Integral(), G21_PTCut_RES_Ang.Integral(), G21_PTCut_DIS_Ang.Integral()]
    G21_event_lists = [G21_PTCut_QEL_List, G21_PTCut_MEC_List, G21_PTCut_RES_List, G21_PTCut_DIS_List]
    G21_event_total = G21_theta_range[0] + G21_theta_range[1] + G21_theta_range[2] + G21_theta_range[3]
    G21_PTCut_Total += G21_event_total
    G21_difference = (value_max - value_min)/2
    for i in range(len(G21_theta_range)):
        if G21_event_total < n_event_cut and G21_event_lists[i] == []:
            G21_event_lists[i].append(0.0)
        if G21_event_total > n_event_cut: 
            G21_event_lists[i].append(G21_theta_range[i]/G21_event_total)
            if G21_PTCut_Angs_List == []:
                G21_PTCut_Angs_List.append(G21_difference)
            if degrees(value_max - G21_difference) not in G21_PTCut_Angs_List:
                G21_PTCut_Angs_List.append(degrees(value_max - G21_difference))
G21_PTCut_Angs_Array = array.array('f', G21_PTCut_Angs_List)
G21_QEL_Ang = array.array('f', G21_PTCut_QEL_List)
G21_MEC_Ang = array.array('f', G21_PTCut_MEC_List)
G21_RES_Ang = array.array('f', G21_PTCut_RES_List)
G21_DIS_Ang = array.array('f', G21_PTCut_DIS_List)

G21_Yerr_QEL_List = []
G21_Yerr_MEC_List = []
G21_Yerr_RES_List = []
G21_Yerr_DIS_List = []
G21_Yerr_Events_Lists = [G21_Yerr_QEL_List, G21_Yerr_MEC_List, G21_Yerr_RES_List, G21_Yerr_DIS_List]
G21_Event_Lists = [G21_PTCut_QEL_List, G21_PTCut_MEC_List, G21_PTCut_RES_List, G21_PTCut_DIS_List]
for event in G21_Event_Lists:
        for fraction in event:
            num = G21_Event_Lists.index(event)
            G21_Yerr_Events_Lists[num].append(sqrt(fraction*(1-fraction)/G21_PTCut_Total))
G21_Yerr_QEL_Array = array.array('f', G21_Yerr_QEL_List)
G21_Yerr_MEC_Array = array.array('f', G21_Yerr_MEC_List)
G21_Yerr_RES_Array = array.array('f', G21_Yerr_RES_List)
G21_Yerr_DIS_Array = array.array('f', G21_Yerr_DIS_List)

G21_PTCut_Sum_P = []
for i in range(len(G21_PTCut_QEL_List)):
    count = 0
    count += G21_PTCut_QEL_List[i] + G21_PTCut_MEC_List[i] + G21_PTCut_RES_List[i] + G21_PTCut_DIS_List[i]
    G21_PTCut_Sum_P.append(count)
G21_PTCut_Sum_P_Array = array.array('f', G21_PTCut_Sum_P)

In [ ]:
#Gem21 Model without Initial State Pions v4
G21_PTCut_QEL_NIP_List = []
G21_PTCut_MEC_NIP_List = []
G21_PTCut_RES_NIP_List = []
G21_PTCut_DIS_NIP_List = []
G21_PTCut_Angs_NIP_List = []
G21_PTCut_NIP_Total = 0
for value in degree_values:
    value_min = radians(value)
    value_max = radians(value+5)
    df_gst_G21_02a_PTAng_NIP = df_gst_GEM21_11b_all.Filter(f"ptl>{lep_accept_pt} && thetazl>{value_min} && thetazl<{value_max} && numi_pions==0")
    G21_PTCut_QEL_Ang_NIP = df_gst_G21_02a_PTAng_NIP.Filter("qel==1").Histo1D(("G21_PTCut_QEL_Ang_NIP", "", bins, x_start, x_end), "ptl")
    G21_PTCut_MEC_Ang_NIP = df_gst_G21_02a_PTAng_NIP.Filter("mec==1").Histo1D(("G21_PTCut_MEC_Ang_NIP", "", bins, x_start, x_end), "ptl")
    G21_PTCut_RES_Ang_NIP = df_gst_G21_02a_PTAng_NIP.Filter("res==1").Histo1D(("G21_PTCut_RES_Ang_NIP", "", bins, x_start, x_end), "ptl")
    G21_PTCut_DIS_Ang_NIP = df_gst_G21_02a_PTAng_NIP.Filter("dis==1").Histo1D(("G21_PTCut_DIS_Ang_NIP", "", bins, x_start, x_end), "ptl")
    G21_theta_range_NIP = [G21_PTCut_QEL_Ang_NIP.Integral(), G21_PTCut_MEC_Ang_NIP.Integral(), G21_PTCut_RES_Ang_NIP.Integral(), G21_PTCut_DIS_Ang_NIP.Integral()]
    G21_event_lists_NIP = [G21_PTCut_QEL_NIP_List, G21_PTCut_MEC_NIP_List, G21_PTCut_RES_NIP_List, G21_PTCut_DIS_NIP_List]
    G21_range_total_NIP = G21_theta_range_NIP[0] + G21_theta_range_NIP[1] + G21_theta_range_NIP[2] + G21_theta_range_NIP[3]
    G21_PTCut_NIP_Total += G21_range_total_NIP
    G21_difference_NIP = (value_max - value_min)/2
    for i in range(len(G21_theta_range_NIP)):
        if G21_range_total_NIP < n_event_cut and G21_event_lists_NIP[i] == []:
            G21_event_lists_NIP[i].append(0.0)
        if G21_range_total_NIP > n_event_cut: 
            G21_event_lists_NIP[i].append(G21_theta_range_NIP[i]/G21_range_total_NIP)
            if G21_PTCut_Angs_NIP_List == []:
                G21_PTCut_Angs_NIP_List.append(G21_difference_NIP)
            if degrees(value_max - G21_difference_NIP) not in G21_PTCut_Angs_NIP_List:
                G21_PTCut_Angs_NIP_List.append(degrees(value_max - G21_difference_NIP))
G21_PTCut_Angs_NIP_Array = array.array('f', G21_PTCut_Angs_NIP_List)
G21_QEL_Ang_NIP = array.array('f', G21_PTCut_QEL_NIP_List)
G21_MEC_Ang_NIP = array.array('f', G21_PTCut_MEC_NIP_List)
G21_RES_Ang_NIP = array.array('f', G21_PTCut_RES_NIP_List)
G21_DIS_Ang_NIP = array.array('f', G21_PTCut_DIS_NIP_List)

G21_Yerr_QEL_NIP_List = []
G21_Yerr_MEC_NIP_List = []
G21_Yerr_RES_NIP_List = []
G21_Yerr_DIS_NIP_List = []
G21_Yerr_Events_NIP_Lists = [G21_Yerr_QEL_NIP_List, G21_Yerr_MEC_NIP_List, G21_Yerr_RES_NIP_List, G21_Yerr_DIS_NIP_List]
G21_Event_NIP_Lists = [G21_PTCut_QEL_NIP_List, G21_PTCut_MEC_NIP_List, G21_PTCut_RES_NIP_List, G21_PTCut_DIS_NIP_List]
for event in G21_Event_NIP_Lists:
        for fraction in event:
            num = G21_Event_NIP_Lists.index(event)
            G21_Yerr_Events_NIP_Lists[num].append(sqrt(fraction*(1-fraction)/G21_PTCut_NIP_Total))
G21_Yerr_QEL_NIP_Array = array.array('f', G21_Yerr_QEL_NIP_List)
G21_Yerr_MEC_NIP_Array = array.array('f', G21_Yerr_MEC_NIP_List)
G21_Yerr_RES_NIP_Array = array.array('f', G21_Yerr_RES_NIP_List)
G21_Yerr_DIS_NIP_Array = array.array('f', G21_Yerr_DIS_NIP_List)

G21_PTCut_Sum_NIP = []
for i in range(len(G21_PTCut_QEL_NIP_List)):
    count = 0
    count += G21_PTCut_QEL_NIP_List[i] + G21_PTCut_MEC_NIP_List[i] + G21_PTCut_RES_NIP_List[i] + G21_PTCut_DIS_NIP_List[i]
    G21_PTCut_Sum_NIP.append(count)
G21_PTCut_Sum_NIP_Array = array.array('f', G21_PTCut_Sum_NIP)

In [ ]:
#G21 Model without final State Pions v4
G21_PTCut_QEL_NP_List = []
G21_PTCut_MEC_NP_List = []
G21_PTCut_RES_NP_List = []
G21_PTCut_DIS_NP_List = []
G21_PTCut_Angs_NP_List = []
G21_PTCut_NP_Total = 0
for value in degree_values:
    value_min = radians(value)
    value_max = radians(value+5)
    df_gst_GEM21_11b_PTAng_NP = df_gst_GEM21_11b_all.Filter(f"ptl>{lep_accept_pt} && thetazl>{value_min} && thetazl<{value_max} && num_pions==0")
    G21_PTCut_QEL_Ang_NP = df_gst_GEM21_11b_PTAng_NP.Filter("qel==1").Histo1D(("G21_PTCut_QEL_Ang_NP", "", bins, x_start, x_end), "ptl")
    G21_PTCut_MEC_Ang_NP = df_gst_GEM21_11b_PTAng_NP.Filter("mec==1").Histo1D(("G21_PTCut_MEC_Ang_NP", "", bins, x_start, x_end), "ptl")
    G21_PTCut_RES_Ang_NP = df_gst_GEM21_11b_PTAng_NP.Filter("res==1").Histo1D(("G21_PTCut_RES_Ang_NP", "", bins, x_start, x_end), "ptl")
    G21_PTCut_DIS_Ang_NP = df_gst_GEM21_11b_PTAng_NP.Filter("dis==1").Histo1D(("G21_PTCut_DIS_Ang_NP", "", bins, x_start, x_end), "ptl")
    G21_theta_range_NP = [G21_PTCut_QEL_Ang_NP.Integral(), G21_PTCut_MEC_Ang_NP.Integral(), G21_PTCut_RES_Ang_NP.Integral(), G21_PTCut_DIS_Ang_NP.Integral()]
    G21_event_lists_NP = [G21_PTCut_QEL_NP_List, G21_PTCut_MEC_NP_List, G21_PTCut_RES_NP_List, G21_PTCut_DIS_NP_List]
    G21_range_total_NP = G21_theta_range_NP[0] + G21_theta_range_NP[1] + G21_theta_range_NP[2] + G21_theta_range_NP[3]
    G21_difference_NP = (value_max - value_min)/2
    G21_PTCut_NP_Total += G21_range_total_NP
    for i in range(len(G21_theta_range_NP)):
        if G21_PTCut_NP_Total < n_event_cut and G21_event_lists_NP[i] == []:
            G21_event_lists_NP[i].append(0.0)
        if G21_range_total_NP > n_event_cut: 
            G21_event_lists_NP[i].append(G21_theta_range_NP[i]/G21_range_total_NP)
            if G21_PTCut_Angs_NP_List == []:
                G21_PTCut_Angs_NP_List.append(G21_difference_NP) 
            if degrees(value_max - G21_difference_NP) not in G21_PTCut_Angs_NP_List:
                G21_PTCut_Angs_NP_List.append(degrees(value_max - G21_difference_NP))
G21_PTCut_Angs_NP_Array = array.array('f', G21_PTCut_Angs_NP_List)
G21_QEL_Ang_NP = array.array('f', G21_PTCut_QEL_NP_List)
G21_MEC_Ang_NP = array.array('f', G21_PTCut_MEC_NP_List)
G21_RES_Ang_NP = array.array('f', G21_PTCut_RES_NP_List)
G21_DIS_Ang_NP = array.array('f', G21_PTCut_DIS_NP_List)

G21_Yerr_QEL_NP_List = []
G21_Yerr_MEC_NP_List = []
G21_Yerr_RES_NP_List = []
G21_Yerr_DIS_NP_List = []
G21_Yerr_Events_NP_Lists = [G21_Yerr_QEL_NP_List, G21_Yerr_MEC_NP_List, G21_Yerr_RES_NP_List, G21_Yerr_DIS_NP_List]
G21_Event_NP_Lists = [G21_PTCut_QEL_NP_List, G21_PTCut_MEC_NP_List, G21_PTCut_RES_NP_List, G21_PTCut_DIS_NP_List]
for event in G21_Event_NP_Lists:
        for fraction in event:
            num = G21_Event_NP_Lists.index(event)
            G21_Yerr_Events_NP_Lists[num].append(sqrt(fraction*(1-fraction)/G21_PTCut_NP_Total))
G21_Yerr_QEL_NP_Array = array.array('f', G21_Yerr_QEL_NP_List)
G21_Yerr_MEC_NP_Array = array.array('f', G21_Yerr_MEC_NP_List)
G21_Yerr_RES_NP_Array = array.array('f', G21_Yerr_RES_NP_List)
G21_Yerr_DIS_NP_Array = array.array('f', G21_Yerr_DIS_NP_List)

G21_PTCut_Sum_NP = []
for i in range(len(G21_PTCut_QEL_NP_List)):
    count = 0
    count += G21_PTCut_QEL_NP_List[i] + G21_PTCut_MEC_NP_List[i] + G21_PTCut_RES_NP_List[i] + G21_PTCut_DIS_NP_List[i]
    G21_PTCut_Sum_NP.append(count)
G21_PTCut_Sum_NP_Array = array.array('f', G21_PTCut_Sum_NP)

In [ ]:
#G21 Model without res/dis v4
G21_PTCut_QEL_NRD_List = []
G21_PTCut_MEC_NRD_List = []
G21_PTCut_Angs_NRD_List = []
G21_PTCut_NRD_Total = 0
for value in degree_values:
    value_min = radians(value)
    value_max = radians(value+5)
    df_gst_G21_02a_PTAng_NRD = df_gst_GEM21_11b_all.Filter(f"ptl>{lep_accept_pt} && thetazl>{value_min} && thetazl<{value_max} && res==0 && dis==0")
    G21_PTCut_QEL_Ang_NRD = df_gst_G21_02a_PTAng_NRD.Filter("qel==1").Histo1D(("G21_PTCut_QEL_Ang_NRD", "", bins, x_start, x_end), "ptl")
    G21_PTCut_MEC_Ang_NRD = df_gst_G21_02a_PTAng_NRD.Filter("mec==1").Histo1D(("G21_PTCut_MEC_Ang_NRD", "", bins, x_start, x_end), "ptl")
    G21_theta_range_NRD = [G21_PTCut_QEL_Ang_NRD.Integral(), G21_PTCut_MEC_Ang_NRD.Integral()]
    G21_event_lists_NRD = [G21_PTCut_QEL_NRD_List, G21_PTCut_MEC_NRD_List]
    G21_range_total_NRD = G21_theta_range_NRD[0] + G21_theta_range_NRD[1]
    G21_PTCut_NRD_Total += G21_range_total_NRD
    G21_difference_NRD = (value_max - value_min)/2
    for i in range(len(G21_theta_range_NRD)):
        if G21_range_total_NRD < n_event_cut and G21_event_lists_NRD[i] == []:
            G21_event_lists_NRD[i].append(0.0)
        if G21_range_total_NRD > n_event_cut: 
            G21_event_lists_NRD[i].append(G21_theta_range_NRD[i]/G21_range_total_NRD)
            if G21_PTCut_Angs_NRD_List == []:
                G21_PTCut_Angs_NRD_List.append(G21_difference_NRD)
            if degrees(value_max - G21_difference_NRD) not in G21_PTCut_Angs_NRD_List:
                G21_PTCut_Angs_NRD_List.append(degrees(value_max - G21_difference_NRD))
G21_PTCut_Angs_NRD_Array = array.array('f', G21_PTCut_Angs_NRD_List)
G21_QEL_Ang_NRD = array.array('f', G21_PTCut_QEL_NRD_List)
G21_MEC_Ang_NRD = array.array('f', G21_PTCut_MEC_NRD_List)

G21_Yerr_QEL_NRD_List = []
G21_Yerr_MEC_NRD_List = []
G21_Yerr_Events_NRD_Lists = [G21_Yerr_QEL_NRD_List, G21_Yerr_MEC_NRD_List]
G21_Event_NRD_Lists = [G21_PTCut_QEL_NRD_List, G21_PTCut_MEC_NRD_List]
for event in G21_Event_NRD_Lists:
        for fraction in event:
            num = G21_Event_NRD_Lists.index(event)
            G21_Yerr_Events_NRD_Lists[num].append(sqrt(fraction*(1-fraction)/G21_PTCut_NRD_Total))
G21_Yerr_QEL_NRD_Array = array.array('f', G21_Yerr_QEL_NRD_List)
G21_Yerr_MEC_NRD_Array = array.array('f', G21_Yerr_MEC_NRD_List)

G21_PTCut_Sum_NRD = []
for i in range(len(G21_PTCut_QEL_NRD_List)):
    count = 0
    count += G21_PTCut_QEL_NRD_List[i] + G21_PTCut_MEC_NRD_List[i]
    G21_PTCut_Sum_NRD.append(count)
G21_PTCut_Sum_NRD_Array = array.array('f', G21_PTCut_Sum_NRD)

In [ ]:
#G21 Model without res/dis/final state pions v4
G21_PTCut_QEL_NRDP_List = []
G21_PTCut_MEC_NRDP_List = []
G21_PTCut_Angs_NRDP_List = []
G21_PTCut_NRDP_Total = 0
for value in degree_values:
    value_min = radians(value)
    value_max = radians(value+5)
    df_gst_G21_02a_PTAng_NRDP = df_gst_GEM21_11b_all.Filter(f"ptl>{lep_accept_pt} && thetazl>{value_min} && thetazl<{value_max} && num_pions==0 && res==0 && dis==0")
    G21_PTCut_QEL_Ang_NRDP = df_gst_G21_02a_PTAng_NRDP.Filter("qel==1").Histo1D(("G21_PTCut_QEL_Ang_NRDP", "", bins, x_start, x_end), "ptl")
    G21_PTCut_MEC_Ang_NRDP = df_gst_G21_02a_PTAng_NRDP.Filter("mec==1").Histo1D(("G21_PTCut_MEC_Ang_NRDP", "", bins, x_start, x_end), "ptl")
    G21_theta_range_NRDP = [G21_PTCut_QEL_Ang_NRDP.Integral(), G21_PTCut_MEC_Ang_NRDP.Integral()]
    G21_event_lists_NRDP = [G21_PTCut_QEL_NRDP_List, G21_PTCut_MEC_NRDP_List]
    G21_range_total_NRDP = G21_theta_range_NRDP[0] + G21_theta_range_NRDP[1]
    G21_PTCut_NRDP_Total += G21_range_total_NRDP
    G21_difference_NRDP = (value_max - value_min)/2
    for i in range(len(G21_theta_range_NRDP)):
        if G21_range_total_NRDP < n_event_cut and G21_event_lists_NRDP[i] == []:
            G21_event_lists_NRDP[i].append(0.0)
        if G21_range_total_NRDP > n_event_cut: 
            G21_event_lists_NRDP[i].append(G21_theta_range_NRDP[i]/G21_range_total_NRDP)
            if G21_PTCut_Angs_NRDP_List == []:
                G21_PTCut_Angs_NRDP_List.append(G21_difference_NRDP)
            if degrees(value_max - G21_difference_NRDP) not in G21_PTCut_Angs_NRDP_List:
                G21_PTCut_Angs_NRDP_List.append(degrees(value_max - G21_difference_NRDP))
G21_PTCut_Angs_NRDP_Array = array.array('f', G21_PTCut_Angs_NRDP_List)
G21_QEL_Ang_NRDP = array.array('f', G21_PTCut_QEL_NRDP_List)
G21_MEC_Ang_NRDP = array.array('f', G21_PTCut_MEC_NRDP_List)

G21_Yerr_QEL_NRDP_List = []
G21_Yerr_MEC_NRDP_List = []
G21_Yerr_Events_NRDP_Lists = [G21_Yerr_QEL_NRDP_List, G21_Yerr_MEC_NRDP_List]
G21_Event_NRDP_Lists = [G21_PTCut_QEL_NRDP_List, G21_PTCut_MEC_NRDP_List]
for event in G21_Event_NRDP_Lists:
        for fraction in event:
            num = G21_Event_NRDP_Lists.index(event)
            G21_Yerr_Events_NRDP_Lists[num].append(sqrt(fraction*(1-fraction)/G21_PTCut_NRDP_Total))
G21_Yerr_QEL_NRDP_Array = array.array('f', G21_Yerr_QEL_NRDP_List)
G21_Yerr_MEC_NRDP_Array = array.array('f', G21_Yerr_MEC_NRDP_List)

G21_PTCut_Sum_NRDP = []
for i in range(len(G21_PTCut_QEL_NRDP_List)):
    count = 0
    count += G21_PTCut_QEL_NRDP_List[i] + G21_PTCut_MEC_NRDP_List[i]
    G21_PTCut_Sum_NRDP.append(count)
G21_PTCut_Sum_NRDP_Array = array.array('f', G21_PTCut_Sum_NRDP)

In [ ]:
#G21 Model without nfa Pions v4
G21_PTCut_QEL_NFA_List = []
G21_PTCut_MEC_NFA_List = []
G21_PTCut_RES_NFA_List = []
G21_PTCut_DIS_NFA_List = []
G21_PTCut_Angs_NFA_List = []
G21_PTCut_NFA_Total = 0
for value in degree_values:
    value_min = radians(value)
    value_max = radians(value+5)
    df_gst_G21_02a_PTAng_NFA = df_gst_GEM21_11b_all.Filter(f"ptl>{lep_accept_pt} && thetazl>{value_min} && thetazl<{value_max} && (nfa_piplus+nfa_piminus+nfa_pi0)==0")
    G21_PTCut_QEL_Ang_NFA = df_gst_G21_02a_PTAng_NFA.Filter("qel==1").Histo1D(("G21_PTCut_QEL_Ang_NFA", "", bins, x_start, x_end), "ptl")
    G21_PTCut_MEC_Ang_NFA = df_gst_G21_02a_PTAng_NFA.Filter("mec==1").Histo1D(("G21_PTCut_MEC_Ang_NFA", "", bins, x_start, x_end), "ptl")
    G21_PTCut_RES_Ang_NFA = df_gst_G21_02a_PTAng_NFA.Filter("res==1").Histo1D(("G21_PTCut_RES_Ang_NFA", "", bins, x_start, x_end), "ptl")
    G21_PTCut_DIS_Ang_NFA = df_gst_G21_02a_PTAng_NFA.Filter("dis==1").Histo1D(("G21_PTCut_DIS_Ang_NFA", "", bins, x_start, x_end), "ptl")
    G21_theta_range_NFA = [G21_PTCut_QEL_Ang_NFA.Integral(), G21_PTCut_MEC_Ang_NFA.Integral(), G21_PTCut_RES_Ang_NFA.Integral(), G21_PTCut_DIS_Ang_NFA.Integral()]
    G21_event_lists_NFA = [G21_PTCut_QEL_NFA_List, G21_PTCut_MEC_NFA_List, G21_PTCut_RES_NFA_List, G21_PTCut_DIS_NFA_List]
    G21_range_total_NFA = G21_theta_range_NFA[0] + G21_theta_range_NFA[1] + G21_theta_range_NFA[2] + G21_theta_range_NFA[3]
    G21_PTCut_NFA_Total += G21_range_total_NFA
    G21_difference_NFA = (value_max - value_min)/2
    for i in range(len(G21_theta_range_NFA)):
        if G21_range_total_NFA < n_event_cut and G21_event_lists_NFA[i] == []:
            G21_event_lists_NFA[i].append(0.0)
        if G21_range_total_NFA > n_event_cut: 
            G21_event_lists_NFA[i].append(G21_theta_range_NFA[i]/G21_range_total_NFA)
            if G21_PTCut_Angs_NFA_List == []:
                G21_PTCut_Angs_NFA_List.append(G21_difference_NFA)
            if degrees(value_max - G21_difference_NFA) not in G21_PTCut_Angs_NFA_List:
                G21_PTCut_Angs_NFA_List.append(degrees(value_max - G21_difference_NFA))
G21_PTCut_Angs_NFA_Array = array.array('f', G21_PTCut_Angs_NFA_List)
G21_QEL_Ang_NFA = array.array('f', G21_PTCut_QEL_NFA_List)
G21_MEC_Ang_NFA = array.array('f', G21_PTCut_MEC_NFA_List)
G21_RES_Ang_NFA = array.array('f', G21_PTCut_RES_NFA_List)
G21_DIS_Ang_NFA = array.array('f', G21_PTCut_DIS_NFA_List)

G21_Yerr_QEL_NFA_List = []
G21_Yerr_MEC_NFA_List = []
G21_Yerr_RES_NFA_List = []
G21_Yerr_DIS_NFA_List = []
G21_Yerr_Events_NFA_Lists = [G21_Yerr_QEL_NFA_List, G21_Yerr_MEC_NFA_List, G21_Yerr_RES_NFA_List, G21_Yerr_DIS_NFA_List]
G21_Event_NFA_Lists = [G21_PTCut_QEL_NFA_List, G21_PTCut_MEC_NFA_List, G21_PTCut_RES_NFA_List, G21_PTCut_DIS_NFA_List]
for event in G21_Event_NFA_Lists:
        for fraction in event:
            num = G21_Event_NFA_Lists.index(event)
            G21_Yerr_Events_NFA_Lists[num].append(sqrt(fraction*(1-fraction)/G21_PTCut_NFA_Total))
G21_Yerr_QEL_NFA_Array = array.array('f', G21_Yerr_QEL_NFA_List)
G21_Yerr_MEC_NFA_Array = array.array('f', G21_Yerr_MEC_NFA_List)
G21_Yerr_RES_NFA_Array = array.array('f', G21_Yerr_RES_NFA_List)
G21_Yerr_DIS_NFA_Array = array.array('f', G21_Yerr_DIS_NFA_List)

G21_PTCut_Sum_NFA = []
for i in range(len(G21_PTCut_QEL_NFA_List)):
    count = 0
    count += G21_PTCut_QEL_NFA_List[i] + G21_PTCut_MEC_NFA_List[i] + G21_PTCut_RES_NFA_List[i] + G21_PTCut_DIS_NFA_List[i]
    G21_PTCut_Sum_NFA.append(count)
G21_PTCut_Sum_NFA_Array = array.array('f', G21_PTCut_Sum_NFA)

In [ ]:
#Plotting relative fraction of events

In [ ]:
#G18 v4
G18_Ang_Canvas = ROOT.TCanvas()

G18_QEL_Ang_Plot = ROOT.TGraph(len(G18_PTCut_Angs_Array), G18_PTCut_Angs_Array, G18_QEL_Ang)
G18_QEL_Ang_Plot.SetTitle("Relative Fraction of Interaction Type Per Angle Cut in G18 Model")
G18_QEL_Ang_Plot.GetXaxis().SetTitle("Angle (Degrees)")
G18_QEL_Ang_Plot.GetYaxis().SetTitle("Percentage")
G18_QEL_Ang_Plot.SetMaximum(1.1)
G18_QEL_Ang_Plot.SetMinimum(0.0)
G18_QEL_Ang_Plot.SetMarkerColor(4)
G18_QEL_Ang_Plot.SetMarkerStyle(21)
G18_QEL_Ang_Plot.Draw("same")

G18_MEC_Ang_Plot = ROOT.TGraph(len(G18_PTCut_Angs_Array), G18_PTCut_Angs_Array, G18_MEC_Ang)
G18_MEC_Ang_Plot.Draw("same")
G18_MEC_Ang_Plot.SetMarkerColor(2)
G18_MEC_Ang_Plot.SetMarkerStyle(21)

G18_RES_Ang_Plot = ROOT.TGraph(len(G18_PTCut_Angs_Array), G18_PTCut_Angs_Array, G18_RES_Ang)
G18_RES_Ang_Plot.Draw("same")
G18_RES_Ang_Plot.SetMarkerColor(3)
G18_RES_Ang_Plot.SetMarkerStyle(21)

G18_DIS_Ang_Plot = ROOT.TGraph(len(G18_PTCut_Angs_Array), G18_PTCut_Angs_Array, G18_DIS_Ang)
G18_DIS_Ang_Plot.Draw("same")
G18_DIS_Ang_Plot.SetMarkerColor(6)
G18_DIS_Ang_Plot.SetMarkerStyle(21)

G18_SUM_Ang_Plot = ROOT.TGraph(len(G18_PTCut_Angs_Array), G18_PTCut_Angs_Array, G18_PTCut_Sum_P_Array)
G18_SUM_Ang_Plot.Draw("same")
G18_SUM_Ang_Plot.SetMarkerColor(1)
G18_SUM_Ang_Plot.SetMarkerStyle(34)

G18_Ang_leg = ROOT.TLegend(0.2, 0.85, 0.80, 0.90)
G18_Ang_leg.SetNColumns(5)
G18_Ang_leg.AddEntry(G18_QEL_Ang_Plot, "QEL")
G18_Ang_leg.AddEntry(G18_MEC_Ang_Plot, "MEC")
G18_Ang_leg.AddEntry(G18_RES_Ang_Plot, "RES")
G18_Ang_leg.AddEntry(G18_DIS_Ang_Plot, "DIS")
G18_Ang_leg.AddEntry(G18_SUM_Ang_Plot, "SUM")
G18_Ang_leg.Draw()

G18_Ang_zeroes = []
for i in range(len(G18_PTCut_Angs_Array)):
    G18_Ang_zeroes.append(2.5)
G18_Ang_zeroes_array = array.array('f', G18_Ang_zeroes)

G18_Yerr_QEL_Angs_Bars = ROOT.TGraphErrors(len(G18_PTCut_Angs_Array), G18_PTCut_Angs_Array, G18_QEL_Ang, G18_Ang_zeroes_array, G18_Yerr_QEL_Array)
G18_Yerr_MEC_Angs_Bars = ROOT.TGraphErrors(len(G18_PTCut_Angs_Array), G18_PTCut_Angs_Array, G18_MEC_Ang, G18_Ang_zeroes_array, G18_Yerr_MEC_Array)
G18_Yerr_RES_Angs_Bars = ROOT.TGraphErrors(len(G18_PTCut_Angs_Array), G18_PTCut_Angs_Array, G18_RES_Ang, G18_Ang_zeroes_array, G18_Yerr_RES_Array)
G18_Yerr_DIS_Angs_Bars = ROOT.TGraphErrors(len(G18_PTCut_Angs_Array), G18_PTCut_Angs_Array, G18_DIS_Ang, G18_Ang_zeroes_array, G18_Yerr_DIS_Array)
G18_Yerr_QEL_Angs_Bars.Draw("same")
G18_Yerr_MEC_Angs_Bars.Draw("same")
G18_Yerr_RES_Angs_Bars.Draw("same")
G18_Yerr_DIS_Angs_Bars.Draw("same")

G18_Ang_Canvas.Draw()

In [ ]:
#G18 without Initial State Pions v4
G18_Ang_Canvas_NIP = ROOT.TCanvas()

G18_QEL_NIP_Ang_Plot = ROOT.TGraph(len(G18_PTCut_Angs_NIP_Array), G18_PTCut_Angs_NIP_Array, G18_QEL_Ang_NIP)
G18_QEL_NIP_Ang_Plot.SetTitle("Relative Fraction of Interaction Type Per Angle Cut w/o Initial State Pions in G18 Model")
G18_QEL_NIP_Ang_Plot.GetXaxis().SetTitle("Angle (Degrees)")
G18_QEL_NIP_Ang_Plot.GetYaxis().SetTitle("Percentage")
G18_QEL_NIP_Ang_Plot.SetMaximum(1.1)
G18_QEL_NIP_Ang_Plot.SetMinimum(0.0)
G18_QEL_NIP_Ang_Plot.SetMarkerColor(4)
G18_QEL_NIP_Ang_Plot.SetMarkerStyle(29)
G18_QEL_NIP_Ang_Plot.Draw("same")

G18_MEC_NIP_Ang_Plot = ROOT.TGraph(len(G18_PTCut_Angs_NIP_Array), G18_PTCut_Angs_NIP_Array, G18_MEC_Ang_NIP)
G18_MEC_NIP_Ang_Plot.Draw("same")
G18_MEC_NIP_Ang_Plot.SetMarkerColor(2)
G18_MEC_NIP_Ang_Plot.SetMarkerStyle(29)

G18_RES_NIP_Ang_Plot = ROOT.TGraph(len(G18_PTCut_Angs_NIP_Array), G18_PTCut_Angs_NIP_Array, G18_RES_Ang_NIP)
G18_RES_NIP_Ang_Plot.Draw("same")
G18_RES_NIP_Ang_Plot.SetMarkerColor(3)
G18_RES_NIP_Ang_Plot.SetMarkerStyle(29)

G18_DIS_NIP_Ang_Plot = ROOT.TGraph(len(G18_PTCut_Angs_NIP_Array), G18_PTCut_Angs_NIP_Array, G18_DIS_Ang_NIP)
G18_DIS_NIP_Ang_Plot.Draw("same")
G18_DIS_NIP_Ang_Plot.SetMarkerColor(6)
G18_DIS_NIP_Ang_Plot.SetMarkerStyle(29)

G18_SUM_NIP_Ang_Plot = ROOT.TGraph(len(G18_PTCut_Angs_NIP_Array), G18_PTCut_Angs_NIP_Array, G18_PTCut_Sum_NIP_Array)
G18_SUM_NIP_Ang_Plot.Draw("same")
G18_SUM_NIP_Ang_Plot.SetMarkerColor(1)
G18_SUM_NIP_Ang_Plot.SetMarkerStyle(34)

G18_Ang_leg_NIP = ROOT.TLegend(0.2, 0.85, 0.80, 0.90)
G18_Ang_leg_NIP.SetNColumns(5)
G18_Ang_leg_NIP.AddEntry(G18_QEL_NIP_Ang_Plot, "QEL")
G18_Ang_leg_NIP.AddEntry(G18_MEC_NIP_Ang_Plot, "MEC")
G18_Ang_leg_NIP.AddEntry(G18_RES_NIP_Ang_Plot, "RES")
G18_Ang_leg_NIP.AddEntry(G18_DIS_NIP_Ang_Plot, "DIS")
G18_Ang_leg_NIP.AddEntry(G18_SUM_NIP_Ang_Plot, "SUM")
G18_Ang_leg_NIP.Draw()

G18_Ang_NIP_zeroes = []
for i in range(len(G18_PTCut_Angs_NIP_Array)):
    G18_Ang_NIP_zeroes.append(2.5)
G18_Ang_NIP_zeroes_array = array.array('f', G18_Ang_NIP_zeroes)

G18_Yerr_QEL_NIP_Angs_Bars = ROOT.TGraphErrors(len(G18_PTCut_Angs_NIP_Array), G18_PTCut_Angs_NIP_Array, G18_QEL_Ang_NIP, G18_Ang_NIP_zeroes_array, G18_Yerr_QEL_NIP_Array)
G18_Yerr_MEC_NIP_Angs_Bars = ROOT.TGraphErrors(len(G18_PTCut_Angs_NIP_Array), G18_PTCut_Angs_NIP_Array, G18_MEC_Ang_NIP, G18_Ang_NIP_zeroes_array, G18_Yerr_MEC_NIP_Array)
G18_Yerr_RES_NIP_Angs_Bars = ROOT.TGraphErrors(len(G18_PTCut_Angs_NIP_Array), G18_PTCut_Angs_NIP_Array, G18_RES_Ang_NIP, G18_Ang_NIP_zeroes_array, G18_Yerr_RES_NIP_Array)
G18_Yerr_DIS_NIP_Angs_Bars = ROOT.TGraphErrors(len(G18_PTCut_Angs_NIP_Array), G18_PTCut_Angs_NIP_Array, G18_DIS_Ang_NIP, G18_Ang_NIP_zeroes_array, G18_Yerr_DIS_NIP_Array)
G18_Yerr_QEL_NIP_Angs_Bars.Draw("same")
G18_Yerr_MEC_NIP_Angs_Bars.Draw("same")
G18_Yerr_RES_NIP_Angs_Bars.Draw("same")
G18_Yerr_DIS_NIP_Angs_Bars.Draw("same")

G18_Ang_Canvas_NIP.Draw()

In [ ]:
#G18 without final State Pions v4
G18_Ang_Canvas_NP = ROOT.TCanvas()

G18_QEL_NP_Ang_Plot = ROOT.TGraph(len(G18_PTCut_Angs_NP_Array), G18_PTCut_Angs_NP_Array, G18_QEL_Ang_NP)
G18_QEL_NP_Ang_Plot.SetTitle("Relative Fraction of Interaction Type Per Angle Cut w/o Final State Pions in G18 Model")
G18_QEL_NP_Ang_Plot.GetXaxis().SetTitle("Angle (Degrees)")
G18_QEL_NP_Ang_Plot.GetYaxis().SetTitle("Percentage")
G18_QEL_NP_Ang_Plot.SetMaximum(1.1)
G18_QEL_NP_Ang_Plot.SetMinimum(0.0)
G18_QEL_NP_Ang_Plot.SetMarkerColor(4)
G18_QEL_NP_Ang_Plot.SetMarkerStyle(29)
G18_QEL_NP_Ang_Plot.Draw("same")

G18_MEC_NP_Ang_Plot = ROOT.TGraph(len(G18_PTCut_Angs_NP_Array), G18_PTCut_Angs_NP_Array, G18_MEC_Ang_NP)
G18_MEC_NP_Ang_Plot.Draw("same")
G18_MEC_NP_Ang_Plot.SetMarkerColor(2)
G18_MEC_NP_Ang_Plot.SetMarkerStyle(29)

G18_RES_NP_Ang_Plot = ROOT.TGraph(len(G18_PTCut_Angs_NP_Array), G18_PTCut_Angs_NP_Array, G18_RES_Ang_NP)
G18_RES_NP_Ang_Plot.Draw("same")
G18_RES_NP_Ang_Plot.SetMarkerColor(3)
G18_RES_NP_Ang_Plot.SetMarkerStyle(29)

G18_DIS_NP_Ang_Plot = ROOT.TGraph(len(G18_PTCut_Angs_NP_Array), G18_PTCut_Angs_NP_Array, G18_DIS_Ang_NP)
G18_DIS_NP_Ang_Plot.Draw("same")
G18_DIS_NP_Ang_Plot.SetMarkerColor(6)
G18_DIS_NP_Ang_Plot.SetMarkerStyle(29)

G18_SUM_NP_Ang_Plot = ROOT.TGraph(len(G18_PTCut_Angs_NP_Array), G18_PTCut_Angs_NP_Array, G18_PTCut_Sum_NP_Array)
G18_SUM_NP_Ang_Plot.Draw("same")
G18_SUM_NP_Ang_Plot.SetMarkerColor(1)
G18_SUM_NP_Ang_Plot.SetMarkerStyle(34)

G18_Ang_leg_NP = ROOT.TLegend(0.2, 0.85, 0.80, 0.90)
G18_Ang_leg_NP.SetNColumns(5)
G18_Ang_leg_NP.AddEntry(G18_QEL_NP_Ang_Plot, "QEL")
G18_Ang_leg_NP.AddEntry(G18_MEC_NP_Ang_Plot, "MEC")
G18_Ang_leg_NP.AddEntry(G18_RES_NP_Ang_Plot, "RES")
G18_Ang_leg_NP.AddEntry(G18_DIS_NP_Ang_Plot, "DIS")
G18_Ang_leg_NP.AddEntry(G18_SUM_NP_Ang_Plot, "SUM")
G18_Ang_leg_NP.Draw()

G18_Ang_NP_zeroes = []
for i in range(len(G18_PTCut_Angs_NP_Array)):
    G18_Ang_NP_zeroes.append(2.5)
G18_Ang_NP_zeroes_array = array.array('f', G18_Ang_NP_zeroes)

G18_Yerr_QEL_NP_Angs_Bars = ROOT.TGraphErrors(len(G18_PTCut_Angs_NP_Array), G18_PTCut_Angs_NP_Array, G18_QEL_Ang_NP, G18_Ang_NP_zeroes_array, G18_Yerr_QEL_NP_Array)
G18_Yerr_MEC_NP_Angs_Bars = ROOT.TGraphErrors(len(G18_PTCut_Angs_NP_Array), G18_PTCut_Angs_NP_Array, G18_MEC_Ang_NP, G18_Ang_NP_zeroes_array, G18_Yerr_MEC_NP_Array)
G18_Yerr_RES_NP_Angs_Bars = ROOT.TGraphErrors(len(G18_PTCut_Angs_NP_Array), G18_PTCut_Angs_NP_Array, G18_RES_Ang_NP, G18_Ang_NP_zeroes_array, G18_Yerr_RES_NP_Array)
G18_Yerr_DIS_NP_Angs_Bars = ROOT.TGraphErrors(len(G18_PTCut_Angs_NP_Array), G18_PTCut_Angs_NP_Array, G18_DIS_Ang_NP, G18_Ang_NP_zeroes_array, G18_Yerr_DIS_NP_Array)
G18_Yerr_QEL_NP_Angs_Bars.Draw("same")
G18_Yerr_MEC_NP_Angs_Bars.Draw("same")
G18_Yerr_RES_NP_Angs_Bars.Draw("same")
G18_Yerr_DIS_NP_Angs_Bars.Draw("same")

G18_Ang_Canvas_NP.Draw()

In [ ]:
#G18 without res/dis v4
G18_Ang_Canvas_NRD = ROOT.TCanvas()

G18_QEL_NRD_Ang_Plot = ROOT.TGraph(len(G18_PTCut_Angs_NRD_Array), G18_PTCut_Angs_NRD_Array, G18_QEL_Ang_NRD)
G18_QEL_NRD_Ang_Plot.SetTitle("Relative Fraction of Interaction Type Per Angle Cut w/o RES/DIS Events in G18 Model")
G18_QEL_NRD_Ang_Plot.GetXaxis().SetTitle("Angle (Degrees)")
G18_QEL_NRD_Ang_Plot.GetYaxis().SetTitle("Percentage")
G18_QEL_NRD_Ang_Plot.SetMaximum(1.1)
G18_QEL_NRD_Ang_Plot.SetMinimum(0.0)
G18_QEL_NRD_Ang_Plot.SetMarkerColor(4)
G18_QEL_NRD_Ang_Plot.SetMarkerStyle(29)
G18_QEL_NRD_Ang_Plot.Draw("same")

G18_MEC_NRD_Ang_Plot = ROOT.TGraph(len(G18_PTCut_Angs_NRD_Array), G18_PTCut_Angs_NRD_Array, G18_MEC_Ang_NRD)
G18_MEC_NRD_Ang_Plot.Draw("same")
G18_MEC_NRD_Ang_Plot.SetMarkerColor(2)
G18_MEC_NRD_Ang_Plot.SetMarkerStyle(29)

G18_SUM_NRD_Ang_Plot = ROOT.TGraph(len(G18_PTCut_Angs_NRD_Array), G18_PTCut_Angs_NRD_Array, G18_PTCut_Sum_NRD_Array)
G18_SUM_NRD_Ang_Plot.Draw("same")
G18_SUM_NRD_Ang_Plot.SetMarkerColor(1)
G18_SUM_NRD_Ang_Plot.SetMarkerStyle(34)

G18_Ang_leg_NRD = ROOT.TLegend(0.2, 0.85, 0.80, 0.90)
G18_Ang_leg_NRD.SetNColumns(3)
G18_Ang_leg_NRD.AddEntry(G18_QEL_NRD_Ang_Plot, "QEL")
G18_Ang_leg_NRD.AddEntry(G18_MEC_NRD_Ang_Plot, "MEC")
G18_Ang_leg_NRD.AddEntry(G18_SUM_NRD_Ang_Plot, "SUM")
G18_Ang_leg_NRD.Draw()

G18_Ang_NRD_zeroes = []
for i in range(len(G18_PTCut_Angs_NRD_Array)):
    G18_Ang_NRD_zeroes.append(2.5)
G18_Ang_NRD_zeroes_array = array.array('f', G18_Ang_NRD_zeroes)

G18_Yerr_QEL_NRD_Angs_Bars = ROOT.TGraphErrors(len(G18_PTCut_Angs_NRD_Array), G18_PTCut_Angs_NRD_Array, G18_QEL_Ang_NRD, G18_Ang_NRD_zeroes_array, G18_Yerr_QEL_NRD_Array)
G18_Yerr_MEC_NRD_Angs_Bars = ROOT.TGraphErrors(len(G18_PTCut_Angs_NRD_Array), G18_PTCut_Angs_NRD_Array, G18_MEC_Ang_NRD, G18_Ang_NRD_zeroes_array, G18_Yerr_MEC_NRD_Array)
G18_Yerr_QEL_NRD_Angs_Bars.Draw("same")
G18_Yerr_MEC_NRD_Angs_Bars.Draw("same")

G18_Ang_Canvas_NRD.Draw()

In [ ]:
#G18 without res/dis/final state pions v4
G18_Ang_Canvas_NRDP = ROOT.TCanvas()

G18_QEL_NRDP_Ang_Plot = ROOT.TGraph(len(G18_PTCut_Angs_NRDP_Array), G18_PTCut_Angs_NRDP_Array, G18_QEL_Ang_NRDP)
G18_QEL_NRDP_Ang_Plot.SetTitle("Relative Fraction of Interaction Type Per Angle Cut w/o RES/DIS/Final State Pions in G18 Model")
G18_QEL_NRDP_Ang_Plot.GetXaxis().SetTitle("Angle (Degrees)")
G18_QEL_NRDP_Ang_Plot.GetYaxis().SetTitle("Percentage")
G18_QEL_NRDP_Ang_Plot.SetMaximum(1.1)
G18_QEL_NRDP_Ang_Plot.SetMinimum(0.0)
G18_QEL_NRDP_Ang_Plot.SetMarkerColor(4)
G18_QEL_NRDP_Ang_Plot.SetMarkerStyle(29)
G18_QEL_NRDP_Ang_Plot.Draw("same")

G18_MEC_NRDP_Ang_Plot = ROOT.TGraph(len(G18_PTCut_Angs_NRDP_Array), G18_PTCut_Angs_NRDP_Array, G18_MEC_Ang_NRDP)
G18_MEC_NRDP_Ang_Plot.Draw("same")
G18_MEC_NRDP_Ang_Plot.SetMarkerColor(2)
G18_MEC_NRDP_Ang_Plot.SetMarkerStyle(29)

G18_SUM_NRDP_Ang_Plot = ROOT.TGraph(len(G18_PTCut_Angs_NRDP_Array), G18_PTCut_Angs_NRDP_Array, G18_PTCut_Sum_NRDP_Array)
G18_SUM_NRDP_Ang_Plot.Draw("same")
G18_SUM_NRDP_Ang_Plot.SetMarkerColor(1)
G18_SUM_NRDP_Ang_Plot.SetMarkerStyle(34)

G18_Ang_leg_NRDP = ROOT.TLegend(0.2, 0.85, 0.80, 0.90)
G18_Ang_leg_NRDP.SetNColumns(3)
G18_Ang_leg_NRDP.AddEntry(G18_QEL_NRDP_Ang_Plot, "QEL")
G18_Ang_leg_NRDP.AddEntry(G18_MEC_NRDP_Ang_Plot, "MEC")
G18_Ang_leg_NRDP.AddEntry(G18_SUM_NRDP_Ang_Plot, "SUM")
G18_Ang_leg_NRDP.Draw()

G18_Ang_NRDP_zeroes = []
for i in range(len(G18_PTCut_Angs_NRDP_Array)):
    G18_Ang_NRDP_zeroes.append(2.5)
G18_Ang_NRDP_zeroes_array = array.array('f', G18_Ang_NRDP_zeroes)

G18_Yerr_QEL_NRDP_Angs_Bars = ROOT.TGraphErrors(len(G18_PTCut_Angs_NRDP_Array), G18_PTCut_Angs_NRDP_Array, G18_QEL_Ang_NRDP, G18_Ang_NRDP_zeroes_array, G18_Yerr_QEL_NRDP_Array)
G18_Yerr_MEC_NRDP_Angs_Bars = ROOT.TGraphErrors(len(G18_PTCut_Angs_NRDP_Array), G18_PTCut_Angs_NRDP_Array, G18_MEC_Ang_NRDP, G18_Ang_NRDP_zeroes_array, G18_Yerr_MEC_NRDP_Array)
G18_Yerr_QEL_NRDP_Angs_Bars.Draw("same")
G18_Yerr_MEC_NRDP_Angs_Bars.Draw("same")

G18_Ang_Canvas_NRDP.Draw()

In [ ]:
#G18 without nfa Pions v4
G18_Ang_Canvas_NFA = ROOT.TCanvas()

G18_QEL_NFA_Ang_Plot = ROOT.TGraph(len(G18_PTCut_Angs_NFA_Array), G18_PTCut_Angs_NFA_Array, G18_QEL_Ang_NFA)
G18_QEL_NFA_Ang_Plot.SetTitle("Relative Fraction of Interaction Type Per Angle Cut w/o NFA Pions in G18 Model")
G18_QEL_NFA_Ang_Plot.GetXaxis().SetTitle("Angle (Degrees)")
G18_QEL_NFA_Ang_Plot.GetYaxis().SetTitle("Percentage")
G18_QEL_NFA_Ang_Plot.SetMaximum(1.1)
G18_QEL_NFA_Ang_Plot.SetMinimum(0.0)
G18_QEL_NFA_Ang_Plot.SetMarkerColor(4)
G18_QEL_NFA_Ang_Plot.SetMarkerStyle(29)
G18_QEL_NFA_Ang_Plot.Draw("same")

G18_MEC_NFA_Ang_Plot = ROOT.TGraph(len(G18_PTCut_Angs_NFA_Array), G18_PTCut_Angs_NFA_Array, G18_MEC_Ang_NFA)
G18_MEC_NFA_Ang_Plot.Draw("same")
G18_MEC_NFA_Ang_Plot.SetMarkerColor(2)
G18_MEC_NFA_Ang_Plot.SetMarkerStyle(29)

G18_RES_NFA_Ang_Plot = ROOT.TGraph(len(G18_PTCut_Angs_NFA_Array), G18_PTCut_Angs_NFA_Array, G18_RES_Ang_NFA)
G18_RES_NFA_Ang_Plot.Draw("same")
G18_RES_NFA_Ang_Plot.SetMarkerColor(3)
G18_RES_NFA_Ang_Plot.SetMarkerStyle(29)

G18_DIS_NFA_Ang_Plot = ROOT.TGraph(len(G18_PTCut_Angs_NFA_Array), G18_PTCut_Angs_NFA_Array, G18_DIS_Ang_NFA)
G18_DIS_NFA_Ang_Plot.Draw("same")
G18_DIS_NFA_Ang_Plot.SetMarkerColor(6)
G18_DIS_NFA_Ang_Plot.SetMarkerStyle(29)

G18_SUM_NFA_Ang_Plot = ROOT.TGraph(len(G18_PTCut_Angs_NFA_Array), G18_PTCut_Angs_NFA_Array, G18_PTCut_Sum_NFA_Array)
G18_SUM_NFA_Ang_Plot.Draw("same")
G18_SUM_NFA_Ang_Plot.SetMarkerColor(1)
G18_SUM_NFA_Ang_Plot.SetMarkerStyle(34)

G18_Ang_leg_NFA = ROOT.TLegend(0.2, 0.85, 0.80, 0.90)
G18_Ang_leg_NFA.SetNColumns(5)
G18_Ang_leg_NFA.AddEntry(G18_QEL_NFA_Ang_Plot, "QEL")
G18_Ang_leg_NFA.AddEntry(G18_MEC_NFA_Ang_Plot, "MEC")
G18_Ang_leg_NFA.AddEntry(G18_RES_NFA_Ang_Plot, "RES")
G18_Ang_leg_NFA.AddEntry(G18_DIS_NFA_Ang_Plot, "DIS")
G18_Ang_leg_NFA.AddEntry(G18_SUM_NFA_Ang_Plot, "SUM")
G18_Ang_leg_NFA.Draw()

G18_Ang_NFA_zeroes = []
for i in range(len(G18_PTCut_Angs_NFA_Array)):
    G18_Ang_NFA_zeroes.append(2.5)
G18_Ang_NFA_zeroes_array = array.array('f', G18_Ang_NFA_zeroes)

G18_Yerr_QEL_NFA_Angs_Bars = ROOT.TGraphErrors(len(G18_PTCut_Angs_NFA_Array), G18_PTCut_Angs_NFA_Array, G18_QEL_Ang_NFA, G18_Ang_NFA_zeroes_array, G18_Yerr_QEL_NFA_Array)
G18_Yerr_MEC_NFA_Angs_Bars = ROOT.TGraphErrors(len(G18_PTCut_Angs_NFA_Array), G18_PTCut_Angs_NFA_Array, G18_MEC_Ang_NFA, G18_Ang_NFA_zeroes_array, G18_Yerr_MEC_NFA_Array)
G18_Yerr_RES_NFA_Angs_Bars = ROOT.TGraphErrors(len(G18_PTCut_Angs_NFA_Array), G18_PTCut_Angs_NFA_Array, G18_RES_Ang_NFA, G18_Ang_NFA_zeroes_array, G18_Yerr_RES_NFA_Array)
G18_Yerr_DIS_NFA_Angs_Bars = ROOT.TGraphErrors(len(G18_PTCut_Angs_NFA_Array), G18_PTCut_Angs_NFA_Array, G18_DIS_Ang_NFA, G18_Ang_NFA_zeroes_array, G18_Yerr_DIS_NFA_Array)
G18_Yerr_QEL_NFA_Angs_Bars.Draw("same")
G18_Yerr_MEC_NFA_Angs_Bars.Draw("same")
G18_Yerr_RES_NFA_Angs_Bars.Draw("same")
G18_Yerr_DIS_NFA_Angs_Bars.Draw("same")

G18_Ang_Canvas_NFA.Draw()

In [ ]:
# GEM21 Model Plots of Angle Cuts V3

In [ ]:
#Gem21 v4
G21_Ang_Canvas = ROOT.TCanvas()

G21_QEL_Ang_Plot = ROOT.TGraph(len(G21_PTCut_Angs_Array), G21_PTCut_Angs_Array, G21_QEL_Ang)
G21_QEL_Ang_Plot.SetTitle("Relative Fraction of Interaction Type Per Angle Cut in GEM21 Model")
G21_QEL_Ang_Plot.GetXaxis().SetTitle("Angle (Degrees)")
G21_QEL_Ang_Plot.GetYaxis().SetTitle("Percentage")
G21_QEL_Ang_Plot.SetMaximum(1.1)
G21_QEL_Ang_Plot.SetMinimum(0.0)
G21_QEL_Ang_Plot.SetMarkerColor(4)
G21_QEL_Ang_Plot.SetMarkerStyle(21)
G21_QEL_Ang_Plot.Draw("same")

G21_MEC_Ang_Plot = ROOT.TGraph(len(G21_PTCut_Angs_Array), G21_PTCut_Angs_Array, G21_MEC_Ang)
G21_MEC_Ang_Plot.Draw("same")
G21_MEC_Ang_Plot.SetMarkerColor(2)
G21_MEC_Ang_Plot.SetMarkerStyle(21)

G21_RES_Ang_Plot = ROOT.TGraph(len(G21_PTCut_Angs_Array), G21_PTCut_Angs_Array, G21_RES_Ang)
G21_RES_Ang_Plot.Draw("same")
G21_RES_Ang_Plot.SetMarkerColor(3)
G21_RES_Ang_Plot.SetMarkerStyle(21)

G21_DIS_Ang_Plot = ROOT.TGraph(len(G21_PTCut_Angs_Array), G21_PTCut_Angs_Array, G21_DIS_Ang)
G21_DIS_Ang_Plot.Draw("same")
G21_DIS_Ang_Plot.SetMarkerColor(6)
G21_DIS_Ang_Plot.SetMarkerStyle(21)

G21_SUM_Ang_Plot = ROOT.TGraph(len(G21_PTCut_Angs_Array), G21_PTCut_Angs_Array, G21_PTCut_Sum_P_Array)
G21_SUM_Ang_Plot.Draw("same")
G21_SUM_Ang_Plot.SetMarkerColor(1)
G21_SUM_Ang_Plot.SetMarkerStyle(34)

G21_Ang_leg = ROOT.TLegend(0.2, 0.85, 0.80, 0.90)
G21_Ang_leg.SetNColumns(5)
G21_Ang_leg.AddEntry(G21_QEL_Ang_Plot, "QEL")
G21_Ang_leg.AddEntry(G21_MEC_Ang_Plot, "MEC")
G21_Ang_leg.AddEntry(G21_RES_Ang_Plot, "RES")
G21_Ang_leg.AddEntry(G21_DIS_Ang_Plot, "DIS")
G21_Ang_leg.AddEntry(G21_SUM_Ang_Plot, "SUM")
G21_Ang_leg.Draw()

G21_Ang_zeroes = []
for i in range(len(G21_PTCut_Angs_Array)):
    G21_Ang_zeroes.append(2.5)
G21_Ang_zeroes_array = array.array('f', G21_Ang_zeroes)

G21_Yerr_QEL_Angs_Bars = ROOT.TGraphErrors(len(G21_PTCut_Angs_Array), G21_PTCut_Angs_Array, G21_QEL_Ang, G21_Ang_zeroes_array, G21_Yerr_QEL_Array)
G21_Yerr_MEC_Angs_Bars = ROOT.TGraphErrors(len(G21_PTCut_Angs_Array), G21_PTCut_Angs_Array, G21_MEC_Ang, G21_Ang_zeroes_array, G21_Yerr_MEC_Array)
G21_Yerr_RES_Angs_Bars = ROOT.TGraphErrors(len(G21_PTCut_Angs_Array), G21_PTCut_Angs_Array, G21_RES_Ang, G21_Ang_zeroes_array, G21_Yerr_RES_Array)
G21_Yerr_DIS_Angs_Bars = ROOT.TGraphErrors(len(G21_PTCut_Angs_Array), G21_PTCut_Angs_Array, G21_DIS_Ang, G21_Ang_zeroes_array, G21_Yerr_DIS_Array)
G21_Yerr_QEL_Angs_Bars.Draw("same")
G21_Yerr_MEC_Angs_Bars.Draw("same")
G21_Yerr_RES_Angs_Bars.Draw("same")
G21_Yerr_DIS_Angs_Bars.Draw("same")

G21_Ang_Canvas.Draw()

In [ ]:
#GEM21 without Initial State Pions v4
G21_Ang_Canvas_NIP = ROOT.TCanvas()

G21_QEL_NIP_Ang_Plot = ROOT.TGraph(len(G21_PTCut_Angs_NIP_Array), G21_PTCut_Angs_NIP_Array, G21_QEL_Ang_NIP)
G21_QEL_NIP_Ang_Plot.SetTitle("Relative Fraction of Interaction Type Per Angle Cut w/o Initial State Pions in GEM21 Model")
G21_QEL_NIP_Ang_Plot.GetXaxis().SetTitle("Angle (Degrees)")
G21_QEL_NIP_Ang_Plot.GetYaxis().SetTitle("Percentage")
G21_QEL_NIP_Ang_Plot.SetMaximum(1.1)
G21_QEL_NIP_Ang_Plot.SetMinimum(0.0)
G21_QEL_NIP_Ang_Plot.SetMarkerColor(4)
G21_QEL_NIP_Ang_Plot.SetMarkerStyle(29)
G21_QEL_NIP_Ang_Plot.Draw("same")

G21_MEC_NIP_Ang_Plot = ROOT.TGraph(len(G21_PTCut_Angs_NIP_Array), G21_PTCut_Angs_NIP_Array, G21_MEC_Ang_NIP)
G21_MEC_NIP_Ang_Plot.Draw("same")
G21_MEC_NIP_Ang_Plot.SetMarkerColor(2)
G21_MEC_NIP_Ang_Plot.SetMarkerStyle(29)

G21_RES_NIP_Ang_Plot = ROOT.TGraph(len(G21_PTCut_Angs_NIP_Array), G21_PTCut_Angs_NIP_Array, G21_RES_Ang_NIP)
G21_RES_NIP_Ang_Plot.Draw("same")
G21_RES_NIP_Ang_Plot.SetMarkerColor(3)
G21_RES_NIP_Ang_Plot.SetMarkerStyle(29)

G21_DIS_NIP_Ang_Plot = ROOT.TGraph(len(G21_PTCut_Angs_NIP_Array), G21_PTCut_Angs_NIP_Array, G21_DIS_Ang_NIP)
G21_DIS_NIP_Ang_Plot.Draw("same")
G21_DIS_NIP_Ang_Plot.SetMarkerColor(6)
G21_DIS_NIP_Ang_Plot.SetMarkerStyle(29)

G21_SUM_NIP_Ang_Plot = ROOT.TGraph(len(G21_PTCut_Angs_NIP_Array), G21_PTCut_Angs_NIP_Array, G21_PTCut_Sum_NIP_Array)
G21_SUM_NIP_Ang_Plot.Draw("same")
G21_SUM_NIP_Ang_Plot.SetMarkerColor(1)
G21_SUM_NIP_Ang_Plot.SetMarkerStyle(34)

G21_Ang_leg_NIP = ROOT.TLegend(0.2, 0.85, 0.80, 0.90)
G21_Ang_leg_NIP.SetNColumns(5)
G21_Ang_leg_NIP.AddEntry(G21_QEL_NIP_Ang_Plot, "QEL")
G21_Ang_leg_NIP.AddEntry(G21_MEC_NIP_Ang_Plot, "MEC")
G21_Ang_leg_NIP.AddEntry(G21_RES_NIP_Ang_Plot, "RES")
G21_Ang_leg_NIP.AddEntry(G21_DIS_NIP_Ang_Plot, "DIS")
G21_Ang_leg_NIP.AddEntry(G21_DIS_NIP_Ang_Plot, "SUM")
G21_Ang_leg_NIP.Draw()

G21_Ang_NIP_zeroes = []
for i in range(len(G21_PTCut_Angs_NIP_Array)):
    G21_Ang_NIP_zeroes.append(2.5)
G21_Ang_NIP_zeroes_array = array.array('f', G21_Ang_NIP_zeroes)

G21_Yerr_QEL_NIP_Angs_Bars = ROOT.TGraphErrors(len(G21_PTCut_Angs_NIP_Array), G21_PTCut_Angs_NIP_Array, G21_QEL_Ang_NIP, G21_Ang_NIP_zeroes_array, G21_Yerr_QEL_NIP_Array)
G21_Yerr_MEC_NIP_Angs_Bars = ROOT.TGraphErrors(len(G21_PTCut_Angs_NIP_Array), G21_PTCut_Angs_NIP_Array, G21_MEC_Ang_NIP, G21_Ang_NIP_zeroes_array, G21_Yerr_MEC_NIP_Array)
G21_Yerr_RES_NIP_Angs_Bars = ROOT.TGraphErrors(len(G21_PTCut_Angs_NIP_Array), G21_PTCut_Angs_NIP_Array, G21_RES_Ang_NIP, G21_Ang_NIP_zeroes_array, G21_Yerr_RES_NIP_Array)
G21_Yerr_DIS_NIP_Angs_Bars = ROOT.TGraphErrors(len(G21_PTCut_Angs_NIP_Array), G21_PTCut_Angs_NIP_Array, G21_DIS_Ang_NIP, G21_Ang_NIP_zeroes_array, G21_Yerr_DIS_NIP_Array)
G21_Yerr_QEL_NIP_Angs_Bars.Draw("same")
G21_Yerr_MEC_NIP_Angs_Bars.Draw("same")
G21_Yerr_RES_NIP_Angs_Bars.Draw("same")
G21_Yerr_DIS_NIP_Angs_Bars.Draw("same")

G21_Ang_Canvas_NIP.Draw()

In [ ]:
#GEM21 without final State Pions v4
G21_Ang_Canvas_NP = ROOT.TCanvas()

G21_QEL_NP_Ang_Plot = ROOT.TGraph(len(G21_PTCut_Angs_NP_Array), G21_PTCut_Angs_NP_Array, G21_QEL_Ang_NP)
G21_QEL_NP_Ang_Plot.SetTitle("Relative Fraction of Interaction Type Per Angle Cut w/o Final State Pions in GEM21 Model")
G21_QEL_NP_Ang_Plot.GetXaxis().SetTitle("Angle (Degrees)")
G21_QEL_NP_Ang_Plot.GetYaxis().SetTitle("Percentage")
G21_QEL_NP_Ang_Plot.SetMaximum(1.1)
G21_QEL_NP_Ang_Plot.SetMinimum(0.0)
G21_QEL_NP_Ang_Plot.SetMarkerColor(4)
G21_QEL_NP_Ang_Plot.SetMarkerStyle(29)
G21_QEL_NP_Ang_Plot.Draw("same")

G21_MEC_NP_Ang_Plot = ROOT.TGraph(len(G21_PTCut_Angs_NP_Array), G21_PTCut_Angs_NP_Array, G21_MEC_Ang_NP)
G21_MEC_NP_Ang_Plot.Draw("same")
G21_MEC_NP_Ang_Plot.SetMarkerColor(2)
G21_MEC_NP_Ang_Plot.SetMarkerStyle(29)

G21_RES_NP_Ang_Plot = ROOT.TGraph(len(G21_PTCut_Angs_NP_Array), G21_PTCut_Angs_NP_Array, G21_RES_Ang_NP)
G21_RES_NP_Ang_Plot.Draw("same")
G21_RES_NP_Ang_Plot.SetMarkerColor(3)
G21_RES_NP_Ang_Plot.SetMarkerStyle(29)

G21_DIS_NP_Ang_Plot = ROOT.TGraph(len(G21_PTCut_Angs_NP_Array), G21_PTCut_Angs_NP_Array, G21_DIS_Ang_NP)
G21_DIS_NP_Ang_Plot.Draw("same")
G21_DIS_NP_Ang_Plot.SetMarkerColor(6)
G21_DIS_NP_Ang_Plot.SetMarkerStyle(29)

G21_SUM_NP_Ang_Plot = ROOT.TGraph(len(G21_PTCut_Angs_NP_Array), G21_PTCut_Angs_NP_Array, G21_PTCut_Sum_NP_Array)
G21_SUM_NP_Ang_Plot.Draw("same")
G21_SUM_NP_Ang_Plot.SetMarkerColor(1)
G21_SUM_NP_Ang_Plot.SetMarkerStyle(34)

G21_Ang_leg_NP = ROOT.TLegend(0.2, 0.85, 0.80, 0.90)
G21_Ang_leg_NP.SetNColumns(5)
G21_Ang_leg_NP.AddEntry(G21_QEL_NP_Ang_Plot, "QEL")
G21_Ang_leg_NP.AddEntry(G21_MEC_NP_Ang_Plot, "MEC")
G21_Ang_leg_NP.AddEntry(G21_RES_NP_Ang_Plot, "RES")
G21_Ang_leg_NP.AddEntry(G21_DIS_NP_Ang_Plot, "DIS")
G21_Ang_leg_NP.AddEntry(G21_SUM_NP_Ang_Plot, "SUM")
G21_Ang_leg_NP.Draw()

G21_Ang_NP_zeroes = []
for i in range(len(G21_PTCut_Angs_NP_Array)):
    G21_Ang_NP_zeroes.append(2.5)
G21_Ang_NP_zeroes_array = array.array('f', G21_Ang_NP_zeroes)

G21_Yerr_QEL_NP_Angs_Bars = ROOT.TGraphErrors(len(G21_PTCut_Angs_NP_Array), G21_PTCut_Angs_NP_Array, G21_QEL_Ang_NP, G21_Ang_NP_zeroes_array, G21_Yerr_QEL_NP_Array)
G21_Yerr_MEC_NP_Angs_Bars = ROOT.TGraphErrors(len(G21_PTCut_Angs_NP_Array), G21_PTCut_Angs_NP_Array, G21_MEC_Ang_NP, G21_Ang_NP_zeroes_array, G21_Yerr_MEC_NP_Array)
G21_Yerr_RES_NP_Angs_Bars = ROOT.TGraphErrors(len(G21_PTCut_Angs_NP_Array), G21_PTCut_Angs_NP_Array, G21_RES_Ang_NP, G21_Ang_NP_zeroes_array, G21_Yerr_RES_NP_Array)
G21_Yerr_DIS_NP_Angs_Bars = ROOT.TGraphErrors(len(G21_PTCut_Angs_NP_Array), G21_PTCut_Angs_NP_Array, G21_DIS_Ang_NP, G21_Ang_NP_zeroes_array, G21_Yerr_DIS_NP_Array)
G21_Yerr_QEL_NP_Angs_Bars.Draw("same")
G21_Yerr_MEC_NP_Angs_Bars.Draw("same")
G21_Yerr_RES_NP_Angs_Bars.Draw("same")
G21_Yerr_DIS_NP_Angs_Bars.Draw("same")

G21_Ang_Canvas_NP.Draw()

In [ ]:
#GEM21 without res/dis v4
G21_Ang_Canvas_NRD = ROOT.TCanvas()

G21_QEL_NRD_Ang_Plot = ROOT.TGraph(len(G21_PTCut_Angs_NRD_Array), G21_PTCut_Angs_NRD_Array, G21_QEL_Ang_NRD)
G21_QEL_NRD_Ang_Plot.SetTitle("Relative Fraction of Interaction Type Per Angle Cut w/o RES/DIS Events in GEM21 Model")
G21_QEL_NRD_Ang_Plot.GetXaxis().SetTitle("Angle (Degrees)")
G21_QEL_NRD_Ang_Plot.GetYaxis().SetTitle("Percentage")
G21_QEL_NRD_Ang_Plot.SetMaximum(1.1)
G21_QEL_NRD_Ang_Plot.SetMinimum(0.0)
G21_QEL_NRD_Ang_Plot.SetMarkerColor(4)
G21_QEL_NRD_Ang_Plot.SetMarkerStyle(29)
G21_QEL_NRD_Ang_Plot.Draw("same")

G21_MEC_NRD_Ang_Plot = ROOT.TGraph(len(G21_PTCut_Angs_NRD_Array), G21_PTCut_Angs_NRD_Array, G21_MEC_Ang_NRD)
G21_MEC_NRD_Ang_Plot.Draw("same")
G21_MEC_NRD_Ang_Plot.SetMarkerColor(2)
G21_MEC_NRD_Ang_Plot.SetMarkerStyle(29)

G21_SUM_NRD_Ang_Plot = ROOT.TGraph(len(G21_PTCut_Angs_NRD_Array), G21_PTCut_Angs_NRD_Array, G21_PTCut_Sum_NRD_Array)
G21_SUM_NRD_Ang_Plot.Draw("same")
G21_SUM_NRD_Ang_Plot.SetMarkerColor(1)
G21_SUM_NRD_Ang_Plot.SetMarkerStyle(34)

G21_Ang_leg_NRD = ROOT.TLegend(0.2, 0.85, 0.80, 0.90)
G21_Ang_leg_NRD.SetNColumns(3)
G21_Ang_leg_NRD.AddEntry(G21_QEL_NRD_Ang_Plot, "QEL")
G21_Ang_leg_NRD.AddEntry(G21_MEC_NRD_Ang_Plot, "MEC")
G21_Ang_leg_NRD.AddEntry(G21_SUM_NRD_Ang_Plot, "SUM")
G21_Ang_leg_NRD.Draw()

G21_Ang_NRD_zeroes = []
for i in range(len(G21_PTCut_Angs_NRD_Array)):
    G21_Ang_NRD_zeroes.append(2.5)
G21_Ang_NRD_zeroes_array = array.array('f', G21_Ang_NRD_zeroes)

G21_Yerr_QEL_NRD_Angs_Bars = ROOT.TGraphErrors(len(G21_PTCut_Angs_NRD_Array), G21_PTCut_Angs_NRD_Array, G21_QEL_Ang_NRD, G21_Ang_NRD_zeroes_array, G21_Yerr_QEL_NRD_Array)
G21_Yerr_MEC_NRD_Angs_Bars = ROOT.TGraphErrors(len(G21_PTCut_Angs_NRD_Array), G21_PTCut_Angs_NRD_Array, G21_MEC_Ang_NRD, G21_Ang_NRD_zeroes_array, G21_Yerr_MEC_NRD_Array)
G21_Yerr_QEL_NRD_Angs_Bars.Draw("same")
G21_Yerr_MEC_NRD_Angs_Bars.Draw("same")

G21_Ang_Canvas_NRD.Draw()

In [ ]:
#GEM21 without res/dis/final state pions v4
G21_Ang_Canvas_NRDP = ROOT.TCanvas()

G21_QEL_NRDP_Ang_Plot = ROOT.TGraph(len(G21_PTCut_Angs_NRDP_Array), G21_PTCut_Angs_NRDP_Array, G21_QEL_Ang_NRDP)
G21_QEL_NRDP_Ang_Plot.SetTitle("Relative Fraction of Interaction Type Per Angle Cut w/o RES/DIS/Final State Pions in GEM21 Model")
G21_QEL_NRDP_Ang_Plot.GetXaxis().SetTitle("Angle (Degrees)")
G21_QEL_NRDP_Ang_Plot.GetYaxis().SetTitle("Percentage")
G21_QEL_NRDP_Ang_Plot.SetMaximum(1.1)
G21_QEL_NRDP_Ang_Plot.SetMinimum(0.0)
G21_QEL_NRDP_Ang_Plot.SetMarkerColor(4)
G21_QEL_NRDP_Ang_Plot.SetMarkerStyle(29)
G21_QEL_NRDP_Ang_Plot.Draw("same")

G21_MEC_NRDP_Ang_Plot = ROOT.TGraph(len(G21_PTCut_Angs_NRDP_Array), G21_PTCut_Angs_NRDP_Array, G21_MEC_Ang_NRDP)
G21_MEC_NRDP_Ang_Plot.Draw("same")
G21_MEC_NRDP_Ang_Plot.SetMarkerColor(2)
G21_MEC_NRDP_Ang_Plot.SetMarkerStyle(29)

G21_SUM_NRDP_Ang_Plot = ROOT.TGraph(len(G21_PTCut_Angs_NRDP_Array), G21_PTCut_Angs_NRDP_Array, G21_PTCut_Sum_NRDP_Array)
G21_SUM_NRDP_Ang_Plot.Draw("same")
G21_SUM_NRDP_Ang_Plot.SetMarkerColor(1)
G21_SUM_NRDP_Ang_Plot.SetMarkerStyle(34)

G21_Ang_leg_NRDP = ROOT.TLegend(0.2, 0.85, 0.80, 0.90)
G21_Ang_leg_NRDP.SetNColumns(3)
G21_Ang_leg_NRDP.AddEntry(G21_QEL_NRDP_Ang_Plot, "QEL")
G21_Ang_leg_NRDP.AddEntry(G21_MEC_NRDP_Ang_Plot, "MEC")
G21_Ang_leg_NRDP.AddEntry(G21_SUM_NRDP_Ang_Plot, "SUM")
G21_Ang_leg_NRDP.Draw()

G21_Ang_NRDP_zeroes = []
for i in range(len(G21_PTCut_Angs_NRDP_Array)):
    G21_Ang_NRDP_zeroes.append(2.5)
G21_Ang_NRDP_zeroes_array = array.array('f', G21_Ang_NRDP_zeroes)

G21_Yerr_QEL_NRDP_Angs_Bars = ROOT.TGraphErrors(len(G21_PTCut_Angs_NRDP_Array), G21_PTCut_Angs_NRDP_Array, G21_QEL_Ang_NRDP, G21_Ang_NRDP_zeroes_array, G21_Yerr_QEL_NRDP_Array)
G21_Yerr_MEC_NRDP_Angs_Bars = ROOT.TGraphErrors(len(G21_PTCut_Angs_NRDP_Array), G21_PTCut_Angs_NRDP_Array, G21_MEC_Ang_NRDP, G21_Ang_NRDP_zeroes_array, G21_Yerr_MEC_NRDP_Array)
G21_Yerr_QEL_NRDP_Angs_Bars.Draw("same")
G21_Yerr_MEC_NRDP_Angs_Bars.Draw("same")

G21_Ang_Canvas_NRDP.Draw()

In [ ]:
#GEM21 without nfa Pions v4
G21_Ang_Canvas_NFA = ROOT.TCanvas()

G21_QEL_NFA_Ang_Plot = ROOT.TGraph(len(G21_PTCut_Angs_NFA_Array), G21_PTCut_Angs_NFA_Array, G21_QEL_Ang_NFA)
G21_QEL_NFA_Ang_Plot.SetTitle("Relative Fraction of Interaction Type Per Angle Cut w/o NFA Pions in GEM21 Model")
G21_QEL_NFA_Ang_Plot.GetXaxis().SetTitle("Angle (Degrees)")
G21_QEL_NFA_Ang_Plot.GetYaxis().SetTitle("Percentage")
G21_QEL_NFA_Ang_Plot.SetMaximum(1.1)
G21_QEL_NFA_Ang_Plot.SetMinimum(0.0)
G21_QEL_NFA_Ang_Plot.SetMarkerColor(4)
G21_QEL_NFA_Ang_Plot.SetMarkerStyle(29)
G21_QEL_NFA_Ang_Plot.Draw("same")

G21_MEC_NFA_Ang_Plot = ROOT.TGraph(len(G21_PTCut_Angs_NFA_Array), G21_PTCut_Angs_NFA_Array, G21_MEC_Ang_NFA)
G21_MEC_NFA_Ang_Plot.Draw("same")
G21_MEC_NFA_Ang_Plot.SetMarkerColor(2)
G21_MEC_NFA_Ang_Plot.SetMarkerStyle(29)

G21_RES_NFA_Ang_Plot = ROOT.TGraph(len(G21_PTCut_Angs_NFA_Array), G21_PTCut_Angs_NFA_Array, G21_RES_Ang_NFA)
G21_RES_NFA_Ang_Plot.Draw("same")
G21_RES_NFA_Ang_Plot.SetMarkerColor(3)
G21_RES_NFA_Ang_Plot.SetMarkerStyle(29)

G21_DIS_NFA_Ang_Plot = ROOT.TGraph(len(G21_PTCut_Angs_NFA_Array), G21_PTCut_Angs_NFA_Array, G21_DIS_Ang_NFA)
G21_DIS_NFA_Ang_Plot.Draw("same")
G21_DIS_NFA_Ang_Plot.SetMarkerColor(6)
G21_DIS_NFA_Ang_Plot.SetMarkerStyle(29)

G21_SUM_NFA_Ang_Plot = ROOT.TGraph(len(G21_PTCut_Angs_NFA_Array), G21_PTCut_Angs_NFA_Array, G21_PTCut_Sum_NFA_Array)
G21_SUM_NFA_Ang_Plot.Draw("same")
G21_SUM_NFA_Ang_Plot.SetMarkerColor(1)
G21_SUM_NFA_Ang_Plot.SetMarkerStyle(34)

G21_Ang_leg_NFA = ROOT.TLegend(0.2, 0.85, 0.80, 0.90)
G21_Ang_leg_NFA.SetNColumns(5)
G21_Ang_leg_NFA.AddEntry(G21_QEL_NFA_Ang_Plot, "QEL")
G21_Ang_leg_NFA.AddEntry(G21_MEC_NFA_Ang_Plot, "MEC")
G21_Ang_leg_NFA.AddEntry(G21_RES_NFA_Ang_Plot, "RES")
G21_Ang_leg_NFA.AddEntry(G21_DIS_NFA_Ang_Plot, "DIS")
G21_Ang_leg_NFA.AddEntry(G21_SUM_NFA_Ang_Plot, "SUM")
G21_Ang_leg_NFA.Draw()

G21_Ang_NFA_zeroes = []
for i in range(len(G21_PTCut_Angs_NFA_Array)):
    G21_Ang_NFA_zeroes.append(2.5)
G21_Ang_NFA_zeroes_array = array.array('f', G21_Ang_NFA_zeroes)

G21_Yerr_QEL_NFA_Angs_Bars = ROOT.TGraphErrors(len(G21_PTCut_Angs_NFA_Array), G21_PTCut_Angs_NFA_Array, G21_QEL_Ang_NFA, G21_Ang_NFA_zeroes_array, G21_Yerr_QEL_NFA_Array)
G21_Yerr_MEC_NFA_Angs_Bars = ROOT.TGraphErrors(len(G21_PTCut_Angs_NFA_Array), G21_PTCut_Angs_NFA_Array, G21_MEC_Ang_NFA, G21_Ang_NFA_zeroes_array, G21_Yerr_MEC_NFA_Array)
G21_Yerr_RES_NFA_Angs_Bars = ROOT.TGraphErrors(len(G21_PTCut_Angs_NFA_Array), G21_PTCut_Angs_NFA_Array, G21_RES_Ang_NFA, G21_Ang_NFA_zeroes_array, G21_Yerr_RES_NFA_Array)
G21_Yerr_DIS_NFA_Angs_Bars = ROOT.TGraphErrors(len(G21_PTCut_Angs_NFA_Array), G21_PTCut_Angs_NFA_Array, G21_DIS_Ang_NFA, G21_Ang_NFA_zeroes_array, G21_Yerr_DIS_NFA_Array)
G21_Yerr_QEL_NFA_Angs_Bars.Draw("same")
G21_Yerr_MEC_NFA_Angs_Bars.Draw("same")
G21_Yerr_RES_NFA_Angs_Bars.Draw("same")
G21_Yerr_DIS_NFA_Angs_Bars.Draw("same")

G21_Ang_Canvas_NFA.Draw()

In [ ]:
#fraction of events by interaction type as function of bins of energy_transfer 
#for ni_pions=0, nf_pions=0, nfa_pions=0 (use lepton pt > 400 MeV)
bins = 100
x_start = 0.0
x_end = 4.0 #should be max e transfer in GeV?
n_event_cut = 20
Etrans_BinVals = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0]

In [ ]:
#g18 energy transfer v1
G18_QEL_Etrans_Pions_List = []
G18_MEC_Etrans_Pions_List = []
G18_RES_Etrans_Pions_List = []
G18_DIS_Etrans_Pions_List = []
G18_XVals_Etrans_Pions_List = []
G18_Total_Count_Etrans_Pions = 0
for value in Etrans_BinVals:
    value_min = value
    value_max = value + 0.1
    df_gst_G18_02a_ET_Pions = df_gst_G18_02a_all.Filter(f"energy_transfer>{value_min} && energy_transfer<{value_max} && ptl<{lep_accept_pt}")
    G18_QEL_Etrans_Pions = df_gst_G18_02a_ET_Pions.Filter("qel==1").Histo1D(("G18_QEL_Etrans_Pions", "", bins, x_start, x_end), "energy_transfer")
    G18_MEC_Etrans_Pions = df_gst_G18_02a_ET_Pions.Filter("mec==1").Histo1D(("G18_MEC_Etrans_Pions", "", bins, x_start, x_end), "energy_transfer")
    G18_RES_Etrans_Pions = df_gst_G18_02a_ET_Pions.Filter("res==1").Histo1D(("G18_RES_Etrans_Pions", "", bins, x_start, x_end), "energy_transfer")
    G18_DIS_Etrans_Pions = df_gst_G18_02a_ET_Pions.Filter("dis==1").Histo1D(("G18_DIS_Etrans_Pions", "", bins, x_start, x_end), "energy_transfer")
    G18_Int_Etrans_Pions_List = [G18_QEL_Etrans_Pions.Integral(), G18_MEC_Etrans_Pions.Integral(), G18_RES_Etrans_Pions.Integral(), G18_DIS_Etrans_Pions.Integral()]
    G18_Total_Run_Etrans_Pions = G18_Int_Etrans_Pions_List[0] + G18_Int_Etrans_Pions_List[1] + G18_Int_Etrans_Pions_List[2] + G18_Int_Etrans_Pions_List[3]
    G18_Events_Etrans_Pions_List = [G18_QEL_Etrans_Pions_List, G18_MEC_Etrans_Pions_List, G18_RES_Etrans_Pions_List, G18_DIS_Etrans_Pions_List]
    G18_Total_Count_Etrans_Pions += G18_Total_Run_Etrans_Pions
    G18_MidPoint_Etrans_Pions = (value_max - value_min)/2
    for i in range(len(G18_Int_Etrans_Pions_List)):
        if G18_Total_Run_Etrans_Pions < n_event_cut and G18_Events_Etrans_Pions_List[i] == []:
            G18_Events_Etrans_Pions_List[i].append(0.0)
        if G18_XVals_Etrans_Pions_List == []:
            G18_XVals_Etrans_Pions_List.append(G18_MidPoint_Etrans_Pions)
        if G18_Total_Run_Etrans_Pions > n_event_cut: 
            G18_Events_Etrans_Pions_List[i].append(G18_Int_Etrans_Pions_List[i]/G18_Total_Run_Etrans_Pions)
            if (value_max - G18_MidPoint_Etrans_Pions) not in G18_XVals_Etrans_Pions_List:
                G18_XVals_Etrans_Pions_List.append(value_max - G18_MidPoint_Etrans_Pions)
G18_XVals_Etrans_Pions_Array = array.array('f', G18_XVals_Etrans_Pions_List) 
G18_QEL_Etrans_Pions_Array = array.array('f', G18_QEL_Etrans_Pions_List)
G18_MEC_Etrans_Pions_Array = array.array('f', G18_MEC_Etrans_Pions_List)
G18_RES_Etrans_Pions_Array = array.array('f', G18_RES_Etrans_Pions_List)
G18_DIS_Etrans_Pions_Array = array.array('f', G18_DIS_Etrans_Pions_List)
G18_Yerr_QEL_Etrans_Pions_List = []
G18_Yerr_MEC_Etrans_Pions_List = []
G18_Yerr_RES_Etrans_Pions_List = []
G18_Yerr_DIS_Etrans_Pions_List = []
G18_Yerr_Events_Etrans_Pions_Lists = [G18_Yerr_QEL_Etrans_Pions_List, G18_Yerr_MEC_Etrans_Pions_List, G18_Yerr_RES_Etrans_Pions_List, G18_Yerr_DIS_Etrans_Pions_List]
for event in G18_Events_Etrans_Pions_List:
        for fraction in event:
            num = G18_Events_Etrans_Pions_List.index(event)
            G18_Yerr_Events_Etrans_Pions_Lists[num].append(sqrt(fraction*(1-fraction)/G18_Total_Count_Etrans_Pions))
G18_Yerr_QEL_Etrans_Pions_Array = array.array('f', G18_Yerr_QEL_Etrans_Pions_List)
G18_Yerr_MEC_Etrans_Pions_Array = array.array('f', G18_Yerr_MEC_Etrans_Pions_List)
G18_Yerr_RES_Etrans_Pions_Array = array.array('f', G18_Yerr_RES_Etrans_Pions_List)
G18_Yerr_DIS_Etrans_Pions_Array = array.array('f', G18_Yerr_DIS_Etrans_Pions_List)

G18_Etrans_Sum_Pions = []
for i in range(len(G18_QEL_Etrans_Pions_List)):
    count = 0
    count += G18_QEL_Etrans_Pions_List[i] + G18_MEC_Etrans_Pions_List[i] + G18_RES_Etrans_Pions_List[i] + G18_DIS_Etrans_Pions_List[i]
    G18_Etrans_Sum_Pions.append(count)
G18_Etrans_Sum_Pions_Array = array.array('f', G18_Etrans_Sum_Pions)

In [ ]:
#g18 energy transfer without initial state pions v1
G18_QEL_Etrans_NIP_List = []
G18_MEC_Etrans_NIP_List = []
G18_RES_Etrans_NIP_List = []
G18_DIS_Etrans_NIP_List = []
G18_XVals_Etrans_NIP_List = []
G18_Total_Count_Etrans_NIP = 0
for value in Etrans_BinVals:
    value_min = value
    value_max = value + 0.1
    df_gst_G18_02a_ET_NIP = df_gst_G18_02a_all.Filter(f"energy_transfer>{value_min} && energy_transfer<{value_max} && ptl<{lep_accept_pt}").Filter("numi_pions==0")
    G18_QEL_Etrans_NIP = df_gst_G18_02a_ET_NIP.Filter("qel==1").Histo1D(("G18_QEL_Etrans_NIP", "", bins, x_start, x_end), "energy_transfer")
    G18_MEC_Etrans_NIP = df_gst_G18_02a_ET_NIP.Filter("mec==1").Histo1D(("G18_MEC_Etrans_NIP", "", bins, x_start, x_end), "energy_transfer")
    G18_RES_Etrans_NIP = df_gst_G18_02a_ET_NIP.Filter("res==1").Histo1D(("G18_RES_Etrans_NIP", "", bins, x_start, x_end), "energy_transfer")
    G18_DIS_Etrans_NIP = df_gst_G18_02a_ET_NIP.Filter("dis==1").Histo1D(("G18_DIS_Etrans_NIP", "", bins, x_start, x_end), "energy_transfer")
    G18_Int_Etrans_NIP_List = [G18_QEL_Etrans_NIP.Integral(), G18_MEC_Etrans_NIP.Integral(), G18_RES_Etrans_NIP.Integral(), G18_DIS_Etrans_NIP.Integral()]
    G18_Total_Run_Etrans_NIP = G18_Int_Etrans_NIP_List[0] + G18_Int_Etrans_NIP_List[1] + G18_Int_Etrans_NIP_List[2] + G18_Int_Etrans_NIP_List[3]
    G18_Events_Etrans_NIP_List = [G18_QEL_Etrans_NIP_List, G18_MEC_Etrans_NIP_List, G18_RES_Etrans_NIP_List, G18_DIS_Etrans_NIP_List]
    G18_Total_Count_Etrans_NIP += G18_Total_Run_Etrans_NIP
    G18_MidPoint_Etrans_NIP = (value_max - value_min)/2
    for i in range(len(G18_Int_Etrans_NIP_List)):
        if G18_Total_Run_Etrans_NIP < n_event_cut and G18_Events_Etrans_NIP_List[i] == []:
            G18_Events_Etrans_NIP_List[i].append(0.0)
        if G18_XVals_Etrans_NIP_List == []:
            G18_XVals_Etrans_NIP_List.append(G18_MidPoint_Etrans_NIP)
        if G18_Total_Run_Etrans_NIP > n_event_cut: 
            G18_Events_Etrans_NIP_List[i].append(G18_Int_Etrans_NIP_List[i]/G18_Total_Run_Etrans_NIP)
            if (value_max - G18_MidPoint_Etrans_NIP) not in G18_XVals_Etrans_NIP_List:
                G18_XVals_Etrans_NIP_List.append(value_max - G18_MidPoint_Etrans_NIP)
G18_XVals_Etrans_NIP_Array = array.array('f', G18_XVals_Etrans_NIP_List) 
G18_QEL_Etrans_NIP_Array = array.array('f', G18_QEL_Etrans_NIP_List)
G18_MEC_Etrans_NIP_Array = array.array('f', G18_MEC_Etrans_NIP_List)
G18_RES_Etrans_NIP_Array = array.array('f', G18_RES_Etrans_NIP_List)
G18_DIS_Etrans_NIP_Array = array.array('f', G18_DIS_Etrans_NIP_List)
G18_Yerr_QEL_Etrans_NIP_List = []
G18_Yerr_MEC_Etrans_NIP_List = []
G18_Yerr_RES_Etrans_NIP_List = []
G18_Yerr_DIS_Etrans_NIP_List = []
G18_Yerr_Events_Etrans_NIP_Lists = [G18_Yerr_QEL_Etrans_NIP_List, G18_Yerr_MEC_Etrans_NIP_List, G18_Yerr_RES_Etrans_NIP_List, G18_Yerr_DIS_Etrans_NIP_List]
for event in G18_Events_Etrans_NIP_List:
        for fraction in event:
            num = G18_Events_Etrans_NIP_List.index(event)
            G18_Yerr_Events_Etrans_NIP_Lists[num].append(sqrt(fraction*(1-fraction)/G18_Total_Count_Etrans_NIP))
G18_Yerr_QEL_Etrans_NIP_Array = array.array('f', G18_Yerr_QEL_Etrans_NIP_List)
G18_Yerr_MEC_Etrans_NIP_Array = array.array('f', G18_Yerr_MEC_Etrans_NIP_List)
G18_Yerr_RES_Etrans_NIP_Array = array.array('f', G18_Yerr_RES_Etrans_NIP_List)
G18_Yerr_DIS_Etrans_NIP_Array = array.array('f', G18_Yerr_DIS_Etrans_NIP_List)

G18_Etrans_Sum_NIP = []
for i in range(len(G18_QEL_Etrans_NIP_List)):
    count = 0
    count += G18_QEL_Etrans_NIP_List[i] + G18_MEC_Etrans_NIP_List[i] + G18_RES_Etrans_NIP_List[i] + G18_DIS_Etrans_NIP_List[i]
    G18_Etrans_Sum_NIP.append(count)
G18_Etrans_Sum_NIP_Array = array.array('f', G18_Etrans_Sum_NIP)

In [ ]:
#g18 energy transfer without final state pions v1
G18_QEL_Etrans_NFP_List = []
G18_MEC_Etrans_NFP_List = []
G18_RES_Etrans_NFP_List = []
G18_DIS_Etrans_NFP_List = []
G18_XVals_Etrans_NFP_List = []
G18_Total_Count_Etrans_NFP = 0
for value in Etrans_BinVals:
    value_min = value
    value_max = value + 0.1
    df_gst_G18_02a_ET_NFP = df_gst_G18_02a_all.Filter(f"energy_transfer>{value_min} && energy_transfer<{value_max} && ptl<{lep_accept_pt}").Filter("num_pions==0")
    G18_QEL_Etrans_NFP = df_gst_G18_02a_ET_NFP.Filter("qel==1").Histo1D(("G18_QEL_Etrans_NFP", "", bins, x_start, x_end), "energy_transfer")
    G18_MEC_Etrans_NFP = df_gst_G18_02a_ET_NFP.Filter("mec==1").Histo1D(("G18_MEC_Etrans_NFP", "", bins, x_start, x_end), "energy_transfer")
    G18_RES_Etrans_NFP = df_gst_G18_02a_ET_NFP.Filter("res==1").Histo1D(("G18_RES_Etrans_NFP", "", bins, x_start, x_end), "energy_transfer")
    G18_DIS_Etrans_NFP = df_gst_G18_02a_ET_NFP.Filter("dis==1").Histo1D(("G18_DIS_Etrans_NFP", "", bins, x_start, x_end), "energy_transfer")
    G18_Int_Etrans_NFP_List = [G18_QEL_Etrans_NFP.Integral(), G18_MEC_Etrans_NFP.Integral(), G18_RES_Etrans_NFP.Integral(), G18_DIS_Etrans_NFP.Integral()]
    G18_Total_Run_Etrans_NFP = G18_Int_Etrans_NFP_List[0] + G18_Int_Etrans_NFP_List[1] + G18_Int_Etrans_NFP_List[2] + G18_Int_Etrans_NFP_List[3]
    G18_Events_Etrans_NFP_List = [G18_QEL_Etrans_NFP_List, G18_MEC_Etrans_NFP_List, G18_RES_Etrans_NFP_List, G18_DIS_Etrans_NFP_List]
    G18_Total_Count_Etrans_NFP += G18_Total_Run_Etrans_NFP
    G18_MidPoint_Etrans_NFP = (value_max - value_min)/2
    for i in range(len(G18_Int_Etrans_NFP_List)):
        if G18_Total_Run_Etrans_NFP < n_event_cut and G18_Events_Etrans_NFP_List[i] == []:
            G18_Events_Etrans_NFP_List[i].append(0.0)#adds zero to beginning of list when events are too low
        if G18_XVals_Etrans_NFP_List == []:
            G18_XVals_Etrans_NFP_List.append(G18_MidPoint_Etrans_NFP)
        if G18_Total_Run_Etrans_NFP > n_event_cut: 
            G18_Events_Etrans_NFP_List[i].append(G18_Int_Etrans_NFP_List[i]/G18_Total_Run_Etrans_NFP)
            if (value_max - G18_MidPoint_Etrans_NFP) not in G18_XVals_Etrans_NFP_List:
                G18_XVals_Etrans_NFP_List.append(value_max - G18_MidPoint_Etrans_NFP)
G18_XVals_Etrans_NFP_Array = array.array('f', G18_XVals_Etrans_NFP_List)
G18_QEL_Etrans_NFP_Array = array.array('f', G18_QEL_Etrans_NFP_List)
G18_MEC_Etrans_NFP_Array = array.array('f', G18_MEC_Etrans_NFP_List)
G18_RES_Etrans_NFP_Array = array.array('f', G18_RES_Etrans_NFP_List)
G18_DIS_Etrans_NFP_Array = array.array('f', G18_DIS_Etrans_NFP_List)
G18_Yerr_QEL_Etrans_NFP_List = []
G18_Yerr_MEC_Etrans_NFP_List = []
G18_Yerr_RES_Etrans_NFP_List = []
G18_Yerr_DIS_Etrans_NFP_List = []
G18_Yerr_Events_Etrans_NFP_Lists = [G18_Yerr_QEL_Etrans_NFP_List, G18_Yerr_MEC_Etrans_NFP_List, G18_Yerr_RES_Etrans_NFP_List, G18_Yerr_DIS_Etrans_NFP_List]
for event in G18_Events_Etrans_NFP_List:
        for fraction in event:
            num = G18_Events_Etrans_NFP_List.index(event)
            G18_Yerr_Events_Etrans_NFP_Lists[num].append(sqrt(fraction*(1-fraction)/G18_Total_Count_Etrans_NFP))
G18_Yerr_QEL_Etrans_NFP_Array = array.array('f', G18_Yerr_QEL_Etrans_NFP_List)
G18_Yerr_MEC_Etrans_NFP_Array = array.array('f', G18_Yerr_MEC_Etrans_NFP_List)
G18_Yerr_RES_Etrans_NFP_Array = array.array('f', G18_Yerr_RES_Etrans_NFP_List)
G18_Yerr_DIS_Etrans_NFP_Array = array.array('f', G18_Yerr_DIS_Etrans_NFP_List)

G18_Etrans_Sum_NFP = []
for i in range(len(G18_QEL_Etrans_NFP_List)):
    count = 0
    count += G18_QEL_Etrans_NFP_List[i] + G18_MEC_Etrans_NFP_List[i] + G18_RES_Etrans_NFP_List[i] + G18_DIS_Etrans_NFP_List[i]
    G18_Etrans_Sum_NFP.append(count)
G18_Etrans_Sum_NFP_Array = array.array('f', G18_Etrans_Sum_NFP)

In [ ]:
#g18 energy transfer without nfa pions v1
G18_QEL_Etrans_NFAP_List = []
G18_MEC_Etrans_NFAP_List = []
G18_RES_Etrans_NFAP_List = []
G18_DIS_Etrans_NFAP_List = []
G18_XVals_Etrans_NFAP_List = []
G18_Total_Count_Etrans_NFAP = 0
for value in Etrans_BinVals:
    value_min = value
    value_max = value + 0.1
    df_gst_G18_02a_ET_NFAP = df_gst_G18_02a_all.Filter(f"energy_transfer>{value_min} && energy_transfer<{value_max} && ptl<{lep_accept_pt}").Filter("nfa_pions==0")
    G18_QEL_Etrans_NFAP = df_gst_G18_02a_ET_NFAP.Filter("qel==1").Histo1D(("G18_QEL_Etrans_NFAP", "", bins, x_start, x_end), "energy_transfer")
    G18_MEC_Etrans_NFAP = df_gst_G18_02a_ET_NFAP.Filter("mec==1").Histo1D(("G18_MEC_Etrans_NFAP", "", bins, x_start, x_end), "energy_transfer")
    G18_RES_Etrans_NFAP = df_gst_G18_02a_ET_NFAP.Filter("res==1").Histo1D(("G18_RES_Etrans_NFAP", "", bins, x_start, x_end), "energy_transfer")
    G18_DIS_Etrans_NFAP = df_gst_G18_02a_ET_NFAP.Filter("dis==1").Histo1D(("G18_DIS_Etrans_NFAP", "", bins, x_start, x_end), "energy_transfer")
    G18_Int_Etrans_NFAP_List = [G18_QEL_Etrans_NFAP.Integral(), G18_MEC_Etrans_NFAP.Integral(), G18_RES_Etrans_NFAP.Integral(), G18_DIS_Etrans_NFAP.Integral()]
    G18_Total_Run_Etrans_NFAP = G18_Int_Etrans_NFAP_List[0] + G18_Int_Etrans_NFAP_List[1] + G18_Int_Etrans_NFAP_List[2] + G18_Int_Etrans_NFAP_List[3]
    G18_Events_Etrans_NFAP_List = [G18_QEL_Etrans_NFAP_List, G18_MEC_Etrans_NFAP_List, G18_RES_Etrans_NFAP_List, G18_DIS_Etrans_NFAP_List]
    G18_Total_Count_Etrans_NFAP += G18_Total_Run_Etrans_NFAP
    G18_MidPoint_Etrans_NFAP = (value_max - value_min)/2
    for i in range(len(G18_Int_Etrans_NFAP_List)):
        if G18_Total_Run_Etrans_NFAP < n_event_cut and G18_Events_Etrans_NFAP_List[i] == []:
            G18_Events_Etrans_NFAP_List[i].append(0.0)#adds zero to beginning of list when events are too low
        if G18_XVals_Etrans_NFAP_List == []:
            G18_XVals_Etrans_NFAP_List.append(G18_MidPoint_Etrans_NFAP)
        if G18_Total_Run_Etrans_NFAP > n_event_cut: 
            G18_Events_Etrans_NFAP_List[i].append(G18_Int_Etrans_NFAP_List[i]/G18_Total_Run_Etrans_NFAP)
            if (value_max - G18_MidPoint_Etrans_NFAP) not in G18_XVals_Etrans_NFAP_List:
                G18_XVals_Etrans_NFAP_List.append(value_max - G18_MidPoint_Etrans_NFAP)
G18_XVals_Etrans_NFAP_Array = array.array('f', G18_XVals_Etrans_NFAP_List)
G18_QEL_Etrans_NFAP_Array = array.array('f', G18_QEL_Etrans_NFAP_List)
G18_MEC_Etrans_NFAP_Array = array.array('f', G18_MEC_Etrans_NFAP_List)
G18_RES_Etrans_NFAP_Array = array.array('f', G18_RES_Etrans_NFAP_List)
G18_DIS_Etrans_NFAP_Array = array.array('f', G18_DIS_Etrans_NFAP_List)
G18_Yerr_QEL_Etrans_NFAP_List = []
G18_Yerr_MEC_Etrans_NFAP_List = []
G18_Yerr_RES_Etrans_NFAP_List = []
G18_Yerr_DIS_Etrans_NFAP_List = []
G18_Yerr_Events_Etrans_NFAP_Lists = [G18_Yerr_QEL_Etrans_NFAP_List, G18_Yerr_MEC_Etrans_NFAP_List, G18_Yerr_RES_Etrans_NFAP_List, G18_Yerr_DIS_Etrans_NFAP_List]
for event in G18_Events_Etrans_NFAP_List:
        for fraction in event:
            num = G18_Events_Etrans_NFAP_List.index(event)
            G18_Yerr_Events_Etrans_NFAP_Lists[num].append(sqrt(fraction*(1-fraction)/G18_Total_Count_Etrans_NFAP))
G18_Yerr_QEL_Etrans_NFAP_Array = array.array('f', G18_Yerr_QEL_Etrans_NFAP_List)
G18_Yerr_MEC_Etrans_NFAP_Array = array.array('f', G18_Yerr_MEC_Etrans_NFAP_List)
G18_Yerr_RES_Etrans_NFAP_Array = array.array('f', G18_Yerr_RES_Etrans_NFAP_List)
G18_Yerr_DIS_Etrans_NFAP_Array = array.array('f', G18_Yerr_DIS_Etrans_NFAP_List)

G18_Etrans_Sum_NFAP = []
for i in range(len(G18_QEL_Etrans_NFAP_List)):
    count = 0
    count += G18_QEL_Etrans_NFAP_List[i] + G18_MEC_Etrans_NFAP_List[i] + G18_RES_Etrans_NFAP_List[i] + G18_DIS_Etrans_NFAP_List[i]
    G18_Etrans_Sum_NFAP.append(count)
G18_Etrans_Sum_NFAP_Array = array.array('f', G18_Etrans_Sum_NFAP)

In [ ]:
#gem21 arrays

In [ ]:
#gem21 energy transfer v1
G21_QEL_Etrans_Pions_List = []
G21_MEC_Etrans_Pions_List = []
G21_RES_Etrans_Pions_List = []
G21_DIS_Etrans_Pions_List = []
G21_XVals_Etrans_Pions_List = []
G21_Total_Count_Etrans_Pions = 0
for value in Etrans_BinVals:
    value_min = value
    value_max = value + 0.1
    df_gst_GEM21_11B_ET_Pions = df_gst_GEM21_11b_all.Filter(f"energy_transfer>{value_min} && energy_transfer<{value_max} && ptl<{lep_accept_pt}")
    G21_QEL_Etrans_Pions = df_gst_GEM21_11B_ET_Pions.Filter("qel==1").Histo1D(("G21_QEL_Etrans_Pions", "", bins, x_start, x_end), "energy_transfer")
    G21_MEC_Etrans_Pions = df_gst_GEM21_11B_ET_Pions.Filter("mec==1").Histo1D(("G21_MEC_Etrans_Pions", "", bins, x_start, x_end), "energy_transfer")
    G21_RES_Etrans_Pions = df_gst_GEM21_11B_ET_Pions.Filter("res==1").Histo1D(("G21_RES_Etrans_Pions", "", bins, x_start, x_end), "energy_transfer")
    G21_DIS_Etrans_Pions = df_gst_GEM21_11B_ET_Pions.Filter("dis==1").Histo1D(("G21_DIS_Etrans_Pions", "", bins, x_start, x_end), "energy_transfer")
    G21_Int_Etrans_Pions_List = [G21_QEL_Etrans_Pions.Integral(), G21_MEC_Etrans_Pions.Integral(), G21_RES_Etrans_Pions.Integral(), G21_DIS_Etrans_Pions.Integral()]
    G21_Total_Run_Etrans_Pions = G21_Int_Etrans_Pions_List[0] + G21_Int_Etrans_Pions_List[1] + G21_Int_Etrans_Pions_List[2] + G21_Int_Etrans_Pions_List[3]
    G21_Events_Etrans_Pions_List = [G21_QEL_Etrans_Pions_List, G21_MEC_Etrans_Pions_List, G21_RES_Etrans_Pions_List, G21_DIS_Etrans_Pions_List]
    G21_Total_Count_Etrans_Pions += G21_Total_Run_Etrans_Pions
    G21_MidPoint_Etrans_Pions = (value_max - value_min)/2
    for i in range(len(G21_Int_Etrans_Pions_List)):
        if G21_Total_Run_Etrans_Pions < n_event_cut and G21_Events_Etrans_Pions_List[i] == []:
            G21_Events_Etrans_Pions_List[i].append(0.0)
        if G21_XVals_Etrans_Pions_List == []:
            G21_XVals_Etrans_Pions_List.append(G21_MidPoint_Etrans_Pions)
        if G21_Total_Run_Etrans_Pions > n_event_cut: 
            G21_Events_Etrans_Pions_List[i].append(G21_Int_Etrans_Pions_List[i]/G21_Total_Run_Etrans_Pions)
            if (value_max - G21_MidPoint_Etrans_Pions) not in G21_XVals_Etrans_Pions_List:
                G21_XVals_Etrans_Pions_List.append(value_max - G21_MidPoint_Etrans_Pions)
G21_XVals_Etrans_Pions_Array = array.array('f', G21_XVals_Etrans_Pions_List) 
G21_QEL_Etrans_Pions_Array = array.array('f', G21_QEL_Etrans_Pions_List)
G21_MEC_Etrans_Pions_Array = array.array('f', G21_MEC_Etrans_Pions_List)
G21_RES_Etrans_Pions_Array = array.array('f', G21_RES_Etrans_Pions_List)
G21_DIS_Etrans_Pions_Array = array.array('f', G21_DIS_Etrans_Pions_List)
G21_Yerr_QEL_Etrans_Pions_List = []
G21_Yerr_MEC_Etrans_Pions_List = []
G21_Yerr_RES_Etrans_Pions_List = []
G21_Yerr_DIS_Etrans_Pions_List = []
G21_Yerr_Events_Etrans_Pions_Lists = [G21_Yerr_QEL_Etrans_Pions_List, G21_Yerr_MEC_Etrans_Pions_List, G21_Yerr_RES_Etrans_Pions_List, G21_Yerr_DIS_Etrans_Pions_List]
for event in G21_Events_Etrans_Pions_List:
        for fraction in event:
            num = G21_Events_Etrans_Pions_List.index(event)
            G21_Yerr_Events_Etrans_Pions_Lists[num].append(sqrt(fraction*(1-fraction)/G21_Total_Count_Etrans_Pions))
G21_Yerr_QEL_Etrans_Pions_Array = array.array('f', G21_Yerr_QEL_Etrans_Pions_List)
G21_Yerr_MEC_Etrans_Pions_Array = array.array('f', G21_Yerr_MEC_Etrans_Pions_List)
G21_Yerr_RES_Etrans_Pions_Array = array.array('f', G21_Yerr_RES_Etrans_Pions_List)
G21_Yerr_DIS_Etrans_Pions_Array = array.array('f', G21_Yerr_DIS_Etrans_Pions_List)

G21_Etrans_Sum_Pions = []
for i in range(len(G21_QEL_Etrans_Pions_List)):
    count = 0
    count += G21_QEL_Etrans_Pions_List[i] + G21_MEC_Etrans_Pions_List[i] + G21_RES_Etrans_Pions_List[i] + G21_DIS_Etrans_Pions_List[i]
    G21_Etrans_Sum_Pions.append(count)
G21_Etrans_Sum_Pions_Array = array.array('f', G21_Etrans_Sum_Pions)

In [ ]:
#gem21 energy transfer without initial state pions v1
G21_QEL_Etrans_NIP_List = []
G21_MEC_Etrans_NIP_List = []
G21_RES_Etrans_NIP_List = []
G21_DIS_Etrans_NIP_List = []
G21_XVals_Etrans_NIP_List = []
G21_Total_Count_Etrans_NIP = 0
for value in Etrans_BinVals:
    value_min = value
    value_max = value + 0.1
    df_gst_GEM21_11B_ET_NIP = df_gst_GEM21_11b_all.Filter(f"energy_transfer>{value_min} && energy_transfer<{value_max} && ptl<{lep_accept_pt}").Filter("numi_pions==0")
    G21_QEL_Etrans_NIP = df_gst_GEM21_11B_ET_NIP.Filter("qel==1").Histo1D(("G21_QEL_Etrans_NIP", "", bins, x_start, x_end), "energy_transfer")
    G21_MEC_Etrans_NIP = df_gst_GEM21_11B_ET_NIP.Filter("mec==1").Histo1D(("G21_MEC_Etrans_NIP", "", bins, x_start, x_end), "energy_transfer")
    G21_RES_Etrans_NIP = df_gst_GEM21_11B_ET_NIP.Filter("res==1").Histo1D(("G21_RES_Etrans_NIP", "", bins, x_start, x_end), "energy_transfer")
    G21_DIS_Etrans_NIP = df_gst_GEM21_11B_ET_NIP.Filter("dis==1").Histo1D(("G21_DIS_Etrans_NIP", "", bins, x_start, x_end), "energy_transfer")
    G21_Int_Etrans_NIP_List = [G21_QEL_Etrans_NIP.Integral(), G21_MEC_Etrans_NIP.Integral(), G21_RES_Etrans_NIP.Integral(), G21_DIS_Etrans_NIP.Integral()]
    G21_Total_Run_Etrans_NIP = G21_Int_Etrans_NIP_List[0] + G21_Int_Etrans_NIP_List[1] + G21_Int_Etrans_NIP_List[2] + G21_Int_Etrans_NIP_List[3]
    G21_Events_Etrans_NIP_List = [G21_QEL_Etrans_NIP_List, G21_MEC_Etrans_NIP_List, G21_RES_Etrans_NIP_List, G21_DIS_Etrans_NIP_List]
    G21_Total_Count_Etrans_NIP += G21_Total_Run_Etrans_NIP
    G21_MidPoint_Etrans_NIP = (value_max - value_min)/2
    for i in range(len(G21_Int_Etrans_NIP_List)):
        if G21_Total_Run_Etrans_NIP < n_event_cut and G21_Events_Etrans_NIP_List[i] == []:
            G21_Events_Etrans_NIP_List[i].append(0.0)
        if G21_XVals_Etrans_NIP_List == []:
            G21_XVals_Etrans_NIP_List.append(G21_MidPoint_Etrans_NIP)
        if G21_Total_Run_Etrans_NIP > n_event_cut: 
            G21_Events_Etrans_NIP_List[i].append(G21_Int_Etrans_NIP_List[i]/G21_Total_Run_Etrans_NIP)
            if (value_max - G21_MidPoint_Etrans_NIP) not in G21_XVals_Etrans_NIP_List:
                G21_XVals_Etrans_NIP_List.append(value_max - G21_MidPoint_Etrans_NIP)
G21_XVals_Etrans_NIP_Array = array.array('f', G21_XVals_Etrans_NIP_List) 
G21_QEL_Etrans_NIP_Array = array.array('f', G21_QEL_Etrans_NIP_List)
G21_MEC_Etrans_NIP_Array = array.array('f', G21_MEC_Etrans_NIP_List)
G21_RES_Etrans_NIP_Array = array.array('f', G21_RES_Etrans_NIP_List)
G21_DIS_Etrans_NIP_Array = array.array('f', G21_DIS_Etrans_NIP_List)
G21_Yerr_QEL_Etrans_NIP_List = []
G21_Yerr_MEC_Etrans_NIP_List = []
G21_Yerr_RES_Etrans_NIP_List = []
G21_Yerr_DIS_Etrans_NIP_List = []
G21_Yerr_Events_Etrans_NIP_Lists = [G21_Yerr_QEL_Etrans_NIP_List, G21_Yerr_MEC_Etrans_NIP_List, G21_Yerr_RES_Etrans_NIP_List, G21_Yerr_DIS_Etrans_NIP_List]
for event in G21_Events_Etrans_NIP_List:
        for fraction in event:
            num = G21_Events_Etrans_NIP_List.index(event)
            G21_Yerr_Events_Etrans_NIP_Lists[num].append(sqrt(fraction*(1-fraction)/G21_Total_Count_Etrans_NIP))
G21_Yerr_QEL_Etrans_NIP_Array = array.array('f', G21_Yerr_QEL_Etrans_NIP_List)
G21_Yerr_MEC_Etrans_NIP_Array = array.array('f', G21_Yerr_MEC_Etrans_NIP_List)
G21_Yerr_RES_Etrans_NIP_Array = array.array('f', G21_Yerr_RES_Etrans_NIP_List)
G21_Yerr_DIS_Etrans_NIP_Array = array.array('f', G21_Yerr_DIS_Etrans_NIP_List)

G21_Etrans_Sum_NIP = []
for i in range(len(G21_QEL_Etrans_NIP_List)):
    count = 0
    count += G21_QEL_Etrans_NIP_List[i] + G21_MEC_Etrans_NIP_List[i] + G21_RES_Etrans_NIP_List[i] + G21_DIS_Etrans_NIP_List[i]
    G21_Etrans_Sum_NIP.append(count)
G21_Etrans_Sum_NIP_Array = array.array('f', G21_Etrans_Sum_NIP)

In [ ]:
#gem21 energy transfer without final state pions v1
G21_QEL_Etrans_NFP_List = []
G21_MEC_Etrans_NFP_List = []
G21_RES_Etrans_NFP_List = []
G21_DIS_Etrans_NFP_List = []
G21_XVals_Etrans_NFP_List = []
G21_Total_Count_Etrans_NFP = 0
for value in Etrans_BinVals:
    value_min = value
    value_max = value + 0.1
    df_gst_GEM21_11b_ET_NFP = df_gst_GEM21_11b_all.Filter(f"energy_transfer>{value_min} && energy_transfer<{value_max} && ptl<{lep_accept_pt}").Filter("num_pions==0")
    G21_QEL_Etrans_NFP = df_gst_GEM21_11b_ET_NFP.Filter("qel==1").Histo1D(("G21_QEL_Etrans_NFP", "", bins, x_start, x_end), "energy_transfer")
    G21_MEC_Etrans_NFP = df_gst_GEM21_11b_ET_NFP.Filter("mec==1").Histo1D(("G21_MEC_Etrans_NFP", "", bins, x_start, x_end), "energy_transfer")
    G21_RES_Etrans_NFP = df_gst_GEM21_11b_ET_NFP.Filter("res==1").Histo1D(("G21_RES_Etrans_NFP", "", bins, x_start, x_end), "energy_transfer")
    G21_DIS_Etrans_NFP = df_gst_GEM21_11b_ET_NFP.Filter("dis==1").Histo1D(("G21_DIS_Etrans_NFP", "", bins, x_start, x_end), "energy_transfer")
    G21_Int_Etrans_NFP_List = [G21_QEL_Etrans_NFP.Integral(), G21_MEC_Etrans_NFP.Integral(), G21_RES_Etrans_NFP.Integral(), G21_DIS_Etrans_NFP.Integral()]
    G21_Total_Run_Etrans_NFP = G21_Int_Etrans_NFP_List[0] + G21_Int_Etrans_NFP_List[1] + G21_Int_Etrans_NFP_List[2] + G21_Int_Etrans_NFP_List[3]
    G21_Events_Etrans_NFP_List = [G21_QEL_Etrans_NFP_List, G21_MEC_Etrans_NFP_List, G21_RES_Etrans_NFP_List, G21_DIS_Etrans_NFP_List]
    G21_Total_Count_Etrans_NFP += G21_Total_Run_Etrans_NFP
    G21_MidPoint_Etrans_NFP = (value_max - value_min)/2
    for i in range(len(G21_Int_Etrans_NFP_List)):
        if G21_Total_Run_Etrans_NFP < n_event_cut and G21_Events_Etrans_NFP_List[i] == []:
            G21_Events_Etrans_NFP_List[i].append(0.0)#adds zero to beginning of list when events are too low
        if G21_XVals_Etrans_NFP_List == []:
            G21_XVals_Etrans_NFP_List.append(G21_MidPoint_Etrans_NFP)
        if G21_Total_Run_Etrans_NFP > n_event_cut: 
            G21_Events_Etrans_NFP_List[i].append(G21_Int_Etrans_NFP_List[i]/G21_Total_Run_Etrans_NFP)
            if (value_max - G21_MidPoint_Etrans_NFP) not in G21_XVals_Etrans_NFP_List:
                G21_XVals_Etrans_NFP_List.append(value_max - G21_MidPoint_Etrans_NFP)
G21_XVals_Etrans_NFP_Array = array.array('f', G21_XVals_Etrans_NFP_List)
G21_QEL_Etrans_NFP_Array = array.array('f', G21_QEL_Etrans_NFP_List)
G21_MEC_Etrans_NFP_Array = array.array('f', G21_MEC_Etrans_NFP_List)
G21_RES_Etrans_NFP_Array = array.array('f', G21_RES_Etrans_NFP_List)
G21_DIS_Etrans_NFP_Array = array.array('f', G21_DIS_Etrans_NFP_List)
G21_Yerr_QEL_Etrans_NFP_List = []
G21_Yerr_MEC_Etrans_NFP_List = []
G21_Yerr_RES_Etrans_NFP_List = []
G21_Yerr_DIS_Etrans_NFP_List = []
G21_Yerr_Events_Etrans_NFP_Lists = [G21_Yerr_QEL_Etrans_NFP_List, G21_Yerr_MEC_Etrans_NFP_List, G21_Yerr_RES_Etrans_NFP_List, G21_Yerr_DIS_Etrans_NFP_List]
for event in G21_Events_Etrans_NFP_List:
        for fraction in event:
            num = G21_Events_Etrans_NFP_List.index(event)
            G21_Yerr_Events_Etrans_NFP_Lists[num].append(sqrt(fraction*(1-fraction)/G21_Total_Count_Etrans_NFP))
G21_Yerr_QEL_Etrans_NFP_Array = array.array('f', G21_Yerr_QEL_Etrans_NFP_List)
G21_Yerr_MEC_Etrans_NFP_Array = array.array('f', G21_Yerr_MEC_Etrans_NFP_List)
G21_Yerr_RES_Etrans_NFP_Array = array.array('f', G21_Yerr_RES_Etrans_NFP_List)
G21_Yerr_DIS_Etrans_NFP_Array = array.array('f', G21_Yerr_DIS_Etrans_NFP_List)

G21_Etrans_Sum_NFP = []
for i in range(len(G21_QEL_Etrans_NFP_List)):
    count = 0
    count += G21_QEL_Etrans_NFP_List[i] + G21_MEC_Etrans_NFP_List[i] + G21_RES_Etrans_NFP_List[i] + G21_DIS_Etrans_NFP_List[i]
    G21_Etrans_Sum_NFP.append(count)
G21_Etrans_Sum_NFP_Array = array.array('f', G21_Etrans_Sum_NFP)

In [ ]:
#gem21 energy transfer without nfa pions v1
G21_QEL_Etrans_NFAP_List = []
G21_MEC_Etrans_NFAP_List = []
G21_RES_Etrans_NFAP_List = []
G21_DIS_Etrans_NFAP_List = []
G21_XVals_Etrans_NFAP_List = []
G21_Total_Count_Etrans_NFAP = 0
for value in Etrans_BinVals:
    value_min = value
    value_max = value + 0.1
    df_gst_GEM21_11b_ET_NFAP = df_gst_GEM21_11b_all.Filter(f"energy_transfer>{value_min} && energy_transfer<{value_max} && ptl<{lep_accept_pt}").Filter("nfa_pions==0")
    G21_QEL_Etrans_NFAP = df_gst_GEM21_11b_ET_NFAP.Filter("qel==1").Histo1D(("G21_QEL_Etrans_NFAP", "", bins, x_start, x_end), "energy_transfer")
    G21_MEC_Etrans_NFAP = df_gst_GEM21_11b_ET_NFAP.Filter("mec==1").Histo1D(("G21_MEC_Etrans_NFAP", "", bins, x_start, x_end), "energy_transfer")
    G21_RES_Etrans_NFAP = df_gst_GEM21_11b_ET_NFAP.Filter("res==1").Histo1D(("G21_RES_Etrans_NFAP", "", bins, x_start, x_end), "energy_transfer")
    G21_DIS_Etrans_NFAP = df_gst_GEM21_11b_ET_NFAP.Filter("dis==1").Histo1D(("G21_DIS_Etrans_NFAP", "", bins, x_start, x_end), "energy_transfer")
    G21_Int_Etrans_NFAP_List = [G21_QEL_Etrans_NFAP.Integral(), G21_MEC_Etrans_NFAP.Integral(), G21_RES_Etrans_NFAP.Integral(), G21_DIS_Etrans_NFAP.Integral()]
    G21_Total_Run_Etrans_NFAP = G21_Int_Etrans_NFAP_List[0] + G21_Int_Etrans_NFAP_List[1] + G21_Int_Etrans_NFAP_List[2] + G21_Int_Etrans_NFAP_List[3]
    G21_Events_Etrans_NFAP_List = [G21_QEL_Etrans_NFAP_List, G21_MEC_Etrans_NFAP_List, G21_RES_Etrans_NFAP_List, G21_DIS_Etrans_NFAP_List]
    G21_Total_Count_Etrans_NFAP += G21_Total_Run_Etrans_NFAP
    G21_MidPoint_Etrans_NFAP = (value_max - value_min)/2
    for i in range(len(G21_Int_Etrans_NFAP_List)):
        if G21_Total_Run_Etrans_NFAP < n_event_cut and G21_Events_Etrans_NFAP_List[i] == []:
            G21_Events_Etrans_NFAP_List[i].append(0.0)
        if G21_XVals_Etrans_NFAP_List == []:
            G21_XVals_Etrans_NFAP_List.append(G21_MidPoint_Etrans_NFAP)
        if G21_Total_Run_Etrans_NFAP > n_event_cut: 
            G21_Events_Etrans_NFAP_List[i].append(G21_Int_Etrans_NFAP_List[i]/G21_Total_Run_Etrans_NFAP)
            if (value_max - G21_MidPoint_Etrans_NFAP) not in G21_XVals_Etrans_NFAP_List:
                G21_XVals_Etrans_NFAP_List.append(value_max - G21_MidPoint_Etrans_NFAP)
G21_XVals_Etrans_NFAP_Array = array.array('f', G21_XVals_Etrans_NFAP_List)
G21_QEL_Etrans_NFAP_Array = array.array('f', G21_QEL_Etrans_NFAP_List)
G21_MEC_Etrans_NFAP_Array = array.array('f', G21_MEC_Etrans_NFAP_List)
G21_RES_Etrans_NFAP_Array = array.array('f', G21_RES_Etrans_NFAP_List)
G21_DIS_Etrans_NFAP_Array = array.array('f', G21_DIS_Etrans_NFAP_List)
G21_Yerr_QEL_Etrans_NFAP_List = []
G21_Yerr_MEC_Etrans_NFAP_List = []
G21_Yerr_RES_Etrans_NFAP_List = []
G21_Yerr_DIS_Etrans_NFAP_List = []
G21_Yerr_Events_Etrans_NFAP_Lists = [G21_Yerr_QEL_Etrans_NFAP_List, G21_Yerr_MEC_Etrans_NFAP_List, G21_Yerr_RES_Etrans_NFAP_List, G21_Yerr_DIS_Etrans_NFAP_List]
for event in G21_Events_Etrans_NFAP_List:
        for fraction in event:
            num = G21_Events_Etrans_NFAP_List.index(event)
            G21_Yerr_Events_Etrans_NFAP_Lists[num].append(sqrt(fraction*(1-fraction)/G21_Total_Count_Etrans_NFAP))
G21_Yerr_QEL_Etrans_NFAP_Array = array.array('f', G21_Yerr_QEL_Etrans_NFAP_List)
G21_Yerr_MEC_Etrans_NFAP_Array = array.array('f', G21_Yerr_MEC_Etrans_NFAP_List)
G21_Yerr_RES_Etrans_NFAP_Array = array.array('f', G21_Yerr_RES_Etrans_NFAP_List)
G21_Yerr_DIS_Etrans_NFAP_Array = array.array('f', G21_Yerr_DIS_Etrans_NFAP_List)

G21_Etrans_Sum_NFAP = []
for i in range(len(G21_QEL_Etrans_NFAP_List)):
    count = 0
    count += G21_QEL_Etrans_NFAP_List[i] + G21_MEC_Etrans_NFAP_List[i] + G21_RES_Etrans_NFAP_List[i] + G21_DIS_Etrans_NFAP_List[i]
    G21_Etrans_Sum_NFAP.append(count)
G21_Etrans_Sum_NFAP_Array = array.array('f', G21_Etrans_Sum_NFAP)

In [ ]:
#plots

In [ ]:
#G18 without initial state pions v4
G18_ETrans_Pions_Canvas = ROOT.TCanvas()

G18_ETrans_QEL_Pions_Plot = ROOT.TGraph(len(G18_XVals_Etrans_Pions_Array), G18_XVals_Etrans_Pions_Array, G18_QEL_Etrans_Pions_Array)
G18_ETrans_QEL_Pions_Plot.SetTitle("Relative Fraction of Interaction Type Per Energy Transfer Cut in G18 Model")
G18_ETrans_QEL_Pions_Plot.GetXaxis().SetTitle("Energy Transfer (GeV)")
G18_ETrans_QEL_Pions_Plot.GetYaxis().SetTitle("Percentage")
G18_ETrans_QEL_Pions_Plot.SetMaximum(1.1)
G18_ETrans_QEL_Pions_Plot.SetMinimum(0.0)
G18_ETrans_QEL_Pions_Plot.SetMarkerColor(4)
G18_ETrans_QEL_Pions_Plot.SetMarkerStyle(29)
G18_ETrans_QEL_Pions_Plot.Draw("same")

G18_ETrans_MEC_Pions_Plot = ROOT.TGraph(len(G18_XVals_Etrans_Pions_Array), G18_XVals_Etrans_Pions_Array, G18_MEC_Etrans_Pions_Array)
G18_ETrans_MEC_Pions_Plot.Draw("same")
G18_ETrans_MEC_Pions_Plot.SetMarkerColor(2)
G18_ETrans_MEC_Pions_Plot.SetMarkerStyle(29)

G18_ETrans_RES_Pions_Plot = ROOT.TGraph(len(G18_XVals_Etrans_Pions_Array), G18_XVals_Etrans_Pions_Array, G18_RES_Etrans_Pions_Array)
G18_ETrans_RES_Pions_Plot.Draw("same")
G18_ETrans_RES_Pions_Plot.SetMarkerColor(3)
G18_ETrans_RES_Pions_Plot.SetMarkerStyle(29)

G18_ETrans_DIS_Pions_Plot = ROOT.TGraph(len(G18_XVals_Etrans_Pions_Array), G18_XVals_Etrans_Pions_Array, G18_DIS_Etrans_Pions_Array)
G18_ETrans_DIS_Pions_Plot.Draw("same")
G18_ETrans_DIS_Pions_Plot.SetMarkerColor(6)
G18_ETrans_DIS_Pions_Plot.SetMarkerStyle(29)

G18_ETrans_Sums_Pions_Plot = ROOT.TGraph(len(G18_XVals_Etrans_Pions_Array), G18_XVals_Etrans_Pions_Array, G18_Etrans_Sum_Pions_Array)
G18_ETrans_Sums_Pions_Plot.Draw("same")
G18_ETrans_Sums_Pions_Plot.SetMarkerColor(1)
G18_ETrans_Sums_Pions_Plot.SetMarkerStyle(34)

G18_ETrans_Leg_Pions = ROOT.TLegend(0.2, 0.85, 0.80, 0.90)
G18_ETrans_Leg_Pions.SetNColumns(5)
G18_ETrans_Leg_Pions.AddEntry(G18_ETrans_QEL_Pions_Plot, "QEL")
G18_ETrans_Leg_Pions.AddEntry(G18_ETrans_MEC_Pions_Plot, "MEC")
G18_ETrans_Leg_Pions.AddEntry(G18_ETrans_RES_Pions_Plot, "RES")
G18_ETrans_Leg_Pions.AddEntry(G18_ETrans_DIS_Pions_Plot, "DIS")
G18_ETrans_Leg_Pions.AddEntry(G18_ETrans_Sums_Pions_Plot, "SUM")
G18_ETrans_Leg_Pions.Draw()

G18_ETrans_Zeroes_Pions = []
for i in range(len(G18_XVals_Etrans_Pions_Array)):
    G18_ETrans_Zeroes_Pions.append(0.05)
G18_ETrans_Zeroes_Pions_array = array.array('f', G18_ETrans_Zeroes_Pions)

G18_Yerr_QEL_Pions_Bars = ROOT.TGraphErrors(len(G18_XVals_Etrans_Pions_Array), G18_XVals_Etrans_Pions_Array, G18_QEL_Etrans_Pions_Array, G18_ETrans_Zeroes_Pions_array, G18_Yerr_QEL_Etrans_Pions_Array)
G18_Yerr_MEC_Pions_Bars = ROOT.TGraphErrors(len(G18_XVals_Etrans_Pions_Array), G18_XVals_Etrans_Pions_Array, G18_MEC_Etrans_Pions_Array, G18_ETrans_Zeroes_Pions_array, G18_Yerr_MEC_Etrans_Pions_Array)
G18_Yerr_RES_Pions_Bars = ROOT.TGraphErrors(len(G18_XVals_Etrans_Pions_Array), G18_XVals_Etrans_Pions_Array, G18_RES_Etrans_Pions_Array, G18_ETrans_Zeroes_Pions_array, G18_Yerr_RES_Etrans_Pions_Array)
G18_Yerr_DIS_Pions_Bars = ROOT.TGraphErrors(len(G18_XVals_Etrans_Pions_Array), G18_XVals_Etrans_Pions_Array, G18_DIS_Etrans_Pions_Array, G18_ETrans_Zeroes_Pions_array, G18_Yerr_DIS_Etrans_Pions_Array)
G18_Yerr_QEL_Pions_Bars.Draw("same")
G18_Yerr_MEC_Pions_Bars.Draw("same")
G18_Yerr_RES_Pions_Bars.Draw("same")
G18_Yerr_DIS_Pions_Bars.Draw("same")

G18_ETrans_Pions_Canvas.Draw()
#add setlinecolor and with everything

In [ ]:
#G18 without initial state pions v4
G18_ETrans_NIP_Canvas = ROOT.TCanvas()

G18_ETrans_QEL_NIP_Plot = ROOT.TGraph(len(G18_XVals_Etrans_NIP_Array), G18_XVals_Etrans_NIP_Array, G18_QEL_Etrans_NIP_Array)
G18_ETrans_QEL_NIP_Plot.SetTitle("Relative Fraction of Interaction Type Per Energy Transfer Cut w/o Initial State Pions in G18 Model")
G18_ETrans_QEL_NIP_Plot.GetXaxis().SetTitle("Energy Transfer (GeV)")
G18_ETrans_QEL_NIP_Plot.GetYaxis().SetTitle("Percentage")
G18_ETrans_QEL_NIP_Plot.SetMaximum(1.1)
G18_ETrans_QEL_NIP_Plot.SetMinimum(0.0)
G18_ETrans_QEL_NIP_Plot.SetMarkerColor(4)
G18_ETrans_QEL_NIP_Plot.SetMarkerStyle(29)
G18_ETrans_QEL_NIP_Plot.Draw("same")

G18_ETrans_MEC_NIP_Plot = ROOT.TGraph(len(G18_XVals_Etrans_NIP_Array), G18_XVals_Etrans_NIP_Array, G18_MEC_Etrans_NIP_Array)
G18_ETrans_MEC_NIP_Plot.Draw("same")
G18_ETrans_MEC_NIP_Plot.SetMarkerColor(2)
G18_ETrans_MEC_NIP_Plot.SetMarkerStyle(29)

G18_ETrans_RES_NIP_Plot = ROOT.TGraph(len(G18_XVals_Etrans_NIP_Array), G18_XVals_Etrans_NIP_Array, G18_RES_Etrans_NIP_Array)
G18_ETrans_RES_NIP_Plot.Draw("same")
G18_ETrans_RES_NIP_Plot.SetMarkerColor(3)
G18_ETrans_RES_NIP_Plot.SetMarkerStyle(29)

G18_ETrans_DIS_NIP_Plot = ROOT.TGraph(len(G18_XVals_Etrans_NIP_Array), G18_XVals_Etrans_NIP_Array, G18_DIS_Etrans_NIP_Array)
G18_ETrans_DIS_NIP_Plot.Draw("same")
G18_ETrans_DIS_NIP_Plot.SetMarkerColor(6)
G18_ETrans_DIS_NIP_Plot.SetMarkerStyle(29)

G18_ETrans_Sums_NIP_Plot = ROOT.TGraph(len(G18_XVals_Etrans_NIP_Array), G18_XVals_Etrans_NIP_Array, G18_Etrans_Sum_NIP_Array)
G18_ETrans_Sums_NIP_Plot.Draw("same")
G18_ETrans_Sums_NIP_Plot.SetMarkerColor(1)
G18_ETrans_Sums_NIP_Plot.SetMarkerStyle(34)

G18_ETrans_Leg_NIP = ROOT.TLegend(0.2, 0.85, 0.80, 0.90)
G18_ETrans_Leg_NIP.SetNColumns(5)
G18_ETrans_Leg_NIP.AddEntry(G18_ETrans_QEL_NIP_Plot, "QEL")
G18_ETrans_Leg_NIP.AddEntry(G18_ETrans_MEC_NIP_Plot, "MEC")
G18_ETrans_Leg_NIP.AddEntry(G18_ETrans_RES_NIP_Plot, "RES")
G18_ETrans_Leg_NIP.AddEntry(G18_ETrans_DIS_NIP_Plot, "DIS")
G18_ETrans_Leg_NIP.AddEntry(G18_ETrans_Sums_NIP_Plot, "SUM")
G18_ETrans_Leg_NIP.Draw()

G18_ETrans_Zeroes_NIP = []
for i in range(len(G18_XVals_Etrans_NIP_Array)):
    G18_ETrans_Zeroes_NIP.append(0.05)
G18_ETrans_Zeroes_NIP_array = array.array('f', G18_ETrans_Zeroes_NIP)

G18_Yerr_QEL_NIP_Bars = ROOT.TGraphErrors(len(G18_XVals_Etrans_NIP_Array), G18_XVals_Etrans_NIP_Array, G18_QEL_Etrans_NIP_Array, G18_ETrans_Zeroes_NIP_array, G18_Yerr_QEL_Etrans_NIP_Array)
G18_Yerr_MEC_NIP_Bars = ROOT.TGraphErrors(len(G18_XVals_Etrans_NIP_Array), G18_XVals_Etrans_NIP_Array, G18_MEC_Etrans_NIP_Array, G18_ETrans_Zeroes_NIP_array, G18_Yerr_MEC_Etrans_NIP_Array)
G18_Yerr_RES_NIP_Bars = ROOT.TGraphErrors(len(G18_XVals_Etrans_NIP_Array), G18_XVals_Etrans_NIP_Array, G18_RES_Etrans_NIP_Array, G18_ETrans_Zeroes_NIP_array, G18_Yerr_RES_Etrans_NIP_Array)
G18_Yerr_DIS_NIP_Bars = ROOT.TGraphErrors(len(G18_XVals_Etrans_NIP_Array), G18_XVals_Etrans_NIP_Array, G18_DIS_Etrans_NIP_Array, G18_ETrans_Zeroes_NIP_array, G18_Yerr_DIS_Etrans_NIP_Array)
G18_Yerr_QEL_NIP_Bars.Draw("same")
G18_Yerr_MEC_NIP_Bars.Draw("same")
G18_Yerr_RES_NIP_Bars.Draw("same")
G18_Yerr_DIS_NIP_Bars.Draw("same")

G18_ETrans_NIP_Canvas.Draw()
#add setlinecolor and with everything

In [ ]:
#G18 without final state pions v4
G18_ETrans_NFP_Canvas = ROOT.TCanvas()

G18_ETrans_QEL_NFP_Plot = ROOT.TGraph(len(G18_XVals_Etrans_NFP_Array), G18_XVals_Etrans_NFP_Array, G18_QEL_Etrans_NFP_Array)
G18_ETrans_QEL_NFP_Plot.SetTitle("Relative Fraction of Interaction Type Per Energy Transfer Cut w/o Final State Pions in G18 Model")
G18_ETrans_QEL_NFP_Plot.GetXaxis().SetTitle("Energy Transfer (GeV)")
G18_ETrans_QEL_NFP_Plot.GetYaxis().SetTitle("Percentage")
G18_ETrans_QEL_NFP_Plot.SetMaximum(1.1)
G18_ETrans_QEL_NFP_Plot.SetMinimum(0.0)
G18_ETrans_QEL_NFP_Plot.SetMarkerColor(4)
G18_ETrans_QEL_NFP_Plot.SetMarkerStyle(29)
G18_ETrans_QEL_NFP_Plot.Draw("same")

G18_ETrans_MEC_NFP_Plot = ROOT.TGraph(len(G18_XVals_Etrans_NFP_Array), G18_XVals_Etrans_NFP_Array, G18_MEC_Etrans_NFP_Array)
G18_ETrans_MEC_NFP_Plot.Draw("same")
G18_ETrans_MEC_NFP_Plot.SetMarkerColor(2)
G18_ETrans_MEC_NFP_Plot.SetMarkerStyle(29)

G18_ETrans_RES_NFP_Plot = ROOT.TGraph(len(G18_XVals_Etrans_NFP_Array), G18_XVals_Etrans_NFP_Array, G18_RES_Etrans_NFP_Array)
G18_ETrans_RES_NFP_Plot.Draw("same")
G18_ETrans_RES_NFP_Plot.SetMarkerColor(3)
G18_ETrans_RES_NFP_Plot.SetMarkerStyle(29)

G18_ETrans_DIS_NFP_Plot = ROOT.TGraph(len(G18_XVals_Etrans_NFP_Array), G18_XVals_Etrans_NFP_Array, G18_DIS_Etrans_NFP_Array)
G18_ETrans_DIS_NFP_Plot.Draw("same")
G18_ETrans_DIS_NFP_Plot.SetMarkerColor(6)
G18_ETrans_DIS_NFP_Plot.SetMarkerStyle(29)

G18_ETrans_Sums_NFP_Plot = ROOT.TGraph(len(G18_XVals_Etrans_NFP_Array), G18_XVals_Etrans_NFP_Array, G18_Etrans_Sum_NFP_Array)
G18_ETrans_Sums_NFP_Plot.Draw("same")
G18_ETrans_Sums_NFP_Plot.SetMarkerColor(1)
G18_ETrans_Sums_NFP_Plot.SetMarkerStyle(34)

G18_ETrans_Leg_NFP = ROOT.TLegend(0.2, 0.85, 0.80, 0.90)
G18_ETrans_Leg_NFP.SetNColumns(5)
G18_ETrans_Leg_NFP.AddEntry(G18_ETrans_QEL_NFP_Plot, "QEL")
G18_ETrans_Leg_NFP.AddEntry(G18_ETrans_MEC_NFP_Plot, "MEC")
G18_ETrans_Leg_NFP.AddEntry(G18_ETrans_RES_NFP_Plot, "RES")
G18_ETrans_Leg_NFP.AddEntry(G18_ETrans_DIS_NFP_Plot, "DIS")
G18_ETrans_Leg_NFP.AddEntry(G18_ETrans_Sums_NFP_Plot, "SUM")
G18_ETrans_Leg_NFP.Draw()

G18_ETrans_Zeroes_NFP = []
for i in range(len(G18_XVals_Etrans_NFP_Array)):
    G18_ETrans_Zeroes_NFP.append(0.05)
G18_ETrans_Zeroes_NFP_array = array.array('f', G18_ETrans_Zeroes_NFP)

G18_Yerr_QEL_NFP_Bars = ROOT.TGraphErrors(len(G18_XVals_Etrans_NFP_Array), G18_XVals_Etrans_NFP_Array, G18_QEL_Etrans_NFP_Array, G18_ETrans_Zeroes_NFP_array, G18_Yerr_QEL_Etrans_NFP_Array)
G18_Yerr_MEC_NFP_Bars = ROOT.TGraphErrors(len(G18_XVals_Etrans_NFP_Array), G18_XVals_Etrans_NFP_Array, G18_MEC_Etrans_NFP_Array, G18_ETrans_Zeroes_NFP_array, G18_Yerr_MEC_Etrans_NFP_Array)
G18_Yerr_RES_NFP_Bars = ROOT.TGraphErrors(len(G18_XVals_Etrans_NFP_Array), G18_XVals_Etrans_NFP_Array, G18_RES_Etrans_NFP_Array, G18_ETrans_Zeroes_NFP_array, G18_Yerr_RES_Etrans_NFP_Array)
G18_Yerr_DIS_NFP_Bars = ROOT.TGraphErrors(len(G18_XVals_Etrans_NFP_Array), G18_XVals_Etrans_NFP_Array, G18_DIS_Etrans_NFP_Array, G18_ETrans_Zeroes_NFP_array, G18_Yerr_DIS_Etrans_NFP_Array)
G18_Yerr_QEL_NFP_Bars.Draw("same")
G18_Yerr_MEC_NFP_Bars.Draw("same")
G18_Yerr_RES_NFP_Bars.Draw("same")
G18_Yerr_DIS_NFP_Bars.Draw("same")

G18_ETrans_NFP_Canvas.Draw()

In [ ]:
#G18 without nfa pions v4
G18_ETrans_NFAP_Canvas = ROOT.TCanvas()

G18_ETrans_QEL_NFAP_Plot = ROOT.TGraph(len(G18_XVals_Etrans_NFAP_Array), G18_XVals_Etrans_NFAP_Array, G18_QEL_Etrans_NFAP_Array)
G18_ETrans_QEL_NFAP_Plot.SetTitle("Relative Fraction of Interaction Type Per Energy Transfer Cut w/o NFA Pions in G18 Model")
G18_ETrans_QEL_NFAP_Plot.GetXaxis().SetTitle("Energy Transfer (GeV)")
G18_ETrans_QEL_NFAP_Plot.GetYaxis().SetTitle("Percentage")
G18_ETrans_QEL_NFAP_Plot.SetMaximum(1.1)
G18_ETrans_QEL_NFAP_Plot.SetMinimum(0.0)
G18_ETrans_QEL_NFAP_Plot.SetMarkerColor(4)
G18_ETrans_QEL_NFAP_Plot.SetMarkerStyle(29)
G18_ETrans_QEL_NFAP_Plot.Draw("same")

G18_ETrans_MEC_NFAP_Plot = ROOT.TGraph(len(G18_XVals_Etrans_NFAP_Array), G18_XVals_Etrans_NFAP_Array, G18_MEC_Etrans_NFAP_Array)
G18_ETrans_MEC_NFAP_Plot.Draw("same")
G18_ETrans_MEC_NFAP_Plot.SetMarkerColor(2)
G18_ETrans_MEC_NFAP_Plot.SetMarkerStyle(29)

G18_ETrans_RES_NFAP_Plot = ROOT.TGraph(len(G18_XVals_Etrans_NFAP_Array), G18_XVals_Etrans_NFAP_Array, G18_RES_Etrans_NFAP_Array)
G18_ETrans_RES_NFAP_Plot.Draw("same")
G18_ETrans_RES_NFAP_Plot.SetMarkerColor(3)
G18_ETrans_RES_NFAP_Plot.SetMarkerStyle(29)

G18_ETrans_DIS_NFAP_Plot = ROOT.TGraph(len(G18_XVals_Etrans_NFAP_Array), G18_XVals_Etrans_NFAP_Array, G18_DIS_Etrans_NFAP_Array)
G18_ETrans_DIS_NFAP_Plot.Draw("same")
G18_ETrans_DIS_NFAP_Plot.SetMarkerColor(6)
G18_ETrans_DIS_NFAP_Plot.SetMarkerStyle(29)

G18_ETrans_Sums_NFAP_Plot = ROOT.TGraph(len(G18_XVals_Etrans_NFAP_Array), G18_XVals_Etrans_NFAP_Array, G18_Etrans_Sum_NFAP_Array)
G18_ETrans_Sums_NFAP_Plot.Draw("same")
G18_ETrans_Sums_NFAP_Plot.SetMarkerColor(1)
G18_ETrans_Sums_NFAP_Plot.SetMarkerStyle(34)

G18_ETrans_Leg_NFAP = ROOT.TLegend(0.2, 0.85, 0.80, 0.90)
G18_ETrans_Leg_NFAP.SetNColumns(5)
G18_ETrans_Leg_NFAP.AddEntry(G18_ETrans_QEL_NFAP_Plot, "QEL")
G18_ETrans_Leg_NFAP.AddEntry(G18_ETrans_MEC_NFAP_Plot, "MEC")
G18_ETrans_Leg_NFAP.AddEntry(G18_ETrans_RES_NFAP_Plot, "RES")
G18_ETrans_Leg_NFAP.AddEntry(G18_ETrans_DIS_NFAP_Plot, "DIS")
G18_ETrans_Leg_NFAP.AddEntry(G18_ETrans_Sums_NFAP_Plot, "SUM")
G18_ETrans_Leg_NFAP.Draw()

G18_ETrans_Zeroes_NFAP = []
for i in range(len(G18_XVals_Etrans_NFAP_Array)):
    G18_ETrans_Zeroes_NFAP.append(0.05)
G18_ETrans_Zeroes_NFAP_array = array.array('f', G18_ETrans_Zeroes_NFAP)

G18_Yerr_QEL_NFAP_Bars = ROOT.TGraphErrors(len(G18_XVals_Etrans_NFAP_Array), G18_XVals_Etrans_NFAP_Array, G18_QEL_Etrans_NFAP_Array, G18_ETrans_Zeroes_NFAP_array, G18_Yerr_QEL_Etrans_NFAP_Array)
G18_Yerr_MEC_NFAP_Bars = ROOT.TGraphErrors(len(G18_XVals_Etrans_NFAP_Array), G18_XVals_Etrans_NFAP_Array, G18_MEC_Etrans_NFAP_Array, G18_ETrans_Zeroes_NFAP_array, G18_Yerr_MEC_Etrans_NFAP_Array)
G18_Yerr_RES_NFAP_Bars = ROOT.TGraphErrors(len(G18_XVals_Etrans_NFAP_Array), G18_XVals_Etrans_NFAP_Array, G18_RES_Etrans_NFAP_Array, G18_ETrans_Zeroes_NFAP_array, G18_Yerr_RES_Etrans_NFAP_Array)
G18_Yerr_DIS_NFAP_Bars = ROOT.TGraphErrors(len(G18_XVals_Etrans_NFAP_Array), G18_XVals_Etrans_NFAP_Array, G18_DIS_Etrans_NFAP_Array, G18_ETrans_Zeroes_NFAP_array, G18_Yerr_DIS_Etrans_NFAP_Array)
G18_Yerr_QEL_NFAP_Bars.Draw("same")
G18_Yerr_MEC_NFAP_Bars.Draw("same")
G18_Yerr_RES_NFAP_Bars.Draw("same")
G18_Yerr_DIS_NFAP_Bars.Draw("same")

G18_ETrans_NFAP_Canvas.Draw()

In [ ]:
#gem21 plots

In [ ]:
#G21 v4
G21_ETrans_Pions_Canvas = ROOT.TCanvas()

G21_ETrans_QEL_Pions_Plot = ROOT.TGraph(len(G21_XVals_Etrans_Pions_Array), G21_XVals_Etrans_Pions_Array, G21_QEL_Etrans_Pions_Array)
G21_ETrans_QEL_Pions_Plot.SetTitle("Relative Fraction of Interaction Type Per Energy Transfer Cut in GEM21 Model")
G21_ETrans_QEL_Pions_Plot.GetXaxis().SetTitle("Energy Transfer (GeV)")
G21_ETrans_QEL_Pions_Plot.GetYaxis().SetTitle("Percentage")
G21_ETrans_QEL_Pions_Plot.SetMaximum(1.1)
G21_ETrans_QEL_Pions_Plot.SetMinimum(0.0)
G21_ETrans_QEL_Pions_Plot.SetMarkerColor(4)
G21_ETrans_QEL_Pions_Plot.SetMarkerStyle(29)
G21_ETrans_QEL_Pions_Plot.Draw("same")

G21_ETrans_MEC_Pions_Plot = ROOT.TGraph(len(G21_XVals_Etrans_Pions_Array), G21_XVals_Etrans_Pions_Array, G21_MEC_Etrans_Pions_Array)
G21_ETrans_MEC_Pions_Plot.Draw("same")
G21_ETrans_MEC_Pions_Plot.SetMarkerColor(2)
G21_ETrans_MEC_Pions_Plot.SetMarkerStyle(29)

G21_ETrans_RES_Pions_Plot = ROOT.TGraph(len(G21_XVals_Etrans_Pions_Array), G21_XVals_Etrans_Pions_Array, G21_RES_Etrans_Pions_Array)
G21_ETrans_RES_Pions_Plot.Draw("same")
G21_ETrans_RES_Pions_Plot.SetMarkerColor(3)
G21_ETrans_RES_Pions_Plot.SetMarkerStyle(29)

G21_ETrans_DIS_Pions_Plot = ROOT.TGraph(len(G21_XVals_Etrans_Pions_Array), G21_XVals_Etrans_Pions_Array, G21_DIS_Etrans_Pions_Array)
G21_ETrans_DIS_Pions_Plot.Draw("same")
G21_ETrans_DIS_Pions_Plot.SetMarkerColor(6)
G21_ETrans_DIS_Pions_Plot.SetMarkerStyle(29)

G21_ETrans_Sums_Pions_Plot = ROOT.TGraph(len(G21_XVals_Etrans_Pions_Array), G21_XVals_Etrans_Pions_Array, G21_Etrans_Sum_Pions_Array)
G21_ETrans_Sums_Pions_Plot.Draw("same")
G21_ETrans_Sums_Pions_Plot.SetMarkerColor(1)
G21_ETrans_Sums_Pions_Plot.SetMarkerStyle(34)

G21_ETrans_Leg_Pions = ROOT.TLegend(0.2, 0.85, 0.80, 0.90)
G21_ETrans_Leg_Pions.SetNColumns(5)
G21_ETrans_Leg_Pions.AddEntry(G21_ETrans_QEL_Pions_Plot, "QEL")
G21_ETrans_Leg_Pions.AddEntry(G21_ETrans_MEC_Pions_Plot, "MEC")
G21_ETrans_Leg_Pions.AddEntry(G21_ETrans_RES_Pions_Plot, "RES")
G21_ETrans_Leg_Pions.AddEntry(G21_ETrans_DIS_Pions_Plot, "DIS")
G21_ETrans_Leg_Pions.AddEntry(G21_ETrans_Sums_Pions_Plot, "SUM")
G21_ETrans_Leg_Pions.Draw()

G21_ETrans_Zeroes_Pions = []
for i in range(len(G21_XVals_Etrans_Pions_Array)):
    G21_ETrans_Zeroes_Pions.append(0.05)
G21_ETrans_Zeroes_Pions_array = array.array('f', G21_ETrans_Zeroes_Pions)

G21_Yerr_QEL_Pions_Bars = ROOT.TGraphErrors(len(G21_XVals_Etrans_Pions_Array), G21_XVals_Etrans_Pions_Array, G21_QEL_Etrans_Pions_Array, G21_ETrans_Zeroes_Pions_array, G21_Yerr_QEL_Etrans_Pions_Array)
G21_Yerr_MEC_Pions_Bars = ROOT.TGraphErrors(len(G21_XVals_Etrans_Pions_Array), G21_XVals_Etrans_Pions_Array, G21_MEC_Etrans_Pions_Array, G21_ETrans_Zeroes_Pions_array, G21_Yerr_MEC_Etrans_Pions_Array)
G21_Yerr_RES_Pions_Bars = ROOT.TGraphErrors(len(G21_XVals_Etrans_Pions_Array), G21_XVals_Etrans_Pions_Array, G21_RES_Etrans_Pions_Array, G21_ETrans_Zeroes_Pions_array, G21_Yerr_RES_Etrans_Pions_Array)
G21_Yerr_DIS_Pions_Bars = ROOT.TGraphErrors(len(G21_XVals_Etrans_Pions_Array), G21_XVals_Etrans_Pions_Array, G21_DIS_Etrans_Pions_Array, G21_ETrans_Zeroes_Pions_array, G21_Yerr_DIS_Etrans_Pions_Array)
G21_Yerr_QEL_Pions_Bars.Draw("same")
G21_Yerr_MEC_Pions_Bars.Draw("same")
G21_Yerr_RES_Pions_Bars.Draw("same")
G21_Yerr_DIS_Pions_Bars.Draw("same")

G21_ETrans_Pions_Canvas.Draw()

In [ ]:
#G21 without initial state pions v4
G21_ETrans_NIP_Canvas = ROOT.TCanvas()

G21_ETrans_QEL_NIP_Plot = ROOT.TGraph(len(G21_XVals_Etrans_NIP_Array), G21_XVals_Etrans_NIP_Array, G21_QEL_Etrans_NIP_Array)
G21_ETrans_QEL_NIP_Plot.SetTitle("Relative Fraction of Interaction Type Per Energy Transfer Cut w/o Initial State Pions in GEM21 Model")
G21_ETrans_QEL_NIP_Plot.GetXaxis().SetTitle("Energy Transfer (GeV)")
G21_ETrans_QEL_NIP_Plot.GetYaxis().SetTitle("Percentage")
G21_ETrans_QEL_NIP_Plot.SetMaximum(1.1)
G21_ETrans_QEL_NIP_Plot.SetMinimum(0.0)
G21_ETrans_QEL_NIP_Plot.SetMarkerColor(4)
G21_ETrans_QEL_NIP_Plot.SetMarkerStyle(29)
G21_ETrans_QEL_NIP_Plot.Draw("same")

G21_ETrans_MEC_NIP_Plot = ROOT.TGraph(len(G21_XVals_Etrans_NIP_Array), G21_XVals_Etrans_NIP_Array, G21_MEC_Etrans_NIP_Array)
G21_ETrans_MEC_NIP_Plot.Draw("same")
G21_ETrans_MEC_NIP_Plot.SetMarkerColor(2)
G21_ETrans_MEC_NIP_Plot.SetMarkerStyle(29)

G21_ETrans_RES_NIP_Plot = ROOT.TGraph(len(G21_XVals_Etrans_NIP_Array), G21_XVals_Etrans_NIP_Array, G21_RES_Etrans_NIP_Array)
G21_ETrans_RES_NIP_Plot.Draw("same")
G21_ETrans_RES_NIP_Plot.SetMarkerColor(3)
G21_ETrans_RES_NIP_Plot.SetMarkerStyle(29)

G21_ETrans_DIS_NIP_Plot = ROOT.TGraph(len(G21_XVals_Etrans_NIP_Array), G21_XVals_Etrans_NIP_Array, G21_DIS_Etrans_NIP_Array)
G21_ETrans_DIS_NIP_Plot.Draw("same")
G21_ETrans_DIS_NIP_Plot.SetMarkerColor(6)
G21_ETrans_DIS_NIP_Plot.SetMarkerStyle(29)

G21_ETrans_Sums_NIP_Plot = ROOT.TGraph(len(G21_XVals_Etrans_NIP_Array), G21_XVals_Etrans_NIP_Array, G21_Etrans_Sum_NIP_Array)
G21_ETrans_Sums_NIP_Plot.Draw("same")
G21_ETrans_Sums_NIP_Plot.SetMarkerColor(1)
G21_ETrans_Sums_NIP_Plot.SetMarkerStyle(34)

G21_ETrans_Leg_NIP = ROOT.TLegend(0.2, 0.85, 0.80, 0.90)
G21_ETrans_Leg_NIP.SetNColumns(5)
G21_ETrans_Leg_NIP.AddEntry(G21_ETrans_QEL_NIP_Plot, "QEL")
G21_ETrans_Leg_NIP.AddEntry(G21_ETrans_MEC_NIP_Plot, "MEC")
G21_ETrans_Leg_NIP.AddEntry(G21_ETrans_RES_NIP_Plot, "RES")
G21_ETrans_Leg_NIP.AddEntry(G21_ETrans_DIS_NIP_Plot, "DIS")
G21_ETrans_Leg_NIP.AddEntry(G21_ETrans_Sums_NIP_Plot, "SUM")
G21_ETrans_Leg_NIP.Draw()

G21_ETrans_Zeroes_NIP = []
for i in range(len(G21_XVals_Etrans_NIP_Array)):
    G21_ETrans_Zeroes_NIP.append(0.05)
G21_ETrans_Zeroes_NIP_array = array.array('f', G21_ETrans_Zeroes_NIP)

G21_Yerr_QEL_NIP_Bars = ROOT.TGraphErrors(len(G21_XVals_Etrans_NIP_Array), G21_XVals_Etrans_NIP_Array, G21_QEL_Etrans_NIP_Array, G21_ETrans_Zeroes_NIP_array, G21_Yerr_QEL_Etrans_NIP_Array)
G21_Yerr_MEC_NIP_Bars = ROOT.TGraphErrors(len(G21_XVals_Etrans_NIP_Array), G21_XVals_Etrans_NIP_Array, G21_MEC_Etrans_NIP_Array, G21_ETrans_Zeroes_NIP_array, G21_Yerr_MEC_Etrans_NIP_Array)
G21_Yerr_RES_NIP_Bars = ROOT.TGraphErrors(len(G21_XVals_Etrans_NIP_Array), G21_XVals_Etrans_NIP_Array, G21_RES_Etrans_NIP_Array, G21_ETrans_Zeroes_NIP_array, G21_Yerr_RES_Etrans_NIP_Array)
G21_Yerr_DIS_NIP_Bars = ROOT.TGraphErrors(len(G21_XVals_Etrans_NIP_Array), G21_XVals_Etrans_NIP_Array, G21_DIS_Etrans_NIP_Array, G21_ETrans_Zeroes_NIP_array, G21_Yerr_DIS_Etrans_NIP_Array)
G21_Yerr_QEL_NIP_Bars.Draw("same")
G21_Yerr_MEC_NIP_Bars.Draw("same")
G21_Yerr_RES_NIP_Bars.Draw("same")
G21_Yerr_DIS_NIP_Bars.Draw("same")

G21_ETrans_NIP_Canvas.Draw()

In [ ]:
#GEM21 without final state pions v4
G21_ETrans_NFP_Canvas = ROOT.TCanvas()

G21_ETrans_QEL_NFP_Plot = ROOT.TGraph(len(G21_XVals_Etrans_NFP_Array), G21_XVals_Etrans_NFP_Array, G21_QEL_Etrans_NFP_Array)
G21_ETrans_QEL_NFP_Plot.SetTitle("Relative Fraction of Interaction Type Per Energy Transfer Cut w/o Final State Pions in GEM21 Model")
G21_ETrans_QEL_NFP_Plot.GetXaxis().SetTitle("Energy Transfer (GeV)")
G21_ETrans_QEL_NFP_Plot.GetYaxis().SetTitle("Percentage")
G21_ETrans_QEL_NFP_Plot.SetMaximum(1.1)
G21_ETrans_QEL_NFP_Plot.SetMinimum(0.0)
G21_ETrans_QEL_NFP_Plot.SetMarkerColor(4)
G21_ETrans_QEL_NFP_Plot.SetMarkerStyle(29)
G21_ETrans_QEL_NFP_Plot.Draw("same")

G21_ETrans_MEC_NFP_Plot = ROOT.TGraph(len(G21_XVals_Etrans_NFP_Array), G21_XVals_Etrans_NFP_Array, G21_MEC_Etrans_NFP_Array)
G21_ETrans_MEC_NFP_Plot.Draw("same")
G21_ETrans_MEC_NFP_Plot.SetMarkerColor(2)
G21_ETrans_MEC_NFP_Plot.SetMarkerStyle(29)

G21_ETrans_RES_NFP_Plot = ROOT.TGraph(len(G21_XVals_Etrans_NFP_Array), G21_XVals_Etrans_NFP_Array, G21_RES_Etrans_NFP_Array)
G21_ETrans_RES_NFP_Plot.Draw("same")
G21_ETrans_RES_NFP_Plot.SetMarkerColor(3)
G21_ETrans_RES_NFP_Plot.SetMarkerStyle(29)

G21_ETrans_DIS_NFP_Plot = ROOT.TGraph(len(G21_XVals_Etrans_NFP_Array), G21_XVals_Etrans_NFP_Array, G21_DIS_Etrans_NFP_Array)
G21_ETrans_DIS_NFP_Plot.Draw("same")
G21_ETrans_DIS_NFP_Plot.SetMarkerColor(6)
G21_ETrans_DIS_NFP_Plot.SetMarkerStyle(29)

G21_ETrans_Sums_NFP_Plot = ROOT.TGraph(len(G21_XVals_Etrans_NFP_Array), G21_XVals_Etrans_NFP_Array, G21_Etrans_Sum_NFP_Array)
G21_ETrans_Sums_NFP_Plot.Draw("same")
G21_ETrans_Sums_NFP_Plot.SetMarkerColor(1)
G21_ETrans_Sums_NFP_Plot.SetMarkerStyle(34)

G21_ETrans_Leg_NFP = ROOT.TLegend(0.2, 0.85, 0.80, 0.90)
G21_ETrans_Leg_NFP.SetNColumns(5)
G21_ETrans_Leg_NFP.AddEntry(G21_ETrans_QEL_NFP_Plot, "QEL")
G21_ETrans_Leg_NFP.AddEntry(G21_ETrans_MEC_NFP_Plot, "MEC")
G21_ETrans_Leg_NFP.AddEntry(G21_ETrans_RES_NFP_Plot, "RES")
G21_ETrans_Leg_NFP.AddEntry(G21_ETrans_DIS_NFP_Plot, "DIS")
G21_ETrans_Leg_NFP.AddEntry(G21_ETrans_Sums_NFP_Plot, "SUM")
G21_ETrans_Leg_NFP.Draw()

G21_ETrans_Zeroes_NFP = []
for i in range(len(G21_XVals_Etrans_NFP_Array)):
    G21_ETrans_Zeroes_NFP.append(0.05)
G21_ETrans_Zeroes_NFP_array = array.array('f', G21_ETrans_Zeroes_NFP)

G21_Yerr_QEL_NFP_Bars = ROOT.TGraphErrors(len(G21_XVals_Etrans_NFP_Array), G21_XVals_Etrans_NFP_Array, G21_QEL_Etrans_NFP_Array, G21_ETrans_Zeroes_NFP_array, G21_Yerr_QEL_Etrans_NFP_Array)
G21_Yerr_MEC_NFP_Bars = ROOT.TGraphErrors(len(G21_XVals_Etrans_NFP_Array), G21_XVals_Etrans_NFP_Array, G21_MEC_Etrans_NFP_Array, G21_ETrans_Zeroes_NFP_array, G21_Yerr_MEC_Etrans_NFP_Array)
G21_Yerr_RES_NFP_Bars = ROOT.TGraphErrors(len(G21_XVals_Etrans_NFP_Array), G21_XVals_Etrans_NFP_Array, G21_RES_Etrans_NFP_Array, G21_ETrans_Zeroes_NFP_array, G21_Yerr_RES_Etrans_NFP_Array)
G21_Yerr_DIS_NFP_Bars = ROOT.TGraphErrors(len(G21_XVals_Etrans_NFP_Array), G21_XVals_Etrans_NFP_Array, G21_DIS_Etrans_NFP_Array, G21_ETrans_Zeroes_NFP_array, G21_Yerr_DIS_Etrans_NFP_Array)
G21_Yerr_QEL_NFP_Bars.Draw("same")
G21_Yerr_MEC_NFP_Bars.Draw("same")
G21_Yerr_RES_NFP_Bars.Draw("same")
G21_Yerr_DIS_NFP_Bars.Draw("same")

G21_ETrans_NFP_Canvas.Draw()

In [ ]:
#GEM21 without nfa pions v4
G21_ETrans_NFAP_Canvas = ROOT.TCanvas()

G21_ETrans_QEL_NFAP_Plot = ROOT.TGraph(len(G21_XVals_Etrans_NFAP_Array), G21_XVals_Etrans_NFAP_Array, G21_QEL_Etrans_NFAP_Array)
G21_ETrans_QEL_NFAP_Plot.SetTitle("Relative Fraction of Interaction Type Per Energy Transfer Cut w/o NFA Pions in GEM21 Model")
G21_ETrans_QEL_NFAP_Plot.GetXaxis().SetTitle("Energy Transfer (GeV)")
G21_ETrans_QEL_NFAP_Plot.GetYaxis().SetTitle("Percentage")
G21_ETrans_QEL_NFAP_Plot.SetMaximum(1.1)
G21_ETrans_QEL_NFAP_Plot.SetMinimum(0.0)
G21_ETrans_QEL_NFAP_Plot.SetMarkerColor(4)
G21_ETrans_QEL_NFAP_Plot.SetMarkerStyle(29)
G21_ETrans_QEL_NFAP_Plot.Draw("same")

G21_ETrans_MEC_NFAP_Plot = ROOT.TGraph(len(G21_XVals_Etrans_NFAP_Array), G21_XVals_Etrans_NFAP_Array, G21_MEC_Etrans_NFAP_Array)
G21_ETrans_MEC_NFAP_Plot.Draw("same")
G21_ETrans_MEC_NFAP_Plot.SetMarkerColor(2)
G21_ETrans_MEC_NFAP_Plot.SetMarkerStyle(29)

G21_ETrans_RES_NFAP_Plot = ROOT.TGraph(len(G21_XVals_Etrans_NFAP_Array), G21_XVals_Etrans_NFAP_Array, G21_RES_Etrans_NFAP_Array)
G21_ETrans_RES_NFAP_Plot.Draw("same")
G21_ETrans_RES_NFAP_Plot.SetMarkerColor(3)
G21_ETrans_RES_NFAP_Plot.SetMarkerStyle(29)

G21_ETrans_DIS_NFAP_Plot = ROOT.TGraph(len(G21_XVals_Etrans_NFAP_Array), G21_XVals_Etrans_NFAP_Array, G21_DIS_Etrans_NFAP_Array)
G21_ETrans_DIS_NFAP_Plot.Draw("same")
G21_ETrans_DIS_NFAP_Plot.SetMarkerColor(6)
G21_ETrans_DIS_NFAP_Plot.SetMarkerStyle(29)

G21_ETrans_Sums_NFAP_Plot = ROOT.TGraph(len(G21_XVals_Etrans_NFAP_Array), G21_XVals_Etrans_NFAP_Array, G21_Etrans_Sum_NFAP_Array)
G21_ETrans_Sums_NFAP_Plot.Draw("same")
G21_ETrans_Sums_NFAP_Plot.SetMarkerColor(1)
G21_ETrans_Sums_NFAP_Plot.SetMarkerStyle(34)

G21_ETrans_Leg_NFAP = ROOT.TLegend(0.2, 0.85, 0.80, 0.90)
G21_ETrans_Leg_NFAP.SetNColumns(5)
G21_ETrans_Leg_NFAP.AddEntry(G21_ETrans_QEL_NFAP_Plot, "QEL")
G21_ETrans_Leg_NFAP.AddEntry(G21_ETrans_MEC_NFAP_Plot, "MEC")
G21_ETrans_Leg_NFAP.AddEntry(G21_ETrans_RES_NFAP_Plot, "RES")
G21_ETrans_Leg_NFAP.AddEntry(G21_ETrans_DIS_NFAP_Plot, "DIS")
G21_ETrans_Leg_NFAP.AddEntry(G21_ETrans_Sums_NFAP_Plot, "SUM")
G21_ETrans_Leg_NFAP.Draw()

G21_ETrans_Zeroes_NFAP = []
for i in range(len(G21_XVals_Etrans_NFAP_Array)):
    G21_ETrans_Zeroes_NFAP.append(0.05)
G21_ETrans_Zeroes_NFAP_array = array.array('f', G21_ETrans_Zeroes_NFAP)

G21_Yerr_QEL_NFAP_Bars = ROOT.TGraphErrors(len(G21_XVals_Etrans_NFAP_Array), G21_XVals_Etrans_NFAP_Array, G21_QEL_Etrans_NFAP_Array, G21_ETrans_Zeroes_NFAP_array, G21_Yerr_QEL_Etrans_NFAP_Array)
G21_Yerr_MEC_NFAP_Bars = ROOT.TGraphErrors(len(G21_XVals_Etrans_NFAP_Array), G21_XVals_Etrans_NFAP_Array, G21_MEC_Etrans_NFAP_Array, G21_ETrans_Zeroes_NFAP_array, G21_Yerr_MEC_Etrans_NFAP_Array)
G21_Yerr_RES_NFAP_Bars = ROOT.TGraphErrors(len(G21_XVals_Etrans_NFAP_Array), G21_XVals_Etrans_NFAP_Array, G21_RES_Etrans_NFAP_Array, G21_ETrans_Zeroes_NFAP_array, G21_Yerr_RES_Etrans_NFAP_Array)
G21_Yerr_DIS_NFAP_Bars = ROOT.TGraphErrors(len(G21_XVals_Etrans_NFAP_Array), G21_XVals_Etrans_NFAP_Array, G21_DIS_Etrans_NFAP_Array, G21_ETrans_Zeroes_NFAP_array, G21_Yerr_DIS_Etrans_NFAP_Array)
G21_Yerr_QEL_NFAP_Bars.Draw("same")
G21_Yerr_MEC_NFAP_Bars.Draw("same")
G21_Yerr_RES_NFAP_Bars.Draw("same")
G21_Yerr_DIS_NFAP_Bars.Draw("same")

G21_ETrans_NFAP_Canvas.Draw()

# random things

In [ ]:
#what the fraction of QE/MEC/RES/DIS events is as a function of the number of protons, neutrons, and protons+neutrons 
bins = 100
x_start = 0
x_end = 4.0 #GeV

In [ ]:
G18_Proton_Percent_List = []
G18_Neutron_Percent_List = []
G18_ProNeu_Percent_List = []
G18_Proton_lists = [G18_Proton_Percent_List, G18_Neutron_Percent_List, G18_ProNeu_Percent_List]
particles = ["nfa_proton", "nfa_neutron", "n_proton_neutron"]
run = 0
for particle in particles:
    df_gst_G18_02a_nfa_par = df_gst_G18_02a_all.Filter("nfa_proton<1")
    G18_Proton_QEL_Ang = df_gst_G18_02a_nfa_par.Filter("qel==1").Histo1D(("G18_Proton_QEL_Ang", "", bins, x_start, x_end), f"{particle}")
    G18_Proton_MEC_Ang = df_gst_G18_02a_nfa_par.Filter("mec==1").Histo1D(("G18_Proton_MEC_Ang", "", bins, x_start, x_end), f"{particle}")
    G18_Proton_RES_Ang = df_gst_G18_02a_nfa_par.Filter("res==1").Histo1D(("G18_Proton_RES_Ang", "", bins, x_start, x_end), f"{particle}")
    G18_Proton_DIS_Ang = df_gst_G18_02a_nfa_par.Filter("dis==1").Histo1D(("G18_Proton_DIS_Ang", "", bins, x_start, x_end), f"{particle}")
    G18_Proton_Counts = [G18_Proton_QEL_Ang.Integral(), G18_Proton_MEC_Ang.Integral(), G18_Proton_RES_Ang.Integral(), G18_Proton_DIS_Ang.Integral()]
    G18_Proton_total = G18_Proton_Counts[0] + G18_Proton_Counts[1] + G18_Proton_Counts[2] + G18_Proton_Counts[3] 
    G18_Proton_lists[run].append(G18_Proton_Counts[0]/G18_Proton_total)
    G18_Proton_lists[run].append(G18_Proton_Counts[1]/G18_Proton_total)
    G18_Proton_lists[run].append(G18_Proton_Counts[2]/G18_Proton_total)
    G18_Proton_lists[run].append(G18_Proton_Counts[3]/G18_Proton_total)
    run += 1
print(G18_Proton_Percent_List, G18_Neutron_Percent_List, G18_ProNeu_Percent_List)

In [ ]:
G21_Proton_Percent_List = []
G21_Neutron_Percent_List = []
G21_ProNeu_Percent_List = []
G21_Proton_lists = [G21_Proton_Percent_List, G21_Neutron_Percent_List, G21_ProNeu_Percent_List]
particles = ["nfp", "nfn", "n_proton_neutron"]
run = 0
for particle in particles:
    G21_Proton_QEL_Ang = df_gst_GEM21_11b_all.Filter("qel==1").Histo1D(("G21_Proton_QEL_Ang", "", bins, x_start, x_end), f"{particle}")
    G21_Proton_MEC_Ang = df_gst_GEM21_11b_all.Filter("mec==1").Histo1D(("G21_Proton_MEC_Ang", "", bins, x_start, x_end), f"{particle}")
    G21_Proton_RES_Ang = df_gst_GEM21_11b_all.Filter("res==1").Histo1D(("G21_Proton_RES_Ang", "", bins, x_start, x_end), f"{particle}")
    G21_Proton_DIS_Ang = df_gst_GEM21_11b_all.Filter("dis==1").Histo1D(("G21_Proton_DIS_Ang", "", bins, x_start, x_end), f"{particle}")
    G21_Proton_Counts = [G21_Proton_QEL_Ang.Integral(), G21_Proton_MEC_Ang.Integral(), G21_Proton_RES_Ang.Integral(), G21_Proton_DIS_Ang.Integral()]
    G21_Proton_total = G21_Proton_Counts[0] + G21_Proton_Counts[1] + G21_Proton_Counts[2] + G21_Proton_Counts[3] 
    G21_Proton_lists[run].append(G21_Proton_Counts[0]/G21_Proton_total)
    G21_Proton_lists[run].append(G21_Proton_Counts[1]/G21_Proton_total)
    G21_Proton_lists[run].append(G21_Proton_Counts[2]/G21_Proton_total)
    G21_Proton_lists[run].append(G21_Proton_Counts[3]/G21_Proton_total)
    run += 1
print(G21_Proton_Percent_List, G21_Neutron_Percent_List, G21_ProNeu_Percent_List)

In [ ]:
X = ['QEL','MEC','RES','DIS']
X_axis = np.arange(len(X))

In [ ]:
plt.bar(X_axis - 0.2, G18_Proton_Percent_List, 0.4, label = 'G18')
plt.bar(X_axis + 0.2, G21_Proton_Percent_List, 0.4, label = 'GEM21', color='r')
plt.ylim(0.0, 1.0)
plt.xticks(X_axis, X)
plt.xlabel("Interaction Types")
plt.ylabel("Relative Percentage")
plt.title("Relative Percentages of Final State Protons in G18 and GEM21 Models ")
plt.legend()
plt.show()

In [ ]:
plt.bar(X_axis - 0.2, G18_Neutron_Percent_List, 0.4, label = 'G18')
plt.bar(X_axis + 0.2, G21_Neutron_Percent_List, 0.4, label = 'GEM21', color='r')
plt.ylim(0.0, 1.0)
plt.xticks(X_axis, X)
plt.xlabel("Interaction Types")
plt.ylabel("Relative Percentage")
plt.title("Relative Percentages of Final State Neutrons in G18 and GEM21 Models ")
plt.legend()
plt.show()

In [ ]:
plt.bar(X_axis - 0.2, G18_ProNeu_Percent_List, 0.4, label = 'G18')
plt.bar(X_axis + 0.2, G21_ProNeu_Percent_List, 0.4, label = 'GEM21', color='r')
plt.ylim(0.0, 1.0)
plt.xticks(X_axis, X)
plt.xlabel("Interaction Types")
plt.ylabel("Relative Percentage")
plt.title("Relative Percentages of Final State Protons and Neutrons in G18 and GEM21 Models ")
plt.legend()
plt.show()

In [ ]:
#limiting final state protons to some num

In [ ]:
bins = 100
x_start = 0
x_end = 4.0 #GeV
G18_Proton_Cut_Percent_List = []
G18_Neutron_Cut_Percent_List = []
G18_ProNeu_Cut_Percent_List = []
G18_Particle_Cut_lists = [G18_Proton_Cut_Percent_List, G18_Neutron_Cut_Percent_List, G18_ProNeu_Cut_Percent_List]
particles = ["nfp", "nfn", "n_proton_neutron"]
run = 0
cut_num = 1 #is this right
for particle in particles:
    df_gst_G18_02a_2Par = df_gst_G18_02a_all.Filter(f"{particle}<={cut_num}")
    G18_Particle_Cut_QEL_Ang = df_gst_G18_02a_2Par.Filter("qel==1").Histo1D(("G18_Particle_Cut_QEL_Ang", "", bins, x_start, x_end), f"{particle}")
    G18_Particle_Cut_MEC_Ang = df_gst_G18_02a_2Par.Filter("mec==1").Histo1D(("G18_Particle_Cut_MEC_Ang", "", bins, x_start, x_end), f"{particle}")
    G18_Particle_Cut_RES_Ang = df_gst_G18_02a_2Par.Filter("res==1").Histo1D(("G18_Particle_Cut_RES_Ang", "", bins, x_start, x_end), f"{particle}")
    G18_Particle_Cut_DIS_Ang = df_gst_G18_02a_2Par.Filter("dis==1").Histo1D(("G18_Particle_Cut_DIS_Ang", "", bins, x_start, x_end), f"{particle}")
    G18_Particle_Cut_Counts = [G18_Particle_Cut_QEL_Ang.Integral(), G18_Particle_Cut_MEC_Ang.Integral(), G18_Particle_Cut_RES_Ang.Integral(), G18_Particle_Cut_DIS_Ang.Integral()]
    G18_Particle_Cut_total = G18_Particle_Cut_Counts[0] + G18_Particle_Cut_Counts[1] + G18_Particle_Cut_Counts[2] + G18_Particle_Cut_Counts[3] 
    G18_Particle_Cut_lists[run].append(G18_Particle_Cut_Counts[0]/G18_Particle_Cut_total)
    G18_Particle_Cut_lists[run].append(G18_Particle_Cut_Counts[1]/G18_Particle_Cut_total)
    G18_Particle_Cut_lists[run].append(G18_Particle_Cut_Counts[2]/G18_Particle_Cut_total)
    G18_Particle_Cut_lists[run].append(G18_Particle_Cut_Counts[3]/G18_Particle_Cut_total)
    run += 1
print(G18_Proton_Cut_Percent_List, G18_Neutron_Cut_Percent_List, G18_ProNeu_Cut_Percent_List)

In [ ]:
bins = 100
x_start = 0
x_end = 4.0 #GeV
G21_Proton_Cut_Percent_List = []
G21_Neutron_Cut_Percent_List = []
G21_ProNeu_Cut_Percent_List = []
G21_Particle_Cut_lists = [G21_Proton_Cut_Percent_List, G21_Neutron_Cut_Percent_List, G21_ProNeu_Cut_Percent_List]
particles = ["nfp", "nfn", "n_proton_neutron"]
run = 0
cut_num = 1 #is this right
for particle in particles:
    df_gst_GEM21_11b_2Par = df_gst_GEM21_11b_all.Filter(f"{particle}<={cut_num}")
    G21_Particle_Cut_QEL_Ang = df_gst_GEM21_11b_2Par.Filter("qel==1").Histo1D(("G21_Particle_Cut_QEL_Ang", "", bins, x_start, x_end), f"{particle}")
    G21_Particle_Cut_MEC_Ang = df_gst_GEM21_11b_2Par.Filter("mec==1").Histo1D(("G21_Particle_Cut_MEC_Ang", "", bins, x_start, x_end), f"{particle}")
    G21_Particle_Cut_RES_Ang = df_gst_GEM21_11b_2Par.Filter("res==1").Histo1D(("G21_Particle_Cut_RES_Ang", "", bins, x_start, x_end), f"{particle}")
    G21_Particle_Cut_DIS_Ang = df_gst_GEM21_11b_2Par.Filter("dis==1").Histo1D(("G21_Particle_Cut_DIS_Ang", "", bins, x_start, x_end), f"{particle}")
    G21_Particle_Cut_Counts = [G21_Particle_Cut_QEL_Ang.Integral(), G21_Particle_Cut_MEC_Ang.Integral(), G21_Particle_Cut_RES_Ang.Integral(), G21_Particle_Cut_DIS_Ang.Integral()]
    G21_Particle_Cut_total = G21_Particle_Cut_Counts[0] + G21_Particle_Cut_Counts[1] + G21_Particle_Cut_Counts[2] + G21_Particle_Cut_Counts[3] 
    G21_Particle_Cut_lists[run].append(G21_Particle_Cut_Counts[0]/G21_Particle_Cut_total)
    G21_Particle_Cut_lists[run].append(G21_Particle_Cut_Counts[1]/G21_Particle_Cut_total)
    G21_Particle_Cut_lists[run].append(G21_Particle_Cut_Counts[2]/G21_Particle_Cut_total)
    G21_Particle_Cut_lists[run].append(G21_Particle_Cut_Counts[3]/G21_Particle_Cut_total)
    run += 1
print(G21_Proton_Cut_Percent_List, G21_Neutron_Cut_Percent_List, G21_ProNeu_Cut_Percent_List)

In [ ]:
plt.bar(X_axis - 0.2, G18_Proton_Cut_Percent_List, 0.4, label = 'G18')
plt.bar(X_axis + 0.2, G21_Proton_Cut_Percent_List, 0.4, label = 'GEM21', color='r')
plt.ylim(0.0, 1.0)
plt.xticks(X_axis, X)
plt.xlabel("Interaction Types")
plt.ylabel("Relative Percentage")
plt.title("Relative Percentages of One Final State Proton in G18 and GEM21 Models ")
plt.legend()
plt.show()

In [ ]:
plt.bar(X_axis - 0.2, G18_Neutron_Cut_Percent_List, 0.4, label = 'G18')
plt.bar(X_axis + 0.2, G21_Neutron_Cut_Percent_List, 0.4, label = 'GEM21', color='r')
plt.ylim(0.0, 1.0)
plt.xticks(X_axis, X)
plt.xlabel("Interaction Types")
plt.ylabel("Relative Percentage")
plt.title("Relative Percentages of One Final State Neutron in G18 and GEM21 Models ")
plt.legend()
plt.show()

In [ ]:
plt.bar(X_axis - 0.2, G18_ProNeu_Percent_List, 0.4, label = 'G18')
plt.bar(X_axis + 0.2, G21_ProNeu_Cut_Percent_List, 0.4, label = 'GEM21', color='r')
plt.ylim(0.0, 1.0)
plt.xticks(X_axis, X)
plt.xlabel("Interaction Types")
plt.ylabel("Relative Percentage")
plt.title("Relative Percentages of One Final State Proton/Neutron in G18 and GEM21 Models ")
plt.legend()
plt.show()

In [ ]:
#G18 Model 
#df_gst_G18_02a_PTAng = df_gst_G18_02a_all.Filter(f"ptl>{lep_accept_pt}")
G18_PTCut_QEL_Ang = df_gst_G18_02a_all.Filter("qel==1").Histo1D(("G18_PTCut_QEL_Ang", "", bins, x_start, x_end), "sum_kef_proton")
G18_PTCut_MEC_Ang = df_gst_G18_02a_all.Filter("mec==1").Histo1D(("G18_PTCut_MEC_Ang", "", bins, x_start, x_end), "sum_kef_proton")
G18_PTCut_RES_Ang = df_gst_G18_02a_all.Filter("res==1").Histo1D(("G18_PTCut_RES_Ang", "", bins, x_start, x_end), "sum_kef_proton")
G18_PTCut_DIS_Ang = df_gst_G18_02a_all.Filter("dis==1").Histo1D(("G18_PTCut_DIS_Ang", "", bins, x_start, x_end), "sum_kef_proton")
G18_theta_range = [G18_PTCut_QEL_Ang.Integral(), G18_PTCut_MEC_Ang.Integral(), G18_PTCut_RES_Ang.Integral(), G18_PTCut_DIS_Ang.Integral()]